[![Binder](https://mybinder.org/badge_logo.svg)](https://lab.mlpack.org/v2/gh/mlpack/examples/master?urlpath=lab%2Ftree%2Fq_learning%2Fbipedal_walker_sac.ipynb)

You can easily run this notebook at https://lab.mlpack.org/

Here, we train a [Soft Actor-Critic](https://arxiv.org/abs/1801.01290) agent to get high scores for the [Bipedal Walker](https://gym.openai.com/envs/BipedalWalker-v2/) environment. 

We make the agent train and test on OpenAI Gym toolkit's GUI interface provided through a distributed infrastructure (TCP API). More details can be found [here](https://github.com/zoq/gym_tcp_api).

A video of the trained agent can be seen in the end.

## Including necessary libraries and namespaces

In [1]:
#include <mlpack/core.hpp>

In [2]:
#include <mlpack/methods/ann/ffn.hpp>
#include <mlpack/methods/reinforcement_learning/sac.hpp>
#include <mlpack/methods/ann/loss_functions/empty_loss.hpp>
#include <mlpack/methods/ann/init_rules/gaussian_init.hpp>
#include <mlpack/methods/reinforcement_learning/environment/env_type.hpp>
#include <mlpack/methods/reinforcement_learning/training_config.hpp>

In [3]:
// Used to run the agent on gym's environment (provided externally) for testing.
#include <gym/environment.hpp>

In [4]:
// Used to generate and display a video of the trained agent.
#include "xwidgets/ximage.hpp"
#include "xwidgets/xvideo.hpp"
#include "xwidgets/xaudio.hpp"

In [5]:
using namespace mlpack;

In [6]:
using namespace mlpack::ann;

In [7]:
using namespace ens;

In [8]:
using namespace mlpack::rl;

## Initializing the agent

In [9]:
// Set up the state and action space.
ContinuousActionEnv::State::dimension = 24;
ContinuousActionEnv::Action::size = 4;

In [10]:
// Set up the actor and critic networks.
FFN<EmptyLoss<>, GaussianInitialization>
    policyNetwork(EmptyLoss<>(), GaussianInitialization(0, 0.01));
policyNetwork.Add(new Linear<>(ContinuousActionEnv::State::dimension, 128));
policyNetwork.Add(new ReLULayer<>());
policyNetwork.Add(new Linear<>(128, 128));
policyNetwork.Add(new ReLULayer<>());
policyNetwork.Add(new Linear<>(128, ContinuousActionEnv::Action::size));
policyNetwork.Add(new TanHLayer<>());
policyNetwork.ResetParameters();

FFN<EmptyLoss<>, GaussianInitialization>
    qNetwork(EmptyLoss<>(), GaussianInitialization(0, 0.01));
qNetwork.Add(new Linear<>(ContinuousActionEnv::State::dimension + ContinuousActionEnv::Action::size, 128));
qNetwork.Add(new ReLULayer<>());
qNetwork.Add(new Linear<>(128, 128));
qNetwork.Add(new ReLULayer<>());
qNetwork.Add(new Linear<>(128, 1));
qNetwork.ResetParameters();

In [11]:
// Set up the replay method.
RandomReplay<ContinuousActionEnv> replayMethod(32, 10000);

In [12]:
// Set up training configurations.
TrainingConfig config;
config.ExplorationSteps() = 3200;
config.TargetNetworkSyncInterval() = 1;
config.UpdateInterval() = 1;

In the cell below, we load a pretrained model by manually assigning values to the parameters of the network.

The model was trained for 620 episodes.

In [13]:
qNetwork.Parameters() = {0.348429, 0.250762, 0.00764727, 0.167121, 0.362195, -0.118183, -0.116947, 0.316634, 0.274908, -0.0897295, 0.0431199, 0.292214, 0.278132, 0.0541102, 0.556199, 0.313855, 0.255741, 0.204104, -0.0137224, -0.283403, 0.0103152, 0.602315, 0.0232525, -0.163334, -0.0133627, 0.538593, 0.279604, -0.28886, -0.0628659, 0.262696, 0.247593, 0.0308673, -0.0737611, -0.0241807, 0.152502, 0.0296908, -0.0986279, 0.0696473, 0.123486, 0.00211643, -0.628114, 0.0973021, 0.00733307, 0.314282, -0.109412, 0.0134389, -0.127444, 0.165048, -0.0169516, 0.320027, 0.254444, 0.00990014, 0.27605, 0.238408, 0.279624, 0.351463, 0.246444, 0.259626, 0.305415, 0.294717, 0.00186069, -0.0202414, -0.0244248, 0.246511, -0.248147, 0.613578, -0.0308112, 0.162584, 0.0371756, 0.12135, 0.287912, 0.106763, 0.416342, -0.0214829, 0.273712, -0.0105721, -0.323492, 0.270683, 0.250824, -0.0383819, 0.282092, -0.0778101, 0.460366, 0.282068, 0.237487, 0.0781438, 0.361283, 0.344164, 0.115724, 0.118839, -0.00246232, 0.00870651, 0.0235675, 0.315704, 0.390625, 0.203323, 0.276126, -0.03874, 0.0839244, 0.403117, -0.0141599, -0.0326628, 0.0417172, 0.0158085, 0.246186, 0.352444, 0.227322, 0.674259, -0.0266094, -0.00953717, 0.0650779, -0.0325739, 0.230889, -0.383747, 0.0509163, 0.233012, 0.0548984, -0.327411, 0.261888, 0.256204, 0.0711636, -0.300284, -0.00389919, 0.269765, 0.114837, -0.0509072, 0.33306, 0.276661, -0.215023, -0.066591, 0.334837, 0.0870963, -0.365146, 0.146019, 0.112168, -0.102239, -0.108508, -0.188638, -0.0423591, -0.0915623, -0.0375486, -0.0112031, -0.144485, -0.111208, -0.0871162, -0.00880978, 0.126054, -0.00398403, 0.0328809, 0.188382, -0.0457659, 0.00812524, 0.0166279, -0.0231946, -0.138651, 0.50766, -9.90326e-05, -0.146416, -0.146538, 0.432348, -0.0512136, -0.0327857, -0.114258, -0.176761, 0.00986546, 0.0521264, 0.127102, -0.0104198, 0.0534913, -0.284187, -0.134105, 0.318943, 0.0953906, 0.0152895, 0.0979048, 0.30469, 0.0321041, -0.0671057, 0.0769446, 0.00764932, -0.0506532, -0.105942, -0.121149, -0.0871768, -0.133737, -0.15335, -0.116942, -0.0325367, -0.0270547, 0.0360258, -0.0846611, -0.154813, 0.0179797, 0.324997, 0.00237472, -0.115589, -0.0161152, 0.126867, -0.117739, -5.80478e-05, 0.0999334, 0.016582, -0.0869547, 0.0289184, 0.0669435, -0.12131, -0.0627046, -0.0240454, -0.0980459, -0.0599776, -0.0983582, -0.0995628, -0.139286, 0.0889608, -0.102332, -0.183286, 0.0177573, 0.130288, -0.00492125, 0.0160005, -0.0408392, -0.12975, 0.90505, -0.0740958, -0.133638, -0.148763, 0.0694549, -0.0863162, 0.449133, 0.104066, 0.052349, -0.0263132, -0.160649, 0.124851, -0.0981323, -0.00412959, 0.165194, 0.032355, -0.0376777, -0.323008, -0.0976919, 0.0455335, -0.0253169, -0.130035, -0.0520023, -0.489224, -0.130326, -0.119511, 0.0998697, 0.631954, 0.157497, -0.0771549, 0.0252212, -0.013515, 0.127305, -0.152247, 0.0251158, -0.0669855, 0.274454, -0.696979, -0.0686733, 0.180437, -0.147739, -0.0659491, -0.136545, -0.0708595, 0.0460642, -0.115838, -0.153287, -0.120868, 0.0889588, -0.0863071, -0.145802, -0.263701, 0.127819, -0.0452432, -0.091742, -0.188499, 0.00119093, 0.436073, 0.0294555, -0.0507952, -0.0950587, 0.138502, 0.0862833, -0.0708271, -0.0761374, -0.259359, 0.0991439, -0.0653571, -0.0256838, 0.203026, -0.0996198, 0.0355808, 0.0708439, -0.0178238, 0.20121, 0.273039, -0.0601821, 0.163892, 0.0200561, -0.0421175, 0.00866367, -0.447757, 0.0747786, -0.0961081, 0.183137, -0.0155458, -0.127815, -0.130287, -0.094424, -0.13592, -0.1437, -0.104191, -0.0803078, -0.226836, 0.015394, 0.0187661, 0.100769, -0.109709, 0.338171, -0.318668, -0.00545371, 0.108274, 0.0236997, 0.0647681, -0.116704, -0.181772, -0.178602, -0.0230838, -0.108203, 0.000197701, 0.204964, -0.127549, -0.0618039, -0.000743997, -0.0743898, 0.00891065, -0.180297, -0.11024, -0.144248, -0.047668, -0.124185, -0.145163, -0.147593, 0.125536, 0.0155093, -0.0117363, -0.00632806, -0.0862564, -0.843881, -0.184548, -0.252468, 0.0255983, -0.158666, -0.0936556, -0.0472094, 0.253456, -0.107091, 0.0167307, -0.0790096, -0.155016, -0.0790912, -0.110237, 0.190669, -0.010581, 0.00815208, -0.0245515, -0.108314, -0.0358968, -0.0352293, -0.162518, -0.105455, 0.0169136, -0.0923783, -0.107741, -0.105125, -0.813217, 0.0158402, -0.147215, 0.245687, 0.010934, -0.038318, -0.0939954, 0.160702, -0.0119068, -0.0479805, -0.815199, 0.457187, -0.108215, -0.137907, 0.0165703, -0.0748028, -0.0843851, 0.0179095, 0.00336316, -0.156458, -0.0288157, -0.41507, -0.0109491, 0.0166424, 0.149527, -0.0577998, 0.103317, 0.131407, 0.0466429, 0.0048987, 0.150438, 0.0438157, -0.319764, 0.0196086, -0.573872, 0.0436607, -0.0186679, -0.0466519, -0.206371, -0.0614021, 0.0637144, -0.261886, 0.241691, 0.0082745, 0.0156508, -0.0663, -0.0196121, -0.115523, 0.0896024, 0.100077, 0.146357, 0.119043, -0.0277939, -0.00504321, -0.54327, -0.0596775, -0.0758994, -0.575658, -0.00258222, -0.0371209, -0.00223187, -0.0128131, 0.000854033, -0.0549928, -0.0576899, -0.0849283, 0.0880988, -0.00734429, -0.08157, -0.0239335, 0.0339964, 0.128317, -0.0637654, 0.0579255, -0.0182022, 0.0263835, -0.0657184, 0.000119436, 0.101028, 0.487459, 0.0353341, -0.0150103, -0.00799408, -0.0883064, -0.0808748, 0.0550816, -0.0205543, 0.00101321, 0.113686, -0.0322477, -0.046205, -0.0153178, -0.0211621, 0.0574103, 0.0513266, 0.176951, 0.0864601, 0.0107276, 0.00284104, 0.0810754, 0.0154951, 0.115989, 0.130042, 0.00144385, 0.0794671, 0.18155, -0.016524, 0.215099, -0.102305, 0.0783232, -0.00357237, -0.0301759, 0.183204, -0.00102663, -0.335206, -0.0293939, -0.00860475, -0.0290379, 0.0396604, 0.00493643, -0.280242, -0.0502549, 0.0388416, -0.00860592, -0.00941093, -0.037238, 0.00654776, 0.0571981, 0.855064, -0.0473353, -0.0758828, 0.333054, 0.0187711, -0.246262, 0.0458198, -0.00437949, 0.111088, -0.138436, -0.324949, 0.134528, -0.899275, 0.669647, 0.115802, 0.135833, 0.475798, -1.91519, 0.112401, -0.0352729, 0.93367, 0.204456, 0.134883, 0.144094, 0.118128, 0.604797, -0.458362, -0.126831, 0.18126, -2.89752, 0.3208, -1.45253, 0.0507811, 0.114484, 0.188581, 0.312731, 0.120164, 0.0916022, 0.345847, -1.59406, -1.35599, 0.00869182, -1.74618, -1.55748, 0.219284, -0.663914, 0.00252106, 0.15529, -0.432664, -0.496817, -1.30121, -0.795224, -1.44122, -1.14543, 0.114514, 1.12202, 0.126871, -0.147785, -0.00529447, 0.111817, 0.107449, 0.141784, 0.137721, 0.128808, 0.120743, 0.102041, 0.110374, 0.00954989, 1.87017, -1.27016, 0.113191, 0.908786, -1.03204, -2.71382, 0.764704, -2.26675, 0.87576, 0.0817871, -1.0808, -0.137005, 1.31759, 0.109231, -0.00329131, -1.57187, 0.12531, 0.936814, -1.80778, 0.132098, -1.24831, 0.164196, 0.10493, 0.123499, 0.406141, 0.0893537, 0.136692, 0.33521, 0.535923, 0.00519051, -0.489725, 1.6909, 0.130047, -0.740303, 0.103159, 0.162657, -1.10875, 0.799551, 0.121849, -0.34323, -0.981235, -1.27411, 0.0249896, 0.111332, 0.17089, 0.041042, -0.1335, 0.0922617, 0.448337, 0.548148, 0.191748, 0.109467, 0.429814, -1.76785, 0.0954035, -1.04046, 0.586795, 0.122647, 0.135266, -0.959546, 0.0286517, 1.16638, 0.10025, -0.0966248, 0.580344, 1.18315, 0.11827, 3.419, 0.185264, -0.214269, -3.44274, 0.705769, -2.48504, 0.798076, 0.235654, 0.22149, 1.79299, -0.128705, 0.188153, -0.108079, -0.29004, 0.445881, 0.200148, 0.229008, 0.244308, 3.46569, -0.434813, 0.590748, 0.0605424, -4.15984, -0.330058, -1.83691, 0.290445, 0.18023, -0.00720027, -5.80772, 0.220435, 0.244126, 0.918821, -2.55225, -4.22345, -0.402373, -5.80169, 2.29461, 1.41568, 1.03943, 0.00963004, -1.31178, 0.757714, -6.04523, -6.33474, 0.135043, -7.90617, -2.23242, 0.75499, 1.35362, 0.270507, 0.295545, 0.0107685, 0.204076, 0.181611, 0.210543, 0.207086, 0.190784, 0.220993, 0.212762, 0.261157, 0.0193642, 4.42145, -0.816631, 0.208131, 0.938814, -0.198474, -4.03307, 1.04931, -6.67467, 2.06194, 0.147783, -0.790537, 1.89273, 5.11116, 0.227985, -0.00905733, -4.14582, 0.212309, 5.25806, -2.1938, 0.226844, -2.36268, 0.563686, 0.210679, 0.214695, 0.871894, 0.253165, 0.362658, -2.38307, 0.861587, 0.00249064, -0.368144, 3.54947, 0.191573, 0.0775985, 0.175373, 0.549772, -5.59934, 1.03391, 0.265276, 1.69732, 0.521136, -2.53566, 0.0548505, 0.186485, 0.136991, -0.143641, 0.317406, 0.822943, -0.431414, 1.02969, -1.6881, 0.20229, -1.85658, -3.35892, 0.197098, -0.604901, 1.34129, 0.188468, 0.216438, -4.70161, 0.557518, -2.75223, 0.180902, -0.123239, 1.66606, 2.09865, 0.179138, 0.647131, -0.132701, -0.454183, -1.06739, -0.0511535, 0.0329364, -3.33061, -0.197809, -0.144059, -1.80374, 0.999857, -0.183493, 0.235243, 0.537317, -0.952042, -0.200171, -0.20664, -0.114698, -0.620842, -0.507352, -0.113881, 0.617429, 2.05223, -0.556889, 0.25733, -1.02904, -0.17437, -0.87008, -3.08133, -0.18135, -0.189412, 0.933247, 0.75941, 0.935581, 0.610971, -0.335647, 2.78817, 0.613843, 1.2968, -0.0314734, 1.60946, 0.452019, 0.548857, -0.224192, -2.06247, 0.48529, 0.893365, -0.863861, 0.306944, -0.264838, -0.455542, 0.0221433, -0.158551, -0.146664, -0.192842, -0.146416, -0.135848, -0.158133, -0.239683, -0.182589, 0.0406292, -0.555308, 2.99117, -0.167136, -0.814927, 0.246982, 1.99479, -1.15192, 2.26353, -1.94239, -0.152085, -0.0941899, 0.200887, -2.34963, -0.209995, 0.0364559, 0.189641, -0.168634, -0.63376, 3.04683, -0.178155, 1.08227, -0.00687923, -0.188447, -0.167324, 0.152785, -0.288994, -0.221216, -0.546809, 0.299309, -0.00959458, 0.971236, -1.35921, -0.203522, 1.08944, -0.248923, -0.329962, -0.527299, -1.69162, -0.254595, -1.2184, 0.539354, 0.240566, 0.093358, -0.151407, -0.209724, -1.21876, -0.466955, -1.19075, -1.10036, 2.8801, 1.65962, -0.166522, -1.09365, 0.998547, -0.20905, 0.844229, -2.50352, -0.169533, -0.160364, 2.53163, 0.0188882, -1.5434, -0.108882, 1.31306, 0.196417, 0.673891, -0.171713, 0.619004, -0.203496, -1.90477, -0.645687, -0.498006, -1.37215, -0.496928, -0.208281, -0.1897, -0.289001, 0.79716, -0.161437, -0.214976, 0.55768, -0.364246, -0.143948, -0.177497, -0.194123, -0.919102, -1.29776, -1.61282, 0.798211, -0.612399, -0.936576, -0.915493, -0.375716, -0.164568, -1.82727, 2.53526, -0.178596, -0.143771, -0.670445, -2.02439, 0.442897, -1.1202, 0.176164, -0.905621, -2.64835, -0.468143, 0.0245745, -1.28479, -0.615465, 1.11337, 0.768147, 0.701352, 2.36899, 1.0167, 0.898821, -0.0330729, -0.153289, -0.729795, 0.0107742, -0.214695, -0.203122, -0.169341, -0.149401, -0.228419, -0.16284, -0.212334, -0.138245, 0.0623308, -0.119704, -1.81237, -0.204269, -0.55234, -0.883069, 0.312979, 0.254866, 0.806856, -0.597945, -0.150304, -0.81552, 0.113144, 0.931378, -0.188232, 0.0301286, 1.3384, -0.184746, 1.3587, -1.14012, -0.121455, -0.297352, -0.433434, -0.17581, -0.155172, -1.8084, -0.212958, -0.110332, -2.99924, -0.820011, 0.0112125, 5.18689, -1.97137, -0.164419, 0.464207, -0.159542, -0.355479, 0.423493, -0.902973, -0.17258, 1.25669, -1.56996, -2.21571, 0.0290835, -0.163747, -0.255979, 0.213575, 0.203962, -0.686025, -1.01197, 2.43471, -3.34408, -0.182504, 1.27003, -1.58158, -0.182079, -1.60147, 0.716574, -0.23301, -0.22472, 1.65844, 0.42511, -2.89369, -0.0888385, -1.65728, -2.37322, -1.66591, -0.151179, -0.41135, 0.0473637, 0.787556, 0.0426354, 0.0920808, 0.237919, 1.1626, 0.0107131, 0.0251858, 1.06405, -0.276419, 0.035284, 0.143614, 1.20817, 0.0656248, 0.0483988, 0.0821027, -0.00340868, 0.522995, 0.820906, 0.254379, 0.224975, -0.364224, 0.260096, -0.117425, 0.111992, -0.013796, 0.427862, 0.202923, 0.0372444, 0.0984373, -0.932437, -1.36982, -1.25974, 0.607206, -0.0712334, 0.0500816, -0.0717349, 0.905004, -0.00083326, -1.64976, -0.501932, -0.0191319, -0.762366, 0.528093, -0.389223, -0.533418, 0.288738, -1.34674, 0.0054891, 0.251551, -0.0033587, 0.045676, 0.00875965, 0.0677931, 0.078948, 0.0370984, 0.0150223, -0.0232718, 0.0578247, 0.0203273, 0.636832, -1.12018, 0.0439916, -0.440715, 0.529527, -1.50304, 0.601876, -0.371678, 0.0611557, 0.0405653, 0.497139, -0.362163, 0.76364, 0.0270256, 0.00465363, -0.190897, 0.03503, -0.128064, -0.861676, 0.0246292, -0.678202, 0.0327104, 0.0258121, 0.0334181, 1.52627, 0.0497624, 0.0215617, -0.457641, 0.228863, 0.00764158, 0.599967, -0.376409, 0.0486194, -0.715226, 0.0194665, 0.22225, -1.27031, 0.32142, 0.113001, -1.75941, 0.450032, -0.641759, 0.0754754, 0.0490455, -0.0185185, -0.247172, 0.0401561, -0.285231, -1.1184, 0.245918, -0.443024, 0.0206356, 0.818102, 0.054898, 0.0385116, 0.100174, -0.320835, 0.0111323, 0.0142225, -0.0629384, -0.457169, 2.48642, -0.00397146, -0.368513, 0.425196, -0.851844, 0.0361059, 0.313838, -0.200758, -0.178284, -0.497327, 0.241234, -0.0325168, 0.176029, -0.153833, -0.157002, -0.190566, -0.0157288, -0.14241, -0.140662, 0.339245, -0.08854, -0.161731, -0.223832, -0.252626, 0.0326944, 0.124618, 0.215437, -1.19037, -0.204826, -0.311402, -0.0495203, -0.205515, -0.125543, -0.0357864, -0.272334, -0.122748, -0.0749766, 0.412038, -0.176201, -0.168729, 0.690982, -0.216324, 0.0130807, -0.00185487, -0.0325439, -0.0011558, 0.395638, 0.565973, -0.267917, -0.000807856, 0.440669, -0.247964, -0.424642, -0.133019, -0.152732, -0.114529, -0.185614, -0.00435387, -0.143867, -0.19146, -0.160854, -0.27493, -0.0746884, -0.101797, -0.00516297, -0.373599, -0.00547672, -0.0289883, 0.145048, -0.147064, 0.00484442, -0.0813111, -0.245486, 0.194276, -0.15582, 0.246582, -0.149325, 0.508903, 0.104502, 0.184649, -0.180707, 0.00219966, -0.3142, -0.148251, 0.143696, -0.223333, -0.188558, 0.0102892, 0.157854, 0.0183798, -0.109458, 0.0902547, -0.137104, -0.313898, 0.140337, -0.33698, 0.028351, 0.187081, 0.0314946, -0.0919907, -0.23974, -0.241567, 0.200568, -0.363803, 0.175358, -0.00491485, -0.00559775, -0.0986354, -0.0601551, -0.0241639, -0.147233, -0.292805, 0.156959, 0.56405, -0.678635, 0.0724751, 0.209078, 0.239359, -0.17177, -0.654063, -0.0146503, -0.121325, -0.0338751, 0.0050969, -0.0977207, -0.15663, -0.154989, 0.0587277, -0.289953, -0.165662, -0.297098, 0.084192, 0.809796, -0.123763, -1.21508, -0.026899, 0.482152, 0.235741, -0.202533, -0.452677, 0.172498, 0.00249786, 0.01417, -0.0199836, -0.225529, 0.0646405, -0.265379, 0.895164, 0.376067, 0.0335905, 0.0997241, -0.0902278, 0.841525, -0.252384, 0.994048, 0.485687, -0.708127, -0.623217, 0.747985, 0.258427, 0.0277774, -0.566543, 0.304217, -0.0378431, -0.0478604, -0.48964, -0.373317, -0.43594, 0.117072, 0.127259, 0.0293073, 0.52045, 0.0652613, 0.00409087, -0.850453, 1.42995, 0.363185, -0.382003, 0.424121, -0.0772384, -0.41342, 0.31627, -0.517785, 0.0057973, -0.359566, 0.00194085, 0.0727683, 0.011213, 0.027029, 0.0483174, 0.0721414, 0.0150032, 0.110775, 0.0380478, 0.0200671, 0.172744, -1.27354, 0.0179213, -0.540731, 0.0380963, -0.599806, 0.529029, -0.313578, 0.766999, 0.0752655, 0.127202, -1.23023, 0.391922, 0.0479664, 0.043961, -0.694129, 0.0610059, -1.21589, -0.764724, 0.00634827, -0.597633, 0.125188, 0.0529113, 0.050512, 0.58875, 0.142267, 0.111227, -0.286626, 0.34414, 0.0717843, 0.0712556, -0.370453, 0.0548757, -0.679886, -0.0175082, -0.0857354, -0.616264, 0.22187, 0.126118, -0.0813967, 0.569112, -0.301471, 0.0119331, 0.0629842, -0.12751, -0.440561, 0.307293, 0.439888, 0.0884418, 0.0660871, -1.63752, 0.0267061, -0.595242, 0.167898, 0.0869812, -0.127385, 0.205276, 0.016077, 0.0271595, 0.207388, -0.0381353, 0.40472, 0.0685219, 0.826315, 0.213986, -0.407382, 0.00142873, -0.209815, -0.0110186, -0.204314, -0.338346, 0.380753, 0.140731, 0.3812, -0.0320297, 0.000942517, -0.0188674, -0.0581877, -0.0275037, -0.356945, 0.0521016, -0.215995, -0.00403096, -0.0095482, -0.0146498, 0.273556, -0.136044, 0.0310049, 1.01307, 0.0135232, -0.179115, -0.00502119, -0.124118, 0.0692646, 0.0574764, -0.0310986, 0.0182821, 0.0581708, 0.774684, -0.0205366, -0.00241887, 0.13886, -0.238685, 0.113877, 0.00608939, 0.0978611, -0.0137429, -0.421061, 0.726635, -0.176609, 0.0572886, 0.36021, -0.12479, -0.18027, 0.0786839, -0.00268359, -0.0585718, -0.116818, 0.00010182, -0.0771314, -0.0235846, -0.00983179, -0.0804548, -0.000230569, 0.0114298, -0.0353031, 0.0424926, 0.0197507, 0.00568727, 0.240753, 0.00387835, 0.123222, 0.194231, -0.00276744, 0.295409, -0.0882068, 0.13775, -0.0241999, 0.438648, 0.212596, 0.0527457, -0.0238148, 0.0149369, -0.225821, 0.0208566, -0.373576, -0.0798813, -0.0453407, -0.0494347, 0.146926, 0.0274412, -0.0233481, 0.0326056, -0.0144543, -0.022043, -0.0216882, -0.1609, 0.0112915, -0.0495527, 0.0266699, -0.00414444, 0.299675, 0.036865, -0.117379, -0.290647, -0.16825, -0.0230085, 0.0529768, -0.174271, -0.110189, 0.0113228, 0.0335035, -0.0563773, 0.217248, -0.426966, 0.537272, -0.0933325, 0.0727439, 0.272817, -0.00423248, -1.11664, -0.0159017, 0.0590363, -0.0581149, -0.21762, -0.00150982, 0.013547, 0.00854249, 0.134665, -0.0880497, -0.0102423, 0.697822, 0.0224245, 0.187428, -0.0382352, -0.822507, -0.000586114, -0.0971854, -0.277746, -0.780147, 0.00803308, 0.496201, 0.0152088, -0.0280537, 0.700653, -0.18539, -0.0105393, -0.36291, 0.163697, -0.107548, 0.0305257, -0.00792483, -0.157635, -0.152631, 0.0549303, -0.421732, -1.74254, -0.185129, 0.319495, -0.459897, -0.225796, 0.013114, -0.269747, 0.0143103, -0.02854, 0.0578058, -0.264148, -0.25376, -0.191317, -0.59597, -0.52339, 0.169148, -0.255993, -0.0519894, -0.0150834, -0.119764, -0.0041951, -1.33879, -0.0264528, 0.0636109, 0.000370543, 0.771243, -1.05092, -0.0848578, 0.0623089, -0.169341, -0.032492, 0.0105037, -0.0241807, 0.0240071, 0.0209654, 0.00152992, -0.030832, -0.211996, -0.0447211, -0.0107913, -0.00523839, 0.347098, -0.00855527, 0.242559, -0.4082, 0.126308, 1.2321, -0.0300418, -0.0698463, 0.00161878, -0.392879, -0.764412, 0.0389622, -0.0104738, -0.0301737, 0.530574, -0.0794564, -0.613778, 0.222661, 0.0306564, -0.0448758, 0.0529038, -0.0839802, -0.0354639, 0.296616, -0.00680439, 0.101213, -0.0334657, -0.527553, -0.0607742, 0.455184, -0.125239, -0.00126928, 0.953248, -0.0977723, -0.592202, -0.101569, -0.076401, -0.205925, -0.0863244, -0.219495, -0.226196, -0.0859923, -0.0444208, -0.00139438, 0.138295, 0.763838, 0.0913069, -0.451527, -0.286099, 0.789428, -0.0140903, 0.779181, -0.0329016, -0.0804445, 0.292982, 0.0916345, -0.132631, -0.0387615, -0.125798, -0.0187542, 0.666524, -0.0121245, 0.322714, 0.453493, 0.364362, -0.0226293, 0.824032, -0.0126187, 0.400041, -0.178725, 0.297559, -0.548256, -0.745157, 0.0319325, -0.0122623, 0.547593, -0.393651, 0.041979, -0.617049, -1.50445, 0.0777188, -0.00661548, 0.0783761, 0.132209, -0.00830879, -0.631167, 0.75368, -0.208703, -0.684246, 1.55244, -0.165614, -0.109183, 0.0196676, 1.11774, -0.00678622, -0.00833897, 0.124482, -0.329795, -0.0869718, 0.187099, -0.319156, -0.0233174, -0.200372, 0.228608, 1.22366, -0.0268735, -0.657412, -0.756489, -0.827104, 0.182424, 0.0714687, -0.495806, -0.263, 0.803962, 1.9696, 0.123404, -0.455872, 0.00269457, 0.0597504, 0.013155, 0.0249116, 0.0688976, 0.036559, -0.0153529, 0.170453, 0.0758899, -0.0123792, 0.62091, 0.630161, -0.00685473, 0.121886, -0.307142, -1.45161, -0.589665, -0.0635056, 0.359269, 0.0397413, -0.56522, -0.0577396, 0.172154, -0.0209129, -0.00767816, -0.435657, 0.0365971, -0.339264, -0.87665, -0.0630199, 0.0464702, -0.0490226, 0.0508209, -0.00964694, -0.34974, 0.0353886, 0.0548286, 2.05756, 0.0306791, -0.0142288, -0.662319, 1.72646, -0.00430926, -0.226658, 0.0313124, 0.0671976, -0.37309, -0.276092, 0.130619, 0.945309, -0.533081, 0.0505046, -0.00567217, 0.0290496, -0.0280922, -0.284149, -1.78443, 0.473153, -1.01102, 0.268645, -0.746759, 0.0658869, -0.222279, -1.3093, 0.0530636, -1.28077, -0.477282, 0.0268711, -0.00521684, -1.09675, -0.659575, -0.611584, 0.104009, -0.385126, -0.399544, -0.544972, 0.0570686, 0.0561456, 0.0733269, 0.614198, -0.534042, -0.0824604, 0.097119, -0.382751, 0.0916869, 0.0785142, 0.0761513, 0.0398133, 0.0704361, 0.29366, -0.209386, 0.198743, 0.0450153, 0.0479214, -0.0228718, -0.135015, 0.0934951, -0.0862981, 0.088385, -0.183996, 0.34052, 0.0300255, 0.316764, 0.00516157, -0.204405, -0.12472, 0.0390415, -0.0626143, 0.0579968, 0.0837877, -0.210214, -0.505673, 0.150344, 0.190218, -0.021334, 0.227113, -0.0134131, -0.154403, 0.316015, -0.123918, -0.170626, -0.514724, -0.314097, -0.139046, -0.766332, 0.131294, 0.122286, 0.00170552, -0.00068875, 0.0475655, 0.0437919, 0.0931307, 0.0809306, 0.0825684, 0.048729, -0.0534975, -0.0395717, 0.00811241, 0.0905114, 0.182878, 0.0289359, 0.297036, -0.128015, -0.100428, 0.224875, -0.11789, -0.189435, 0.0387525, -0.323652, -0.0435886, 0.122555, 0.0607709, -0.029186, -0.0544548, 0.0589942, -0.0268947, -0.180959, 0.0354522, -0.170373, 0.213198, 0.0180724, 0.0213268, 0.295274, 0.0314977, -0.15456, -0.281528, 0.452151, 0.0191189, 0.215794, 0.051255, -0.00674342, 0.783189, -0.0458119, 0.0365633, -0.177429, 0.180591, 0.204493, 0.636151, 0.153493, -0.174955, -0.0164172, 0.0757634, -0.121731, 0.0576532, -0.136397, -0.505488, -0.0767684, 0.1318, 0.158919, 0.0363788, 0.883496, -0.266687, -0.120828, -0.0330431, -0.398114, 0.0690273, 0.114556, -0.204525, -0.684927, -0.0461551, 0.155844, 0.615226, 0.109501, -0.170456, 0.0466916, 1.30209, 0.166765, 0.718791, -0.398101, -0.658183, -0.557389, -0.268418, 0.101329, 0.148269, 0.0696847, -0.639412, 0.0579521, 0.295546, -0.804669, 0.343647, 0.0377302, 0.0616961, -0.0429702, -0.499716, 0.785326, 0.223675, 0.224444, -0.441262, -1.13216, 0.316281, 0.265674, 0.0221406, -0.548364, -0.052608, 0.156868, 0.131001, -0.913504, -0.0114541, 0.206233, -0.0992346, 0.344383, -0.574009, 0.512208, -0.36733, 0.00640667, -0.654895, 0.0165471, -0.46363, 0.341538, -1.10599, -0.297052, 0.222939, -0.558183, 0.628316, 0.128305, 0.112304, -0.00326704, 0.12398, 0.0376018, 0.014167, 0.01543, 0.165507, 0.0935681, 0.122616, 0.0201816, -0.01873, 0.30458, -0.0572775, 0.0952476, 0.0765584, 0.739944, -0.322398, -0.656987, -0.112897, 0.160393, 0.0676337, 0.599162, 0.446724, -0.18274, 0.0679078, 0.0137353, 0.0962484, 0.123581, 0.0906757, -0.246264, 0.103999, 0.0682259, 0.106448, 0.167796, 0.0521134, 0.703729, 0.0454297, 0.081234, -0.0328748, 0.017837, 0.0444957, -0.645101, 0.77492, 0.0579238, -0.442773, -0.0121796, 0.113646, 0.129559, 0.782351, 0.122376, -0.277576, -1.69295, 0.00125803, -0.00525653, 0.084751, -0.201914, -0.356971, 1.33983, 0.296154, -1.55119, -0.352456, -0.610175, 0.0889806, 0.239595, -0.554093, 0.0819581, -1.11878, -0.820572, 0.139757, 0.157264, -0.448946, 0.351015, -0.30229, 0.086086, -0.428935, -0.136055, -1.09845, 0.0404638, -0.00294281, -0.118239, -0.366102, -0.162169, -0.0421348, 0.0617671, -0.0381985, -0.160219, -0.0479997, 0.285672, 0.0327722, -0.179584, 0.320342, -0.0397862, -0.199091, -0.164412, -0.163131, -0.218724, 0.0253017, 0.23331, -0.0156983, -0.225589, -0.110425, 0.23214, 0.0364494, -0.153675, -0.0533588, 0.102413, -0.0896955, -0.118345, -0.119979, -0.157778, -0.0490446, -0.138191, 0.0466875, -0.151713, 0.126345, -0.0125171, 0.0532316, 0.0173824, 0.0112716, -0.0734657, -0.0860092, -0.205103, -0.159788, -0.148869, -0.124612, -0.60348, -0.0741661, -0.00368753, -0.131053, -0.00901092, -0.082414, -0.12644, -0.157426, -0.153969, -0.0797909, -0.11816, 0.0170796, -0.136613, 0.0112114, 0.0527503, 0.0502398, -0.101753, 0.583192, -0.0948807, 0.00466719, -0.579272, -0.0442503, -0.112649, -0.114391, 0.282139, 0.638926, -0.00911784, -0.144775, 0.0273802, -0.290385, -0.0645271, 0.0157131, 0.026154, -0.166378, 0.0841867, 0.116425, -0.0253972, -0.0751373, 0.159892, -0.121915, -0.118593, -0.255319, -0.298588, 0.0287491, 0.017006, 0.120924, -0.17671, -0.0970322, -0.15677, -0.192966, -0.0363386, 0.0205104, -0.0984763, 0.113207, -0.18668, 0.0437375, 0.00938615, -0.0958498, -0.15974, -0.022265, -0.229611, -0.194296, 0.0973349, 0.0109137, -0.0345929, -0.112894, -0.0541935, 0.015112, -0.0674892, 0.170236, -0.274675, -0.029957, -0.0759687, -0.0723466, 0.441203, 0.0660394, -0.0536317, -0.0929464, 0.0178603, 0.140426, -0.129138, 0.624619, -0.0229609, 0.633586, -0.631796, -0.115684, -0.162584, -0.259847, -0.0159206, 0.00342437, -0.102034, -0.0630594, 0.00494759, 0.0121444, -0.0201344, -0.634517, -0.0452966, 0.0251181, 0.0455207, 0.366846, -0.0800365, -0.0787241, -0.312104, 0.0329916, 0.315713, -0.109055, -0.585345, 0.0388432, -0.723791, -0.256028, -0.0548652, -0.0642738, -0.632493, -0.0403344, -0.0526335, -0.628822, 0.426843, 0.10807, -0.234355, -0.259674, 0.00808254, -0.146084, -1.50143, -0.132366, 0.0314079, 0.326508, 0.0585238, 0.309763, -0.609734, 0.00126923, -0.00537366, -0.56864, -0.0108777, -0.0546838, 0.012116, -0.0051673, 0.0352519, -0.0582098, -0.0351503, 0.142973, 0.0343786, -0.0201983, 0.0797437, -0.187124, 0.0145029, -0.266549, -0.513149, 0.0512204, 0.0174188, -0.0439517, -0.512434, 0.00323731, -1.15047, -0.263279, 0.104901, -0.015164, -0.0153678, -0.460801, -0.0562332, -0.18276, 0.149155, -0.0600007, 0.299424, 0.0207639, 0.0174465, -0.0471996, -0.589886, 0.158031, 0.0504631, 1.31583, -0.0817805, -0.0377212, -0.0393584, 0.238249, -0.0670959, -0.133196, 0.0265929, -0.663282, -0.0677964, -0.0530845, -0.0962782, -1.72674, -0.642325, -0.245012, -0.0648955, 0.0380604, 0.0720422, -0.112058, -3.23611, 0.476307, 0.145081, -0.0698282, 0.511251, -0.00946064, 0.262897, 0.126688, 0.0965059, 0.551807, 0.0904142, -0.0434692, 0.00362835, 0.163017, -0.0976343, -0.186226, -0.0579591, -0.638375, -0.0292489, 0.718785, -0.0432937, 0.272071, -0.290548, -0.327137, 0.107369, -0.648414, -0.123316, -0.0323619, -0.314772, -0.308972, -0.260319, 0.0427586, -0.310159, -0.252118, 0.00797473, -0.642086, -0.319685, -0.301602, -0.388428, -0.145217, -0.522411, -0.507222, -0.20664, -0.26127, -0.551712, -0.397401, -0.627983, -0.338945, -0.354468, 0.378719, -0.312599, -0.362501, -0.47947, -0.298619, 0.131313, -0.624637, 0.028191, -0.192386, -0.521462, -0.3274, -0.0608739, -0.264885, 0.0405952, 0.0758096, -0.0703306, -0.117175, 0.223107, 0.202042, -0.405313, 0.157733, -0.33366, -0.66492, -0.0346087, -0.330302, -0.33276, -0.317705, -0.317339, -0.312649, -0.349166, -0.408555, -0.349847, -0.0531731, 0.240095, -0.446482, -0.311133, -0.29173, -0.251804, -0.338257, -0.295332, -0.155313, -0.667995, -0.33472, -0.236782, 0.000797754, 0.320595, -0.309951, -0.032404, -0.187265, -0.352733, 0.395333, -0.391493, -0.368258, -0.0173527, -0.496625, -0.336128, -0.348963, -0.296241, -0.404305, -0.415307, -0.326298, -0.851044, -0.108676, 0.204721, 0.111326, -0.331518, -0.479985, -0.346131, -0.507962, 0.0939195, -0.471349, -0.349711, 0.0490495, -0.0523941, -0.431487, -0.124608, -0.337119, -0.460786, -0.530093, -0.281467, -0.659417, -0.227649, 0.243041, -0.649254, -0.316504, -0.327222, -0.0293634, -0.355879, -0.317703, -0.230174, -0.328354, -0.328282, 0.119553, -0.408044, -0.136633, -0.324307, -0.186963, -0.497958, -0.418144, -0.319816, 0.25712, -0.288021, -0.332707, 0.0946577, -0.650516, -0.103305, -0.0229077, -0.333733, -0.336574, -0.272343, 0.0474991, -0.310827, -0.2404, 0.0237612, -0.651802, -0.320332, -0.304628, -0.407394, -0.139252, -0.518821, -0.51311, -0.200552, -0.291401, -0.559943, -0.381396, -0.615564, -0.347353, -0.36154, 0.36166, -0.316853, -0.351813, -0.479056, -0.328509, 0.102588, -0.625906, -0.00313495, -0.237685, -0.547184, -0.336167, -0.0562278, -0.272458, 0.0405658, 0.0648942, -0.079379, -0.128669, 0.223806, 0.197083, -0.402839, 0.168681, -0.34631, -0.679009, -0.0466329, -0.328583, -0.334413, -0.31766, -0.329689, -0.304191, -0.331617, -0.410095, -0.346812, -0.0445389, 0.265033, -0.462285, -0.321508, -0.298751, -0.223677, -0.361326, -0.311889, -0.164791, -0.668266, -0.334505, -0.218754, -0.00617823, 0.327706, -0.330275, -0.0451653, -0.214637, -0.333015, 0.375129, -0.365764, -0.364051, -0.0449583, -0.52412, -0.335882, -0.334905, -0.261564, -0.40232, -0.399848, -0.305082, -0.838777, -0.111428, 0.228676, 0.0944141, -0.34073, -0.501977, -0.346304, -0.496702, 0.0711251, -0.47605, -0.369735, 0.0637065, -0.0312522, -0.442453, -0.125505, -0.322369, -0.458278, -0.537278, -0.307682, -0.662983, -0.228071, 0.273417, -0.692805, -0.319983, -0.33659, -0.0194469, -0.332695, -0.324797, -0.226288, -0.323512, -0.344109, 0.10275, -0.413868, -0.100918, -0.342825, -0.180493, -0.446703, -0.42264, -0.308931, 0.283873, -0.307077, -0.326147, 0.0978309, -0.651811, -0.133088, -0.0265985, -0.312365, -0.354224, -0.262866, 0.0499091, -0.291127, -0.245837, 0.0365629, -0.639614, -0.326587, -0.289977, -0.38732, -0.0952474, -0.514723, -0.511787, -0.203263, -0.295558, -0.56221, -0.36509, -0.623298, -0.3362, -0.366962, 0.354323, -0.319758, -0.354718, -0.495494, -0.3307, 0.0710968, -0.639685, -0.00283265, -0.234982, -0.526333, -0.330266, -0.0463442, -0.277975, 0.0556551, 0.0634616, -0.0878055, -0.143345, 0.197601, 0.207601, -0.393523, 0.181968, -0.348203, -0.667034, -0.0284886, -0.297679, -0.321122, -0.317345, -0.3115, -0.301487, -0.343093, -0.407707, -0.339509, -0.0509347, 0.27625, -0.499085, -0.314647, -0.280454, -0.214117, -0.338948, -0.305269, -0.165222, -0.656303, -0.342977, -0.200704, -0.018234, 0.335636, -0.318321, -0.0445439, -0.210606, -0.347077, 0.381323, -0.39142, -0.3801, -0.0414001, -0.503723, -0.338271, -0.321749, -0.230709, -0.383153, -0.39333, -0.293105, -0.845147, -0.100253, 0.211194, 0.104559, -0.336604, -0.502765, -0.384718, -0.51661, 0.0640801, -0.450737, -0.354367, 0.0871824, -0.052065, -0.43621, -0.138777, -0.318232, -0.466623, -0.544543, -0.292314, -0.669115, -0.236435, 0.287917, -0.668305, -0.303879, -0.350591, -0.0615803, -0.344355, -0.333219, -0.235828, -0.335466, -0.332551, 0.105629, -0.415288, -0.0878029, -0.323483, -0.192989, -0.420463, -0.426949, -0.299174, 0.265562, -0.299251, -0.329714, 0.0711986, -0.67664, -0.164189, -0.00343712, -0.311907, -0.325856, -0.254235, 0.0611658, -0.296752, -0.223888, 0.0589284, -0.64349, -0.339749, -0.305809, -0.392697, -0.081227, -0.522092, -0.499842, -0.172577, -0.289452, -0.538724, -0.364461, -0.593605, -0.33334, -0.359751, 0.338454, -0.322077, -0.331972, -0.499794, -0.344277, 0.0810886, -0.619835, -0.00497493, -0.288585, -0.507047, -0.348496, -0.0598448, -0.284412, 0.0436159, 0.0539948, -0.108968, -0.107826, 0.191878, 0.179922, -0.390457, 0.156268, -0.336685, -0.660097, -0.028777, -0.316031, -0.345437, -0.348378, -0.307399, -0.330366, -0.327719, -0.417416, -0.333621, -0.0561031, 0.295739, -0.485202, -0.286517, -0.290461, -0.237345, -0.334617, -0.308405, -0.168594, -0.622111, -0.318876, -0.195544, -0.0268783, 0.35958, -0.324233, -0.0526058, -0.245146, -0.331846, 0.388858, -0.388621, -0.361267, -0.0359583, -0.51625, -0.323968, -0.333312, -0.207729, -0.407965, -0.412092, -0.292054, -0.846441, -0.107126, 0.236684, 0.117492, -0.314492, -0.518026, -0.354339, -0.49543, 0.0464862, -0.446083, -0.371035, 0.0860041, -0.0476966, -0.435401, -0.106025, -0.317968, -0.455489, -0.535056, -0.308794, -0.660549, -0.240492, 0.32577, -0.666035, -0.299818, -0.32136, -0.0739713, -0.334224, -0.336032, -0.237453, -0.328826, -0.339634, 0.097307, -0.425891, -0.0563878, -0.338847, -0.189161, -0.40869, -0.428578, -0.307462, 0.255463, -0.298268, -0.314656, 0.0851313, -0.673548, -0.132172, -0.0111897, -0.330939, -0.32622, -0.266128, 0.0649346, -0.315098, -0.231283, 0.0648225, -0.635026, -0.336962, -0.303775, -0.408467, -0.0611293, -0.525973, -0.511805, -0.167848, -0.292541, -0.534315, -0.372279, -0.620684, -0.329856, -0.363493, 0.327141, -0.315367, -0.342113, -0.492365, -0.36591, 0.0679208, -0.656679, -0.0224467, -0.298045, -0.504397, -0.33101, -0.041882, -0.294963, 0.0493516, 0.0625482, -0.0957589, -0.139861, 0.175578, 0.16541, -0.379658, 0.12177, -0.342358, -0.668263, -0.0455386, -0.312126, -0.316729, -0.332937, -0.333433, -0.303881, -0.325633, -0.425889, -0.33571, -0.0709556, 0.297201, -0.500196, -0.310671, -0.289888, -0.233614, -0.293444, -0.30047, -0.173145, -0.61135, -0.332715, -0.15855, -0.0552202, 0.348133, -0.2984, -0.0532353, -0.242836, -0.321954, 0.398989, -0.349618, -0.368974, -0.0345131, -0.509687, -0.340686, -0.34063, -0.185186, -0.413516, -0.381811, -0.292591, -0.843601, -0.108621, 0.231083, 0.127875, -0.335471, -0.529321, -0.362818, -0.512747, 0.0663154, -0.427325, -0.375852, 0.0661575, -0.0607851, -0.460137, -0.109259, -0.351233, -0.476339, -0.537164, -0.3216, -0.680081, -0.220401, 0.385825, -0.674701, -0.308247, -0.337859, -0.0748989, -0.326931, -0.343319, -0.205123, -0.315286, -0.353315, 0.119929, -0.42132, -0.0577023, -0.332791, -0.191925, -0.370314, -0.438845, -0.316435, 0.302371, -0.308008, -0.346228, 0.0543052, -0.669751, -0.149383, -0.012569, -0.303599, -0.344144, -0.283871, 0.0624967, -0.299956, -0.218131, 0.0784484, -0.652741, -0.326117, -0.29378, -0.395157, -0.0200841, -0.535897, -0.52565, -0.163263, -0.302408, -0.539648, -0.362161, -0.628503, -0.328458, -0.372051, 0.299954, -0.287258, -0.339654, -0.48375, -0.357341, 0.0654907, -0.66393, -0.0683792, -0.343973, -0.507014, -0.331226, -0.0511925, -0.300212, 0.0397623, 0.0402732, -0.125967, -0.130385, 0.169815, 0.179422, -0.39676, 0.121602, -0.344626, -0.68293, -0.0297315, -0.304348, -0.339549, -0.323373, -0.320313, -0.314978, -0.336492, -0.389256, -0.354281, -0.0518436, 0.293844, -0.515384, -0.299832, -0.286012, -0.221211, -0.290052, -0.295738, -0.204119, -0.622069, -0.315933, -0.137855, -0.0714837, 0.347651, -0.322286, -0.064582, -0.254538, -0.326821, 0.393677, -0.363927, -0.371512, -0.00720477, -0.531986, -0.34027, -0.328864, -0.150552, -0.400362, -0.404914, -0.28211, -0.815526, -0.115067, 0.260095, 0.115215, -0.347982, -0.521726, -0.34656, -0.509464, 0.0482038, -0.441152, -0.354345, 0.0847655, -0.0789093, -0.451234, -0.131777, -0.334868, -0.471425, -0.552098, -0.335935, -0.662011, -0.21583, 0.411429, -0.652031, -0.306543, -0.346378, -0.120309, -0.328334, -0.346382, -0.20758, -0.340065, -0.360193, 0.123522, -0.409083, -0.0481259, -0.332184, -0.197763, -0.33966, -0.412043, -0.336037, 0.301344, -0.300262, -0.359927, 0.0659196, -0.634271, -0.129226, 0.0182544, -0.312185, -0.333485, -0.271753, 0.0418283, -0.305141, -0.225338, 0.10176, -0.627425, -0.318866, -0.284788, -0.394003, -0.0157581, -0.570042, -0.523703, -0.156707, -0.273061, -0.54439, -0.372408, -0.602608, -0.326473, -0.384825, 0.294492, -0.310114, -0.369214, -0.51095, -0.374061, 0.0840666, -0.683172, -0.0678025, -0.371505, -0.515762, -0.309871, -0.0531356, -0.273873, 0.0218983, 0.0450914, -0.119986, -0.147005, 0.175514, 0.196075, -0.3647, 0.112766, -0.33538, -0.674775, -0.0327704, -0.314063, -0.347568, -0.34047, -0.314413, -0.330064, -0.325315, -0.403018, -0.351122, -0.0647281, 0.320774, -0.470546, -0.313098, -0.276646, -0.233677, -0.291293, -0.257589, -0.218845, -0.58551, -0.334167, -0.14962, -0.0850294, 0.321734, -0.309633, -0.047606, -0.227064, -0.32815, 0.43259, -0.343221, -0.359159, 0.04834, -0.523414, -0.335785, -0.340447, -0.154904, -0.393227, -0.400458, -0.254928, -0.835017, -0.0972525, 0.248848, 0.103317, -0.338076, -0.537294, -0.361567, -0.519208, 0.0505824, -0.465428, -0.364651, 0.0508341, -0.0744052, -0.477513, -0.137171, -0.323017, -0.468236, -0.523578, -0.340968, -0.687877, -0.216465, 0.461482, -0.6292, -0.313556, -0.349654, -0.122883, -0.345964, -0.33686, -0.193263, -0.331386, -0.336623, 0.154454, -0.40151, -0.0879533, -0.339779, -0.224768, -0.279661, -0.398686, -0.311193, 0.240648, -0.315938, -0.358949, 0.158384, -0.650964, -0.0933906, 0.0191303, -0.334948, -0.330975, -0.285134, -0.0152398, -0.292737, -0.208386, 0.0742327, -0.643225, -0.32754, -0.291319, -0.404902, -0.0726968, -0.5935, -0.496975, -0.194917, -0.243305, -0.523563, -0.367981, -0.592508, -0.338368, -0.374952, 0.399939, -0.305495, -0.357742, -0.504708, -0.283189, 0.156682, -0.697128, 0.00882569, -0.36474, -0.507803, -0.247505, -0.0624717, -0.253203, -0.016948, 0.102551, -0.0503658, -0.143624, 0.253806, 0.221387, -0.40085, 0.0341866, -0.341489, -0.662681, -0.0185527, -0.315515, -0.324377, -0.322908, -0.317444, -0.315823, -0.329919, -0.415237, -0.347466, -0.0566749, 0.239869, -0.437323, -0.313658, -0.271987, -0.261313, -0.252783, -0.239169, -0.194242, -0.556838, -0.337726, -0.223417, -0.0797236, 0.239001, -0.310694, -0.0335526, -0.202906, -0.312707, 0.393043, -0.269425, -0.367617, 0.0620052, -0.503548, -0.328964, -0.347786, -0.214649, -0.408814, -0.403483, -0.282061, -0.810888, -0.122963, 0.180758, 0.103745, -0.355884, -0.551445, -0.351135, -0.500478, 0.120755, -0.449994, -0.38135, 0.0187255, -0.0878401, -0.383265, -0.125865, -0.324243, -0.451917, -0.536733, -0.286451, -0.670192, -0.237097, 0.390182, -0.649443, -0.302303, -0.392031, -0.062102, -0.338443, -0.288483, -0.200006, -0.328422, -0.3306, 0.186164, -0.42759, -0.158807, -0.349276, -0.259172, -0.303894, -0.364864, -0.308729, -0.100008, -0.286918, -0.478166, -0.0896948, -0.629905, 0.239217, -0.246527, -0.308734, -0.31351, -0.263473, 0.0333503, -0.297138, -0.227663, -0.2482, -0.656349, -0.338413, -0.29945, -0.382248, 0.0143292, -0.597212, -0.419738, -0.289345, -0.265598, -0.460066, -0.261929, -0.624426, -0.321762, -0.414707, -0.213562, -0.309037, -0.342237, -0.35485, 0.0324275, 0.141259, -0.697746, -0.181448, -0.312596, -0.394298, 0.0135473, -0.0571763, -0.200667, -0.0291549, -0.155399, -0.0234647, -0.189602, -0.318638, 0.367045, -0.350472, -0.00198524, -0.344176, -0.699409, -0.0423231, -0.304214, -0.323318, -0.326159, -0.308235, -0.319869, -0.335403, -0.426781, -0.325705, -0.0676371, 0.159309, 0.0648599, -0.326908, -0.203115, -0.254282, -0.116597, -0.177011, -0.333117, -0.565852, -0.332867, -0.109446, -0.336826, 0.128598, -0.317542, -0.0503537, 0.0102252, -0.331199, -0.153737, -0.0247341, -0.359296, 0.330413, -0.505672, -0.365504, -0.34496, -0.17894, -0.39991, -0.384875, -0.123354, -0.765086, -0.10864, 0.0806377, 0.0196802, -0.32692, -0.136206, -0.364197, -0.490493, 0.0810914, -0.395743, -0.375208, -0.102587, 0.0473237, 0.0576427, -0.131812, -0.334494, -0.453185, -0.513935, -0.109711, -0.661058, -0.177105, 0.707228, -0.400041, -0.284683, -0.349287, -0.122508, -0.349338, 0.0257974, -0.482717, -0.325989, -0.3422, 0.020329, -0.436546, -0.190648, -0.324036, -0.326382, 0.289598, -0.157895, -0.313557, -0.515972, -0.203272, -0.0139233, -0.105641, -0.454669, 0.832177, -0.415523, -0.222872, -0.210159, -0.266322, -0.643171, -0.200328, -0.0537347, -0.536561, -0.538861, -0.219041, -0.196573, -0.252032, -0.0850644, -0.487322, -0.310798, -0.0413615, -0.153882, 0.142276, -0.0209584, -0.53593, -0.224047, -0.0363117, -0.388509, -0.221478, -0.239327, 0.0528192, 0.89218, 0.136876, -0.344987, 0.142226, -0.163407, 0.187608, 0.189624, -0.0726797, 0.273544, 0.00302138, 0.294064, 0.273789, -0.222798, -0.655687, 0.651718, -0.30779, -0.0434825, -0.252291, -0.597562, -0.0396993, -0.206392, -0.225946, -0.212056, -0.230512, -0.20469, -0.243868, -0.302861, -0.228013, -0.0574222, -0.273224, 0.616138, -0.193487, -0.299628, -0.0792147, 0.0930194, -0.232367, -0.283112, 0.0532467, -0.224884, -0.0225295, -0.664957, -0.174448, -0.220076, -0.0724051, 0.174486, -0.228477, -0.646185, 0.0215393, -0.279221, 0.522203, -0.25783, -0.246238, -0.229062, -0.117364, -0.301697, -0.282917, -0.407812, -0.775878, -0.0703302, -0.358258, -0.346256, -0.245182, 0.18511, -0.231167, -0.354193, 0.307874, -0.240625, -0.282932, -0.33649, 0.0377536, 0.806367, -0.110802, -0.210627, -0.24167, -0.191297, -0.0413095, -0.615097, 0.125695, 0.594947, 0.166613, -0.200492, -0.168203, 0.398202, -0.242077, 0.570757, -0.441252, -0.225143, -0.231436, 0.274648, -0.180148, -0.220413, -0.234671, -0.223946, 0.76227, 0.0477913, -0.203095, -0.429463, -0.0725796, 0.168629, 0.0381831, -0.328488, 0.30823, -0.357732, -0.0871093, -0.0902989, -0.0968015, -0.77696, -0.0784215, -0.195274, -0.424875, -0.338736, -0.111552, -0.0799294, -0.113829, 0.161345, -0.10625, 0.552287, -0.121055, 0.0532647, 0.326023, -0.380147, -0.312293, -0.10275, -0.0382055, -0.520802, -0.0836097, -0.124651, 0.127971, 1.0082, -0.0961984, 0.50013, 0.0699649, -0.277544, 0.874538, 0.291545, -0.0423851, 0.237151, 0.0331315, 0.140201, 0.104124, -0.139299, -0.479656, 0.379942, -0.441739, -0.132763, -0.076582, -0.00247364, -0.050136, -0.0668834, -0.1031, -0.0896129, -0.12059, -0.0707805, -0.137625, -0.124343, -0.129417, -0.0743122, -0.114418, 0.2421, -0.0919843, 0.134641, -0.318581, 0.339689, -0.109813, -0.0199133, 0.333782, -0.0851926, -0.214922, -0.642663, 0.0500518, -0.0824651, -0.0651791, -0.0662406, -0.119982, -0.614758, 0.176186, -0.13983, 0.288653, -0.141132, -0.102697, -0.107429, 0.181811, -0.105701, -0.132066, -0.409822, -0.228059, -0.072263, -0.821919, -0.115193, -0.0953448, 0.367486, -0.13644, -0.234614, 0.132675, 0.170915, -0.168766, -0.215825, -0.375265, 1.02429, -0.0876202, -0.109404, -0.121618, -0.2178, 0.0832534, -0.0683354, 0.175877, 0.135518, 0.0208356, -0.0906234, -0.361864, 0.660046, -0.0956411, 0.21742, -0.404594, -0.103252, -0.101495, 0.266467, -0.156651, -0.189512, -0.132159, -0.402263, 0.954178, 0.229769, -0.0706004, 0.11698, 0.22805, 0.00218302, -0.0172282, -0.0465846, -0.567181, -0.535971, 0.119743, -0.0232611, -0.114253, -0.673938, -0.619006, -0.437965, -0.0128079, -0.0012806, -0.656821, -0.580019, -0.913032, 0.0398591, 0.159594, -0.292073, -0.554308, -0.286449, 0.026531, -0.63631, -0.145536, -0.307432, 0.0315328, -0.298071, -0.0193389, -0.0113085, -0.531977, -0.00865641, -0.487916, 0.205875, 0.131326, -0.00740846, 1.45469, -0.00614197, 0.224332, -0.45521, -0.00942348, 0.260058, 0.232935, -0.00238846, 0.0388335, -0.0870536, -0.534548, -0.247679, -0.063154, -0.442693, -0.00611346, -0.0187377, -0.127914, -0.606274, 0.0206411, -0.017545, -0.359999, 0.376548, 0.186938, -0.852818, -0.407336, -0.225395, -0.513809, -0.18586, -0.0232046, -0.155922, -0.311304, -0.000297523, -0.971891, -0.543537, -0.360403, -0.0236809, -0.015873, -0.00985795, -0.572149, -0.0651726, -0.391251, -0.531561, 0.00607094, -0.0211283, -0.317167, 0.367787, -0.124672, -0.0159105, -0.253213, -0.808732, -0.554751, -0.551156, -0.724285, -0.34492, 0.167254, 0.146016, 0.0873242, -0.0246018, 0.0102426, -1.68152, -0.484142, 0.0733867, 0.319885, 0.00587285, 0.274115, -0.664606, -0.0992198, -0.0220567, -0.0143664, 0.341777, -0.00033516, -0.441024, -0.379686, -0.279578, 0.346611, 0.38057, -0.236774, -0.191459, 0.184656, -0.0428123, -0.570488, -0.317444, 0.216493, -0.0677453, -0.312696, 0.254039, -0.205269, -0.212537, -0.0176861, -0.878452, 0.00241276, -0.0257244, 0.0114595, -0.0224356, 0.00520168, -0.00758153, 0.0124176, 0.0442283, 0.0214126, -0.0201277, 0.0321732, -0.0543465, 0.0196939, 0.0246596, -0.0124382, -0.00325102, 0.0211485, 0.0343447, 0.0142809, -0.0318664, 0.039992, -0.20308, 0.017721, 0.025164, 0.0206089, 0.0209558, 0.0175953, 0.0257961, 0.0253433, 0.0483835, -0.0907212, 0.00492088, 0.0479902, -0.019206, 0.0424676, 0.0346915, -0.0350835, -0.00133063, 0.0246456, -0.0140469, -0.0474579, -0.0592756, 0.0119078, -0.0195057, -0.0652083, -0.000840205, -0.149516, 0.0223309, 0.0398047, 0.0316731, -0.0409831, 0.0298925, -0.0190579, -0.0293196, 0.0321583, 0.042173, -0.0111268, -0.0153123, 0.0298377, -0.0507923, 0.0556033, -0.0526492, 0.0273332, 0.0369283, 0.0364304, 0.0142675, -0.00317944, 0.047179, 0.0289992, -0.0138194, 0.030235, 0.0200677, 0.0190153, -0.0107929, -0.0230664, 0.018522, 0.0531693, -0.00998711, 0.0476537, 0.0307719, 0.0036145, -0.177175, 0.0486507, -0.0572445, -0.0508697, 0.0101434, 0.0259611, 0.0409465, 0.0560277, 0.0393787, -0.0773473, 0.0362389, 0.0255604, 0.0337667, 0.0421069, 0.00638316, 0.00308677, -0.171924, -0.038704, 0.0375845, 0.0076082, -0.0222644, 0.00290782, 0.0215374, 0.0398872, -0.195557, -0.00609102, -0.0130518, -0.000620009, -0.201761, 0.0249579, 0.0159586, 0.0318396, -0.061042, 0.0519658, 0.0315792, 0.0318208, 0.0141448, 0.0286055, 0.0245608, 0.0203665, -0.0813684, 0.0382453, 0.0246532, 0.0322772, 0.0239847, 0.00389635, -0.0581747, -0.0197478, 0.446251, 0.166321, -0.00784914, -0.0164236, -0.0216111, 0.452205, -0.279166, -0.151584, -0.0116334, 0.404207, -0.868012, 0.489025, -0.465923, -0.0193396, -0.00469829, 0.505708, 0.496506, 0.0362409, 0.446615, 0.1341, -0.455417, 0.501515, 0.486792, -0.000962121, 0.581143, -0.251888, -0.36494, -0.456446, -0.375719, -0.0542895, 0.00207852, 0.484565, -0.00576659, 0.380671, 0.210997, 0.4825, 0.00176966, 0.187252, -0.0148773, 0.16291, -0.697633, -0.00375297, 0.419569, 0.272583, -0.0213931, 0.146817, 0.193783, 0.380913, 0.459589, -0.0722793, -0.332219, -0.0119204, -0.0412914, -0.307045, 0.427118, 0.457223, -0.00879301, -0.453019, 0.172448, 0.35718, -0.726599, -0.485076, 0.307907, 0.433116, -0.463205, -0.00568545, 0.343276, -0.295696, -0.0204968, -0.949857, 0.601055, -0.115678, -0.00527294, 0.000670628, -0.00583004, 0.464436, -0.121276, -0.452533, -0.119985, -0.00954319, -0.283501, -0.244928, 0.0894903, 0.275871, -0.012443, -0.542692, -0.607713, 0.49896, 0.448828, 0.149553, 0.512281, -0.558726, -0.3088, 0.257458, -0.0172972, -0.00247617, -1.85093, -0.0763512, -0.53303, -0.352006, -0.00298466, -0.0754926, 0.444793, 0.290646, -0.369842, -0.00973607, 0.629081, -0.0124839, 0.131991, -0.444643, -0.4267, 0.312828, -0.23194, -0.21821, 0.400287, 0.248639, 0.0139761, 0.447625, -0.662215, 0.250491, -0.166775, -0.452843, -0.382689, -0.545292, -0.492282, -0.0162744, -1.02395, -0.00403533, -0.04435, -0.735883, -0.0114595, 0.00554497, 0.011181, -0.369805, -0.13094, -0.72777, -0.0128482, -0.0723501, 0.187628, -0.295908, 0.382264, -0.0229902, -0.00314436, -0.422913, -0.392129, 0.665252, -0.0821478, 0.0401174, -0.190072, -0.415447, -0.275498, -0.0789027, -0.0259974, -0.381416, -0.117448, -0.0981742, 0.222335, -0.0388798, 0.0018529, -0.279665, -0.0158908, -0.285152, -0.801069, -0.0219081, -0.00991656, -0.945014, -0.0262468, -0.474535, -0.657895, -0.0109922, -1.11045, -0.107591, -0.00579188, -0.705251, -0.631378, -0.304788, -0.151799, 0.00642076, -0.698876, -0.0135759, -0.0429908, -0.448802, -0.25272, -0.070121, 0.0103165, 0.296885, -0.310111, -0.0443506, 0.543315, -0.395881, -0.140435, -0.297746, -0.351332, -0.0149866, -0.00193104, 0.293998, -0.00208737, -0.910556, -0.280452, -0.723322, -0.000976899, -0.00330868, -0.020551, -0.344228, -0.0472463, 0.129608, -0.0549982, -0.0128308, -0.331881, 0.419217, -0.0426919, 0.464462, -0.0120169, 0.155699, -0.0137205, -0.358938, -0.376049, 0.779658, -0.271659, -0.569892, -0.441707, -0.104489, -0.00979446, -0.017318, 0.573857, 0.545945, -0.14577, -0.499766, -0.0145327, -0.324089, -0.474386, -0.0544953, -0.389264, -0.00975317, -0.0638884, -0.0145786, 0.0424943, 0.0570672, 0.178398, 0.190497, -0.329088, 0.223271, -0.00965647, 0.157089, -0.0798102, -0.344103, -0.269248, 0.00452402, 0.166206, 0.226153, 0.110269, -0.402728, -0.0191228, -0.0253983, -0.191149, 0.00921918, 0.619532, 0.00564338, -0.0145445, -0.0177302, -0.0203854, 0.0615653, 0.0699548, 0.0412197, 0.0179526, 0.103056, 0.223343, 0.0556088, 0.0845043, -0.00269168, -0.00183775, 0.0659494, 0.055234, -0.466449, 0.603275, 0.0293123, 0.059908, 0.106645, 0.0822055, -0.0476477, 0.0890047, 0.0678266, 0.101741, 0.0768233, 0.115209, -0.111155, -0.00616215, 0.137338, 0.0154216, 0.101373, 0.0328831, 0.602453, 0.000744572, -0.191755, -0.00440682, 0.499251, -0.0106367, 0.00624197, 0.0579774, 0.842393, -0.0139384, 0.453717, 0.0723753, 0.105084, 0.0870185, -0.0485876, 0.0629669, -0.0123375, -0.0222813, 0.056535, 0.0910021, 0.610431, -0.0117841, 0.0716545, 0.291518, 0.094845, 0.450273, 0.0958199, 0.104788, 0.1151, 0.083446, -0.00512084, 0.100368, 0.0544156, 0.00472073, 0.0848025, 0.0942495, 0.0621911, -0.0217164, 0.000916394, 0.000564296, 0.123467, -0.028662, 0.0739669, 0.0513018, -0.0212152, 0.554, 0.0828981, 0.180149, 0.992889, 0.0128533, 0.105594, 0.0475176, 0.0782842, 0.121098, 0.124151, 0.111876, 0.0332291, 0.0688806, 0.106663, -0.0134071, 0.00275728, 0.685967, 1.13512, -0.0329555, 0.0489755, -0.015703, 0.275542, 0.0469044, 0.102824, 0.165851, -0.00991248, 0.237522, -0.0137351, 0.418562, 0.0661434, 0.0177103, 0.0730541, 0.468083, 0.0735749, 0.0578529, 0.11677, -0.0420659, 0.0643208, 0.0977038, 0.0894132, 0.178504, 0.0659548, 0.0537457, 0.0725604, 0.0773467, -0.000195983, -0.147505, -0.0147681, -0.159078, -0.715442, -0.0358204, -0.00631395, -0.0421925, -1.50795, -0.33355, -0.494805, -0.0157572, -0.539143, -0.439532, -1.46337, -0.945659, -0.0316843, -0.00948862, -1.52712, -1.41187, 0.290344, -0.0245465, -0.885247, -0.0272969, -1.16352, -1.33401, -0.0125753, -1.16447, -0.820631, -0.264457, -0.345038, -0.395054, -0.0782047, 0.00991, -1.3562, -0.0330631, -1.36633, -0.761632, -0.20918, -0.0436701, 0.363193, -0.0175461, 0.243035, -0.341806, -0.00965979, -0.904192, -0.103796, -0.0281552, 0.139498, -1.65278, -1.4281, -0.64227, -0.0143298, -0.0661003, -0.0181524, -0.0462051, -0.412524, -1.28131, -0.102535, -0.00969558, -0.285314, 0.0455612, -0.781467, 0.535606, 0.00228993, -0.953911, -1.39805, -0.245733, -0.0213804, -0.78456, -0.513379, -0.0394013, -0.219282, -1.08085, -0.367466, -0.0139765, 0.00329301, -0.0323963, -1.31789, -0.178372, -0.413486, -0.15259, -0.00525117, 0.27963, -0.498267, 0.0105712, 0.407711, -0.038303, -0.503196, -0.583193, -1.42003, -1.3879, -0.125413, -1.37831, -0.583158, -0.218732, -0.940974, -0.0333638, -0.0162055, 0.519741, 0.270927, -0.106137, -0.670835, -0.0199889, 0.220885, -1.35752, -0.909644, 0.085935, -0.0259164, 0.48045, -0.0240565, 0.307015, -0.515708, -0.365679, -0.900105, -0.0870184, -0.0912616, -0.799516, -1.6815, -0.0399082, -1.43251, -0.49692, -0.680695, -0.0852393, -0.490734, -0.717643, -0.396677, -0.242574, -0.0186558, -0.67318, -0.0111918, 0.124809, -0.177361, -0.0156799, -0.0118557, -0.0314802, 0.324654, -0.0814245, -0.245865, -0.0223484, 0.0215406, -0.165028, 0.215727, 0.204959, -0.00673555, 0.00383679, 0.264182, 0.285625, -1.01951, 0.146009, 0.331277, -0.32508, -0.112065, 0.0702864, -0.00936936, -0.523522, -0.00898347, -0.721703, -0.109426, -0.200464, -0.126617, -0.012441, 0.0305371, 0.0207895, 0.0370272, -0.158322, 0.0663043, -0.0240991, 0.663873, -0.00550571, 0.0383566, -0.0750546, -0.0085728, 0.076026, 0.49542, 0.004038, -0.258032, -0.104364, 0.177329, -0.00882748, -0.0348431, -0.543252, -0.000668703, -0.00101919, -0.103782, 0.0918398, 0.140008, -0.00332368, -0.445097, 0.0157088, 0.0599104, -0.869782, -0.287794, -0.239001, 0.192052, -0.11695, -0.016956, -0.16074, -0.34498, 0.00607078, -0.351913, 0.0232372, -0.546892, 0.00611005, 0.00443149, -0.00405813, 0.0529667, -0.603242, -0.0568577, 0.383889, 0.00135891, -0.239927, -0.330949, -0.095339, -1.9482, 0.011693, -0.00949376, 0.00585549, 0.20952, 0.0407526, -0.571057, 0.0519128, -0.0352101, -0.159554, -0.126815, 0.0124617, -0.0352184, -2.0429, -2.31011, 0.247543, -0.0153937, -0.0161392, -0.037036, 0.18298, -0.0615618, -0.169437, -0.0388858, 0.0943872, -0.0338343, 0.185234, -0.257127, -0.293654, 0.464582, 0.0143283, -0.374996, -0.0795115, 0.591819, -0.04129, 0.189926, -0.113726, 0.344739, -0.207949, -0.206876, 0.194, -0.691048, -0.205073, -0.00601007, 0.850418, -0.00815785, -0.0189271, 0.0111178, 0.0059172, -0.00487196, 0.0203317, 0.0123665, 0.0320374, 0.00696515, 0.0106826, 0.0184839, -0.151736, 0.0204691, -0.000850771, -0.00133493, 0.00320941, -0.00765855, 0.0272306, -0.109006, -0.0233972, 0.0297533, 0.0907475, 0.0323064, 0.0296503, -0.0176227, 0.0330873, 0.0117611, 0.00842228, 0.0383588, 0.0255503, -0.0614824, 0.00484998, 0.0239564, -0.0156155, 0.0385585, 0.00265262, -0.0339484, 0.0102537, 0.0608902, 0.00528877, -0.132156, -0.085863, 0.00384341, 0.06332, -0.173631, -0.00566425, -0.235073, 0.00505002, 0.011257, 0.021241, -0.0306215, 0.0389347, 0.00270367, -0.00578186, 0.00706916, 0.0337372, 0.00869014, 0.000662354, 0.013842, -0.174475, 0.0302392, -0.151702, 0.023627, 0.0093863, 0.0154364, 0.00885216, -0.00203106, 0.0233534, 0.0319218, 0.0042246, 0.0296245, 0.0176969, -0.0135568, -0.0121129, -0.0119683, 0.0116995, 0.0363422, 0.00878676, 0.0379847, 0.0287739, -0.0104527, -0.423173, 0.0301647, -0.104595, -0.0944258, -0.0207651, 0.03606, 0.0147458, 0.0158047, 0.0108213, -0.191058, 0.0211648, 0.00705311, 0.0269074, 0.0396828, -0.0180489, 0.0143349, -0.123893, -0.0561145, 0.00994776, 0.0102396, 0.0131545, -0.0844509, 0.00571265, 0.0296147, -0.0182003, 0.0067832, -0.058039, -0.0206706, -0.42254, 0.0105587, 0.0253741, 0.0392044, -0.160665, 0.0208244, 0.0275889, 0.0212345, -0.0143241, 0.0359121, 0.0227669, 0.0224361, -0.177687, 0.0278168, -0.00780797, 0.0187365, 0.00998985, 0.00263453, -0.0720995, -0.0163588, 0.0129239, 0.0816402, 0.0119736, -0.00757633, 0.00639465, 0.0919199, 0.121808, 0.109125, 0.00181212, 0.119741, 0.00290625, 0.101965, 0.0909955, 0.00075462, 0.00121641, 0.0972813, 0.096324, -0.0523283, 0.0471551, 0.104299, 0.0834641, 0.11448, 0.104893, -0.0345208, 0.0893972, 0.0953525, 0.113943, 0.120469, 0.108796, -0.0360945, -0.00986298, 0.127223, 0.00748309, 0.113332, 0.0949816, 0.0118541, 0.000735092, 0.0939634, 0.00618592, 0.165402, 0.0274778, 0.00730601, 0.0430996, 0.0892171, 0.0012051, -0.167696, 0.0986888, 0.0990111, 0.0994071, -0.0123416, 0.105392, 0.00224935, -0.0305019, 0.101493, 0.121078, 0.0404784, -0.00239107, 0.105473, 0.0639563, 0.11888, -0.114855, 0.112562, 0.104217, 0.105172, 0.135599, -0.0236541, 0.10651, 0.0947291, -0.0117592, 0.133272, 0.0923562, 0.0851526, -0.00524809, -0.00378669, 0.00768971, 0.102804, 0.0125258, 0.104767, 0.0889315, 0.00785144, -0.311676, 0.0947802, 0.153786, -0.00542208, -0.0142471, 0.112008, 0.123062, 0.0939228, 0.109129, -0.000852107, 0.12336, 0.0979754, 0.124252, 0.105356, -0.00417262, -0.00347124, -0.212685, 0.080032, 0.151214, 0.0754541, -0.00458685, 0.114339, 0.104192, 0.102611, 0.0985907, -0.0144672, 0.0508253, -0.0205439, -0.28177, 0.0848539, 0.112281, 0.115327, 0.130355, 0.129941, 0.119716, 0.107848, -0.00426215, 0.115581, 0.106476, 0.120043, 0.0351016, 0.101148, 0.106899, 0.101146, 0.104654, 0.00126933, 0.000349141, 0.00311528, -0.0649822, 0.278454, -0.0291439, -0.0234939, -0.0134212, 0.225684, -0.254274, 0.252574, -0.0206936, 0.126287, -0.256774, 0.203159, 0.197459, -0.0148781, -0.00461165, 0.200761, 0.201581, 0.404621, -0.0384641, 0.298045, -0.191628, -0.0954968, 0.011182, -0.000745703, 0.347686, -0.0501991, 0.0523126, -0.150112, -0.0842361, -0.0413597, 0.00227981, 0.033969, 0.000897884, 0.06666, 0.271881, -0.130122, -0.0323857, 0.579413, -0.0231406, -0.573043, 0.679797, -0.00118288, 0.171952, 0.0752437, -0.0258279, -0.9734, 0.290558, 0.112242, 0.179835, -0.0159761, 0.11287, 0.00611885, -0.0152185, 0.0421505, 0.0697349, -0.0345801, -0.003136, -0.080989, -0.35987, 0.237912, -0.495549, -0.352142, -0.0565683, 0.161025, -0.136519, 0.0121275, 0.00281093, 0.0482291, -0.0270051, 0.221957, -0.0383944, -0.0357971, -0.0173927, -0.00706371, -0.0210792, -0.0274686, 0.715428, -0.18398, 0.150609, 0.000383158, -2.16106, -0.00688943, -0.17525, -1.53011, -0.00768518, -0.245556, 0.139578, 0.134547, 0.0202019, 0.0264844, 0.0495288, 0.0490828, -0.267613, 0.0388345, 0.0294899, -0.0218565, -0.473272, -1.57698, 0.53268, -0.0329497, -0.00341536, -0.0821782, 0.131914, 0.025198, -0.0792383, -0.00590107, 0.0810587, -0.0158941, -0.696439, -0.115135, -0.0545015, 0.223205, 0.31509, -0.0940198, -0.00800532, 0.166673, -0.0781746, 0.218597, -0.172191, 0.286293, 0.379215, -0.0637558, -0.0237931, 0.267603, -0.0950068, -0.0176066, -0.6156, 0.00510717, -1.17732, -1.38987, -0.0012851, -0.00436354, -0.0018701, -1.24445, -1.24655, -1.30779, 0.00338285, -1.32173, 0.505263, -1.29837, -1.59392, 0.00823118, 0.0102509, -1.28725, -1.28909, 1.26131, -1.15278, -1.57207, 0.385967, -1.31196, -1.36624, -0.0182258, -1.33985, -1.32223, -1.32015, -1.43598, -1.55559, 0.00390293, 0.0014332, -1.35822, 0.00575358, -1.29745, -1.26644, -1.22665, -0.00380797, -2.23223, -0.00600217, 0.0474103, -1.22705, 0.0118116, -0.00639014, 1.10396, -0.0200573, -1.09491, -0.977582, -1.31788, -1.52418, 0.0153059, -1.32215, 0.0158488, -0.0145294, -1.45518, -1.28586, -1.1501, 0.00582194, -1.65761, -0.467222, -1.32507, -2.98585, -1.33153, -1.39048, -1.2966, -1.25975, -0.000655831, -1.37895, -1.38675, -0.00117976, -1.20626, -1.36389, -1.23762, -0.0143261, 0.00253302, -0.0156312, -1.31898, -0.349983, -1.4082, -1.5203, -0.0173841, -1.06561, -1.51045, 0.302393, -0.764259, -0.0130439, -1.48556, -1.47986, -1.28753, -1.36463, -0.285833, -1.36597, -0.980506, -1.34264, -1.335, 0.00666392, -0.0073592, -3.31494, -0.988085, -0.951875, -1.57717, 0.00265944, -0.180382, -1.28045, -1.32576, -0.101706, -0.000132432, 0.418195, 0.0153034, -1.04651, -1.55574, -1.70795, -1.57335, -0.759117, -1.58499, -1.55639, -1.17464, -0.00245623, -1.29361, -1.26294, -1.51982, -0.847961, -1.50906, -1.72979, -1.28831, -1.51547, -0.00415318, -0.583603, -0.0158528, -0.0404799, 0.0217245, 0.0031004, 0.0105044, -0.000245453, 0.0337872, 0.0419984, 0.045647, 0.00737921, 0.0248907, -0.188848, 0.0651545, 0.028147, 0.0120076, -0.00312148, 0.0398917, 0.0082539, -0.0598029, -0.0256943, 0.0531656, 0.216343, 0.0257242, 0.0256216, -0.0481328, 0.0412439, 0.0353668, 0.0376982, 0.0460207, 0.0558083, -0.0348979, -0.00876464, 0.0349142, -0.00853068, 0.0337808, 0.034702, -0.0563546, 0.00284295, 0.0408031, -0.0253057, -0.169884, -0.087324, -0.00984111, 0.0106224, -0.193269, 0.00557113, -0.236855, 0.0508155, 0.0599623, 0.0217996, -0.0635893, 0.0448556, -0.00100689, -0.00877217, 0.0306403, 0.0519877, 0.00286986, -0.0106311, 0.0521394, -0.200096, 0.0111254, -0.224992, 0.0594112, 0.0296101, 0.03548, 0.042387, 0.00990873, 0.018329, 0.0285451, -0.00771492, 0.0426808, 0.049498, 0.0501399, 0.00137392, 0.000738768, -0.000300846, 0.0458711, 0.013248, 0.0269682, 0.0238338, -0.00621216, -0.47683, 0.0544766, -0.129393, -0.0346465, 0.000552029, 0.0445637, 0.0207497, 0.046576, 0.0413026, -0.224899, 0.0535214, 0.0352541, 0.0372726, 0.0107959, -0.0292964, 0.00830523, -0.302226, -0.051192, 0.0271309, 0.0310959, 0.00870144, -0.117152, 0.0262724, 0.0317012, 0.0274504, -0.00463371, -0.115032, -0.0283973, -0.42642, 0.0349517, 0.049923, 0.0347459, -0.190245, 0.0318691, 0.0359449, 0.0546565, -0.0162113, 0.0506224, 0.0301803, 0.0435924, -0.206409, 0.0270342, 0.0425414, 0.0227576, 0.0465064, -0.00913583, -0.0226813, -0.00441773, -0.0202364, -0.125479, 0.00599534, -0.0140129, 0.0052084, 0.0667512, 0.0510366, -0.188755, -0.00232545, -0.0875346, 0.124722, 0.109442, 0.0230225, 0.00984474, -0.00949119, 0.0533903, 0.0804087, 0.101381, -0.0350374, -0.0841741, 0.110107, 0.069596, -0.0152444, -0.0529031, 0.0473962, -0.0642702, 0.00549329, 0.0100564, -8.66906e-05, -0.0525447, -0.0015753, 0.0785817, -0.00785958, 0.0744654, -0.130078, -0.10682, 0.0029508, 0.0817964, 0.00151838, 0.0547363, -0.139778, -0.0198582, 0.125224, -0.053916, -0.00132688, -0.256329, -0.116605, 0.0833077, -0.0467574, -0.0636363, -0.0278029, -0.0183774, -0.0311948, 0.0330191, 0.0913193, -0.0271933, -0.00241529, 0.00759787, 0.0646291, -0.0742541, -0.200441, -0.00350406, 0.0188008, 0.115062, 0.00874467, 0.00339281, 0.073443, 0.00982744, -0.0131452, -0.00685302, 0.0416724, 0.0799348, -0.0076957, 0.0113317, 0.00322291, 0.0746894, 0.0478677, 0.00620138, -0.084136, -0.0274059, -0.3853, 0.0164226, 0.0649989, -0.214312, -0.0107239, 0.00674291, -0.0179372, 0.0900072, 0.0731577, 0.0803024, 0.0415223, 0.126875, 0.179359, 0.0957073, -0.00849986, 0.0046942, -0.241319, -0.158439, -0.182473, -0.00740832, 0.00768792, 0.036861, 0.0852719, 0.0656167, 0.0320985, -0.00819301, 0.113932, -0.0264983, -0.393384, 0.00666059, -0.00558467, -0.0665238, -0.215011, -0.0231333, 0.0497319, -0.0327545, 0.0186582, 0.0715588, -0.0241432, -0.0798943, 0.23514, 0.00168668, -0.010286, 0.00521647, -0.0151174, -0.0137879, -0.037423, -0.00692282, 0.197442, -0.203606, -0.0346026, -0.0163462, -0.00851148, -0.24278, -0.652316, -0.311731, 0.00564672, -0.406618, -1.04858, -0.318975, -0.261217, -0.00157905, -0.00411444, -0.265198, -0.203434, -3.43052, 0.142371, -0.167365, 0.217054, -0.468388, -0.408144, -0.0195495, -1.13318, -0.669466, -1.26085, -1.2216, -0.678468, -0.0580797, -0.0518123, -0.397891, -0.00434638, -0.356218, -0.121235, 0.142159, -0.0245806, 0.941392, -0.0151575, 0.255046, -1.56203, -0.0101688, 0.0542677, 0.212691, -0.0212839, 0.356369, -0.211871, -0.353796, -0.320917, -0.0266307, -0.898501, -0.01341, -0.0241176, -0.592004, -0.456621, 0.10971, -0.0127559, -1.07829, 0.280604, -0.34787, -0.991431, -0.672134, -0.42827, -0.344861, -0.689846, -0.019227, -0.451821, -1.06601, -0.0102031, -0.545516, -0.419362, -0.502173, -0.00234839, -0.0141368, -0.0136086, -0.376519, -0.263822, -0.601365, -0.234342, -0.0213321, 0.496198, -1.06457, 0.127634, -1.57489, -0.00436456, -0.658383, -0.409449, -0.293639, -0.389362, -0.204738, -0.461192, -0.267365, -1.47242, -0.426339, -0.0212483, -0.0233921, -2.3438, -2.52379, -0.142182, -0.612768, -0.00231952, 0.210806, -0.286129, -0.403106, 0.19313, -0.015631, 0.149438, -0.00487206, 0.431803, -0.591812, -0.935629, -0.19465, 0.0283206, -1.02227, -0.386742, -0.329278, 0.00156162, -0.28112, -0.52505, -0.212958, -0.760754, -0.765053, -0.52335, -1.15837, -0.757915, -0.0132227, 0.215417, -0.0214905, 0.920846, 0.707353, -0.023075, -0.00908934, -0.0196695, 0.683251, -0.0118264, 0.22544, -0.00952477, 0.620675, -0.422562, 0.677835, -0.00165688, 0.0129937, -0.0263044, 0.682878, 0.667161, -0.18623, 0.916416, 0.651046, -0.889827, 0.563783, 0.604678, -0.000211421, 0.611566, -0.0303708, 0.0150831, 0.00108567, -0.0146756, -0.0228872, 0.00768867, 0.690819, -0.00892305, 0.583549, 0.610379, 0.902643, 0.00491661, 0.743591, -0.0209304, -0.677838, 0.163797, -0.00599326, -0.270669, -0.0488697, -0.0158823, 0.872026, 0.672757, 0.619384, 0.702848, -0.0602075, 0.0577811, 0.00360896, 0.00170214, -0.0309132, 0.60713, 0.939209, 0.00706582, -0.0196569, -0.692921, 0.630849, 0.130929, -0.00532654, 0.685783, 0.623997, -0.0158742, -0.0258178, 0.545337, -0.03821, 0.0135472, 0.0155182, 0.586722, 0.209531, -0.00638128, -0.0128423, -0.00973282, 0.667269, -0.0135452, 0.0120605, -0.00936331, -0.0224842, 0.0344208, 0.00129111, -0.753937, 0.927499, 0.00601563, -0.00983778, -0.0388418, 0.690376, 0.592037, -0.30595, 0.623384, 0.589271, 0.0126163, 0.589402, -0.000277862, -0.00157108, -0.870981, -0.0633668, 0.11125, -0.0199494, 0.0287382, -0.684103, 0.557239, 0.588575, -0.905712, -0.0197647, -0.654289, -0.0183377, 0.666623, -0.00185679, -0.00513594, 0.709064, -0.0438226, -0.0177494, 0.653424, 0.507195, 0.0105706, 0.671433, -0.000447437, 0.659599, -0.399058, 0.00963684, -0.0307072, -0.0261702, 0.00567597, -0.00757242, -0.333081, -0.00616925, -0.0185555, -0.00307438, -0.00311146, -0.0122729, 0.0179381, 0.0228737, 0.0223992, 0.0263242, -0.00244415, 0.0348562, -0.157842, 0.000980564, 0.0103284, 0.00633122, 0.0111045, 0.0357579, -6.3754e-05, -0.0405998, -0.00706349, 0.00214356, -0.0220769, 0.0020988, 0.0229753, -0.0162553, 0.0129059, 0.0102254, 0.013302, 0.035284, -0.0104558, -0.044102, -0.0230108, 0.0322056, -0.0137925, 0.00841767, 0.013396, -0.00180102, 0.00354419, 0.019003, -0.00045985, -0.107555, -0.096269, 0.00291816, 0.0565927, -0.14945, -0.0117367, -0.183396, -0.00129501, 0.0246699, 0.00800308, -0.0324061, 0.0128025, 0.00827363, -0.00452864, 0.00598629, 0.00267956, -0.0158018, -0.0205525, 0.00343085, -0.129278, 0.0145816, -0.203306, 0.0258607, -7.03272e-05, 0.0149774, 0.00815236, 0.00844854, 0.00786077, 0.00039542, -0.00269053, 0.0149209, 0.0124303, -0.00110902, 0.0182056, 0.00717693, -0.00027555, 0.0250524, -0.00129683, 0.0196365, 0.00548877, -0.0184195, -0.225462, 0.0480279, -0.089048, -0.0232644, -0.0020076, -0.0020803, -0.00170657, 0.0128503, 0.023492, -0.186255, 0.0103225, 0.000147421, 0.02846, 0.0217399, -0.00833628, 0.00862404, -0.134978, -0.0671105, 0.00332596, 0.0122584, 0.00906536, -0.0773261, 0.00922996, 0.0175221, -0.0183978, -0.000792062, -0.00606498, -0.0119222, -0.286693, 0.0163277, 0.00371241, 0.00695878, -0.153887, 0.0184538, 0.0218132, 0.0112102, -0.00449084, 0.0126921, 0.0144873, 0.00817914, -0.199919, 0.0139586, 0.0309565, -0.00355367, 4.16248e-05, -0.00152324, -0.100818, 0.00890299, -0.0346241, -0.0178067, 0.00624213, -0.00762024, 0.00374166, -0.0167192, 0.012491, 0.00387732, -0.000404374, 0.00156265, -0.124053, 0.00302666, -0.00437061, 0.000795404, -0.00919938, -0.00959968, -0.00217518, 0.00835134, -0.0427436, 0.00165736, 0.0485654, -0.0110227, 0.00359246, -0.00621881, 0.0158925, -0.019606, -0.00134406, 0.00254602, -0.00923627, -0.0513532, 0.00937563, 0.0112944, -0.00655795, 0.0143965, 0.0040537, -0.0376752, 0.00700101, 0.0118285, -0.0052962, -0.101352, -0.0795874, 0.00338047, 0.0271538, -0.112761, -0.0157412, -0.166017, 0.00754113, -0.00379186, -0.0214719, -0.0214317, 0.0109629, -0.00511402, -0.0192619, -0.00596345, 0.0178536, -0.00639353, -0.0143591, 0.00367192, -0.132763, -0.00938376, -0.129058, -0.000604882, -0.00399506, 0.00257249, 0.0113975, -0.00628799, 0.00459154, 0.00800885, 0.0113025, 0.00579961, -0.00344465, 0.0148666, -0.000888829, 0.00825936, 0.0137244, 0.00704606, 0.00252857, 0.00106756, 0.0123429, -0.00294972, -0.281322, -0.00641972, -0.0444705, -0.0453406, 0.0196017, 0.00881553, -0.00540768, 0.00226823, 0.0268618, -0.11541, 0.00678989, -0.00105898, 0.002399, -0.0014027, -0.0210766, 0.000709499, -0.158231, -0.022539, 0.00815647, -0.0188124, -0.00743071, -0.0570796, -0.0110938, -0.0230565, -0.0649073, -0.0131549, -0.0234734, -0.00767095, -0.304966, -0.00426991, -0.00740065, 0.0282425, -0.107816, -0.0119566, 0.0186494, -0.00365787, 0.00562989, 0.00813679, -0.00690379, 0.0276512, -0.140806, -0.0103328, -0.00233373, 0.00699975, -0.00946676, -0.015455, -0.0533552, -0.020475, 0.107348, 0.195356, 0.00616133, -0.00420019, -0.0130008, 0.213597, 0.225113, 0.203666, -0.0158284, 0.213696, -0.101579, 0.201455, 0.190883, -0.01977, 0.00644174, 0.201273, 0.197068, -0.0567442, 0.111099, 0.211438, -0.231953, 0.200242, 0.193978, 0.0102154, 0.213018, 0.199912, 0.201623, 0.216735, 0.199635, -0.0379653, 0.00164669, 0.21441, -0.0272593, 0.206753, 0.213442, 0.119965, -0.0249093, 0.169402, -0.015793, -0.0639747, 0.127792, -0.0079591, -0.0390915, -0.141995, -0.0227293, -0.0613041, 0.208236, 0.218503, 0.184097, -0.0444661, 0.206294, 0.00868476, 0.0119226, 0.219704, 0.231842, 0.102513, -0.0247217, 0.211775, -0.071253, 0.201861, -0.111897, 0.226748, 0.211967, 0.213768, 0.207134, -0.0301642, 0.210864, 0.217533, -0.0169424, 0.214731, 0.219938, 0.196372, 0.0213467, -0.00508886, -0.000496287, 0.203617, -0.0176352, 0.219012, 0.214574, -0.00497326, -0.13244, 0.222267, 0.0327655, 0.103496, -0.0106506, 0.201584, 0.20618, 0.224952, 0.190496, -0.0930829, 0.206714, 0.191217, 0.18603, 0.205521, 0.0415953, -0.00600447, -0.560024, -0.0420735, 0.214643, 0.187773, -0.00541984, 0.035766, 0.18025, 0.190482, -0.172031, -0.0189158, 0.100893, -0.0211043, -0.152783, 0.201791, 0.19922, 0.204755, -0.118401, 0.214756, 0.216391, 0.204371, 0.0160008, 0.200686, 0.197346, 0.218524, -0.165837, 0.21586, 0.218165, 0.222027, 0.214828, -0.000608867, -0.0302017, 0.00202852, 0.17014, 0.214531, -0.0497749, -0.0220026, -0.0217468, 0.0618276, 0.292626, 0.286333, -0.007842, 0.278241, -1.68635, 0.081194, 0.232013, -0.026837, 0.000738123, 0.025131, 0.112753, -1.82876, 0.12842, 0.196778, 0.502122, 0.169662, 0.233816, -0.0720499, -0.0313653, 0.343937, -0.0279079, -0.0612423, -0.0316778, -0.0481321, -0.0233319, 0.129851, -0.0314246, 0.13364, 0.356162, 0.121924, -0.0376494, 0.357316, -0.00476134, -0.0189424, -0.468837, -0.00146976, 0.556925, -0.29722, -0.0286199, 0.149786, 0.325578, 0.0750106, 0.223257, 0.0117408, 0.220597, 0.00240052, -0.0381935, 0.0930279, 0.142192, 0.124694, -0.0324475, -0.167241, -0.0119384, 0.293599, -0.332741, 0.270132, -0.0331413, 0.123304, 0.112586, 0.0106714, 0.172114, -0.132368, -0.0165006, -0.034228, 0.227263, 0.144471, 0.00437634, 0.00181881, -0.0267413, 0.113307, -0.325552, -0.0659938, 0.498579, -0.0374354, 0.199408, -0.175389, 0.0730469, -0.765705, 0.011425, 0.0207113, 0.0320602, 0.0876586, 0.125669, -0.0117295, 0.256172, -0.130097, 0.00444639, 0.18937, -0.00577636, -0.0355117, -0.978319, -0.683823, 0.112993, 0.0120157, -0.0227312, 0.239315, 0.0739542, 0.202734, 0.468586, -0.0171141, 0.250435, -0.0130245, 0.205085, -0.018994, -0.173608, 0.197344, -0.315284, 0.100331, 0.185634, 0.130373, -0.0466746, 0.0395422, 0.123534, 0.199815, -1.06873, -0.0898574, 0.189427, -0.00749271, -0.0090981, -0.026253, 0.0487921, -0.00321963, 0.080916, 0.0763583, 0.0195652, -0.0162468, 0.0117678, 0.148693, -0.127584, -0.00108075, 0.0129839, 0.135001, 0.726685, 0.157604, -0.305001, -0.00250318, -0.0186743, 0.152011, 0.155089, -1.21169, 0.106003, 0.139907, 0.100684, 0.168152, 0.171022, -0.0372442, 0.159281, -0.0949238, -0.100702, -0.0842092, -0.200396, -0.00436265, 0.00336115, 0.20542, -0.0350814, 0.127692, 0.113198, 0.0273822, 0.00518302, 0.333882, -0.00094431, -0.359619, -0.291934, -0.00594582, -0.92427, 0.154009, -0.0405623, 0.0645769, -0.154506, 0.116042, 0.223088, -0.0512343, -0.182894, -0.00586709, -0.0122364, -0.0315234, 0.105495, 0.101237, -0.00418423, -0.105612, -0.283532, 0.187331, -0.325265, -0.160129, 0.0410633, 0.169534, -0.1448, -0.0126022, 0.105309, -0.0678527, -0.00756684, -0.324777, 0.166334, -0.122332, -0.00288575, 0.00670387, 0.00877347, 0.178072, -0.0243765, -0.159384, 0.0314919, -0.0126762, -0.518152, -0.104942, -0.0886934, 0.0286548, 0.011096, -0.1665, -0.317282, 0.146635, 0.162904, 0.514453, 0.181782, -0.358045, -0.0997158, 0.144625, -0.0176058, 0.00498012, -0.475121, 0.330569, -0.450362, -0.0668021, 0.00514615, -0.00276125, 0.131987, 0.141086, 0.0365011, 0.0148116, -0.0623256, -0.0371961, -0.620404, -0.260423, -0.0828597, 0.175265, 0.0502883, -0.161886, 0.184076, 0.209498, -0.000137244, 0.110855, -0.105371, 0.218499, 0.233774, -0.185354, -0.0595847, -0.138136, -0.164928, 0.00121793, -0.436726, -0.0114404, 0.421771, -0.219337, -0.0268811, -0.00390267, -0.0058786, -0.146768, -0.831949, -0.306995, 0.00415298, -0.478548, 0.445862, -0.186794, -0.62997, 0.00551841, -0.00466063, -0.16921, -0.123538, -0.443048, 0.388621, -0.266009, 0.0192028, -0.205347, -0.131547, -0.0267806, -0.382935, -0.308184, -0.441318, -0.804895, -0.640061, -0.0800061, -0.00583493, -0.145472, -0.0261196, -0.172923, -0.212981, 0.434726, 0.00192744, -0.216188, -0.0247809, -0.27382, -0.831759, -0.0176541, -0.207206, 0.10517, 0.00386691, 1.05811, -0.29344, -0.179028, -0.499971, -0.0378232, -0.74144, -0.00989929, -0.00830101, -0.641226, -0.14653, 0.413497, -0.00395282, -0.516861, -0.0746195, -0.181465, -0.567792, -0.693121, -0.361969, -0.159723, -0.538414, -0.000503933, -0.340725, -0.401221, -0.0106076, -0.69838, -0.173501, -0.849498, -0.00363247, -0.00667244, -0.0196751, -0.195734, -0.115519, -0.673389, -0.44474, -0.0159055, 0.69951, -0.345171, -0.00466058, 0.441114, -0.0161986, -0.586853, -0.517005, -0.164546, -0.146222, 0.224208, -0.144455, -0.242507, -0.832168, -0.224105, -0.00772156, 0.00602913, -0.192806, 0.703916, -0.128397, -0.517953, -0.0329163, 0.132471, -0.151508, -0.251997, 0.0517123, -0.0111562, 0.2533, -0.0217944, 0.581658, -0.627007, -0.42713, -0.191323, 0.0883218, -0.516712, -0.174796, -0.114113, -0.0328637, -0.164153, -0.520858, -0.256017, 0.132632, -0.667724, -0.456197, -0.443037, -0.515872, -0.0165902, -0.362053, 0.0101206, 0.442027, -0.0978049, -0.0114529, 0.00395302, 0.0449575, -0.178829, -0.23308, -0.173578, -0.0192799, 0.0464525, -0.674576, -0.0673303, 0.203566, 0.00726578, 0.00485465, -0.203294, -0.00345815, -0.626556, 0.462789, -0.228247, 0.113584, -0.0296232, -0.162718, 0.0127351, -0.462058, -0.239509, -0.0821732, -0.0620783, 0.0385296, -0.027242, 0.0122486, -0.0434505, 0.00240044, -0.102259, -0.155845, 0.459127, -0.0181784, 0.417933, 0.0147171, -0.308174, -0.38711, -0.0135547, -0.27375, 0.0809864, -0.0122574, 0.299679, -0.603449, -0.053757, 0.238083, -0.00966475, -0.0154617, 0.00139179, -0.0369514, -0.527872, -0.0692439, 0.484058, 0.00315916, -0.0227431, 0.288594, 0.122596, -1.1435, 0.0622168, -0.460933, -0.0708989, -0.177901, 0.00828984, 0.0111179, 0.241063, 0.000578202, -0.766977, -0.0553947, -0.413361, -0.010167, -0.0204836, 0.00051337, -0.036848, -0.273453, 0.0698369, -0.0495425, -0.00972598, 0.165552, 0.246372, -0.130944, -0.427691, -0.0158292, -0.00174989, -0.315095, -0.0681178, -0.0342111, 0.115565, -0.238474, -0.100355, 0.107367, 0.00270347, 0.00624548, -0.00380596, -0.998285, -0.378618, 0.0558755, -0.519032, -0.0274714, 0.227286, -0.0165658, -0.0501189, 0.689499, -0.0113292, 0.0893608, -0.0260316, -0.00492819, -0.12126, -0.0204531, -0.152974, -0.0451697, 0.277294, 0.055, -0.427239, -0.0300648, 0.0123781, -0.110756, -0.106625, -0.333959, 0.210974, -0.293736, -0.194972, -0.119483, 0.0161286, -0.535094, -0.0100809, 0.0124964, -1.29031, -0.0136514, 0.00911339, 0.0133219, -0.325724, -0.827501, -1.25047, 0.000940676, -0.542031, -1.09764, -0.440217, -0.927924, -0.0119156, -0.0199112, -0.373591, -0.455615, 1.23408, 0.0535573, -1.07206, -3.09891, -0.550701, -0.545877, -0.0961279, -0.598609, -1.10449, -0.96016, -0.834499, -0.866784, -0.0269369, 0.00201327, -0.576857, -0.00568528, -0.397227, -1.27115, -0.0516134, -0.0107509, -10.1918, 0.00179363, -3.18022, -2.09559, -0.0227911, 0.356679, -0.781785, -0.0194818, 0.0846027, -1.18651, -0.456449, -0.605569, -0.0219641, -1.2971, -0.0106308, -0.00476517, -1.02484, -0.440777, 0.000443352, -0.0282271, -0.977054, -2.69162, -0.572944, -3.01406, -1.10721, -0.58948, -0.566356, -1.08407, -0.000384293, -0.549562, -0.594484, -0.00596565, -1.10473, -0.696021, -1.11724, -0.0234266, -0.0116166, -0.0182861, -0.492839, -0.188262, -0.693803, -1.03192, -0.000905554, 0.162002, -0.738855, -2.83777, 0.765764, 0.00795966, -0.715878, -1.02557, -0.418845, -0.514877, -1.28205, -0.476187, -4.31363, -0.795739, -0.556495, 0.000716521, -0.0170662, -2.07342, 0.482545, -4.33074, -1.03274, -0.0481979, -3.01833, -0.319139, -0.655056, -2.81578, 0.0196674, -2.06279, 0.0102019, 0.177461, -1.00486, -1.06325, -0.684761, -2.90891, -0.82634, -0.459817, -0.26351, -0.0232635, -0.325719, -1.03662, -0.515651, -2.61613, -0.749095, -1.00009, -1.13847, -0.973055, -0.0103496, -0.088862, -0.0187169, 0.150193, 0.48589, -0.0257525, 0.0110642, 0.00538172, 0.797634, -0.0314855, 0.397427, 0.0152193, 0.378484, 0.806614, 0.731051, -0.541782, 0.0156088, -0.000916414, 0.785257, 0.808628, 0.458848, 0.135336, 0.24063, -0.449181, 0.636981, 0.721804, 0.00714062, 0.65011, 0.490282, 0.205396, 0.01724, -0.00647229, -0.0823156, 0.00509561, 0.690891, -0.0181159, 0.725386, 0.473652, 0.202835, -0.0201704, -0.232687, 0.00439431, -0.278575, -0.948333, -0.0103283, -0.510214, 0.496122, -0.00161723, -0.839631, 0.499141, 0.669515, 0.300835, -0.0776962, 0.0660957, -0.001953, -0.0183694, 0.22341, 0.699081, 0.146303, -0.00973522, 0.167434, -0.120712, 0.719003, -0.0850025, 0.0521773, 0.655625, 0.719687, -0.0616484, -0.0160941, 0.500911, 0.188312, 0.00959694, -0.490509, 0.683569, 0.0496165, -0.002591, 0.00281925, 0.00790855, 0.657492, -0.716872, -0.0271754, 0.0191419, -0.0424301, -1.17268, 0.244107, -0.48311, 0.77985, -0.0235404, -0.047436, -0.448285, 0.734892, 0.658457, -0.216618, 0.732066, -0.0893796, 0.0610489, 0.68632, -0.008842, -0.0181328, 1.18102, 0.998697, -1.07619, 0.230867, -0.0143908, -0.217694, 0.764383, 0.552373, -0.249295, 0.00303557, -0.31674, -0.0059124, -0.779038, -0.13594, 0.0966365, 0.539203, -0.573956, 0.0427116, 0.479053, 0.923822, -0.00721616, 0.815551, -0.144921, 0.629145, -0.431229, -0.187232, 0.268014, 0.0311798, 0.0538173, -0.0073394, -0.161486, -0.000459355, -0.854196, -0.457947, -0.0146529, -0.0165673, -0.0119285, -0.316989, -0.34736, -0.382408, -0.0174971, -0.445227, 0.0636271, -0.384972, -0.581212, 0.00155465, -0.0121699, -0.325219, -0.399196, 0.782542, -0.832799, -0.538619, -0.932621, -0.444199, -0.410939, -0.0261318, -0.466842, -0.330056, -0.326603, -0.445578, -0.560868, -0.0797693, -0.00745389, -0.405595, -0.00795543, -0.326323, -0.358723, -0.880903, -0.0103754, -0.0225736, -0.00275202, -0.735328, -0.740335, -0.019834, -0.148244, -0.232701, -0.0117465, -0.908, -0.296995, -0.409922, -0.626946, -0.0314668, -0.367682, -0.00655947, -0.0238555, -0.405017, -0.322434, -0.826897, 0.0017566, -0.513675, -0.972528, -0.424843, -0.393868, -0.289813, -0.420491, -0.338575, -0.318354, -0.0259442, -0.438957, -0.450989, 0.00407846, -0.357271, -0.452897, -0.323727, 0.00533276, -0.0150621, -0.00610197, -0.401099, -0.146239, -0.582862, -0.395685, -0.026422, -0.839463, -0.57657, -0.380773, -0.964568, 0.0141295, -0.502666, -0.595347, -0.342438, -0.381646, 0.33682, -0.289799, -0.828161, -0.413317, -0.439482, -0.0128824, 0.0168006, -0.678202, -0.80407, -0.299867, -0.4636, -0.0138567, -0.7105, -0.328637, -0.488707, -1.0178, -0.0105095, -0.830188, -0.0179798, -0.907407, -0.554712, -0.558758, -0.580458, -0.562925, -0.490996, -0.467476, -0.1348, 0.0213831, -0.325301, -0.314487, -0.515952, -0.297952, -0.531952, -0.588903, -0.355783, -0.423205, 0.00661045, -0.574132, -0.00341106, 1.0346, 0.815876, 0.017023, -0.00155515, -0.0280995, 0.773328, 0.0304306, 0.283631, -0.00374688, 0.703736, -0.405365, 0.772662, 0.0412534, 0.00432302, -0.00220745, 0.770478, 0.740344, 0.025803, 1.03091, 0.763873, -0.870561, 0.667056, 0.682707, -0.0350563, 0.699321, 0.0329062, 0.0367903, 0.0175179, 0.039329, -0.00804815, -0.0284278, 0.765807, -0.0167365, 0.677483, 0.728765, 0.947133, -0.0216177, 0.715907, -0.0209122, -0.57563, 0.212086, -0.0152498, -0.221732, 0.334544, -0.00137672, 0.915854, 0.751636, 0.718452, 0.773555, -0.039206, 0.0962303, -0.00399529, -0.0208983, 0.0413704, 0.720813, 1.06582, -0.00681318, 0.0400675, -0.662312, 0.717971, 0.210124, 0.0374025, 0.751531, 0.710476, 0.04488, -0.0218345, 0.634768, 0.00839371, -0.0029788, 0.09539, 0.686362, 0.251443, -0.00502605, 0.00959023, -0.00760205, 0.745505, 0.0020904, 0.0257513, 0.0305271, -0.0099634, -0.031099, 0.0299749, -0.67242, 0.922114, -0.00538751, 0.0335545, 0.0335647, 0.774676, 0.688707, -0.296643, 0.709343, 0.742753, 0.0389009, 0.676795, -0.00279432, 0.0036357, -0.788902, -0.0616192, 0.204011, 0.0274195, -8.34654e-05, -0.661372, 0.677734, 0.634774, -0.946761, 0.000311436, -0.66316, -0.0188857, 0.604273, 0.0570313, 0.0316708, 0.781033, 0.208738, 0.0315711, 0.741879, 0.578358, -0.0167029, 0.752821, 0.0577903, 0.738839, -0.449753, 0.028254, 0.0441562, 0.0349423, 0.0214941, -0.00360927, -0.408662, -0.0140623, -0.0471351, -0.00974835, 0.00440188, -0.00372635, 0.00784175, -0.0077982, 0.0351, 0.0322691, 0.00143263, 0.0299382, -0.177107, 0.0103905, 0.00363485, -0.00461855, -0.000145029, 0.0262544, 0.0128817, -0.0298713, -0.0232435, 0.0104961, 0.0581115, 0.0284219, 0.029013, -0.0452672, 0.0127561, 0.00493252, 0.0343542, 0.0279379, 0.0414824, -0.0563386, -0.00819039, 0.0150449, -0.0106121, 0.0259637, 0.0283328, -0.027083, 0.00725783, 0.0283326, 0.00560248, -0.0628443, -0.0548403, 0.0120677, 0.0306625, -0.0883531, -0.000537829, -0.102944, 0.0419797, 0.0186521, -0.000230246, -0.0503356, 0.0321761, -0.00347201, -0.0345824, 0.0389065, 0.0106108, -0.0113078, -0.00325496, 0.0282498, -0.153079, 0.0394754, -0.228216, 0.0189217, -0.00344718, 0.0157281, 0.0203559, -0.00654953, 0.0238944, 0.0204549, 0.00126596, 0.0445699, 0.0479459, 0.0260172, 0.00717837, -0.0149676, 0.00900948, 0.0131359, -0.0126195, 0.042331, 0.0111837, -0.00320524, -0.325833, 0.00422551, -0.00973097, 0.0211769, 0.0195284, 0.0140718, 0.0206925, 0.0250039, 0.0315361, -0.203156, 0.0260934, 0.00623444, 0.0420441, 0.0176061, -0.0068279, 0.00211279, -0.248444, 0.0229196, 0.0453109, -0.0101196, 0.0156163, -0.0659166, -0.00334601, -0.00211206, 0.0663779, 0.00533608, -0.0336446, -0.00993368, -0.373262, 0.0186511, 0.0141137, 0.0237669, -0.070108, 0.0173785, 0.0142194, 0.0146428, 0.0216581, 0.0125532, 0.0042599, 0.0255721, -0.167921, 0.0103586, 0.0278397, 0.0390473, 0.0262615, 0.00269139, -0.0392401, -0.0113716, -0.0474457, -0.247009, 0.00437051, -0.0115182, -0.0658862, 0.454319, -0.0787574, -0.147543, -0.00438678, -0.0986309, 0.0694613, 0.44219, -0.675894, -0.00982711, -0.00451432, 0.464674, 0.45732, 0.785445, -0.034676, -0.0258392, -0.0769041, 0.418047, 0.491028, -0.0111104, -0.192392, 0.422253, -0.013388, -0.395838, 0.0137232, -0.0572926, -0.00142753, 0.38381, -0.00737276, 0.317315, -0.147205, -0.066407, -0.0640727, -0.165477, -0.00873819, -0.0904573, 0.520495, -0.0215236, -0.210059, 1.57924, 0.00165744, -0.266972, 0.307292, 0.372098, -0.260265, -0.0278659, 0.00632204, -0.00209473, -0.0265849, 0.0281956, 0.372042, -0.0357328, 0.0039809, 0.07554, -0.326445, 0.171834, -1.54847, 0.220203, 0.0391603, 0.460678, -0.0297011, -0.020264, -0.0968198, 0.0336684, 0.0113501, -0.340385, 0.430349, -0.294014, -0.00168698, 0.00398323, 0.00285241, 0.351577, -0.601578, 0.0254656, -0.0308365, -0.00566409, 0.27422, 0.143162, -0.57647, 0.274908, -0.0112996, 0.0312142, -0.707714, 0.416836, 0.36921, 1.05265, 0.413967, -0.100957, -0.2893, 0.00704855, -0.0195656, 0.00926674, -1.03212, 0.249136, -0.395003, 0.238867, 0.00566948, -0.0577237, 0.411447, -0.0390956, 0.0527744, -0.0011721, 0.0634001, -0.0181339, 0.905668, -0.0654569, -0.156632, 0.227955, -0.0920915, 0.0861715, -0.0588661, 0.945709, -0.0224937, 0.410625, -0.0936294, -0.0478722, -0.549815, -0.213329, 0.122536, -0.11612, -0.0468009, 0.00579008, 0.114776, -0.0114099, -0.030793, -0.651998, 0.0160646, 0.005129, -0.00908176, -1.36354, -0.0134853, -0.731304, 0.00486486, -0.277786, 1.29101, -0.925698, 0.0737907, -0.0129122, 0.0146398, -1.35775, -1.25576, 0.918824, -0.0905313, -0.655835, -0.639143, -0.695979, -0.540337, -0.00675807, -1.36489, 0.132412, -0.00890413, 0.0983302, -0.333875, -0.00954195, 0.0164307, -0.745516, -0.01714, -0.879695, -0.633851, 0.0107561, 0.0369303, -1.71772, 0.00641287, -0.564556, -1.80069, 0.00547778, 1.09956, 0.290533, -0.018487, 0.0482311, -0.0306719, -1.0555, -0.279199, 0.00268091, -0.519612, -0.00163745, 0.000894147, -0.270597, -0.846695, -0.0883504, -0.00184235, -0.22187, -0.38842, -0.510503, 1.59855, -0.425826, -1.25838, -0.868822, -0.135242, -0.0161717, -0.615758, -0.00458445, 0.0158875, 0.200236, -0.71975, -0.417359, -0.00800425, -0.00727688, -0.0109676, -0.907465, 0.741321, -0.349404, -0.152215, -0.01731, 0.170732, -0.0818767, -0.381465, 1.34253, -0.00260843, -0.0877053, 0.0881356, -1.38217, -0.870599, 0.87848, -0.395645, -0.126836, 0.133374, -0.734522, 0.00219469, 0.000497456, 1.40803, 1.88727, -1.14139, -0.31208, 0.0105161, -0.510459, -1.34067, -0.411912, -0.484027, 0.00586297, 0.111395, 0.00320076, -0.0649057, -0.556532, -0.195748, -0.729899, -0.386822, -0.25917, -0.570856, -0.518603, 0.0295735, -1.22868, -0.0884812, -0.442146, -0.242008, -0.244698, -0.375539, -0.196155, -0.043921, 0.00339456, -0.684032, -0.00646327, -0.00803728, 0.0297702, -0.00849682, -0.00784098, 0.0350415, 0.0300222, 0.049212, 0.045419, -0.00286014, 0.0468642, -0.0394021, 0.0266931, 0.0396564, -0.00301688, -0.00121417, 0.0472342, 0.0400088, 0.00735662, 0.000191654, 0.05972, -0.0967579, 0.0246867, 0.0258664, -0.00184309, 0.0439341, 0.0369226, 0.027527, 0.0482576, 0.0394457, -0.0962926, -0.00434931, 0.0508742, -0.0210388, 0.0265452, 0.0368956, -0.008318, -0.00502812, 0.0203393, -0.00552158, -0.0211652, -0.0479846, -0.00596885, 0.0113844, -0.0752678, 0.00155951, -0.212766, 0.0425663, 0.036639, 0.0450656, -0.0340293, 0.0332875, 0.00172028, -0.0117485, 0.0444555, 0.0485708, -0.0157418, -0.00475836, 0.0228955, -0.0461289, 0.0409852, -0.186689, 0.0597628, 0.0444798, 0.0274201, 0.0497336, 0.00329985, 0.0488173, 0.0569883, -0.0154158, 0.0468361, 0.0486746, 0.0452802, 0.0218054, 0.00522118, 0.00657114, 0.0598877, -0.0172583, 0.0484879, 0.0504812, 7.63614e-05, -0.255833, 0.0433503, -0.0205337, -0.0131197, 0.00199136, 0.0581272, 0.0303865, 0.0228229, 0.0587285, -0.0406135, 0.0587081, 0.0228637, 0.0558811, 0.0460001, 0.00219743, 0.00342185, -0.240287, 0.0103591, 0.0343471, 0.035935, -0.0110372, 0.0307755, 0.0209107, 0.0447037, -0.0880743, -0.00547272, 0.010312, -0.0152357, -0.247377, 0.036203, 0.0356969, 0.0346797, -0.0207718, 0.0321649, 0.049884, 0.0319191, -0.00248799, 0.0347276, 0.0323915, 0.0440972, -0.033418, 0.0476233, 0.0362621, 0.0431805, 0.0411582, -0.0251602, -0.0411043, -0.00573021, -0.028969, 0.0852592, 0.0144087, -0.0273054, 0.0175951, 0.0949195, 0.148812, 0.0819523, -0.023673, 0.0936037, -0.110534, 0.0768815, 0.108419, 0.00446112, 0.00645933, 0.109789, 0.0801372, -0.0730958, -0.0118569, 0.106788, -0.109561, 0.110351, 0.0632146, -0.0505697, 0.202442, 0.0894216, 0.109072, 0.025808, 0.096177, -0.0405194, 0.0141461, 0.0988355, -0.0174099, 0.0939856, 0.0964056, -0.0269588, -0.00598423, 0.0743157, 0.0118402, -0.115909, 0.0118491, -0.00765354, 0.0272979, -0.0568429, 0.00987415, -0.209348, 0.0711322, 0.0906612, 0.0963251, -0.0230819, 0.0362569, -0.00555659, -0.0328596, 0.0735839, 0.0817551, 0.00678098, -0.0139285, 0.0530767, -0.0403894, 0.0992318, -0.0997471, 0.142315, 0.0739258, 0.0980345, 0.113173, 0.0107466, 0.0968916, 0.11185, -0.000426638, 0.0847257, 0.0916187, 0.127463, -0.0188377, -0.010187, 0.00509378, 0.0964385, -0.0808788, 0.10782, 0.198535, 0.00103552, -0.255979, 0.117683, -0.104436, 0.0245169, -0.00341167, 0.0735899, 0.119488, 0.0735698, 0.0936961, 0.0222897, 0.0949979, 0.07069, 0.00885742, 0.0842858, -0.0142355, 0.0136981, -0.169169, 0.0332198, 0.152197, 0.0900972, -0.00953737, 0.0183347, 0.0796077, 0.0855237, -0.03824, 0.00268389, -0.0593539, -0.0245566, -0.251423, 0.097705, 0.10419, 0.0854923, 0.0347431, 0.117097, 0.106887, 0.0937163, -0.000114298, 0.0984733, 0.0749144, 0.101502, 0.0372009, 0.103471, 0.108628, 0.0609663, 0.0954158, -0.0126096, -0.0795076, -0.00315175, 0.00554315, 0.0246868, -0.0229632, 0.0102404, -0.00198663, 0.0663679, -0.240221, -0.0384848, -0.00170647, -0.0641742, -0.0772379, 0.0706244, 0.255268, 0.00630202, -0.0046536, 0.129591, 0.118402, 0.484297, -0.0170467, -0.0118113, -0.555442, 0.0253596, 0.0585199, 0.0329552, 0.0713148, -0.107812, -0.203406, -0.880455, -0.177309, -0.0357946, -0.00669053, -0.013194, 0.00680848, 0.0851515, -0.01063, -0.121283, -0.0328643, -0.948419, -0.0121793, -0.394966, -0.454956, -0.00503504, -1.05017, -0.607438, -0.00144874, -0.0122834, -0.395743, 0.0206447, 0.0849371, -0.0399704, -0.562582, 0.00870879, -0.01335, -0.0760443, -0.0453225, -0.0023389, 0.00447101, -0.28465, 0.224308, 0.181522, -0.349388, -0.399345, 0.18862, -0.0104245, -0.342603, 0.0106019, -0.0100986, 0.0263716, 0.00429449, -0.386878, 0.103412, -0.267687, -0.00168159, -0.00620266, -0.0152956, 0.0616271, -0.02428, -0.163637, -0.208343, -0.0236481, -0.0739894, 0.0458587, -0.264988, 0.641348, -0.0133404, -0.297277, -0.212592, 0.108554, 0.0610212, 0.49834, -0.0350117, -0.266839, -0.973193, 0.135991, -0.0193119, -0.000578922, -0.533113, 0.583682, -1.07693, -0.0767535, 0.00138577, -0.268959, 0.0699056, 0.144405, -0.404888, -0.0106782, 0.338243, -0.000932956, -0.0716257, -0.304588, -0.318724, 0.146603, 0.0235582, -0.286881, 0.175149, 0.112574, -0.0209422, 0.0921492, -0.363665, 0.17815, -0.425741, -0.143901, -0.088605, -0.222865, -0.286301, -0.017385, -0.00657004, 0.00261361, 0.89716, -1.93758, -0.00376573, -0.0240993, 0.0175156, -1.09583, -1.61214, -2.49766, 0.0232486, -1.74299, 0.591974, -1.26081, -1.9972, -0.0235425, -0.023951, -0.954271, -0.89383, 0.495728, 0.970728, -2.00008, -0.330285, -1.23596, -1.26339, -0.0503755, -1.20425, -1.62211, -1.29945, -1.25365, -0.852352, -0.0341165, -0.0209235, -1.08293, -0.0289932, -1.0241, -2.1922, 0.837298, -0.0581396, -4.62236, -0.00254877, -0.298719, -1.86115, -0.0103892, -0.214317, -1.47775, -0.0101882, 1.02764, -1.96182, -1.03484, -1.69178, -0.0139446, -1.87082, 0.0105546, -0.00513705, -1.91232, -1.22263, 0.876528, -0.0154453, -1.11674, -0.648018, -1.73892, -0.0503143, -1.45217, -0.983761, -1.14928, -1.36195, -0.00716575, -1.53697, -1.2595, 0.00574484, -1.79837, -1.34242, -2.37297, -0.0361314, -0.00651626, -0.0240346, -1.06208, -0.395377, -0.817018, -1.77955, -0.0130674, 1.17286, -1.21114, -0.407954, 1.55131, -0.00860625, -1.18694, -1.44053, -0.980912, -1.02226, 0.536396, -1.2925, -1.40378, -1.60145, -1.55353, -0.00287786, -0.0203494, -0.0843676, 1.46285, -0.649377, -1.79787, 0.0337015, -0.21974, -0.992389, -1.57565, -0.126853, 0.012454, -1.65284, -0.00330027, 0.894469, -0.8957, -1.25202, -1.93224, 0.103859, -1.32557, -1.69722, -1.20754, -0.00181583, -1.2225, -1.26168, -1.74571, 0.718713, -0.962086, -2.01601, -1.64017, -1.36272, -0.00780002, -0.520694, -0.00898067, 0.0387847, -1.74523, -0.0375685, -0.00130243, -0.028596, -1.77138, -1.57628, -1.76046, -0.0242633, -2.04418, 0.357786, -2.0654, -1.83384, -0.0122952, -0.0235231, -1.80952, -1.95713, 0.588687, 0.0442948, -1.81716, -0.182391, -1.96265, -2.04328, -0.0210187, -1.72799, -1.82668, -1.59623, -1.86522, -1.68267, -0.0287041, -0.0134221, -2.05431, -0.0225476, -1.94517, -1.67192, 0.00935739, 0.00800605, 0.723469, -0.0133286, -0.102014, -1.10842, -0.00217322, -1.02362, 0.537297, -0.0328031, 0.298998, -1.55957, -1.98449, -2.11151, -0.0792874, -1.84757, -0.0217647, -0.0359237, -1.73241, -1.8867, 0.0471076, -0.0414413, -1.7409, 0.0149985, -2.12531, 0.32689, -1.78603, -1.17813, -1.81597, -1.82339, -0.0651743, -1.99927, -1.64739, -0.0217139, -1.37739, -1.97638, -1.74184, -0.0168113, -0.0250936, -0.0188375, -1.8243, 0.0255328, -1.4675, -1.96879, -0.037809, 0.450465, -1.56714, 0.0423157, 0.0901827, -0.0134613, -1.80176, -1.7054, -1.88291, -1.89649, 0.565036, -2.24998, 0.0782868, -2.17096, -2.24218, -0.0477891, -0.00616846, 0.327779, 0.373917, 0.402977, -1.50726, -0.0335323, -0.0618739, -1.76625, -2.05239, -0.168644, -0.0203758, -0.738371, -0.0310813, 0.267011, -1.62454, -1.94694, -2.06174, 0.190159, -1.85035, -1.75971, -1.63103, -0.050095, -1.87034, -1.88013, -1.90648, 0.847252, -1.43296, -2.01772, -1.60277, -1.6554, -0.0176427, -0.958462, -0.0215371, 0.382023, -0.787232, -0.00699705, -0.00234803, -0.0226818, -0.0523923, -0.433068, -0.747597, 0.0031935, -0.182629, 0.401205, -0.107063, -0.104274, -0.0199973, 0.00437973, -0.0772175, -0.06098, -0.691793, 0.355981, -0.583545, -0.0572397, -0.206524, -0.135174, 0.0315265, 0.0156175, -0.482975, -0.323957, -0.185086, -0.176112, -0.0310257, -0.00869526, -0.169068, 0.0017965, -0.0625449, -0.8229, 0.334045, -0.00420065, -0.430887, -0.010567, -0.231635, -0.344418, -0.016382, 0.219414, -0.165466, -0.0170941, 0.226993, -0.843811, -0.0305649, -0.234379, -0.0660148, -0.325007, 0.000293004, -0.0183943, -0.511555, -0.0666044, 0.361196, -0.00186035, -0.308027, 0.0709962, -0.190253, -0.0834698, -0.351342, -0.263061, -0.073222, -0.422937, -0.0126526, -0.288769, -0.117376, 0.00351717, -0.667558, -0.309323, -0.72444, -0.003733, -0.00376363, -0.0101582, -0.139184, -0.0185093, -0.133506, -0.374471, -0.0248316, 0.162109, -0.00354809, 0.0742553, -0.138565, 0.00315278, -0.231573, -0.155365, -0.0509812, -0.145677, 0.404555, -0.0892722, -0.886223, -0.185649, -0.221291, -0.0153316, 0.00444306, 0.0522583, 0.404141, -0.164119, -0.450773, -0.00927459, 0.185992, -0.0582633, -0.240754, 0.0499181, -0.0101112, 0.346033, -0.0328875, 0.173854, -0.22648, -0.383387, -0.377386, -0.0564428, -0.0215919, -0.280225, -0.0483992, -0.0293832, -0.123736, -0.446686, -0.278451, 0.454333, -0.143693, -0.380916, -0.369888, -0.338124, 0.00569186, 0.349902, -0.0162887, 0.112892, -0.0284586, -0.0168213, 0.0147256, -0.034489, -1.72847, 0.37538, -0.320621, -0.0165012, 0.17348, 0.764206, -1.37112, -0.550726, -0.0299515, -0.0115222, -1.76936, -1.66234, 0.842673, 0.14016, -0.261158, 0.000150388, -0.0869885, -0.144818, 0.0380107, -0.837555, -0.258761, -0.0880841, -0.440193, -0.675569, 0.00487818, -0.0252078, -0.640402, -0.0250525, -0.868268, -0.298343, 0.080103, -0.00930494, -1.36457, -0.00984474, -0.101815, -1.59419, -0.0338828, -0.238292, -1.23612, -0.0152491, 0.231958, -0.246759, -1.34478, -0.207086, -0.106931, 0.00221493, -0.0153325, -0.0218765, -0.240201, -0.817432, 0.122445, -0.0372093, -0.279892, -0.0114422, -0.0727333, 0.617392, -0.197962, -1.08296, -1.40323, -0.0194617, -0.0629951, -0.312708, -0.466829, -0.00261758, -0.212565, -0.31473, -0.126211, -0.0118505, -0.0215766, -0.0212528, -0.79128, -0.0531513, -0.767795, -0.36827, -0.0348784, 0.314101, -0.410002, 0.160945, 0.516028, -0.0195816, -0.18471, -0.731124, -1.70187, -0.729607, 0.797464, -0.479102, 0.45011, -0.0941617, -0.177792, -0.0165558, -0.0146825, 0.990468, 0.855577, 0.18654, -0.255079, -0.0207899, -0.0260364, -1.62005, -0.0303048, -0.240006, -0.016328, -0.324981, -0.0226475, 0.212141, -0.772477, -0.14435, -0.495581, 0.366215, -0.383802, -0.0138764, -0.304218, -0.0118909, -1.43563, -0.0195204, -0.23004, 0.16001, -0.716576, -0.126857, 0.0635491, 0.057686, -0.00374022, -0.813302, -0.0198923, -0.0474917, -0.658212, 0.000585448, 0.00100419, -0.0531791, -0.345832, 0.0654106, -0.0830616, -0.00210131, -0.471443, 0.208115, -0.348994, 0.194675, -0.00466268, -0.00590103, -0.384708, -0.256311, 0.642857, -0.0544198, -1.43161, -1.03963, -0.421852, -0.521117, -0.0250964, 0.010425, -0.960574, -0.232013, -0.482238, 0.0161515, -0.053773, 0.0123982, -0.17905, -0.000822359, -0.215591, -0.72372, -0.117972, -0.032607, -2.9487, -0.00522439, -1.14909, -0.473652, -0.00471468, -0.0446373, -0.403804, 0.00406613, -0.191517, -1.45051, -0.428502, -0.418714, -0.022064, -0.0255072, 0.00331522, -0.0307829, -0.801489, -0.410888, -0.0458543, -0.0100679, 0.0126596, -1.15348, -0.477784, 0.555133, -0.211604, -0.333641, -0.268438, -0.298248, 0.00675464, -0.282453, -0.0911134, -0.0122306, 0.228204, -0.697756, -0.182304, -0.0021486, 0.0179739, -0.00634043, -0.190144, 1.13284, 0.134092, 0.286692, -0.0107797, 0.000311073, -0.044557, -1.08219, 0.206295, -0.000991636, -0.0465588, 0.222651, -0.228407, -0.262237, -0.14612, -0.452705, -2.09307, -0.536635, -0.446951, -0.0116581, -0.00483432, -0.464224, 0.0596972, -1.71216, -0.918401, 0.00237847, -0.884601, -0.217736, -0.339788, -1.29347, -0.00392877, -0.972632, -0.0104868, 0.0277176, 0.258568, -0.12907, -1.22647, -2.25845, -0.460683, -0.488989, -1.24026, 0.00357162, -0.276503, -0.506254, -1.16231, -0.737072, 0.000726569, -0.965464, -0.408698, -0.0742936, 0.00644537, -0.842215, 0.00168446, 0.862519, 0.46801, -0.0028568, -0.0123628, 0.00903955, 0.944725, -0.137361, 0.310041, 0.0192973, 0.496922, 0.113969, 0.949631, -0.279465, -0.0285365, 0.0156835, 0.965412, 0.947022, -0.152402, 0.921484, 0.48198, -0.471846, 0.727561, 0.708731, -0.142317, 0.837844, 0.402888, -0.11068, -0.537234, -0.062645, -0.100427, -0.00157023, 0.819929, -0.0314178, 0.79722, 0.443438, 0.75729, -0.0233497, 0.266919, -0.0139034, -0.359155, -0.392153, -0.0090961, -0.464594, 0.656532, -0.0018213, 0.305116, 0.374588, 0.906385, 0.401917, -0.00469098, -0.104779, 0.00265657, -0.00610363, 0.202464, 0.89956, 0.889053, -0.00590969, 0.0239987, -0.563287, 0.503619, -0.840029, -0.318028, 0.595479, 0.889702, -0.11798, 0.00298585, 0.358544, 0.0701355, -0.000524567, -0.479475, 0.780007, 0.139058, -0.0446692, -0.00843975, -0.0280394, 0.811158, -0.15758, -0.181852, 0.244916, 0.00967345, 0.232041, 0.210704, -0.273129, 0.76276, -0.0312857, -0.210884, -0.493206, 0.929839, 0.850001, 0.3867, 0.774094, 0.259554, -0.483161, 0.391498, -0.0193712, -0.0151652, -3.09434, 0.684716, -0.564203, 0.0691035, 0.0109855, -0.241452, 0.899772, 0.390566, -0.372122, -0.00668637, -0.289833, -0.0137402, -0.229605, -0.0722034, -0.0616858, 0.52697, 0.391712, -0.180912, 0.48451, 0.855821, -0.0137669, 0.977403, -0.227392, 0.547839, 0.735152, -0.128894, 0.346865, -0.11288, -0.0977413, 0.000824951, -1.12152, 0.000718092, 0.683684, 0.280058, -0.0059686, 0.0130492, -0.0155326, 0.108812, 0.744125, 0.480487, -0.0447132, 0.642246, -0.634374, 0.178552, -0.0576654, 0.00983477, -0.00266207, -0.0234181, 0.0891639, -0.623854, 0.668066, 0.106652, 0.312987, 0.345413, 0.365815, -0.0597823, 0.541242, 0.357292, 0.345883, 0.237122, 0.179723, -0.0564038, -0.0205334, 0.354091, -0.0231969, 0.254111, 0.45359, 0.6281, -0.0149219, -0.207005, -0.00804664, -0.167177, -0.567855, -0.00609974, 0.209463, -0.221856, 0.0095356, -0.0482519, 0.211582, 0.0794286, 0.338436, 0.014243, 0.559868, 0.0127842, -0.0130027, 0.123921, 0.374279, 0.671019, -0.00503388, 0.373845, -0.391291, 0.59887, 0.0515007, 0.699742, 0.425873, 0.228676, 0.455039, -0.0321449, 0.481209, 0.387531, 0.00560193, 0.0959626, 0.507294, 0.366947, -0.0262257, -0.00656039, -0.0139403, 0.276845, -0.0250291, 0.234994, 0.414482, -0.032652, -0.135827, 0.322564, 0.167049, 0.71995, -0.00105447, 0.465346, -0.0687358, 0.164592, 0.270211, -0.120078, 0.402294, 0.0154218, 0.298306, 0.543937, -0.0546949, -0.0163197, -1.54999, 0.534357, -0.0845094, 0.19059, -0.0292447, 0.146189, 0.201798, 0.457724, 0.108845, -0.00302794, 0.10985, -0.0194204, 0.0884539, 0.141228, 0.314761, 0.101082, 0.209216, 0.617869, 0.348282, 0.259785, -0.0341064, 0.301148, 0.509007, -0.00193923, 0.461914, 0.264306, 0.170047, 0.471778, 0.519608, -0.0177677, -0.846152, -0.00851989, -0.0354037, -0.00129834, 0.00893749, 0.00681804, 0.00462568, 0.0132925, -0.000925156, 0.0271361, 0.00914528, 0.00617654, -0.0439942, 0.0359885, 0.00473043, -0.0107403, -0.000392496, 0.00611829, 0.0217517, -0.0141837, -0.0215292, 0.0224291, -0.0264152, 0.00372259, 0.00965198, -0.00646255, 0.00912216, 0.00593975, 0.0223439, 0.0317527, 0.0158392, 0.0193424, -0.00456754, 0.016723, -0.0051061, 0.00602504, 0.0197167, 0.000455048, -0.0056725, -0.00111078, 0.0069071, -0.0104009, 0.00137261, -0.0151294, 0.0175784, -0.00400591, -0.00553513, 0.00525038, -0.00173676, 0.0143918, 0.00819505, 0.000474594, 0.02648, -0.000604252, -0.0132103, 0.01289, 0.0156111, -0.0132485, 0.0107099, -0.00193788, -0.0178972, 0.0216329, -0.0126415, 0.0107568, 0.00867776, 0.0149481, 0.0146217, 0.00274504, 0.00941557, 0.00374444, -0.0170159, -0.00145764, 0.0199557, 0.011465, -0.00165254, 0.0023543, -0.0214948, -0.0221063, 0.00741558, 0.00920067, 0.0166178, -0.0191184, -0.0078104, 0.0150441, -0.0171965, -0.000867372, 0.00352123, 0.0216039, 0.0157738, -0.00284203, 0.0302399, -0.00769749, 0.0295729, 0.0181801, 0.0268254, 0.0206412, 0.00224497, -0.0134229, 0.020164, 0.0142759, 0.0190933, 0.018587, 0.00698054, -0.0185753, 0.0191859, 0.00486311, -0.0158639, -0.0123602, -0.0444544, 0.0118121, 0.00189325, 0.000640489, 0.00555259, 0.00813555, -0.00942681, 0.0211552, 0.0158781, 0.00944563, 0.002913, 0.015401, 0.0362697, -0.00578867, -0.0097737, 0.0090296, 0.00235231, 0.00725452, 0.0233163, -0.00864104, 0.0143901, -0.0201417, -0.477277, 0.214806, 0.0126686, 0.00294698, 0.011175, 0.313207, 0.368957, 0.597277, 0.00214486, 0.382724, -0.417533, 0.208975, 0.00918274, 0.00551296, -0.0078276, 0.311426, 0.191727, 0.691747, -0.411214, 0.0592742, -0.0849933, 0.222493, 0.334652, -0.0385922, 0.40116, -0.20734, -0.0397197, 0.00845297, 0.162834, -0.0376586, -0.00332856, 0.140695, -0.00621273, 0.0413628, 0.300405, -0.611694, 0.00946289, -0.954017, 0.0118722, -0.52853, -0.903784, -0.00772936, -0.687178, -0.590729, -0.0128866, -0.161517, -0.141311, 0.215511, 0.110295, -0.00202902, -0.0646423, -0.0104248, -0.00554259, 0.0129891, 0.0723214, -0.44034, 0.00799697, 0.208809, -0.352421, 0.419028, -1.25418, 0.256153, 0.165857, 0.0702247, 0.30832, 0.00999502, 0.0630385, 0.0741313, 0.0126171, 0.196532, 0.115599, 0.13625, -0.00680079, -0.015283, -0.00440067, 0.177447, -0.390013, -0.0221441, 0.559946, -0.00198723, -0.468597, 0.23189, -0.473302, 1.24651, -0.0143642, 0.390603, -0.154904, 0.245466, 0.0108942, 0.088018, 0.312287, -0.40357, -0.171229, 0.214897, -0.00567311, 0.000214934, 0.756611, 0.481232, -0.309649, 0.172206, -0.0118426, -0.235022, 0.273419, 0.106847, 0.00531236, 0.000813907, -0.336547, -0.0167285, -0.976709, -0.0641631, 0.213168, 0.425481, -0.140951, -0.126169, 0.258991, 0.217597, 0.0230367, 0.257501, 0.336877, 0.495516, 0.288734, 0.0798983, 0.0589065, -0.0650158, 0.238499, -0.0120601, 0.13149, 0.00448419, 0.0886988, 0.183817, -0.0416755, -0.0050689, -0.0253095, -0.0392405, 0.0734122, 0.0070747, -0.0234182, 0.0673748, -0.108679, 0.0732559, -1.18641, 0.014326, -0.00356048, 0.0408359, 0.0606136, -0.167691, 0.0892472, 0.114085, 0.168334, 0.195846, 0.175469, 0.0418078, 0.360164, -0.272433, -0.0455013, 0.120751, -0.241904, -0.025092, -0.00631562, 0.108744, 0.0100581, 0.029131, 0.223621, 0.104881, 0.00967909, 0.573777, -0.0155255, -1.88304e-05, 0.480482, -0.00326292, 0.103846, 0.183522, -0.0314709, 0.411464, -0.276234, 0.00800109, 0.187313, -0.0656073, 0.406643, -0.00448128, -0.0292236, -0.251872, -0.0292331, 0.0843008, 0.00796973, -0.300313, -0.0134558, 0.0087791, -0.0694081, 0.279759, -0.269062, 0.0473531, -0.02761, -0.0135078, -0.159926, -0.305598, -0.0297053, 0.0623253, 0.306011, -0.272838, -0.0186794, -0.00343791, 0.0145234, 0.0607339, -0.333662, -0.172484, 0.409118, -0.0224348, 0.144202, -0.169276, 0.290814, 0.0339701, 0.00696053, 0.154058, -0.662103, 0.00152583, 0.0191461, -0.255076, 0.097053, 0.0328937, 0.17644, -0.130887, -0.0237378, 0.0157919, -2.37915, -0.247889, -0.110867, -0.329216, -0.015512, 0.170525, 0.00110173, -0.131047, 0.210633, -0.00921343, 0.132965, -0.0166144, 0.185738, -0.414467, 0.0556075, -0.0210256, -0.070141, 0.272149, 0.100021, -0.0580834, -0.0418832, -0.0528531, 0.00331821, 0.116096, 0.00671296, -0.235628, -0.412408, -0.143295, 0.0226409, 0.00329076, 0.255913, -0.00446346, 0.259811, -0.779055, -0.0603974, 0.00565745, -0.0121489, -1.5315, -0.440177, -0.93846, -0.029598, -0.657722, -0.68504, -1.68048, -1.23588, -0.018462, -0.0286771, -1.41028, -1.42, 0.511633, 0.19595, -1.06509, -0.155679, -1.13524, -0.95596, 0.0618328, -1.10751, -0.847284, -0.801444, -1.35361, -0.936757, 0.111965, -0.0207118, -1.48233, -0.0551507, -1.44043, -0.882591, 0.320281, -0.0184192, -0.684501, -0.0261771, -0.21532, -1.17558, -0.0117292, -0.730813, 0.753728, -0.0375708, 0.479219, -1.00004, -1.44934, -0.557239, -0.0633601, -0.516952, -0.017837, -0.00662014, -0.561585, -1.59662, 0.191105, 0.00227265, -1.00935, -0.0944951, -0.599444, 0.507171, -0.589156, -1.01732, -1.4723, -0.444291, -0.0361603, -0.780484, -0.977676, -0.0258343, -0.843907, -1.21812, -0.591985, -0.0113345, -0.010904, -0.0232892, -1.40799, -0.127412, -0.80193, -0.128427, 0.000768712, 0.69497, -0.932845, -0.0897506, 0.291986, -0.0315601, -0.80124, -0.880462, -1.53062, -1.32493, -0.571906, -1.23442, -1.43837, -1.18521, -0.554585, -0.02724, -0.0142921, 0.587031, 0.74779, -1.11938, -0.532503, -0.0793757, -0.160635, -1.34988, -0.495641, -0.389229, -0.0193723, 0.395381, -0.00469071, 0.593892, -1.07733, -0.925287, -1.22311, -0.305481, -1.14611, -0.601322, -1.85164, -0.0140444, -1.84682, -0.43771, -0.890809, -0.450232, -0.854621, -0.560644, -0.849317, -0.649394, -0.00918083, -1.15138, -0.0173249, 0.197188, -2.20294, -0.0389607, -0.0169896, -0.0416807, -1.24282, -1.36261, -1.82907, -0.0195685, -1.73051, 0.735825, -1.30362, -0.971199, -0.00907262, -0.0128956, -1.21194, -1.29345, 0.836893, 0.204244, -2.04633, -0.245707, -0.714336, -0.542666, -0.00099747, -0.8969, -1.14395, -1.18503, -0.694064, -0.517491, -0.000320307, -0.0304907, -0.467388, -0.0242839, -0.857102, -2.18772, 0.185729, -0.0650514, -1.83288, -0.00821657, -0.443484, -2.90614, -0.049957, -0.379544, -1.01949, -0.0184832, 0.319278, -1.97211, -1.16704, -1.441, -0.0532187, -1.27578, -0.0192043, -0.0155726, -1.56182, -0.66159, 0.240528, -0.0210501, -0.768216, -0.473277, -1.41881, 0.45276, -1.26419, -1.0095, -1.20375, -1.09018, -0.0508602, -1.03958, -0.695299, -0.0237124, -1.30712, -0.793729, -1.63439, -0.0251332, -0.0189361, -0.0201096, -0.763457, -0.166154, -0.520663, -0.915579, -0.00955478, 0.417279, -0.64603, -0.0169186, 0.353683, -0.0263134, -0.783904, -0.791993, -1.28129, -0.622248, 0.849359, -0.48157, -0.778396, -1.3363, -1.28719, -0.0482551, -0.0164834, 0.843508, 0.653172, -0.361223, -1.35137, -0.0263377, -0.222808, -1.05066, -1.33174, -0.398368, -0.0501884, -1.63388, -0.0289476, 0.366104, -0.576785, -0.995307, -1.55017, 0.390481, -0.62756, -1.2525, -0.538324, -0.0403438, -1.25951, -1.0397, -1.74905, 0.638413, -0.581067, -1.84028, -1.46332, -1.16708, -0.0183258, -0.814547, -0.0353767, 0.268295, -0.779215, -0.00265158, -0.013638, -0.000927955, 0.217949, -0.0903161, -0.604634, -0.0109634, -0.154757, 0.0706032, 0.0308382, -0.109398, -0.0137812, 0.00683118, 0.0776514, 0.21218, -0.283185, 0.336068, 0.146755, 0.221351, -0.286787, 0.039859, -0.0592651, -0.47197, 0.160264, -0.613259, -0.267309, -0.346436, -0.0792391, 9.27738e-05, 0.0567172, 0.00748157, -0.0151196, -0.636809, 0.276809, -0.0554062, 0.499818, -0.0133256, 0.146105, -0.931057, -0.01795, 0.355831, -0.135242, -0.0210373, -0.180026, -0.106762, -0.0393259, -0.0254855, -0.0691809, -1.45534, 0.0071314, -0.000937759, -0.218969, 0.176916, 0.303696, -0.0107187, -0.508389, 0.102326, -0.168807, -1.10822, -0.704398, 0.010497, 0.0810621, -0.800041, 0.00624451, -0.190363, -0.375299, -0.0133011, -1.64516, -0.0105489, -0.734792, -0.00529891, 0.00386773, -0.00918904, 0.0202806, -0.0599763, -0.301347, -0.170955, -0.0227921, 0.234987, -0.493675, 0.0456433, 0.228775, -0.00438003, -0.552949, -0.357195, 0.137365, -0.0115965, -0.848214, 0.0423752, 0.431397, -0.402043, -0.240485, -0.00783677, -0.00804089, -0.0562918, -0.412979, -0.196197, -0.193122, -0.0119129, 0.168344, 0.0625578, -0.265055, 0.347942, -0.0139226, 0.488942, -0.0128177, -0.304983, -0.464878, -0.457424, 0.334104, 0.0803051, -0.353749, -0.0758944, 0.344108, 0.0111564, -0.0901126, -0.601351, 0.17168, -0.360857, -0.355225, 0.0400968, -0.856092, -0.663756, -0.00788462, 0.868589, -0.00499363, -0.328402, -0.1963, 0.00606282, -0.0103654, -0.0185534, -0.0862454, -0.108941, -0.21386, -0.0292654, -0.141068, -2.50949, -0.100943, -0.125083, -0.00809471, -0.0106537, -0.130142, -0.138148, 1.91834, -0.310346, -0.184149, -3.45613, -0.130386, -0.129871, -0.0200051, -0.161724, -0.181974, -0.0758566, -0.10622, -0.11547, -0.0235522, 0.00506842, -0.132868, -0.0221104, -0.116951, -0.243914, -0.366593, -0.0142132, -8.05049, -0.00764862, -3.91819, -1.82951, -0.0123331, 0.0700626, 0.781416, -0.00619372, -0.246557, -0.252951, -0.118442, -0.131184, -0.00687376, -0.192236, -0.0160022, 0.0177213, -0.188378, -0.133991, -0.305795, -0.00366979, -0.142374, -3.61143, -0.127089, 0.250626, -0.189538, -0.146235, -0.133398, -0.190407, -0.05296, -0.145364, -0.141485, 0.00968438, -0.200283, -0.144755, -0.196692, -0.0177844, -0.00278328, -0.0160201, -0.135194, -0.89676, -0.1539, -0.164406, -0.00837026, -0.245044, -0.143223, -3.93157, 0.430938, -0.00359589, -0.155827, -0.150623, -0.137213, -0.151045, -3.26712, -0.122355, -2.45992, -0.106092, -0.13985, 0.00315686, -0.0113732, -0.422144, 0.585332, -3.09378, -0.128299, -0.00293598, -3.31436, -0.118053, -0.17215, -3.39626, -0.00372876, -2.20765, -0.000675941, -0.182342, -0.16561, -0.181996, -0.156534, -4.63614, -0.117927, -0.127225, -0.116762, -0.0218602, -0.0960676, -0.156759, -0.124506, -3.9627, -0.152691, -0.17311, -0.225229, -0.12139, -0.00951348, 0.00434182, -0.0210544, 0.0784864, -0.437812, -0.0596675, -0.0258251, -0.0355257, -2.04891, -0.677716, -0.243482, -0.0374683, -0.415969, 0.534653, -1.77471, 0.304592, -0.0169059, -0.00553455, -1.97613, -1.98861, 0.329766, 0.0859834, -1.11986, 0.278432, -1.02529, -1.05515, -0.019215, -1.05252, -1.68699, -0.519137, -0.601739, -0.771306, -0.0299632, -0.0168336, -1.77244, -0.0449091, -1.64323, -0.530898, 0.0558544, -0.0565763, 0.357281, -0.0100132, 0.346467, 0.863033, -0.0427951, -0.949787, 0.0516811, -0.0175227, 0.389924, -1.54253, -2.12579, -0.356505, -0.0702899, -0.150605, -0.00446702, -0.0321305, -0.662384, -1.904, 0.0869925, -0.0288119, -0.250641, 0.306694, -0.583651, 0.451072, -0.934073, -1.02103, -1.91427, -0.650629, -0.0648624, -0.515053, -0.401503, -0.0167133, 0.24, -1.40914, -0.0909118, -0.0169693, -0.00692438, -0.0242254, -1.56766, 0.226678, -0.645595, -0.602528, -0.0322841, 0.436754, -0.148719, 0.355913, 0.246567, -0.0288132, -0.572988, 0.269452, -1.74922, -1.68226, -0.0674002, -1.2725, 0.294259, -0.271615, -0.861297, -0.0375834, -0.0306472, 0.454286, 0.552749, 0.126792, -0.889384, -0.0206246, 0.311895, -1.57548, -0.750061, 0.302389, -0.0284875, 0.0974205, -0.0287968, 0.315309, -0.754228, -0.462936, -1.4267, 0.440425, -0.14649, -0.561791, -2.35776, -0.0530041, -1.57292, -0.69524, -1.0898, 0.338109, -0.27842, -1.04687, -0.335837, -0.520614, -0.0338215, -1.32062, -0.0202805, -0.260276, 0.16497, -0.0107697, -0.000493471, -0.00861197, 0.286592, 0.279988, 0.345634, -0.00862876, 0.553128, -0.534686, 0.456893, 0.499603, -0.0163343, -0.0157583, 0.241938, 0.268178, -0.141625, -0.253587, 0.427314, 0.550644, 0.264617, 0.399913, -0.0456411, -0.0290692, 0.410796, 0.273038, 0.313407, 0.32405, -0.0365475, -0.0170157, 0.402885, 0.000981096, 0.194131, 0.320447, -0.25138, -0.0216583, 0.189219, -0.00550592, -1.16603, -0.774716, -0.0205042, -0.309872, 0.163054, 0.00569324, -1.20336, 0.303754, 0.343512, 0.546351, 0.0343652, 0.327025, 0.000189166, -0.0195548, 0.0491452, 0.545132, -0.267293, -0.0104857, 0.339354, -0.768709, 0.49036, 0.248708, 0.142211, 0.0488264, 0.553101, -0.0552664, -0.0313483, 0.515659, 0.430841, -0.00234525, 0.0585426, 0.495054, 0.219378, -0.0212227, -0.0296312, -0.00853711, 0.264872, 0.145531, 0.335412, 0.357316, -0.000160487, -1.34436, 0.496207, -0.0713785, 0.651138, -0.000577398, 0.00215097, 0.071592, 0.255608, 0.240779, 0.250851, 0.420368, -0.255664, 0.220285, 0.473524, -0.00546247, -0.00185546, -0.316972, -0.172051, 0.163213, 0.042697, -0.0255918, 0.227655, 0.146846, 0.496606, 0.459174, -0.000107289, 0.24582, -0.0201376, 0.0219916, 0.2546, 0.272117, 0.48653, -0.17775, 0.513538, 0.508758, 0.569813, 0.010319, 0.299371, 0.138098, 0.428383, -0.108782, 0.409274, 0.537499, 0.35778, 0.0305011, -0.00728688, -0.0139729, 0.00675017, 0.494045, -0.520526, -0.00526404, -0.00792998, 0.0202274, 0.0638929, -0.814987, -0.526525, 0.00592907, -0.339576, 0.0698602, 0.090683, -1.01791, -0.0146971, 0.0121334, 0.0800414, 0.117518, -1.35281, 0.549414, -0.632841, 0.0723621, 0.0120236, 0.139339, -0.0381022, 0.150306, -0.126872, -0.275422, 0.022684, -0.611559, -0.0477962, -0.0158611, 0.0271815, -0.00107938, 0.0668671, -0.51322, 0.478483, -0.0204767, 0.0135776, -0.013626, 0.564373, -1.03655, -0.000142092, -0.0389919, -1.22124, -0.0297228, 0.600577, -0.059669, 0.069001, -0.414215, -0.0647462, -0.549282, -0.00272274, -0.0131728, -0.615814, 0.0727605, 0.581883, -0.0089135, -0.330617, 0.785032, -0.201244, -0.454202, -0.644512, 0.0670015, 0.0643706, -0.560575, -0.00979791, -0.395465, -0.264471, 0.00106365, -0.879692, -0.00171077, -0.587684, -0.00952247, 0.0177193, -0.0101121, -0.0265297, 0.275957, -0.525827, -0.637221, -0.0241627, 0.555022, -0.169815, 0.423766, -1.89939, -0.00171162, -0.353324, -1.11098, 0.0899398, 0.0246927, 0.218686, 0.184437, 0.205962, -0.00807377, -0.312858, -0.0109177, -0.016472, -0.68902, -1.70799, 0.858288, -0.445354, -0.00971371, -0.0262475, 0.0676112, -0.437802, 0.171549, 0.00421096, -1.16494, -0.0169351, -0.326196, -0.635674, -0.449416, -0.32665, 0.643941, -0.575194, -0.357489, 0.192453, -0.0237759, 0.190922, -0.573257, -0.35993, 0.58187, -0.615609, -0.560898, -0.409996, -0.477464, -0.0270457, -0.677365, -0.000283787, 0.11734, 0.0449394, -0.00337267, 0.00339056, -0.00849051, 0.0370724, 0.0367193, 0.0257988, -0.0021465, 0.0247286, -0.172735, 0.0309962, 0.0270057, -0.0144008, 0.00130079, 0.0461672, 0.0420566, -0.0919166, 0.119604, 0.0256555, -0.234586, 0.0389255, 0.02253, 0.0364765, 0.0420101, 0.0443109, 0.0514783, 0.0348614, 0.0554028, -0.0706015, -0.0107673, 0.0636772, 0.0174087, 0.0231432, 0.0297251, 0.114643, 0.0210242, 0.0377148, -0.0158403, -0.11427, -0.038538, -0.00132693, 0.00656771, -0.177872, 0.00138931, -0.0513416, 0.053958, 0.023274, 0.0197559, -0.0627599, 0.0247447, 0.00111598, -0.0320318, 0.0445185, 0.0532334, 0.125598, -0.0105229, 0.0414526, -0.122658, 0.0433448, -0.0136318, 0.0561342, 0.0243408, 0.0509089, 0.0514294, -0.0110224, 0.0351739, 0.041391, 0.000292221, 0.0638736, 0.0316285, 0.0253735, 0.00507121, -0.0113591, -0.00563965, 0.0275679, -0.0191634, 0.0422268, 0.0320466, -0.0165664, -0.129269, 0.0361004, -0.13093, 0.022253, -0.00204794, 0.038225, 0.0478598, 0.0422792, 0.0662227, -0.141013, 0.0501513, 0.0152387, 0.0400808, 0.0344737, 0.00146913, -0.00837826, -0.133681, 0.0959691, 0.0254014, 0.0430267, -0.0168438, -0.0866393, 0.0161694, 0.0303486, -0.224803, -0.0137978, -0.102555, -0.0159393, -0.143891, 0.035817, 0.0303813, 0.0437978, -0.124546, 0.0480295, 0.0448166, 0.0521989, 0.0186497, 0.0378782, 0.0440677, 0.0490903, -0.143411, 0.039112, 0.0261696, 0.0398733, 0.0379268, -0.00217884, -0.027878, -0.0154963, 0.436602, 0.139009, -0.0326271, 0.000606667, -0.0465617, 0.0446462, -0.248662, 0.00254846, -0.0203461, 0.0377306, 0.0291917, 0.104582, -0.38508, -0.000138294, 3.20124e-05, 0.0432496, 0.0962227, -0.0389497, 0.40185, 0.0752594, 0.0530646, 0.0168433, 0.0469422, 0.0246126, 0.0422156, -0.260522, -0.303728, -0.278683, -0.351057, -0.0755355, -0.018321, 0.13531, -0.0178742, 0.058134, 0.111598, 0.390698, -0.019599, -0.199737, -0.000735667, 0.218359, -0.328053, 0.00165027, -0.32274, 0.218787, -0.0441124, 0.850382, 0.0832132, 0.0152916, 0.14056, -0.0280633, -0.285151, 0.00715008, 0.0077715, -0.234619, 0.041717, 0.408536, -0.0211778, -0.311428, 0.382888, 0.0834566, -0.260287, -0.273111, 0.123637, 0.0442625, -0.270535, -0.00589999, 0.00260823, -0.307452, -0.00911786, -0.186701, 0.0362142, -0.00745711, 0.00634098, -0.00506684, 0.000531816, 0.078738, -0.0238294, -0.325846, -0.248413, -0.0242906, 0.735279, -0.264962, 0.512772, 0.745389, 0.000222403, -0.260606, -0.374489, 0.0618379, 0.0670851, 0.134419, 0.078926, 0.0452992, -0.287305, 0.0607261, 0.00258201, -0.00515844, -0.195158, 0.230632, -0.537085, -0.251998, -0.00786646, 0.393153, 0.00508756, 0.0525974, -0.00541976, -0.0116979, 0.38887, -0.0224063, 0.670117, -0.289837, -0.293422, 0.114916, 0.417935, -0.308771, 0.0975634, -0.0862358, 0.00678127, 0.0827581, -0.269606, 0.105478, 0.0459217, -0.323129, -0.269358, -0.307898, -0.27685, 0.0033336, -0.675698, -0.0132588, 0.0096956, 0.023591, -0.00210276, -0.00653612, 0.0132724, 0.0056041, -0.00283006, -0.00600413, -0.00319262, 0.00358732, -0.0075322, 0.0116075, 0.00668725, 0.00916666, -0.0157009, -0.011266, -0.0153645, -0.00858911, 0.00750942, 0.00316044, -0.00155116, 0.00225471, 0.00279596, -0.0057248, 0.00429625, 0.00928738, 0.0129045, -0.00430633, -0.0126429, 0.00774093, 0.00886888, -0.00385245, -0.0219579, -0.00432359, 0.00264959, -0.00981258, -0.0172109, 0.000845731, 0.00586376, -0.0107141, -0.0188093, -0.000771592, -0.00249287, -0.00804409, -0.00368319, 0.0032369, 0.00482849, -0.0139042, 0.0163667, -0.00734626, 0.00734984, 0.0101953, -9.78161e-05, -0.0101974, -0.0144128, -0.0125745, -0.00577785, -0.000298061, -0.0083928, 0.000165468, -0.0142828, -0.0178396, -0.00918194, -2.24958e-05, 0.0164647, -0.00799027, 0.00966453, 0.00804493, -0.0113119, 0.000693957, -0.000978922, -0.00340214, -0.00689453, -0.00403072, -0.00469139, 0.00625345, -0.00777719, 0.00578523, -0.00416042, 0.000726977, 0.0142434, -0.0189145, -0.0018601, 0.0127787, 0.00494679, -0.0128682, -0.012759, 0.00228997, 0.000666272, 0.0134324, 0.0123043, 0.000456929, 0.0120054, 0.00161291, -0.00719911, -0.00484195, -0.00602779, -0.000777276, -0.00139268, 0.00059352, -0.00365347, 0.00984291, 0.0147904, -0.00503675, -0.00633207, 0.00110652, 0.00513624, 0.00734503, 0.000224381, -0.000798848, 0.00273784, -0.00255381, -0.00530626, 0.00289856, -0.00179305, -0.0116438, -0.0117589, 0.000748458, -0.00786573, -0.0119703, -0.00280036, 0.00995622, 0.0175386, 0.000723316, 0.015125, 0.00131006, -0.0058699, 0.00406806, 0.0133498, 0.0681814, 0.00379815, 0.0079609, -0.0217831, 0.0707928, 0.072972, 0.0761257, 0.00469418, 0.0736473, -0.0316296, 0.0846787, 0.0792333, -0.0170042, -0.00386033, 0.067887, 0.0702105, -0.0547901, 0.0227496, 0.0663736, -0.013811, 0.0944834, 0.0708406, -0.0773101, 0.077708, 0.0645094, 0.0598694, 0.0736844, 0.0840655, -0.0619414, -0.0214547, 0.0986463, -0.00282505, 0.0597672, 0.0690989, 0.0136966, 0.00332989, 0.076018, -0.0142214, -0.0198903, -0.0666518, 0.0121669, 0.0159563, -0.0509546, -0.00382456, -0.0941779, 0.0434706, 0.0913259, 0.0701683, -0.0671197, 0.069528, -0.0116493, -0.0486013, 0.0790989, 0.0918616, 0.0193026, -0.0109547, 0.0693801, -0.107648, 0.0763937, -0.16471, 0.0548162, 0.0781123, 0.0774272, 0.053725, -0.0120905, 0.0803501, 0.0730085, 0.0117942, 0.0774075, 0.0731388, 0.0693009, 0.00346923, -0.0329368, 0.00261353, 0.0920795, -0.00230692, 0.0821963, 0.0480016, 0.000554638, -0.226703, 0.0824505, 0.0155861, -0.0343465, -0.00442117, 0.10341, 0.0677312, 0.0705679, 0.0793699, -0.0126547, 0.0749146, 0.0676286, 0.0736984, 0.0985913, -0.00911961, 0.000665859, -0.107537, 0.0670503, 0.0882253, 0.0450683, -0.0084155, 0.0491011, 0.0533639, 0.0690662, -0.0198643, -0.00359106, 0.0250208, -0.016656, -0.283705, 0.0518319, 0.0576509, 0.0825189, -0.0396088, 0.0591431, 0.0878686, 0.079308, -0.0258061, 0.0758294, 0.0725413, 0.0800888, -0.017842, 0.0709385, 0.0638574, 0.0669076, 0.0550079, -0.0182074, -0.0189275, -0.00194801, -0.0229153, 0.018379, 0.0202978, 0.0292038, 0.00305886, 0.0225905, 0.0262646, 0.0159655, -0.00361807, 0.0216834, -0.0590072, 0.0134644, 0.0011629, -0.00950194, -0.00933623, 0.0199139, 0.00410936, 0.0350943, 0.0137751, 0.00204285, 0.0810101, 0.0134463, -0.0122412, -0.0807237, 0.0195418, 0.00953649, 0.0180931, -0.00253458, 0.00339778, -0.0579073, -0.0111829, 0.0317211, 0.00858365, -0.00271234, -0.0104215, -0.00481998, -0.0106083, 0.0177666, -0.0168179, -0.0592527, -0.0889008, 0.00717672, 0.0110581, -0.0747862, 0.0114051, -0.16004, 0.00518446, 0.0110503, 0.0113998, -0.0590558, 0.00918473, -0.00162913, -0.00934989, -0.0050829, 0.00809161, 0.00221931, 0.0113956, 0.0119963, -0.122962, -0.00266184, -0.138754, 0.0225718, 0.0100043, 0.0249175, 0.0118383, -0.00155878, 0.0278915, -0.000668312, 0.00524502, 0.00681371, 0.0299126, 0.0270963, -0.00459364, 0.019347, -0.00373678, 0.0412319, -0.00671463, 0.00768019, -0.00490615, 0.00713196, -0.295052, 0.0129762, -0.0136356, -0.0266894, -0.000223945, 0.0318361, 0.0214392, 0.0274495, 0.0258911, -0.111802, 0.00402436, 0.0181057, 0.0198046, -0.00357872, -0.0181032, -0.00380294, -0.191315, -0.0407518, 0.0122183, 0.00391602, -0.0112659, -0.0213223, -0.00741945, 0.00751685, 0.0104835, -0.0116191, -0.0409449, -0.0201852, -0.305958, 0.010475, -0.00597896, 0.00953912, -0.0705549, 0.0122325, 0.00497749, 0.028109, 0.00253424, 0.0242118, 0.0177143, 0.0115722, -0.105876, -0.000817868, 0.0121632, 0.0158862, 0.0049399, -0.0275937, -0.0693881, -0.00195245, -0.0133486, 0.00656448, -0.00311832, 0.00650734, 0.0151953, 0.00628989, 0.030837, 0.018902, 0.00752474, 0.0317628, -0.181362, 0.0135473, -0.00341123, -0.00472656, 0.0112201, 0.0117725, 0.0106364, -0.103515, -0.0276213, 0.0175296, 0.179224, 0.0238897, 0.0122423, -0.063224, 0.0261133, 0.0229411, 0.00871009, 0.00789496, 0.0181661, -0.0580433, 0.00303059, 0.0285622, -0.0163469, 0.0144925, 0.020422, -0.0161566, 0.0031364, 0.0140402, -0.000414789, -0.112114, -0.0963307, -0.0175448, 0.0598664, -0.165946, 0.0066237, -0.202255, 0.0186578, 0.0179506, 0.0128534, -0.0516809, 0.0290041, -0.0135283, -0.0165822, 0.000598653, 0.0253141, -0.00419467, 0.0126126, 0.00677514, -0.153777, 0.0183277, -0.260542, 0.0361742, 0.0263081, 0.0279653, 0.0120033, -0.0182527, 0.0428095, 0.00245839, -0.0244882, 0.0391574, 0.0331608, 0.00230243, -0.0149728, -0.00670448, -0.00692441, 0.0342182, 0.00272433, 0.0285684, 0.00866825, -0.0307048, -0.4225, 0.0203586, -0.0819393, -0.0640846, -0.0246281, 0.0277681, 0.012369, 0.028322, 0.0215348, -0.194702, 0.0379945, -0.0077373, 0.0348813, 0.0172334, -0.00354258, -0.0200347, -0.0910609, -0.19109, 0.0252842, 0.00751624, -0.0178052, -0.0584888, 0.00974669, 0.0139435, 0.0692383, 0.0205316, -0.0310736, -0.00887296, -0.417821, 0.0202382, 0.00826015, 0.0120019, -0.173828, 0.0118604, 0.0254583, 0.0231937, -0.0164694, 0.0304096, 0.0260864, 0.0463922, -0.189958, 0.0141192, -0.000910959, 0.017565, 0.0310069, 0.0254839, -0.0915563, -0.00506199, 0.0104649, 0.00656002, -0.00560887, -0.0042286, -0.0323405, 0.0288032, 0.0403036, 0.0125079, 0.00104832, 0.0341644, -0.185898, 0.0208163, 0.00227169, -0.0058421, -0.00274295, 0.0350208, 0.0446841, -0.0426901, -0.00245601, 0.0168617, -0.239521, 0.0276836, 0.0355161, 0.0178197, 0.0406955, 0.00522901, 0.0152444, 0.047717, 0.0185017, 0.00737908, -0.000734098, 0.027107, -0.017559, 0.0323464, 0.0142093, 0.0124922, -0.00776603, 0.0155503, -0.011663, -0.180669, -0.0769082, -0.0137498, 0.0864141, -0.211357, 0.0141243, -0.128263, 0.0252073, 0.0321243, 0.0043477, -0.0525979, 0.0188379, -0.0162697, -0.0121501, 0.0231791, 0.0394934, 0.0150195, 0.00111689, 0.0336373, -0.151423, 0.036212, -0.102767, 0.0292126, 0.0210437, 0.00805858, 0.0228258, 0.00601363, 0.0285326, 0.0243813, -0.0127377, 0.0125654, 0.021351, -0.00207864, -0.0063397, -0.00439163, 0.00612846, 0.0211942, 0.00596811, 0.0464855, 0.0272003, -0.0106972, -0.110404, 0.0428284, -0.0842036, -0.0186758, -0.0130301, 0.036276, 0.0224213, 0.0228596, 0.031437, -0.215344, 0.0351179, 0.00498118, 0.012328, 0.0341278, -0.00983449, 0.00604608, -0.210445, 0.0241444, 0.0141228, 0.0116446, -0.0132715, -0.0977217, 0.0172067, 0.0216178, -0.328928, -0.00944061, -0.0491964, 0.00783169, -0.0944265, 0.013486, 0.0212487, 0.0217197, -0.190739, 0.033963, 0.026815, 0.0093905, -0.0129249, 0.00715321, 0.0153609, 0.0344335, -0.208083, 0.0283103, 0.0411885, 0.0183436, 0.0364395, -0.00131232, -0.129544, -0.0101672, 0.0361715, 0.110615, -0.0122628, 0.0202869, -0.00584413, 0.132557, 0.131476, 0.122201, 0.00779241, 0.116151, -0.0647083, 0.107495, 0.127447, -0.0152566, -0.000637034, 0.137417, 0.11493, -0.0615922, 0.074714, 0.123952, 0.140429, 0.143972, 0.126879, -0.0828072, 0.109998, 0.118018, 0.141587, 0.123091, 0.117382, -0.0402666, -0.0135624, 0.126842, 0.0101182, 0.135556, 0.116323, 0.0416522, -0.0144542, 0.146872, -0.00740358, -0.0133045, 0.0172231, -0.015637, 0.0349574, -0.042369, 0.000184286, -0.209717, 0.0974223, 0.128479, 0.12409, -0.0637339, 0.116174, -0.00123616, -0.0297904, 0.108701, 0.134605, 0.0861519, 0.0140575, 0.132711, -0.0501003, 0.141894, -0.206385, 0.140306, 0.110602, 0.157404, 0.112575, -0.0292993, 0.13903, 0.136231, -0.014957, 0.133321, 0.146542, 0.117289, -0.0113692, -0.00420237, -0.00679732, 0.128642, 0.017754, 0.128512, 0.10858, -0.00122226, -0.580171, 0.145578, -0.00842522, 0.0218061, -0.0113686, 0.135318, 0.103633, 0.126318, 0.145939, -0.0477101, 0.135518, 0.0978085, 0.119228, 0.124515, -0.0162521, 0.00164201, -0.400811, 0.0854072, 0.138204, 0.123808, -0.0102722, 0.042759, 0.0806261, 0.125976, 0.0428424, -0.0226416, -0.0219095, -0.0260417, -0.57745, 0.135286, 0.122332, 0.147191, -0.0387157, 0.124314, 0.134999, 0.124469, -0.00323139, 0.134757, 0.126482, 0.134286, -0.0375748, 0.119249, 0.114617, 0.115376, 0.118265, -0.000493047, -0.0162833, -0.00425171, 0.0419404, 0.0774118, 0.00508207, -0.0223373, 0.0151729, 0.0705179, 0.0970182, 0.0934736, -0.00720707, 0.103274, -0.0629869, 0.0800881, 0.0952899, 0.0164485, 0.00227079, 0.0978535, 0.0947663, -0.0705956, 0.0312639, 0.075544, 0.206893, 0.110323, 0.0851879, -0.00681361, 0.101184, 0.0978372, 0.0983082, 0.0999061, 0.0798381, -0.0609003, 0.00362048, 0.10193, -0.00903386, 0.0908472, 0.0777719, 0.0233816, -0.00956516, 0.0868126, -0.00583341, -0.000119808, 0.00492431, 0.00038499, 0.0614171, -0.0849078, -0.00488013, -0.246844, 0.10295, 0.105309, 0.0916696, -0.0235202, 0.0977956, 0.00566496, -0.00567417, 0.0916767, 0.113986, 0.0489311, -0.00526598, 0.110989, -0.0426876, 0.0977633, -0.106552, 0.0938631, 0.0941317, 0.0972667, 0.096897, 0.00923247, 0.0771131, 0.0875259, -0.0153879, 0.106381, 0.115971, 0.0974154, -0.0126604, -0.00877913, -0.00443817, 0.0858578, -0.0170013, 0.104954, 0.0880669, -0.00658688, -0.335129, 0.110204, 0.00861683, 0.0580112, -0.0128878, 0.106654, 0.0962996, 0.0988829, 0.0798993, -0.0620755, 0.0966573, 0.0922752, 0.109565, 0.0937835, -0.0106387, 0.0262012, -0.100399, 0.018134, 0.0811029, 0.112286, -0.00261686, 0.0426432, 0.0924705, 0.0930932, 0.0725331, -0.0169703, -0.0120334, -0.0185542, -0.336435, 0.0939598, 0.101081, 0.120815, -0.0271852, 0.0957761, 0.10582, 0.087866, 0.00471204, 0.098705, 0.0991216, 0.110318, -0.0390127, 0.0965697, 0.0874367, 0.08439, 0.114313, -0.00955949, -0.00816085, -0.0100892, 0.0379213, 0.0382612, 0.0118435, 0.0114783, -0.000718059, 0.0525049, 0.0791515, 0.0775216, -0.0135674, 0.0582594, -0.0541957, 0.0685323, 0.0603194, -0.00727247, 0.0134973, 0.0776547, 0.0685876, -0.0600227, 0.101372, 0.0609449, 0.147584, 0.0656479, 0.052553, -0.0738457, 0.0655631, 0.049435, 0.078394, 0.0669338, 0.0544288, 0.001031, -0.0375935, 0.0465068, 0.00599969, 0.0768715, 0.0519983, 0.0523075, 0.00662449, 0.058763, 0.00123977, 0.0116259, -0.0392323, -0.0154435, -0.0345649, -0.0894961, 0.0198483, 0.0259983, 0.0631673, 0.0774845, 0.0592957, -0.036547, 0.0270756, 0.00699357, -0.0202625, 0.0586277, 0.0604082, 0.127426, -0.012904, 0.0654995, 0.061859, 0.05373, -0.0244067, 0.053196, 0.0833207, 0.0770134, 0.0557632, -0.025843, 0.0720704, 0.0697892, -0.0131856, 0.0618426, 0.0721775, 0.0645416, 0.00713569, 0.0073315, 0.0149027, 0.0869619, 0.00510027, 0.0795791, 0.0552526, -0.00588437, -0.110147, 0.0635584, 0.155609, 0.0931992, 0.00202638, 0.0736543, 0.0845421, 0.0710622, 0.0884665, -0.0908125, 0.0737402, 0.0558822, 0.0711593, 0.071401, -0.0216031, -0.0157081, -0.147223, 0.075189, 0.0790593, 0.0540674, -0.00279384, 0.079314, 0.0347358, 0.0714177, -0.0132073, -0.0113622, 0.0314174, -0.0230099, -0.0867275, 0.0628194, 0.0494427, 0.0695724, -0.118483, 0.0828919, 0.0574234, 0.0757694, -0.0302791, 0.0527635, 0.067957, 0.072869, -0.101744, 0.0613214, 0.0616759, 0.0556192, 0.0644321, -0.0111113, -0.0775077, 0.00313045, -0.0285321, 0.0280083, -0.000974379, -0.014421, -0.0602248, 0.0596636, 0.0843398, 0.0197014, 0.00643747, 0.0578181, -0.163349, 0.0739905, 0.0643464, -0.00739523, 0.00291709, 0.0578176, 0.0623831, -0.0445745, -0.00326381, 0.0454214, 0.03772, 0.0551274, 0.0663056, 0.0213554, 0.0783925, 0.0448483, 0.0677518, 0.0587421, 0.0440092, -0.0472174, 0.0100532, 0.0731134, -0.0160576, 0.0699627, 0.0357282, -0.0150894, -0.00653567, 0.0511299, -0.0147913, -0.117761, -0.0658427, -0.0162367, 0.0542511, -0.172538, -0.0156213, -0.434434, 0.0215913, 0.0778199, 0.0728445, -0.0225026, 0.048476, -0.00515844, -0.0245173, 0.0621138, 0.10382, -0.00527778, -0.00931875, 0.0330168, -0.153649, 0.0826239, -0.237518, 0.0480963, 0.0649834, 0.0696738, 0.0450429, -0.00167511, 0.0862652, 0.0740994, -0.0122014, 0.0343126, 0.06055, 0.0292142, -0.00160147, 0.0060467, -0.0172828, 0.0881156, 0.0103098, 0.0834316, 0.0474552, -0.025129, -0.569723, 0.0878139, -0.0861854, 0.00593127, -0.00565771, 0.0713095, 0.0280869, 0.076261, 0.0772778, -0.194034, 0.0762266, 0.0351637, 0.0857197, 0.0806315, -0.0112236, -0.00712277, -0.466039, -0.294268, 0.05044, 0.061256, -0.00580313, -0.0528176, 0.055726, 0.0639399, 0.00280747, 0.00831801, -0.0560926, -0.00933564, -0.556993, 0.066672, 0.047674, 0.0711929, -0.1307, 0.0485695, 0.0903906, 0.0695462, -0.0138233, 0.087053, 0.0617784, 0.0711469, -0.18606, 0.0651054, 0.0387122, 0.0365472, 0.0426748, 0.00183421, -0.01123, 0.0156575, -0.0274512, -0.00110879, 0.0149438, 4.10731e-05, 0.0125252, -0.0185091, -0.0147608, -0.0165131, 0.00628399, -0.020665, -0.000416236, -0.0113785, -0.0162878, -0.00476372, 0.00280197, -0.00475541, -0.0102621, 0.0170746, -0.0110834, -0.019822, 0.000832283, -0.0228608, -0.0184941, 0.00720694, -0.0205881, -0.00407026, -0.0202291, -0.0313009, -0.0243973, -0.00106406, 0.0108821, -0.0167039, 0.000221229, -0.0342595, -0.0035272, -0.0136427, 0.00521651, -0.0263529, -0.00770518, 0.000967697, -0.0168664, 0.0173518, 0.0334016, -0.00850965, 0.0110183, 0.00737065, -0.0243068, -0.0199772, -0.0165319, -0.0165388, -0.0160976, -0.0091587, 0.00240497, -0.00996039, -0.0139773, -0.0302788, -0.000863498, -0.0126304, -0.0123694, -0.00818138, -0.0161323, -0.025289, -0.0223481, -0.0128083, -0.0193246, -0.0162941, -0.0246488, -0.0108319, -0.00187308, -0.00774083, -0.0225994, -0.0278828, -0.0142436, 0.0124926, 0.00551793, -0.0256305, -0.0113034, -0.011439, -0.00192512, -0.00910316, -0.00146822, 0.0255171, -0.00417805, -0.0176376, -0.00532652, -0.013198, -0.0148034, -0.0111094, -0.0149021, -0.015977, -0.0146754, -0.00683644, -0.0210061, -0.0196294, -0.00696129, -0.0115172, -0.000554401, -0.0326618, -0.0284669, -0.0117237, 0.0155077, 0.00283014, -0.0117685, -0.0132653, 0.00445894, 0.000200415, 0.0166012, 0.0238484, 0.0188904, -0.0204112, -0.00549514, -0.00977808, -0.0267379, -0.0209332, -0.000606984, -0.0305035, -0.00221957, -0.0154122, -0.0243798, -0.00988692, 0.0117516, -0.00883559, -0.0153933, -0.025046, -0.0294613, -0.0110858, -0.0170286, -0.0124761, -0.263551, 0.357035, -0.0211861, -0.0408667, -0.0680717, 0.224203, 0.123093, 0.277341, -0.0170397, 0.271879, -2.52823, 0.273282, 0.351795, -0.0184208, -0.0114967, 0.215069, 0.227384, -0.587678, -0.273591, 0.4184, 0.144343, 0.248851, 0.345998, -0.060993, 0.204036, 0.598467, 0.134453, 0.426546, 0.0467364, -0.0394159, 0.000572345, 0.268662, -0.00256879, 0.196259, 0.390388, -0.316007, -0.0107747, 0.32541, -0.0535116, -0.06664, -0.0834765, -0.00858021, 0.310229, -0.208153, -0.00955383, -0.388834, 0.688493, 0.201704, 0.167415, -0.00446047, 0.181069, 0.00272116, -0.0074281, 0.155139, 0.253071, -0.271173, -0.0149868, 0.0429492, 0.154946, 0.270987, -1.19293, 0.218951, 0.0959414, 0.280832, 0.099115, -0.0367535, 0.0443133, 0.00871055, -0.0145855, 0.142956, 0.266919, 0.0936603, -0.0194081, -0.0141279, -0.00597398, 0.219502, -0.38928, -0.0171933, 0.440322, -0.0176179, -0.637957, 0.0129189, 0.148169, -2.98026, -0.0269396, 0.073721, 0.175999, 0.21266, 0.205298, -0.193478, 0.394634, 0.523325, 0.528816, 0.0804144, -0.0383719, 0.000107652, -0.538811, -3.09006, 0.295864, 0.21051, -0.0108283, 0.0190003, 0.163312, 0.0775162, 0.196882, -0.0124523, 0.0568689, -0.0263041, -1.0469, 0.0903959, 0.0600461, 0.343279, -0.0744436, 0.199696, 0.0200583, 0.478736, -0.0193779, 0.302001, 0.108311, 0.324824, -0.722826, 0.0147628, 0.293, 0.151079, 0.0510694, -0.0244151, 0.282214, 0.000952593, -0.497232, -0.462517, -0.0442363, -0.0358646, -0.043783, -1.04961, -1.12044, -0.589928, -0.0415634, -0.580472, -0.821842, -1.18213, -0.566662, -0.0359601, -0.0192452, -0.918577, -1.04911, 1.42689, -0.4637, -0.80595, -1.12282, -1.1912, -1.3374, -0.0172524, -0.286156, -0.939118, -0.727741, -1.48691, -0.707856, 0.00640846, -0.0173443, -1.16182, -0.0255376, -1.13628, -0.534223, -0.524559, -0.0178198, -2.63842, -0.0311371, -0.857187, -1.15181, -0.0528838, -1.34908, 0.253746, -0.0295691, -0.17179, -1.06172, -1.10299, -0.855621, -0.0485174, -1.1956, -0.0271378, -0.0446593, -0.624802, -1.06349, -0.431068, -0.0134665, -0.890871, -0.680295, -0.589579, 0.0181378, -1.19304, -0.341092, -1.07207, -0.977806, -0.0425641, -0.615276, -0.596322, -0.0251866, -1.02752, -1.23036, -0.474519, -0.0442253, -0.0182748, -0.0114871, -1.09567, -0.960001, -0.610577, -0.909703, -0.0111549, -0.0927535, -0.78708, -0.239274, 0.657208, -0.0247806, -1.00351, -0.317897, -0.999292, -0.965903, 0.298461, -1.28023, -1.98892, -1.10431, -0.732769, -0.0347763, -0.0283112, 0.391525, 0.686935, -0.000510768, -0.85463, -0.0320435, -1.10661, -0.99595, -0.712598, -1.13016, -0.0224057, -1.2831, -0.0278312, 0.0244589, -0.450297, -0.886409, -0.771908, 0.331501, -1.40284, -1.0104, -0.890167, -0.0425661, -1.00943, -1.06903, -0.894484, 0.822837, -0.741402, -0.71796, -0.69479, -0.93378, -0.0303488, 0.0786769, -0.0115523, 0.049322, 0.0208174, -0.026522, 0.000523537, 0.00803524, 0.0512276, 0.0612954, 0.0458161, 0.00992377, 0.0625862, -0.0624802, 0.0638049, 0.0311517, -0.0169086, 0.0149874, 0.0320764, 0.0575449, 0.00822348, 0.0376935, 0.0524771, -0.0160411, 0.0613173, 0.0339848, -0.00883473, 0.0554991, 0.0482585, 0.0476362, 0.0437887, 0.057463, -0.0726931, -0.00787879, 0.0492694, -0.00248939, 0.0462381, 0.0491155, 0.0598343, -0.0160499, 0.0610916, -0.00824997, -0.0611622, -0.0330976, -0.0151887, 0.0160092, -0.0617353, -0.000387331, -0.154063, 0.0415777, 0.0465748, 0.0331901, -0.0374959, 0.0455122, -0.010615, -0.035341, 0.0294559, 0.0460399, 0.0495988, 0.00998981, 0.0480017, -0.0915727, 0.0292785, -0.0921359, 0.051615, 0.032341, 0.0260225, 0.0374807, -0.0243402, 0.0444372, 0.0445892, -0.00761731, 0.0525798, 0.0510448, 0.035396, 0.0143264, -0.00110612, -0.00801113, 0.0637607, -0.0014193, 0.0468562, 0.0289865, -0.0128849, -0.201492, 0.0369933, -0.0367767, 0.0073223, 0.0198449, 0.0451035, 0.0320327, 0.0524138, 0.0575819, -0.0813808, 0.0521152, 0.0323707, 0.0538418, 0.0187676, -5.37538e-05, -0.0184187, -0.120712, -0.0249841, 0.0550057, 0.0300104, -0.00121101, 0.0025087, 0.0414692, 0.051783, -0.0926328, -0.0202989, -0.04468, -0.0107627, -0.194819, 0.021276, 0.0519405, 0.0501887, -0.0453346, 0.0647741, 0.0633295, 0.0302224, -0.00752813, 0.0455891, 0.0371337, 0.0427661, -0.0542973, 0.0287062, 0.0284935, 0.0291522, 0.0633419, -0.00230489, -0.0389004, 0.0042556, -0.247139, -0.186807, 0.00582994, -0.00714472, -0.0177211, -0.11753, -0.320344, -0.134415, 0.00383039, -0.0964483, -0.104084, -0.152009, -0.687008, -0.00195799, -0.00320682, -0.149204, -0.173611, -0.28721, -0.127516, -0.265592, 0.0486662, -0.231552, -0.114375, -0.0142297, -0.263652, -0.254188, -0.561558, -0.408789, -0.404796, -0.0556501, -0.0207537, -0.224789, -0.0111494, -0.209016, -0.0970849, -0.236925, -0.0164443, -0.456136, -0.00146683, 0.103315, -0.751662, -0.00637134, -0.261188, -0.888727, -0.000919675, 0.0210185, -0.170709, -0.195299, -0.294163, -0.0689581, -0.511988, -0.00562736, -0.0131, -0.312951, -0.17403, -0.123558, 0.0083815, -0.361124, -0.0656915, -0.0793824, -0.267194, -0.623966, -0.24135, -0.219674, -0.343785, -0.0076704, -0.217091, -0.213122, -0.00116697, -0.644903, -0.23865, -0.200323, 0.0101459, -0.021018, 0.0106982, -0.217117, 0.0360813, -0.42773, -0.272845, -0.012985, 0.278388, -0.199233, 0.250214, -0.0884898, -0.00712319, -0.310696, -0.717387, -0.150255, -0.232695, -0.517826, -0.0779041, -0.454903, -0.403114, -0.329437, -0.00667771, -0.0197043, -0.190453, 0.278622, -0.993821, -0.313727, 0.0165525, 0.0763556, -0.14975, -0.353724, 0.0198501, -0.00427418, -0.0171106, -0.0401689, -0.505275, -0.452098, -0.430563, -0.0824868, -0.260431, -0.587469, -0.292662, -0.0136002, -0.0231726, -0.0684055, -0.440273, -0.0871535, -0.193003, -0.412786, -0.158329, -0.482414, -0.297195, 0.000187742, -0.904822, -0.0172032, 0.211734, 0.417781, 0.00167695, -0.000486363, 0.0263458, 0.196111, 0.673946, 0.0303582, -0.00797759, 0.642861, -0.137135, 0.202074, 0.1025, -0.00257329, -0.00110074, 0.209088, 0.24294, 0.205856, 0.285594, 0.465603, -0.119937, 0.467274, 0.502575, -0.0763163, 0.416168, 0.313787, 0.233821, 0.0311521, 0.190317, -0.0278656, 0.00525205, 0.329608, -0.0117878, 0.190954, 0.36956, 0.18769, -0.00343555, 0.580403, 0.0100852, 0.239685, -0.300254, -0.00274663, -0.349247, -0.250915, -0.00647596, -0.294816, 0.176717, 0.211838, 0.348979, -0.0495941, 0.0514937, -0.0095359, -0.0129327, 0.502769, 0.314789, 0.317997, 0.0165092, 0.194521, 0.309191, 0.576269, -1.66042, 0.210227, 0.07549, 0.26686, 0.272303, 0.0118226, 0.421116, 0.229799, 0.0147524, -0.131287, 0.417646, 0.517306, 0.00648234, -0.0212336, 0.0188145, 0.274094, -0.958259, 0.222179, 0.342656, -0.000192372, -0.371465, 0.259022, -0.0843388, -0.0204777, -0.0146126, 0.198518, 0.222383, 0.321556, 0.238063, -0.0120934, 0.454254, -0.124591, 0.223356, 0.420066, -0.0135532, -0.0238065, -0.601587, -0.191384, -0.471689, 0.4266, -0.0149929, -0.307918, 0.228448, 0.525974, -0.164487, 0.00854701, -0.0897215, -0.0263147, -0.330641, 0.167839, 0.202944, 0.350869, 0.1699, 0.155, 0.357447, 0.304686, 0.0168536, 0.285022, 0.357882, 0.3583, 0.369605, 0.205802, 0.428236, 0.097193, 0.199326, -0.0112452, 0.242869, -0.00256737, 0.347314, -0.566165, 0.0233453, 0.00460847, 0.0192948, -0.169542, -0.278007, -0.518043, 0.0115397, -0.145114, -2.13431, -0.170599, -0.323904, -0.0102106, -0.00950461, -0.178324, -0.173804, 1.21949, 0.307981, -0.413869, -3.98748, -0.167296, -0.153611, -0.00491662, -0.171344, -0.439564, -0.339178, -0.298115, -0.333264, -0.0301606, -0.0158948, -0.172724, -0.0217493, -0.135156, -0.463473, 0.259325, -0.0244201, -8.45332, 0.00699859, -3.8574, -1.45438, -0.0175474, 0.196921, -2.39387, -0.00986168, 0.426549, -0.483038, -0.167192, -0.177919, -0.00950462, -0.545887, 0.00626502, 0.018489, -0.396912, -0.17446, 0.276071, 0.0323904, -0.389192, -4.20755, -0.193606, -1.75482, -0.406893, -0.208684, -0.161462, -0.41774, 0.00456214, -0.188313, -0.183509, -0.000807267, -0.424296, -0.201097, -0.408236, -0.0111208, 0.0115576, -0.0278494, -0.182377, 0.127561, -0.154762, -0.371737, -0.0140636, 0.362879, -0.159471, -3.96772, 0.880931, 0.0085888, -0.185646, -0.399743, -0.168975, -0.18136, -1.97414, -0.180218, -1.18998, -0.239574, -0.189919, -0.0186524, -0.009593, -4.64168, 0.639118, -3.58235, -0.380231, -0.00800626, -3.76546, -0.207647, -0.170073, -3.69983, -0.0159393, -3.49842, -0.0136482, 0.429898, -0.384165, -0.418614, -0.191287, -4.03171, -0.275546, -0.175656, -0.181065, -0.0381445, -0.182981, -0.39385, -0.182084, -4.16128, -0.218677, -0.372299, -0.430482, -0.3675, -0.0229103, -0.116073, -0.0195927, -0.0574123, 0.0901276, -0.0152706, -0.00357405, -0.0895458, 0.332166, -0.121272, 0.0340168, 0.00970165, 0.238506, -0.690502, 0.289226, -0.26278, -0.00611476, -0.00033641, 0.373004, 0.321292, -1.15791, -0.00190789, 0.501135, -0.0787884, 0.149092, 0.303616, -0.000315861, 0.264001, -0.00684369, -0.652685, -0.946151, -0.143536, -0.0553139, -0.00612836, 0.176576, -0.01464, 0.267258, 0.105143, -0.188995, -0.0177193, -0.0447593, -0.0132338, -0.0886176, 0.440916, -0.0117306, 0.14756, 0.386529, -0.00959916, -0.482738, -0.13324, 0.301339, 0.279235, -0.0318306, -1.18159, -0.00159279, -0.0286143, 0.156917, 0.252212, -0.0410862, -0.0170578, -0.471232, -0.0575107, 0.273211, -1.07165, -0.847776, 0.268318, 0.352617, -0.334708, 0.00451276, 0.139218, -0.450835, -0.0113321, -0.915627, 0.412142, -0.0345718, -0.0160962, -0.0198704, 0.00468228, 0.219345, -0.229162, -0.109654, 0.112301, -0.0234059, -0.0197065, -0.419465, -0.536699, -0.2356, -0.0155901, -0.0133372, -0.346856, 0.312205, 0.204561, 0.135435, 0.276039, -0.784489, -0.876226, 0.195578, -0.0220611, -0.0123213, -1.20633, -0.133385, -0.00164417, 0.244281, -0.0338968, -0.0539516, 0.276521, 0.19681, -0.00779336, -0.0330551, 0.125131, -0.0162358, 0.160442, -0.176432, -0.444007, 0.755494, -0.453142, -0.852718, 0.24012, 0.455067, -0.0381423, 0.245977, -0.403, 0.510711, -0.0723244, -0.265198, 0.354859, -0.910937, 0.0864574, 0.0032223, -0.114471, 0.00140632, 0.138178, -0.375479, -0.00891183, -0.014402, -0.000346629, -0.208131, -0.286545, -0.324516, 0.00354098, -0.236574, -1.75151, -0.25137, -0.333908, -0.0272756, -0.00521369, -0.232965, -0.250656, 1.48561, 0.170408, -0.308546, -4.20466, -0.237988, -0.26106, -0.022686, -0.233001, -0.346307, -0.311898, -0.303291, -0.300474, 1.70507e-06, 0.0176351, -0.240588, -0.0305294, -0.257176, -0.344921, 0.0897369, -0.0330495, -6.60218, -0.000889563, -3.72633, -0.915617, -0.0155258, 0.140686, -2.37355, -0.0200606, 0.209535, -0.351582, -0.243317, -0.225232, -0.0141768, -0.37347, -0.00198297, 0.0155648, -0.306019, -0.244806, 0.2646, 0.00650193, -0.321777, -3.50673, -0.25033, -2.54138, -0.34448, -0.227107, -0.266065, -0.341927, -0.00121289, -0.248108, -0.266043, -0.0178305, -0.32455, -0.261362, -0.327037, -0.0280122, -0.000423645, -0.0228848, -0.269365, -0.512454, -0.260094, -0.303721, -0.0263153, 0.242426, -0.247427, -3.47679, 0.826784, 0.0204403, -0.228688, -0.322098, -0.24408, -0.227213, -2.2079, -0.259903, -2.35137, -0.279028, -0.241262, 0.015698, 0.00234902, -4.24859, 0.528946, -6.03985, -0.311874, -0.00750451, -3.82045, -0.214497, -0.258489, -4.31852, 0.00942051, -4.17731, 0.0133487, 0.275524, -0.305532, -0.315627, -0.235938, -3.6795, -0.305308, -0.251032, -0.221161, 0.0157262, -0.211848, -0.315584, -0.227579, -2.18748, -0.281371, -0.290323, -0.362825, -0.328424, -0.0189566, 0.0173968, -0.00135994, 0.282565, 0.147335, 0.00342414, 0.00256949, 0.00649417, 0.490674, -0.41708, -0.0526814, -0.00152031, 0.00231493, 0.0184205, 0.529067, -0.277755, -0.0086048, -0.00913055, 0.505092, 0.481299, -0.374519, 0.303136, 0.231208, -0.624535, 0.308586, 0.398449, -0.0379021, 0.215563, 0.220239, -0.225609, -0.124062, -0.32036, -0.0771794, -0.002528, 0.421828, -0.00595738, 0.338117, 0.11024, 0.264767, -0.00171948, -0.0797449, -0.00183393, -1.95693, -0.800907, -0.0126977, -0.682012, 0.319787, -0.00720883, -4.29369, 0.399277, 0.450528, 0.0479835, -0.0382045, -0.573693, -0.00480477, -0.0166452, -0.136925, 0.432386, 0.269877, 0.00601028, -0.402225, -1.96801, 0.0429928, -0.399774, -0.765393, -0.134639, 0.455724, -0.403729, -0.0239548, -0.143929, -0.309862, -0.00758597, -0.790533, 0.399428, -0.0799154, -0.0258441, -0.0279095, -0.00297806, 0.391031, -0.243266, -0.34982, -0.117478, -0.0148669, -5.11252, -0.204945, -0.875566, 0.391696, 0.0119949, -0.303162, -0.469544, 0.47249, 0.36668, -1.0828, 0.466524, 0.24178, -0.0992521, -0.0944641, -0.0029354, -2.58049e-05, -0.514737, -0.0656677, -0.246063, -0.233996, -0.0201211, -0.511383, 0.335436, -0.0961426, -0.49732, -0.00716695, -0.192149, -0.0130474, -3.66921, -0.483308, -0.320252, 0.197521, -1.46415, -0.504864, -0.0874865, 0.576581, 0.0149713, 0.497614, -0.222457, 0.155797, -1.05111, -0.469347, -0.135929, -0.234774, -0.495644, 0.00688057, -0.111045, -0.00674064, -0.0755292, -0.0216005, -0.00798502, 0.0111659, -0.00656014, -0.0270336, -0.00788617, -0.0225696, -0.0122844, -0.00299335, -0.13402, 0.0045186, -0.0168108, 0.00752444, -0.0163384, -0.029011, -0.0207197, -0.0104526, -0.0736165, -0.00330406, 0.0867265, -0.0064671, -0.00929282, -0.015296, -0.0200545, -0.0142783, 0.00786422, 0.00988642, -0.00992626, -0.0476469, -0.0037096, -0.0028532, -0.00272198, -0.0141521, -0.0153937, -0.0846163, -0.00449557, -0.0195985, -0.00820133, -0.117242, -0.11149, -0.0133181, 0.0605022, -0.135936, -0.00866794, -0.208697, -0.0036677, -0.0176677, -0.0247631, -0.0508841, -0.0110261, -0.00762912, -0.01991, 0.0061438, -0.0207477, -0.0814474, 0.00960592, 0.00291166, -0.121239, 0.00493361, -0.220382, 0.00193077, -0.00169199, -0.00514824, -0.0260921, -0.011354, -0.00755787, -0.000754704, -0.00515273, -0.0113552, -0.0143076, -0.0276284, -0.0288469, -0.00794618, -0.00129015, -0.00402504, -0.0174995, -0.0162706, -0.000183374, -0.00505701, -0.318247, 0.017752, -0.0580638, -0.0680364, 0.0108928, -0.00332329, -0.00309373, -0.0237666, -0.0154133, -0.171135, -0.00635332, -0.0269934, 0.00343673, -0.0138967, -0.0211975, 0.00749082, -0.208202, -0.0291722, -0.013765, -0.022072, -0.0123907, -0.0547191, -0.0428152, -0.0190929, 0.0124707, -0.000766952, -0.0054836, 0.000941958, -0.318475, -0.0108601, -0.0191556, -0.0156884, -0.146495, -0.0147268, -0.00536967, -0.0270636, 0.00332036, -0.0231737, -0.00439881, 0.00510712, -0.16877, -4.80407e-05, -0.00351022, -0.0170047, -0.00299062, -0.000765445, -0.0633829, -0.0117766, -0.0317753, -0.217836, 0.00252563, 0.0111807, 0.00342866, -0.105679, 0.119206, -0.425703, -0.0229823, 0.0976694, -0.3726, -0.15025, -0.232898, -0.013578, 0.00117146, -0.0526381, 0.0474925, -0.0987341, -0.00876365, -0.210837, 0.134308, -0.0437282, -0.122499, -0.0932036, 0.0763255, -0.501778, -0.108856, 0.0579585, -0.0274763, -0.0577569, -0.0215379, -0.174999, 0.00506869, -0.181167, -0.246582, -0.0658668, -0.00993286, 1.46375, -0.0106903, 0.0613087, 1.01332, -0.0146183, 0.311674, 0.02626, 0.00524322, -0.264643, -0.442888, -0.0254477, 0.0239925, -0.0203641, -0.104522, -0.0160364, -0.0315489, -0.379241, -0.0548403, 0.0426619, 0.00538855, -0.0519072, 0.00660257, 0.131773, -0.981342, 0.0671134, -0.237408, -0.095349, -0.0699804, -0.035084, 0.130402, -0.0419093, -0.0100864, 0.00728075, -0.119627, -0.260598, -0.0255847, 0.000837288, 0.00324742, -0.205267, -0.0169313, -0.0499096, -0.260637, -0.0142237, -0.547124, -0.0240569, 0.0435392, -0.224692, 0.00336643, 0.0129546, -0.123253, 0.0839506, -0.0683107, -0.37707, -0.0609556, 1.1634, 0.151568, 0.0103326, -0.000109647, -0.00134503, 0.156771, -0.347352, -0.08398, -0.289827, 0.00552095, -0.17438, 0.0602861, -0.0516639, -0.0565533, -0.00343068, -0.155163, -0.00524455, -0.450635, -0.161451, 0.0227187, 0.0363465, 1.10837, 0.0676639, -0.0638553, 0.0993413, -0.0158088, 0.0362367, -0.14127, 0.0631108, 0.437139, -0.0175299, -0.354721, -0.0654779, -0.14116, -0.0132651, -0.353551, -0.00634494, 0.43841, -0.0141679, -0.0400336, -0.0238904, -0.0281842, -0.162322, -0.300163, -0.104092, 0.00343375, -0.146626, -0.225032, -0.157166, -0.0504766, -0.0336572, 0.00714107, -0.178289, -0.101916, -0.423566, 0.409108, -0.340921, -0.181736, -0.264332, -0.264519, -0.0164896, 0.224784, -0.580879, 0.0275586, 0.0909978, -0.236919, -0.0231221, -0.0194582, -0.186832, -0.0355608, -0.120458, -0.0832031, 0.467387, 0.0130523, 0.952919, -0.0376936, -0.0348177, 0.466628, 0.000569293, -0.0744647, 0.590719, -0.0174127, 0.383876, -0.695217, -0.12703, -0.199325, 0.0135406, -0.00980053, -0.00266129, -0.0127762, -0.110556, -0.207944, 0.439703, -0.00715883, -0.0350824, 0.314969, -0.233416, -0.765623, 0.0324831, -0.0940978, -0.123916, 0.0617187, -0.00298006, -0.184367, -0.0872142, 0.00708584, -0.237849, -0.191562, -0.0986568, -0.00698522, -0.0126553, -0.0153689, -0.15637, -0.550262, -0.245069, -0.373512, -0.00812392, 0.619182, -0.0824926, 0.256643, -0.0194324, -0.017906, -0.127943, -0.215502, -0.209698, -0.115648, 0.294174, -0.332095, -0.115546, 0.2312, -0.234379, -0.00200932, -0.0194842, -0.916478, -0.690383, 0.550346, -0.275351, -0.0181561, 0.133748, -0.154108, -0.163468, 0.0592419, -0.000970683, 0.434527, -0.00134772, 0.272161, -0.186936, -0.0477244, -0.292611, 0.327902, -0.0509046, -0.0885857, -0.550227, -0.0715549, -0.262826, 0.0215093, -0.242278, 0.0572445, -0.268836, -0.275746, 0.0270858, 0.0260197, -0.000669385, -0.448223, 0.00129483, 0.0484062, 0.351157, 0.00223329, 6.05582e-05, -0.0485586, 0.351536, 0.0987129, 0.301464, -0.012701, 0.251898, -1.09036, 0.379568, 0.498609, -0.0180255, -0.0204112, 0.355933, 0.354929, -0.815585, 0.117825, 0.498721, -0.1278, 0.24346, 0.34129, -0.0863215, 0.161203, 0.626533, 0.0194591, 0.410886, 0.0404223, -0.0569115, -0.0168014, 0.31471, -0.0184914, 0.260375, 0.383423, -0.0411742, -0.0197323, 0.273693, -0.0059119, -0.366309, 0.383359, 0.00763227, 0.324804, -0.924829, -0.0221221, -0.51106, 0.781225, 0.304831, 0.157073, -0.000471227, 0.218219, -0.00221618, -0.0242598, 0.148379, 0.338564, 0.067843, 0.0247173, -0.00585645, -0.381284, 0.198227, -1.59286, 0.124379, 0.055744, 0.405542, 0.00108018, -0.0166089, 0.0444443, -0.0184339, -0.0268372, 0.358279, 0.345713, 0.106107, -0.0380063, -0.00761366, -6.94167e-05, 0.275441, -0.501404, 0.0117077, 0.560992, -0.0137027, -0.704567, -0.04754, -0.123164, -2.02338, -0.0225023, -0.016016, 0.325146, 0.357938, 0.256838, -0.0721484, 0.374665, 0.597889, 0.517583, -0.00885701, -0.0487052, -0.0134894, -1.67455, -4.74436, 0.0759559, 0.230564, -0.0143679, -0.147012, 0.267945, 0.0435663, -0.150156, -0.0273673, 0.00150291, -0.00854491, -0.995386, 0.0754247, 0.0599725, 0.384964, -0.233821, 0.241415, 0.00893214, 0.514207, 0.0373139, 0.397648, 0.0082283, 0.324622, -0.539625, 0.0274501, 0.302966, 0.113893, -0.0533501, -0.00642121, 0.406036, -0.00809792, 0.0310943, 0.0470085, -0.00577607, -0.00416425, 0.00765454, 0.0592949, 0.0636349, 0.0643851, 0.0135705, 0.044579, -0.115493, 0.0521234, 0.0726864, -0.0194063, 0.000561373, 0.0700131, 0.0584801, -0.0622522, 0.0404299, 0.0603169, 0.00447781, 0.0625148, 0.0639349, -0.0207618, 0.055638, 0.0585177, 0.0748197, 0.0505293, 0.067947, -0.0948391, -0.0154425, 0.0735198, 0.00067743, 0.0650144, 0.062179, 0.0276611, 0.000623042, 0.0278299, -0.0129609, -0.0821108, -0.00440351, 0.0097614, 0.00208312, -0.147442, 0.00402139, -0.204841, 0.062748, 0.0872298, 0.0615057, -0.054607, 0.0744554, 0.00104081, -0.0313598, 0.0676336, 0.0724152, 0.0575031, -0.0191926, 0.0583814, -0.101692, 0.0799791, -0.21138, 0.0587028, 0.0777327, 0.0276922, 0.0547357, -0.0203464, 0.0701161, 0.0566289, -0.0131594, 0.0766196, 0.0609867, 0.0587865, -0.0055554, 0.00858991, 0.0151588, 0.0637338, 0.00421045, 0.0578663, 0.0564074, -0.00652784, -0.33343, 0.0606123, -0.0711931, 0.0334152, -0.00195962, 0.0721536, 0.0628167, 0.0592882, 0.0713431, -0.116817, 0.0556016, 0.0610235, 0.0480483, 0.0576202, -0.0113818, -0.00808994, -0.215186, -0.0360757, 0.0584842, 0.0561369, 0.0150428, -0.0641784, 0.0623116, 0.0509987, -0.0242259, -0.00510785, -0.0749783, -0.00606684, -0.353717, 0.0579544, 0.0750615, 0.0575371, -0.0849433, 0.0486596, 0.0707097, 0.0621379, 0.000158784, 0.0634713, 0.0747968, 0.0648833, -0.113858, 0.0673848, 0.0456729, 0.0767082, 0.0906087, -0.00812036, -0.0268236, 0.00234808, -0.00535098, 0.011712, 0.0109077, -0.00292315, -0.00240871, 0.0273197, 0.0118998, -0.00782214, 0.0129714, 0.0118555, 0.00897125, 0.00887352, 0.0268643, 0.00262371, 7.22116e-05, 0.0298211, 0.00364369, -0.0206604, 0.00473203, -0.00345693, -0.000730366, 0.0163339, -0.00768508, 0.00563949, 0.0105833, 0.00680132, 0.0096042, -0.00750512, 0.0153686, -0.0105418, -0.0122636, 0.000882899, 0.00614901, 0.0173826, -0.00617073, -0.00505447, 0.0103522, -0.00481835, -0.00604688, 0.0215625, -0.0190088, -0.00427683, -0.00311255, 0.0186647, -0.0226709, 0.00531576, 0.0126288, 0.0036103, -0.00106902, 0.021054, 0.0193098, -0.0148065, -0.00353672, 0.00432483, -0.0104256, 0.020925, -0.00210833, 0.0154, 0.0151185, 0.00869953, 0.00617585, 0.00214561, -0.00336946, 0.0217396, 0.0258279, -0.0125997, 0.0165007, 0.000637254, 0.0104466, 0.036751, 0.0177606, -0.00262013, 0.00841585, -0.00107273, -0.00725412, -0.0140331, 0.0027584, 0.00469768, 0.00306152, 0.0115949, -0.0224545, 0.0225951, 0.000438702, 0.0272252, 0.00632935, 0.0110914, 0.00029944, 0.000416498, 0.00873178, 0.0220438, -0.0101908, -0.00941663, 0.00224967, 0.0141366, 0.00836973, -0.0176383, -0.00415067, 0.017518, -0.0249487, 0.0167337, -0.0125223, 0.0203736, 0.00607924, -0.0199711, 0.0285358, -0.00916472, 0.017686, 0.00114268, -0.000285271, 0.00905015, 0.00692479, 0.00333017, 0.0264597, -0.021521, 0.0178339, 0.015969, -0.00612486, 0.0284874, 0.0124582, 0.00741337, -0.00452179, 0.0179961, 0.0219071, -0.0152186, 0.00267182, 0.000995647, 0.00146917, 0.00533982, -0.155824, -0.480254, -0.0394844, 0.00985193, -0.0338281, -0.957209, -0.758002, -0.323511, -0.0459029, -0.804653, 0.562818, -1.1117, -1.09691, -0.00435645, -0.0226398, -0.880632, -1.13288, 0.594059, -0.0848357, -0.784886, 0.101012, -0.413627, -0.570227, 3.96533e-05, -1.40499, -0.370398, -0.934263, -1.01822, -1.13696, -0.0355336, -0.00905827, -0.996018, -0.0228059, -1.08454, -0.181928, -0.204481, -0.0456989, 0.120141, -0.0161723, -0.0736691, 0.779436, -0.0292082, -0.644972, 0.973998, -0.0327386, 0.17199, -0.554532, -1.13888, -0.766611, -0.0548966, -0.640826, -0.0305149, -0.0494185, -0.714287, -1.10157, -0.065716, -0.0159162, -1.20867, -0.0162448, -0.779031, 0.399857, -0.741049, -1.06737, -1.18624, -0.657177, -0.0519774, -0.967692, -1.18452, -0.0374929, -0.412306, -1.07228, -0.364133, -0.0319969, -0.0111783, -0.0163002, -1.0849, 0.150204, -1.18556, -0.734807, -0.0544286, 0.205002, -1.05651, 0.0592501, 0.043095, -0.0373011, -0.945259, -1.24274, -0.976256, -1.09999, -0.276543, -0.466693, 1.28134, -0.764869, -0.754416, -0.0390627, -0.0323016, 0.323536, 0.406836, -0.27916, -0.689476, -0.0275447, 0.13577, -0.972267, -0.703658, 0.129943, -0.0325855, 0.497477, -0.0463177, 0.0903778, -1.21763, -0.884739, -0.888099, -0.0754386, -0.905444, -0.598469, -1.46755, -0.0238406, -1.15909, -0.792261, -0.998527, 0.155913, -1.24812, -0.815787, -0.725237, -0.707676, -0.0138449, -1.26993, -0.0134574, -0.0370431, 0.0881539, 0.00979489, 5.29949e-05, 0.0201565, 0.0806827, 0.0958731, 0.0860076, 0.00776722, 0.0864552, -0.0247758, 0.0876874, 0.0806109, -0.00143811, 0.00106574, 0.0905849, 0.0953799, -0.0302154, 0.0218934, 0.0812802, 0.147121, 0.0999837, 0.0835777, -0.0819679, 0.0727284, 0.0906576, 0.0831838, 0.085009, 0.100554, -0.0336108, -0.0232098, 0.0959657, -0.0152233, 0.0821059, 0.0891628, -0.0296414, -0.0101106, 0.0943576, -0.00625972, 0.152214, -0.00182426, 0.00815194, -0.028275, 0.0956889, 0.00166464, -0.168794, 0.0883346, 0.0713398, 0.101145, -0.0407531, 0.0841913, -0.0117973, -0.0355552, 0.0770784, 0.0892643, 0.0529184, -0.00216046, 0.082619, 0.0125085, 0.0947446, -0.13338, 0.0936981, 0.0896468, 0.0783075, 0.0920873, -0.0161639, 0.0971361, 0.0748078, 0.00107982, 0.0987794, 0.1243, 0.088015, -0.0229248, -0.00121968, -0.001789, 0.0969641, -0.00793717, 0.10602, 0.0758256, 0.00980448, -0.373173, 0.0883053, 0.164921, 0.0538698, 0.00366267, 0.101151, 0.0984731, 0.0923738, 0.0938752, -0.0140084, 0.0940808, 0.0882723, 0.104292, 0.0938409, -0.008679, 0.000718866, -0.228567, 0.0750833, 0.111909, 0.0697224, 0.0137026, 0.112437, 0.0809481, 0.0937779, 0.0425202, -0.00279487, 0.0484508, -0.009901, -0.426916, 0.09138, 0.0913095, 0.0939648, 0.179731, 0.0827351, 0.0866047, 0.0709306, -0.00551549, 0.0887304, 0.0904963, 0.0990051, 0.0226815, 0.114992, 0.0817869, 0.0976316, 0.0939438, -0.0111681, -0.0415164, -0.00418625, -0.0714441, -0.238783, -0.0120526, -0.00425887, 0.0194644, -0.1032, -0.347787, 0.0335872, -0.00200474, -0.0515503, -0.552973, -0.167741, 0.0910651, -0.00298974, -0.00640246, -0.144066, -0.209107, -2.91942, -0.0742026, -0.0638569, 0.363618, -0.286965, -0.100768, -0.0468939, -0.697387, 0.0910942, -0.438954, 0.058661, -0.179696, -0.0280188, -0.00619979, -0.307653, -0.0112485, -0.324261, -0.139548, -0.105986, -0.00575102, 1.06373, -0.00173085, -0.318693, 0.420419, 0.00182414, 0.374891, 0.0584273, -0.0166857, -0.734641, 0.0851091, -0.220272, -0.340873, -0.0264137, -0.690121, 0.0130415, -0.0179755, -0.165174, -0.365053, -0.0975475, -0.00592032, -0.466023, -0.502405, -0.102856, -1.76705, -0.342418, -0.330594, -0.242482, -0.231475, -0.00338759, -0.457507, -0.569632, -0.00300076, 0.101013, -0.35274, -0.074087, -0.0104217, -0.00426535, -0.0149448, -0.335473, -0.669458, -0.246842, -0.0516401, -0.0213255, -0.420339, -0.397994, -0.0707193, -1.66212, 0.00195895, -0.280432, -0.0146608, -0.210994, -0.402859, -0.514888, -0.119078, 0.649667, 0.255764, -0.374941, -0.00230032, -0.0099104, -0.273988, -2.99062, -0.364178, 0.0516667, 0.0113023, 0.16504, -0.286533, -0.471415, 0.339978, 0.0116357, 0.516564, 0.0107754, -0.548472, -0.201334, -0.415896, 0.0185814, 0.357455, -0.518562, -0.514479, 0.121526, -0.0727653, -0.157644, -0.350918, -0.066934, -0.0881478, -0.353281, 0.0278141, -0.344838, -0.395137, -0.00361047, 0.512487, 0.00333969, -0.989886, -0.519292, -0.0140075, -0.00129093, -0.0154596, -0.0809727, -0.11431, -0.353216, 0.00324557, -0.111874, -2.2692, -0.0938809, -0.151549, -0.0144992, 0.00312357, -0.0887914, -0.122854, 1.94339, -0.986868, -0.235402, -1.31613, -0.102446, -0.106424, -0.021696, -0.126681, -0.28586, -0.174462, -0.150494, -0.207389, 0.00452374, -0.00447486, -0.12787, 0.00233312, -0.0810791, -0.384461, -1.05026, -0.0187314, -5.55943, -0.00409215, -1.14172, -0.756158, -0.00341773, -0.000589112, 0.321526, -0.0312746, -0.959014, -0.25152, -0.0877837, -0.151401, -0.000915633, -0.368671, -0.00531657, 0.0115393, -0.231104, -0.138707, -1.00076, -0.0255066, -0.205501, -0.477677, -0.152895, 0.211265, -0.239889, -0.171647, -0.096733, -0.224203, 0.00974582, -0.114099, -0.120979, -0.00906858, -0.19425, -0.150094, -0.229266, -0.0273069, 0.0186669, -0.0177526, -0.114192, 0.574271, -0.134441, -0.2282, -0.0144751, -0.948784, -0.150438, -0.840237, 0.68857, -0.0127544, -0.18298, -0.240154, -0.114477, -0.10556, -2.66171, -0.120954, -0.986128, -0.129995, -0.139836, 0.0210954, -0.0241893, 1.14604, 0.669794, -4.58775, -0.209843, -0.000945725, -0.997311, -0.0613863, -0.161409, -1.16876, -0.00368771, -0.360359, 0.00823734, -0.898142, -0.238471, -0.236562, -0.193563, -2.38028, -0.136779, -0.0947968, -0.0787046, -0.016289, -0.0895738, -0.253005, -0.114277, -2.00415, -0.157166, -0.215171, -0.297061, -0.178141, -0.0216037, -0.000849683, -0.0204833, -0.0196494, 0.0068833, -0.00548896, -0.0123938, 0.0413657, 0.0285897, 0.0166915, -0.00196775, -0.00927333, 0.0181728, -0.144347, 0.0142768, 0.0195508, 0.00178676, -0.00408513, 0.0250257, 0.0091069, -0.0959164, -0.0178117, 0.0278985, 0.210803, 0.019222, 0.0184852, -0.0288654, 0.0104166, 0.00372866, 0.0193213, 0.037913, 0.00136816, -0.028516, -0.0062376, 0.0128578, -0.0332258, 0.0091844, 0.00690738, -0.0329136, -0.0185773, 0.0265973, -0.0117169, -0.101355, -0.0824193, -0.0155464, 0.013562, -0.152595, 0.0247638, -0.23762, -0.000678271, 0.00396845, 0.0156396, -0.0243115, 0.0326788, 0.00980986, -0.00685301, 0.0148579, 0.0399752, -0.0283223, -0.0146013, 0.0334521, -0.139967, 0.0369865, -0.163624, 0.0130394, 0.0118918, 0.04001, -0.00454361, 0.0223515, -0.0053821, 0.00349674, -0.0239964, 0.0176896, 0.0255429, 0.0228938, -0.0102141, 0.00283231, 0.00895066, 0.0402285, 0.0188462, 0.00129223, 0.00955221, -0.0284208, -0.358083, 0.00788179, -0.0772434, -0.05782, -0.0055255, 0.0307119, 0.00582988, 0.0236253, 0.0339756, -0.18827, 0.0328666, -0.00508096, 0.0226578, 0.0260053, -0.0137699, -0.0153728, -0.0380859, -0.148649, 0.00522301, 0.00152504, 0.00726284, -0.0445268, 0.0189921, 0.00918141, 0.109584, -0.0159769, -0.01437, -0.0209434, -0.360612, 0.0200884, 0.0112294, 0.0312315, -0.145775, -0.007191, 0.0342143, 0.0139094, 0.00567609, 0.0201482, 0.019478, 0.0214276, -0.144642, 0.0070036, 0.0172625, 0.00659478, 0.0155208, -0.00609088, -0.114016, -0.0076446, -0.104192, -0.88534, -0.0418099, -0.017805, -0.025616, -1.87597, -0.477368, -0.893189, -0.0307942, -0.537624, 0.104046, -2.01032, -0.501261, -0.0222393, -0.00310172, -1.82264, -2.01106, 0.496378, -0.0641647, -1.27595, 0.193473, -1.3004, -1.49324, -0.0107974, -1.16464, -1.5336, -0.902747, -0.989307, -0.359139, -0.0327505, -0.0290798, -1.7357, -0.0325903, -1.75808, -0.638169, -0.119957, -0.0421638, 0.134448, -0.0176639, 0.296609, 0.0926332, -0.0197029, -0.870276, 0.0745459, -0.0309753, 0.278375, -1.80891, -1.95527, -0.460959, -0.100551, -0.36106, -0.0113657, -0.0313245, -0.94249, -1.75534, -0.0925935, -0.0282916, -0.597434, 0.326216, -0.78531, 0.18072, -1.11958, -1.1188, -1.919, -0.548284, -0.0465875, -0.453559, -0.699384, -0.00880698, -0.611948, -1.2535, -0.475503, -0.0295629, -0.0219792, -0.0410914, -1.6639, 0.165519, -0.350269, -1.2248, -0.0222158, 0.35004, -0.715934, 0.338438, 0.323681, -0.0237617, -0.566058, -0.603975, -1.89143, -1.72702, 0.0415428, -1.64088, 0.435145, -0.54278, -0.889031, -0.0430598, -0.00862151, 0.467107, 0.489987, 0.0970882, -0.913429, -0.0104199, 0.28923, -1.78122, -0.842804, 0.261276, -0.0209072, 0.153594, -0.0528654, 0.23883, -0.414136, -0.569773, -1.2439, 0.289991, -0.510344, -0.792351, -1.97232, -0.0534526, -1.99598, -0.65999, -1.08564, 0.40278, -0.317394, -1.09612, -0.505369, -0.539004, -0.0147969, -1.49185, -0.0196258, 0.453793, 0.0579387, -0.017293, -0.0233309, -0.0001296, -0.203205, 0.0659831, 0.0501509, 0.0033191, -0.152312, 0.111734, -0.247138, 0.0578698, -0.0131634, -0.0116664, -0.18265, -0.261048, -0.262093, 0.428467, 0.0509306, 0.224819, -0.0480245, -0.0562537, 0.0273534, -0.0945185, 0.0669794, 0.0779567, 0.0750863, 0.063692, -0.0289861, -0.00892045, -0.194255, 0.00197866, -0.109436, 0.0629149, 0.436079, 0.00432368, -0.0191012, 0.00310127, 0.199312, 0.0254563, -0.010778, 0.461179, 0.109481, -0.0286099, 0.0683809, 0.0373279, -0.225496, -0.169667, -0.0227378, 0.077147, -0.00223655, 0.00473663, 0.0498206, -0.190697, 0.423387, -0.00930082, 0.0558561, 0.211269, -0.0504235, -0.169797, 0.0832865, -0.0695411, -0.18659, 0.0839428, -0.0132747, -0.00922604, 0.0527824, -0.0119418, 0.0347729, 0.0420421, 0.0493921, -0.0368435, -0.0026074, -0.0132757, -0.0890221, 0.0117466, 0.0435269, 0.0730957, -0.0179848, 0.184312, 0.0513436, 0.136703, 0.443551, -0.00997774, 0.0612987, 0.0662537, -0.212172, -0.015286, 0.108786, -0.158889, 0.0335861, 0.0741768, -0.0491597, -0.00217451, 0.00774996, 0.271837, 0.423191, -0.0173156, 0.076119, -0.0106986, 0.247423, -0.202055, -0.0102999, 0.336641, 0.0201876, -0.0725832, -0.0204128, 0.21701, 0.0631138, 0.0484002, 0.0372956, 0.185864, 0.0689764, -0.0282168, -0.217193, -0.0326611, -0.236586, 0.0796278, -0.17586, 0.169549, 0.101859, 0.0641886, 0.062582, 0.0768936, -0.00640601, 0.145307, -0.00475977, -0.0274736, 0.00275482, 0.00352757, -0.0153492, 0.00593507, 0.00708452, 0.0459333, 0.0239323, -0.0207237, 0.0399239, -0.0526815, 0.0292424, 0.00133307, -0.0170518, -0.00314987, 0.0350128, 0.0172668, 0.000980307, -0.0533605, 0.0249661, 0.0236994, 0.0481754, 0.0325796, -0.00645147, 0.0321614, 0.0188736, 0.0365163, 0.0399235, 0.0257126, -0.0376812, 0.0146443, 0.0287279, -0.0162737, 0.0524056, 0.0132693, -0.0202904, -0.0079484, -0.00769219, -0.0195174, 0.154517, -0.143125, -0.00611059, 0.0332591, 0.110121, -0.000498948, -0.100346, 0.0242437, 0.0393041, 0.0320998, -0.0674242, 0.020421, -0.00428933, -0.0121725, 0.00607638, 0.0335245, -0.0384566, -0.000750566, 0.021775, 0.031031, 0.0245315, -0.177005, 0.0258343, 0.0369686, 0.0516936, 0.023903, -0.015101, 0.0297883, 0.0337827, 0.00410992, 0.0204953, 0.0582003, 0.00556104, -0.00861293, -0.01773, 0.0139197, 0.0218163, 0.00265127, 0.0412946, 0.0376707, -0.0163765, -0.288849, 0.0307001, 0.132684, -0.0169414, -0.00355974, 0.0285684, 0.021816, 0.0527714, 0.033387, -0.0335852, 0.0411939, 0.00331804, 0.038847, 0.0502491, 0.00437923, -0.00628189, -0.251976, 0.103405, 0.00834072, 0.0258653, 0.00358999, 0.0944614, 0.0176785, 0.037594, 0.0689512, -0.0178931, 0.0631083, -0.0204967, -0.192078, 0.0275846, 0.0215192, 0.0359674, 0.179395, 0.0181538, 0.0401821, 0.0337383, 0.00730503, 0.0201374, 0.0263378, 0.0428866, -0.015614, 0.0513555, 0.027241, 0.00707444, 0.0308113, -0.00555175, -0.0880559, -0.00230477, 0.00637842, 0.0713144, 0.00932132, -0.00655168, 0.00704745, 0.0411583, 0.0636731, 0.0631114, 0.0105242, 0.0691324, -0.0703446, 0.0640152, 0.0611689, 0.00107127, -0.00131259, 0.0778221, 0.0514307, -0.0567096, 0.0326623, 0.058477, 0.207203, 0.0526218, 0.0519314, -0.0372185, 0.0126229, 0.0582795, 0.0684354, 0.0489119, 0.074973, -0.0570444, 0.00235575, 0.0789525, 0.00343306, 0.0591811, 0.0673562, 0.0243002, 0.0213024, 0.035332, -0.00238023, -0.0351615, -0.022003, -0.0058431, 0.0865736, -0.0775571, 0.014228, -0.225621, 0.0513439, 0.0722388, 0.0517528, -0.04276, 0.0568375, -0.000517624, -0.0476466, 0.0444595, 0.0524628, 0.0588222, -0.00290657, 0.0762607, -0.0816259, 0.0698483, -0.183485, 0.062908, 0.0625244, 0.0356142, 0.0752735, -0.00368048, 0.0592415, 0.0856886, -0.0182116, 0.0620512, 0.074424, 0.0702478, 0.0219277, -0.0157377, 0.00107384, 0.0675532, -0.00234518, 0.0749754, 0.0558338, -0.00697789, -0.421966, 0.0698522, -0.0314596, 0.0392991, -0.00351384, 0.0591644, 0.0624522, 0.067234, 0.0533972, -0.0961762, 0.0630434, 0.0467716, 0.0565636, 0.046365, -0.0162123, -0.0160533, -0.238543, 0.0369463, 0.05256, 0.0541698, -0.000926208, 0.0233264, 0.0508654, 0.0727358, 0.0332064, -0.00894523, 0.00123219, -0.0293789, -0.442332, 0.0397106, 0.0466336, 0.07659, -0.0512096, 0.0588841, 0.0694085, 0.0487958, 0.0109147, 0.0647986, 0.0629535, 0.0430342, -0.0835061, 0.0501494, 0.0702169, 0.066156, 0.0616433, -0.00870578, -0.058419, -0.0163772, 0.181416, 0.540165, -0.0318324, -0.00972763, -0.0226484, 0.0713554, 0.0901151, 0.592919, 0.000132515, 0.429154, -0.72562, 0.136493, -0.208361, 0.00472942, -0.018792, 0.0429897, 0.0955451, -2.15408, 0.215324, 0.266507, 0.332454, 0.324116, 0.323772, -0.0378955, 0.26987, 0.101378, -0.0581292, -0.378955, -0.270979, -0.0374775, 0.00297988, 0.185205, 0.00672243, 0.151025, 0.683285, 0.152102, -0.0163325, 0.760279, -0.00709279, 0.0909169, -0.75273, -0.0103039, -0.646884, -0.539726, -0.00286633, 0.352484, 0.0757068, 0.0963187, 0.392736, -0.0150297, 0.229409, 0.00858155, -0.0501768, 0.0434185, 0.210005, 0.221234, -0.00294682, -0.101672, 0.133362, 0.486663, 0.0191967, 0.343508, -0.000452675, 0.146609, 0.0940026, -0.0066574, 0.196657, -0.162709, -0.00587449, -0.364265, 0.405089, 0.231265, -0.0243638, 0.0020519, -0.0221735, 0.186099, -0.117799, -0.355569, 0.573409, -0.0317455, 0.140735, -0.130231, 0.0308352, -0.914731, -0.0361215, -0.150878, -0.408877, 0.0667063, 0.154346, -1.06682, 0.351069, 0.0707721, -0.28208, 0.248822, 0.0112184, -0.00206348, -0.810163, -0.624282, 0.567429, -0.00314754, -0.0220565, 0.114171, 0.0813533, 0.295927, 0.373847, -0.0273576, 0.458062, -0.024307, -0.163284, -0.0401417, -0.0947452, 0.145885, -0.0765595, 0.0199056, 0.371239, 0.0425756, -0.00387926, 0.125161, 0.108666, 0.273637, -0.409718, -0.221506, -0.000814422, -0.0151053, 0.0412939, -0.00876222, -0.863697, -0.00517645, 0.199958, -0.0397718, -0.0168813, 0.00798146, -0.00249021, 0.00279971, -0.00329133, -0.02218, 0.00346296, -0.0127989, -0.019625, 0.00408616, -0.0335319, -0.0159499, -0.0101997, -0.00602154, 0.00126825, -0.176373, 0.180031, -0.022927, -0.240234, -0.00368564, -0.0283887, 0.00920019, -0.0131959, -0.0238024, -0.0251805, -0.00390112, -0.0276341, -0.087706, 0.00462936, 0.00553679, -0.00754963, -0.0174663, -0.0235549, 0.196894, 0.00267398, -0.0146143, 0.00316045, 0.122331, -0.142837, -0.00821699, -0.0165343, 0.098641, -0.0105888, 0.117145, -0.032894, -0.0189187, -0.0325027, -0.0523808, -0.0320488, -0.0194399, -0.0376877, -0.0334991, -0.00444184, 0.155768, 0.00495803, -0.0390305, 0.012068, -0.0070761, 0.0423523, -0.0166736, -0.00425554, 0.0043227, -0.0202093, -0.00712421, 0.0065082, -0.0145383, -0.0157716, -0.00956192, -0.0194214, -0.0313648, -0.010064, 0.00363991, -0.010382, -0.00818836, -0.0162552, 0.00169965, -0.0475769, -0.0196085, 0.051872, -0.0094226, 0.115645, 0.133286, -0.0171395, -0.0192268, -0.0292091, -0.0227042, 0.000922694, -0.0134522, 0.00524863, -0.0334932, 0.00767799, -0.0144029, -0.0100078, -0.0109757, -0.0963907, 0.159755, -0.0446818, -0.0462357, -0.0162256, 0.0688763, -0.0237176, -0.0123049, -0.247147, -0.0200657, 0.0891453, -0.00214939, 0.197777, -0.0210768, -0.0171723, 0.00482225, 0.0967395, -0.0251643, 0.00356848, -0.0080552, 0.00037315, -0.0318013, -0.00594542, -0.00563734, -0.0438215, -0.00145199, -0.0156029, -0.0443446, -0.0382683, -0.00239323, -0.0122664, 0.00576159, 0.134904, 0.0554791, -0.0277792, -0.0163503, 0.00173425, 0.0702586, 0.085627, 0.0268989, -0.0169593, 0.0806735, 0.0123787, 0.0805323, 0.0653761, 2.38187e-06, 0.00620814, 0.0555825, 0.0676242, 0.00860277, 0.132888, 0.0442093, -0.141055, 0.0736022, 0.0503888, 0.020851, 0.0584441, 0.0466945, 0.0691643, 0.0930852, 0.0776113, -0.0645335, 0.0231691, 0.0688388, 0.00941517, 0.0767848, 0.0422582, 0.173841, 0.00226293, 0.0293653, -0.0155822, 0.00366352, -0.0300383, -0.00509732, 0.0107039, 0.112251, -0.00848773, 0.00382133, 0.0644432, 0.0769248, 0.076819, -0.0534482, 0.0660081, -0.0294571, -0.0287631, 0.0628208, 0.0919591, 0.145207, -0.0126916, 0.0652116, 0.148937, 0.0714917, -0.194611, 0.0510212, 0.0818441, 0.0830845, 0.0701239, 0.000664404, 0.0762203, 0.0749583, -0.00339476, 0.047967, 0.0757259, 0.0537573, 0.0102925, -0.00390699, 0.00478748, 0.0742958, -0.0046599, 0.0831307, 0.0760563, -0.0145155, 0.0423861, 0.0817108, 0.0455823, 0.144982, 0.0027706, 0.0945715, 0.056042, 0.0659, 0.0737969, 0.0266159, 0.0767193, 0.0479992, 0.0749549, 0.0563859, -0.0208547, 0.0042312, -0.206095, 0.0776997, 0.0641998, 0.0400863, -0.00865691, 0.110036, 0.049729, 0.0702689, -0.050046, -0.00446536, -0.00685039, -0.00885922, 0.0385581, 0.0579195, 0.0529072, 0.0706652, 0.0394302, 0.0652753, 0.0749105, 0.067972, -0.00372374, 0.0672919, 0.0766546, 0.105166, 0.0469972, 0.0780616, 0.0552988, 0.052522, 0.0712398, -0.0230184, -0.0155472, -0.00828588, 0.382167, 0.568794, -0.0295888, 0.00682588, 0.0162243, -0.0516184, 0.887438, 0.994775, -0.0170934, 1.0223, -0.305955, 0.184205, 0.132125, 0.00176737, -0.00279387, -0.106603, 0.0254434, -0.245796, 0.337885, 0.552725, 0.245373, 0.695918, 0.791402, -0.0829196, 0.319067, 0.974505, 0.471176, 0.392263, 0.432293, -0.00941616, 0.00602325, 0.64329, -0.00465847, 0.371131, 0.8847, 0.429423, -0.0246305, 0.930602, 0.0147778, 0.452629, -0.213653, 0.00591416, 1.24628, -0.497918, -0.0259852, 0.395009, 0.40422, 0.321689, 1.04909, -0.0477929, -0.199281, 0.00644533, 0.0232227, 0.722798, 0.538556, 0.335437, -0.00895735, 0.288925, 0.351448, 1.10877, -0.574083, -0.180679, 0.783795, 0.190837, 0.256042, -0.00645721, 1.21975, 0.322049, 0.0035676, -0.228873, 0.554879, 0.963905, 0.00847817, -0.00483595, 0.00745918, 0.537433, 0.172095, 0.504394, 0.664854, -0.00905914, -0.00638625, 0.338637, 0.736526, -0.611279, -0.00911122, 0.395747, 0.00818056, 0.0573363, 0.658912, -0.826298, 1.13601, 2.24164, 0.478917, 1.17257, -0.0622488, -0.00448785, -0.841034, -0.98305, 2.15885, 0.384947, -0.0149099, 0.426893, 0.00365272, 1.14475, 0.426652, 0.00708257, -0.545522, -0.0084523, -1.11528, 0.0622985, 0.237493, 0.673551, 0.93099, 0.400213, 1.2371, 0.174446, -0.0799691, 0.0482448, 0.210551, 0.934565, 0.891865, 0.410345, 0.646903, 0.266937, 0.209912, -0.0265979, -0.890302, -0.0121858, -0.16926, -0.392802, -0.00424245, 0.00458981, -0.0775146, -0.322305, -0.457245, -0.347457, -0.00324876, -0.389394, -0.144391, -0.325289, -0.416848, -0.00726508, -0.0215612, -0.345736, -0.297656, -0.536405, -0.129346, -0.299099, 0.388683, -0.377864, -0.366753, -0.0663264, -0.443605, -0.302814, -0.345149, -0.363571, -0.354667, -0.0485598, -0.0139714, -0.357001, 0.00209559, -0.382633, -0.326461, -0.193849, 0.0100329, -0.212556, -0.00237673, -1.0796, -0.619989, -0.0113776, -0.56747, -0.696076, -0.00808406, 0.00383513, -0.316242, -0.331861, -0.391263, -0.0519873, -0.331202, -0.000379294, -0.0317496, -0.322516, -0.330535, -0.137219, -0.00521016, -0.270032, -0.666398, -0.312803, 0.631304, -0.413015, -0.32341, -0.356328, -0.329067, -0.0116309, -0.359081, -0.257201, -0.0211533, -0.420623, -0.34555, -0.382956, -0.0237519, -0.00434511, 0.00613071, -0.35776, -0.0145809, -0.320754, -0.528968, -0.00938108, 0.335325, -0.15556, -0.718794, 0.345835, -0.0266446, -0.424974, -0.480976, -0.34607, -0.374404, -0.62822, -0.344707, -0.347032, -0.276454, -0.362121, 0.00486222, -0.00160874, -0.259773, 0.57457, -0.623568, -0.295743, 0.00159625, 0.0650856, -0.361737, -0.342024, 0.184309, -0.00532016, 0.115099, -0.0153538, 0.735511, -0.387354, -0.357592, -0.256333, -0.406335, -0.199866, -0.293898, -0.331249, -0.0134564, -0.31995, -0.400221, -0.253657, -0.200893, -0.344191, -0.288011, -0.380977, -0.329465, -0.00730702, -0.272614, 0.00724165, 0.0287823, 0.0178069, -0.00993766, -0.00750213, 0.00120128, 0.0105052, -0.013946, -0.00284837, 0.00785324, 0.0146648, 0.00249559, -0.00308028, 0.00113376, -0.0143511, 0.00601031, -0.0220886, 0.0010239, -0.00633156, 0.0311052, -0.00949865, 0.0111505, 0.00476803, -0.00308019, -0.00715989, 0.0019148, 0.00577047, -0.00697915, 0.00603004, 0.000919578, 0.00560073, 0.00228263, -0.00224979, 0.00365601, 0.012742, 0.0165684, 0.0314945, 0.0002186, -0.0137357, 0.0101864, 0.0359832, -0.0201282, -0.0134832, -0.00820765, 0.0468079, 0.019776, 0.0378189, 0.0197338, -0.00151845, -0.000226108, 0.00596133, -0.0043215, 0.000232778, -0.00173916, -0.00291969, 0.000431707, 0.024216, 0.0021933, -0.000538734, 0.0442145, -0.0296447, -0.0150421, 0.00918679, -0.00190633, -0.00143452, -0.0118923, -0.00283045, 0.0090839, 0.00598247, 0.00558706, 0.00880615, -0.0114615, 0.0192333, 0.002937, -0.0124981, -0.0181005, -0.00700974, -0.00947015, -0.00732716, 0.0154255, -0.0110637, 0.0412036, -0.00173205, 0.0213294, 0.0415853, 0.00925209, -0.00522442, -0.010219, 0.0102314, 0.00201677, -0.0249365, 0.000793284, 0.0368149, -0.00337033, -0.00640874, 0.011, -0.0262792, -0.0062515, 0.0215582, 0.00312228, 0.00099752, -0.0149404, 0.0296654, -0.00323181, 0.0091786, 0.0180964, -0.0118812, 0.0187469, 0.0140172, 0.0281265, -0.00535065, 0.00331491, 0.00998411, 0.0711277, 0.00472847, 0.00670145, -0.00773096, -0.00390162, 0.00437431, 0.00794787, -0.0134518, 0.00413044, 0.00580694, 0.0182492, -0.00273506, 0.0212245, 0.00208395, -0.00324103, 0.00389108, 0.280665, -1.69863, -0.0692138, -0.0174599, -0.0174626, -1.63395, -1.34535, -1.25815, -0.0154632, -1.78193, -1.07798, -1.59404, -0.898797, -0.0279688, -0.0334546, -1.57363, -1.58558, -0.33025, 0.292616, -1.27077, 0.302089, -2.31435, -2.20617, -0.123214, -1.33806, -1.46519, -1.58478, -1.60809, -1.25968, -0.144522, -0.00249421, -2.11083, -0.0391321, -2.02931, -1.27391, 0.118229, -0.0300143, 1.11678, -0.00114656, 0.275882, -0.162625, -0.0342583, -0.0161739, -0.081883, -0.0354213, 0.409134, -0.763142, -1.77921, -1.66987, -0.0296694, -1.06725, -0.0465548, -0.0997095, -1.54671, -1.83648, 0.301958, -0.0188724, -1.30171, 0.205056, -1.93164, 0.0146488, -1.60056, -1.76414, -1.79536, -1.43945, 0.00863375, -1.8834, -1.18896, -0.0143747, -1.38779, -1.85633, -1.72816, -0.0209311, -0.0179134, -0.00619828, -2.03967, 0.567287, -1.34594, -0.786743, -0.0112805, 0.548409, -1.19991, -0.0247082, -1.07794, -0.0161257, -1.65214, -1.19305, -1.56812, -2.13928, -1.27205, -2.3534, 1.67252, -1.49673, -2.00377, -0.0611579, -0.00915507, -1.0571, -0.812683, -0.930249, -1.49342, -0.0314847, 0.233783, -1.5132, -1.97428, 0.388255, -0.0166426, 0.50638, -0.0352164, 0.262493, -1.44209, -0.991323, -1.66363, 0.467867, -1.12967, -1.91101, -1.90584, -0.0387782, -1.55091, -1.53679, -1.76563, -0.476312, -1.20641, -1.32025, -1.1759, -1.31262, -0.0198517, -1.2925, -0.0170574, -0.169331, 0.306381, -0.0514544, -0.034904, -0.002045, 0.148034, -0.0553189, 0.20029, -0.00260401, 0.0626158, -1.21675, 0.176301, -0.192695, -0.0264894, 0.00493914, 0.124716, 0.114055, 0.580703, -0.166757, 0.210402, -0.500376, 0.146252, 0.309644, 0.0108391, 0.144698, 0.690205, 0.05576, 0.231209, -0.217868, -0.0671244, -0.0104118, 0.166778, -0.00197802, 0.116701, 0.284647, -0.189967, 0.016937, -0.301953, -0.00680671, -0.0265137, -0.737049, -0.0203703, 0.27323, -0.532462, -0.0233306, -1.14165, 0.745414, 0.132269, -0.0301743, -0.0215919, 0.0533775, -0.0120552, -0.0472517, 0.0190458, 0.185412, -0.162229, 0.00387469, -0.0704465, 0.0288494, 0.0587507, -0.908009, -0.0460193, 0.022422, 0.176768, -0.104073, 0.0264733, -0.163309, -0.097751, 0.000313064, -0.15614, 0.221119, 0.0372779, -0.0215446, -0.00391328, -0.0185328, 0.138832, -0.30344, -0.242915, 0.0433219, -0.00546059, -1.0391, -0.0966357, 0.080611, -4.63068, -0.00881742, -0.169487, -0.19752, 0.119802, 0.129523, -0.227262, 0.326444, 0.645521, 0.353276, -0.147203, 0.00846135, -0.0149508, -1.61217, -6.58732, 0.435609, 0.142693, -0.00107507, -0.177348, 0.0963965, -0.182647, -0.262847, -0.00402486, -0.101364, -0.0119683, -2.90238, -0.173673, -0.0202908, 0.229747, -0.00409029, 0.0117884, -0.13847, 0.496568, -0.0257777, 0.230295, -0.0951902, 0.0585261, -0.133417, -0.26312, 0.173599, 0.11, -0.114136, -0.00890638, 0.418943, -0.026299, -0.0418812, -0.00391992, 0.0124627, -0.00782425, 0.028632, 0.0141306, 0.0180837, 0.000380028, -0.00634607, 0.00546329, -0.138478, 0.00617405, -0.0253711, -0.0117602, -0.0011311, -0.00245627, -0.00260343, -0.101186, -0.0329065, -0.0199654, -0.0775974, -0.00331881, -0.0211314, -0.0322094, -0.00103226, -0.0161812, 0.00107721, 0.0183194, 0.00169635, -0.0396325, -0.00075198, 0.0102007, -0.0105236, 0.00195429, -0.00548547, -0.0388787, 0.00633922, -0.0053702, -0.0193595, -0.115951, -0.0939842, -0.00638413, 0.0839121, -0.164474, 0.0199938, -0.131345, 0.00635744, -0.0239532, -0.014494, -0.042936, -0.00935319, -0.00500756, -0.0133811, -0.0155622, 0.0135333, -0.0251524, -0.00893276, 0.0041594, -0.155246, -0.00613702, -0.186541, -0.00102386, 0.0199199, 0.00225951, -0.00381366, -0.0118928, 0.00127224, -0.0104633, -0.00391629, -0.00302075, -0.00390362, -0.0287234, -0.0144504, -0.00857183, 0.000555405, -0.00678674, 0.00901109, 0.0074679, 0.00975011, -0.0214344, -0.248955, 0.0180978, -0.0975577, -0.0382421, 0.0121145, 0.0185427, 0.0118501, -0.0072724, 0.00913316, -0.264138, 0.00600677, -0.0171724, 0.0120958, 0.00168272, -0.0123228, -0.0032826, -0.139957, -0.00563288, 0.00910259, -0.00832189, -0.000990729, -0.0343119, -0.0265121, -0.00137559, -0.0255449, 0.0114217, -0.00618845, 0.00293991, -0.255446, -0.00169353, -0.0181472, -0.000964298, -0.211552, -0.0140487, 0.0011166, 6.08362e-05, -0.00143227, -0.00823213, 0.0114763, 0.019706, -0.236503, -0.0265591, -0.0018473, -0.00991145, -0.0123064, -0.0138003, -0.111062, 0.0164499, -0.567524, 0.14262, -0.0117162, -0.0205815, 0.00787382, 0.228249, 0.0102971, 0.320872, -0.0215317, 0.224744, 0.234275, 0.206339, 0.106669, 0.00788142, 0.00243005, 0.0684328, 0.205492, 0.878138, -0.565925, 0.186487, -0.327782, -0.271155, -0.607701, -0.0493722, 0.216825, -0.287822, -0.149459, -0.469453, 0.0710412, -0.0319489, -0.0057979, 0.0721244, 0.00428329, 0.0739674, 0.060549, -0.638769, -0.0269844, -0.932302, 0.000915458, -0.38806, -0.554832, -0.0165995, -0.953237, 0.161841, -0.0175779, -0.927986, -0.337016, 0.0507575, 0.196943, -0.0188594, -0.946924, -0.00653362, -0.00821834, 0.349903, 0.0216124, -0.634802, 0.0102189, 0.0610814, -0.592514, 0.169618, 0.191383, -0.460038, 0.117862, 0.0949638, -0.305544, -0.0105148, 0.32149, 0.214783, -0.00539475, -0.422748, -0.146391, 0.54691, 0.00163022, 0.0157581, 0.0180947, 0.0818983, 0.072824, 0.290464, -0.0375395, -0.00495177, -0.4262, 0.115642, -0.407637, 0.751353, -0.0126335, -0.0952816, 0.166951, 0.473351, 0.00383291, 0.437268, -0.570181, -0.800399, -0.512827, 0.158641, -0.0296554, 0.0109622, 0.40063, 0.760771, -0.778146, -0.0529927, -0.00720508, -0.267055, 0.40754, 0.128245, -0.454419, -0.0119332, -0.142652, 0.00132183, -0.377586, 0.0990469, -0.0889297, 0.162021, -0.137646, -0.249465, 0.26436, -0.523243, -0.0569455, 0.443084, -0.176233, -0.369839, -0.356943, 0.156996, 0.429237, -0.0559282, -0.192053, 0.00749552, 0.289019, -0.0286794, 0.0630778, 0.167107, 0.0211763, -0.0230526, -0.0338143, 0.227817, 0.248197, 0.189198, 0.00954231, 0.24485, -0.129478, 0.228927, 0.197706, -0.0038392, -0.00267325, 0.224478, 0.221989, -0.117363, 0.0817306, 0.217984, 0.106137, 0.215117, 0.214074, -0.057307, 0.243836, 0.228327, 0.237833, 0.251962, 0.233153, -0.0497874, -0.0165506, 0.252465, 0.00568725, 0.249019, 0.180963, 0.0307154, -0.0124279, 0.16871, -0.00664382, -0.0964739, 0.0424159, -0.00153496, 0.00706232, -0.143852, 0.0194236, -0.330453, 0.221925, 0.245438, 0.235967, -0.00857426, 0.192311, 0.00075061, 0.0160763, 0.233857, 0.236188, 0.110609, -0.0116468, 0.233649, -0.12514, 0.230052, -0.308693, 0.23351, 0.231002, 0.237576, 0.220782, -0.00103698, 0.223523, 0.222898, -0.019368, 0.203227, 0.224986, 0.227131, 0.00247425, -0.00939299, -0.00140877, 0.230873, -0.00429871, 0.257017, 0.209161, -0.0103778, -0.6457, 0.234903, -0.0877125, 0.0996106, 0.00881704, 0.251021, 0.219258, 0.217498, 0.237249, -0.0977477, 0.241308, 0.164155, 0.229995, 0.235739, 0.0120268, 0.0064472, -0.562917, 0.0292849, 0.185016, 0.211162, -0.00386481, -0.0200157, 0.207115, 0.227283, 0.0675646, -0.0175805, -0.0286455, -0.0316774, -0.592798, 0.213143, 0.204595, 0.242694, -0.106716, 0.228974, 0.246212, 0.214401, -0.00529386, 0.236839, 0.206083, 0.258297, -0.134163, 0.228759, 0.226261, 0.205936, 0.219912, -0.00936799, -0.00130752, -0.00432446, 0.413904, 0.149374, -0.00952184, -0.00530005, 0.0331672, 0.172505, 0.154849, 0.141905, 0.0128904, 0.146372, 0.193467, 0.204703, 0.141004, -0.000451872, -0.0027156, 0.176129, 0.187708, -0.0286458, 0.400517, 0.146814, -0.0263024, 0.16431, 0.123969, -0.0326496, 0.149429, 0.159176, 0.173355, 0.0977773, 0.129188, -0.107634, 0.0134245, 0.162521, -0.021899, 0.144484, 0.138022, 0.41438, 0.0138952, 0.145071, 0.0064496, 0.249655, 0.0946221, -0.0111972, -0.391425, 0.296211, -0.0266595, 0.23333, 0.133375, 0.146981, 0.152813, -0.0643622, 0.185478, -0.0135488, -0.0344354, 0.151719, 0.127528, 0.410382, 0.00832127, 0.0781359, 0.317506, 0.146622, 0.0267856, 0.164601, 0.135776, 0.151585, 0.175928, -0.00332423, 0.146368, 0.1523, -0.00881416, 0.166742, 0.100233, 0.154699, 0.00926581, -0.0112192, 0.00341989, 0.135169, 0.00312221, 0.133131, 0.0943734, -0.0112477, 0.305781, 0.152137, 0.231352, 0.395612, 0.0102292, 0.104564, 0.0996168, 0.175465, 0.144308, 0.192977, 0.12543, 0.123479, 0.183903, 0.151952, -0.0233527, 0.0155439, -0.40028, 0.346429, 0.142313, 0.0862969, -0.0259322, 0.18465, 0.134887, 0.151174, 0.0450149, -0.00151775, 0.382159, -0.0187428, 0.270883, 0.122907, 0.130284, 0.152133, 0.408796, 0.117474, 0.146521, 0.104257, 0.00864494, 0.169282, 0.146326, 0.124048, 0.216124, 0.134346, 0.0808245, 0.16311, 0.0498046, -0.0105285, -0.28555, 0.00885094, 0.104899, -2.10531, -0.0429915, -0.026117, -0.0128798, -0.637816, -1.81965, -1.91192, -0.0276023, -1.5587, 0.530843, -0.814977, -1.35751, -0.0188363, -0.0211281, -0.754484, -0.762746, 0.649247, 0.179925, -2.05857, -0.409557, -0.788939, -0.798474, 0.000356602, -1.05883, -1.78768, -1.62242, -1.19193, -1.48756, -0.0133434, -0.0311876, -0.823624, -0.036733, -0.844019, -2.2326, 0.118363, -0.0108567, -1.28893, -0.0228404, -0.499208, -1.12927, -0.0297933, -1.03408, 0.698742, -0.0412109, 0.423513, -1.73707, -0.812505, -1.81215, -0.0485644, -1.78383, -0.0220664, -0.0372289, -2.26836, -0.785474, 0.181448, -0.0206969, -1.27134, -0.437789, -1.88714, 0.667295, -1.64328, -1.13636, -0.895011, -1.74559, -0.0438802, -1.69024, -1.20947, 0.00115766, -1.45578, -0.807801, -1.9093, -0.0394128, -0.0220605, -0.0352194, -0.730893, -0.150286, -1.34015, -1.88199, -0.012093, 0.602315, -1.17788, -0.300804, 0.437456, -0.0348157, -1.35592, -1.47082, -0.74933, -0.849332, 0.708823, -0.855582, -0.876774, -1.18641, -1.82177, -0.0135799, -0.0257591, 0.804103, 0.81483, -0.321535, -1.3046, -0.0352976, -0.391294, -0.604013, -1.78116, -0.306581, -0.0167415, -0.425574, -0.0121891, 0.29506, -1.50703, -1.61214, -2.03826, -0.0636998, -1.47485, -1.83073, -0.711581, -0.0311023, -0.767131, -1.54083, -2.02053, 0.453194, -1.44355, -1.96399, -1.94512, -1.33766, -0.0116114, -0.489074, -0.0262673, 0.252578, 0.0638659, 0.00959784, -0.000188016, -0.00869302, 0.181522, -0.362873, -0.00292504, -0.0159, 0.0512334, -0.149452, 0.162893, -0.86724, 0.000187425, -0.00693888, 0.161841, 0.133905, -0.216204, 0.321011, 0.0146138, -0.214735, 0.0497467, 0.120214, -0.0691283, 0.194902, -0.0331508, -0.151871, 0.14418, -0.319362, -0.018276, 0.0124281, 0.0664229, -0.000258879, 0.0618909, 0.152344, 0.230958, -0.0320051, 0.189556, 0.000349639, -0.557356, -0.363773, -0.0061136, -0.593932, -0.175581, -0.0221913, -2.88243, 0.0724779, 0.107383, -0.0289807, -0.0572297, 0.0198434, -0.00465933, -0.025522, -0.288012, 0.136022, 0.320086, 0.00266124, -0.20351, -1.03355, 0.113462, -0.140555, -0.243086, -0.435358, 0.102399, -0.257514, -0.0153046, -0.139641, -0.14665, -0.00130705, -0.404836, 0.0788569, -0.227036, -0.00117582, -0.0169989, 0.0205428, 0.0260861, -0.165752, -0.375951, -0.0489716, -0.00121668, -1.57915, -0.0884085, -0.522099, -0.68058, 0.0029316, -0.268256, -0.493076, 0.122594, 0.0700689, -0.478357, 0.181044, 0.0347366, 0.218663, -0.122337, 0.00237345, 0.0116622, -0.0118086, 6.8281e-05, -0.453037, -0.343801, -0.00493255, -0.134274, 0.138013, -0.144496, -0.217395, 0.00304276, -0.391118, -0.00861113, -1.85808, -0.368146, -0.22431, 0.0713864, -0.923859, -0.0789738, -0.101085, 0.267038, -0.00645888, 0.146995, -0.254764, 0.153617, -0.814765, -0.35566, -0.266108, -0.084357, -0.425905, -0.00727236, -2.21493, -0.0199583, 0.181683, 0.0490387, -0.00558668, 0.00520503, 0.0196382, 0.0308708, 0.0685717, 0.0264093, -0.011875, 0.0457944, -0.0717538, 0.0488216, 0.0416639, -0.000167242, 0.00273841, 0.026166, 0.0707024, -0.0281307, 0.162051, 0.0408285, -0.105161, 0.044091, 0.0321611, -0.00394501, 0.0699884, 0.0298488, 0.0599506, 0.0666873, 0.0475364, -0.0433384, -0.00823409, 0.062835, -0.00928163, 0.0411914, 0.0221951, 0.18114, -0.00123229, 0.0425296, 0.00499639, -0.0909656, -0.0026917, -0.00608294, 0.0860247, -0.0400905, -0.01079, 0.0632721, 0.0201913, 0.0212339, 0.0455891, -0.0593089, 0.0810534, 0.00104992, -0.0120154, 0.0474376, 0.0458606, 0.160924, -0.00152005, 0.0512515, -0.0520103, 0.0489348, -0.153415, 0.0587162, 0.0609555, 0.0429371, 0.0416115, -0.0149807, 0.0500335, 0.0546145, -0.00519181, 0.0464945, 0.0505837, 0.0333275, -0.00102467, -0.000645282, -0.010871, 0.0365837, 0.00448416, 0.0745137, 0.0298368, -0.0359663, 0.123549, 0.0684537, -0.108627, 0.17329, -0.0070107, 0.0633884, 0.0419395, 0.0405954, 0.0421254, -0.101081, 0.0610345, 0.0518211, 0.0684592, 0.0335019, 0.00224381, 0.00265106, -0.502833, 0.0692734, 0.0723552, 0.0489729, -0.0220129, -0.0477134, -0.0122656, 0.0644552, -0.168474, 0.00806482, -0.0107588, -0.00126669, 0.152502, 0.0410352, 0.0420028, 0.0537333, -0.106103, 0.05597, 0.022343, -0.0386778, -0.0131213, 0.052488, 0.0541138, 0.0460726, -0.103348, 0.0781422, 0.0349822, 0.0423342, 0.038685, -0.00148473, -0.06744, 0.00429026, 0.740662, -1.11053, 0.018411, 0.0137586, 0.014676, 0.182005, -0.87891, -1.21971, 0.00158455, -1.07693, 0.0669896, 0.095345, -0.399865, -0.0102336, -0.0139083, 0.183922, 0.203983, -0.780528, 0.824443, -1.10303, 0.019441, -0.387709, -0.330321, -0.0192162, -0.251583, -0.251146, -0.673974, -0.0147914, -0.307982, -0.0255399, 0.00723101, -0.0234516, -0.0075054, 0.0260638, -1.09896, 0.733015, -0.0297623, -0.821531, -0.000182067, 0.0119947, -1.15279, -0.00284637, -0.644595, -0.341675, -0.0266846, 0.367544, -0.0432163, 0.128562, -0.946286, -0.0398725, -0.698535, -0.00134479, -0.00617845, -0.961642, -0.0628541, 0.799384, -0.0127485, -0.349459, 0.101102, -1.13517, -1.30062, -0.871139, -0.126386, 0.00485881, -0.825717, -0.00355609, -0.929787, -0.264446, 0.0126584, -0.202346, -0.229853, -0.93355, -0.00966438, 0.00182564, -0.0136616, -0.0664501, -0.1558, -0.0277776, -0.717654, -0.00148199, 0.443061, -0.306503, -0.0205063, -1.26727, -0.02096, -0.497158, 0.0792991, 0.118809, -0.0593955, -0.0015991, -0.297386, -0.512297, -0.224138, -1.03079, -0.0118676, 0.0123423, -1.70909, -0.305225, 0.33639, -0.781636, 0.000589119, -0.0873598, 0.119013, -0.913062, 0.0338908, 0.00320947, -0.834735, -0.0213358, 0.15515, -0.107216, -0.300935, -1.10083, 0.168436, -0.49859, -0.874944, 0.0475059, 0.0218143, 0.140684, -0.835878, -1.11366, 0.496291, 0.0652182, -1.0997, -0.593645, -0.551369, -0.0133875, 0.415511, -0.00959798, 0.721985, -0.569152, 0.00142006, -0.000647954, 0.00320816, -0.0645958, 0.0594715, -0.0416241, 0.00793237, -0.0671131, 0.102035, -0.171184, 0.146375, -0.00511317, -0.0049228, -0.098642, -0.0677791, 0.725988, 0.722938, -0.994736, 0.350228, -0.211704, -0.31606, -0.0554758, 0.395804, -0.725333, 0.155833, 0.315667, 0.142072, -0.0501882, 0.00407032, -0.313772, 0.017883, -0.208886, -0.369803, 0.662617, 0.00337053, 0.639494, 0.0108795, 0.375658, 1.10183, 0.0101312, -1.17335, 0.0425568, -0.0152377, 0.609941, -0.935354, -0.278774, -0.378001, 0.0209098, 0.568893, -0.0156153, 0.00981835, -0.548463, -0.210057, 0.749392, -0.0148654, 0.28982, 0.339976, -0.181123, -0.570472, 0.254166, -0.0698896, -0.177931, -0.0464378, -0.0312673, -0.0839766, 0.525752, -0.00336, 0.105374, -0.00227823, -0.259902, -0.00698751, -0.0153746, -0.00448045, -0.22429, -0.383251, 0.286172, 0.215801, -0.0213537, 1.12929, 0.454094, 0.145492, -0.177535, -0.0159608, 0.0349907, -0.12788, -0.0452468, -0.277501, 0.693353, -0.348952, 0.297293, 0.0766051, -0.18105, 0.0567291, -0.011602, 0.207765, -0.343808, 1.6028, -0.501163, -0.00857521, 0.347847, -0.0273643, -0.236396, 0.265498, -0.010181, 0.260947, -0.0171026, 0.696552, 0.270642, 0.26657, -0.849867, 0.0933093, 0.477708, -0.243812, -1.141, 0.013215, -0.128892, -0.138025, -0.706959, 0.452306, 0.345614, -0.439585, 0.165978, -0.0718687, 0.00265406, 0.122106, -0.00422858, 0.56872, 0.0536112, 0.0116479, 0.0132023, -0.00820378, -0.0789361, 0.218549, -0.296977, 0.00664082, -0.198981, 0.753889, -0.126173, -0.347514, -0.0126567, -0.0044666, -0.0762397, -0.0747915, 0.582795, 0.545086, -0.326623, 0.000517107, -0.210086, -0.170974, -0.00228712, 0.0726561, -0.0736443, 0.427195, 0.58225, 0.0424928, -0.0491135, -0.00486054, -0.167542, -0.0313512, -0.211967, -0.066652, 0.493441, -0.0331023, -2.16238, -0.0177819, -0.0925852, -0.450187, -0.00364299, -0.585314, -0.686294, -0.0417073, 0.680935, -0.247213, -0.0930172, -0.187616, -0.0587375, 0.134942, -0.0126335, 0.00961285, -0.238798, -0.204144, 0.544898, 0.00194058, 0.563208, -0.412809, -0.178286, -0.597894, 0.286526, -0.419956, -0.245191, 0.411833, -0.012362, -0.241187, 0.087649, -0.0145036, -0.225888, -0.205143, -0.0583803, -0.0107636, -0.0075486, -0.0155622, -0.0897524, -0.606383, -0.0654277, -0.173507, -0.0232359, 0.658912, 0.137203, 0.157837, 1.11541, -0.00119649, 0.192363, -0.307305, -0.139331, -0.214746, 0.70545, -0.133005, -0.225698, 0.521183, -0.312413, 0.0114973, 0.0128439, -0.436709, 1.03805, 0.260584, -0.287011, 0.00334296, -0.142233, -0.0647478, -0.234225, 0.05076, 0.00922634, -1.4436, 0.0113843, 0.368377, 0.0708051, 0.262696, -0.465388, 0.476688, 0.231253, -0.258634, -0.424605, -0.0161256, -0.212606, 0.431602, -0.194632, 0.682806, 0.00759362, -0.3351, 0.241811, 0.439566, -0.0170905, 0.136294, -0.02014, -0.0085458, 0.00872457, -0.00305098, -0.00654823, -0.00483334, -0.0131857, -0.000664046, -0.0109114, -0.0145236, -0.0120883, -0.012829, -0.0110178, -0.00184577, -0.00778587, 0.00828313, -0.0300278, -0.00252002, -0.003236, 0.0147054, -0.00884516, 0.0121523, -0.0140978, -0.0117173, 0.0110087, -0.00853368, -0.0316537, -0.0178829, -0.0189621, 0.00818288, 0.00436192, -0.00903508, -0.00918324, 0.00126282, -0.0238307, -0.0137788, -0.0143326, -0.00147677, -0.00606568, 0.00544078, 0.0998469, -0.00923996, -0.0108188, 0.0217897, 0.0672272, 0.00425805, -0.0361722, -0.00198791, -3.34939e-05, -0.000450923, 0.0052986, -0.0455148, -0.0120263, -0.0124846, 0.00570972, -0.00216751, 0.0113489, -4.12337e-05, -0.000813469, 0.0688508, -0.0260701, 0.000421934, -0.000745289, -0.012336, -0.0124465, -0.00772058, -0.0264621, -0.0180847, -0.0109196, 0.0124849, -0.0159707, 0.000666672, -0.0175722, -0.00976958, -0.0124766, 0.00916171, -0.0177636, -0.00938371, 0.0101332, -0.0112035, 0.0126869, -0.038885, -0.0248875, 0.000884965, 0.01785, -0.00245273, -0.00860878, -0.0116699, -0.00988354, 0.000685767, -0.017908, -0.0165352, -0.0219558, -0.00605198, -0.0051506, -0.00152816, -0.0130524, -0.00716507, 0.0295305, -0.0508573, -0.0157692, -0.015605, 0.019295, -0.00612422, -0.0173429, 0.0194653, -0.00870117, 0.0107258, 0.0117518, -0.0120531, -0.00337286, 0.00210622, -0.00405953, 0.11022, -0.00793631, -0.00686717, -0.0121958, -0.00543234, -0.0015974, -0.0188292, -0.0115097, 0.00569438, -0.00847731, -0.0178796, -0.020949, -0.00435445, 0.014822, -0.0533528, -0.0168043, 0.0250203, 0.0537078, -0.00591638, 0.00343073, -0.00427355, 0.0670518, 0.0680138, 0.0658784, 0.0177284, 0.0569855, -0.044625, 0.0496176, 0.0547807, 0.00252455, 0.0135565, 0.073531, 0.0530101, -0.0604297, 0.0525372, 0.0673101, 0.139328, 0.0684363, 0.0653967, -0.0320557, 0.0722632, 0.0499172, 0.0689993, 0.0791424, 0.0813621, -0.0664863, -0.00899859, 0.0695066, 0.00650165, 0.0736748, 0.0503321, 0.0552286, -0.0131982, 0.0368435, -0.000204052, 0.051279, 0.00741537, -8.27721e-05, 0.0768162, -0.014128, -0.0231752, -0.141784, 0.0565823, 0.048762, 0.0433478, -0.0129298, 0.0757862, -0.0068575, -0.0337462, 0.0434582, 0.0555591, 0.0762475, -0.0176436, 0.0539059, 0.0249332, 0.0628297, -0.134938, 0.0511638, 0.0679807, 0.0573943, 0.0639424, -0.0262314, 0.0591654, 0.070443, 0.00814559, 0.0731558, 0.0618551, 0.0462317, -0.0164579, -0.00988766, 0.00448084, 0.0446103, 0.00111945, 0.0455791, 0.063058, -0.0114162, -0.213852, 0.0859494, 0.0445672, 0.0476702, 0.00864787, 0.0619454, 0.064254, 0.0556451, 0.0694866, -0.0529573, 0.0693911, 0.0557087, 0.0580469, 0.0583999, -0.00498345, -0.00563056, -0.089312, 0.0438822, 0.0689445, 0.0672413, -0.000537813, 0.0565412, 0.0477629, 0.0539132, 0.0806281, -0.00494866, 0.0283808, -0.0282604, -0.240233, 0.0414521, 0.0504794, 0.0658924, 0.034927, 0.0479123, 0.0796235, 0.0492936, 0.00842, 0.0521443, 0.071479, 0.0382085, -0.00613964, 0.0460696, 0.0520782, 0.0607146, 0.0527485, -0.00828114, -0.0120203, 0.0041837, 0.0158507, -0.000123716, 0.00534484, 0.000637534, -0.0302244, 0.261991, 0.239384, 0.066968, -0.0355195, 0.260813, -0.107254, 0.275124, 0.212405, -0.000742625, -0.00617918, 0.267534, 0.279584, 0.0898822, 0.0879799, 0.131174, 0.122214, 0.268567, 0.266911, -0.0653771, 0.287646, 0.144951, 0.190173, 0.243141, 0.201202, -0.0173356, -0.0145281, 0.26674, 0.011073, 0.263088, 0.0231966, 0.0480605, 0.0153758, -0.232114, -0.0149712, -0.0168612, -0.0781489, 0.00136476, 0.227184, -0.0207861, 0.02649, -0.182069, -0.0540913, 0.265945, 0.212223, -0.0578603, 0.144521, 0.00204775, -0.00573167, 0.181042, 0.284112, 0.0906427, 0.00714812, 0.183182, -0.00105966, 0.264109, -0.254609, 0.172126, 0.253715, 0.272346, 0.155962, 0.0282645, 0.244638, 0.203458, 0.0125228, 0.0943915, 0.208846, 0.0729388, -0.0215662, -0.0188271, 0.000705724, 0.277546, 0.00231716, 0.23449, 0.183316, -7.77516e-05, -0.255536, 0.22771, 0.137136, 0.128291, 0.00424911, 0.245495, 0.185119, 0.280842, 0.27458, -0.093456, 0.27985, -0.045097, 0.215918, 0.262731, 0.0153748, -0.0170751, -0.855924, 0.0564114, 0.134355, 0.192103, 0.0166985, 0.116805, 0.238871, 0.237293, 0.131852, -0.00708026, 0.196212, -0.0177389, -0.180917, 0.1899, 0.188019, 0.21296, -0.030318, 0.231563, 0.208703, 0.239504, 0.00586918, 0.27686, 0.195302, 0.225487, -0.0872156, 0.210412, 0.185656, 0.141968, 0.187102, -0.0147551, 0.0621979, -0.00644263, -0.0637643, -0.0551561, -0.0349968, 0.0062029, -0.00397977, 0.0379201, -0.00665992, -0.00569643, -0.00285714, -0.0279604, 0.886104, 0.0127304, 0.0287283, -0.0084172, -0.00358838, 0.00838785, 0.0272859, -1.00628, -0.0769594, -0.0257405, -0.0651303, 0.0239411, -0.00596424, -0.0105575, 0.0181264, -0.0116401, 0.007911, 0.0157955, 0.0279859, -0.101254, -0.00878158, -0.0146091, -0.0100074, 0.029447, -0.0484269, -0.0604044, -1.77617e-05, -0.0318739, -0.00883178, 0.155526, -0.094613, -0.022186, -0.418098, 0.835826, -0.00281335, -0.211237, -0.0452475, 0.0214976, -0.0072606, -0.034731, -0.0209066, -0.0149763, -0.0315752, -0.00166469, 0.0251472, -0.0732266, 0.0102388, 0.0319343, 0.220865, -0.0253768, 0.150295, 0.0247103, -0.00372591, 0.0214501, 0.036442, 0.00490774, -0.0137824, 0.0201469, -0.0136792, 0.0126584, 0.0315769, 0.0156845, -0.0206032, 0.000672367, 0.00404631, -0.0124283, -0.0102987, 0.0208408, 0.00211142, -0.0205342, -0.285241, 0.0242785, 0.0184368, 0.53308, -0.0131135, 0.0358273, 0.0261124, 0.0040967, 0.0157679, 0.587252, -0.00800814, -0.0669929, 0.00245438, -0.0162703, -0.0114755, -0.024106, 0.381098, 0.0690275, -0.0261833, -0.0167032, -0.0145772, 0.0570129, 0.0138462, -0.0208664, -0.0259515, -0.00728003, 0.119295, -0.0141533, -0.4483, 0.0327969, 0.00272758, 0.020065, 0.6783, 0.0246694, 0.0116693, -0.0119142, -0.000692221, 0.0423641, 0.0225888, 0.00453691, 0.448588, 0.0575489, -0.0245147, -0.0209342, 0.00280402, -0.0199511, 0.472327, -0.0148239, 0.234066, 0.120832, -0.0386221, -0.00923742, -0.00631137, 0.14918, -0.238772, -0.0659657, -0.0077179, 0.519705, -0.431765, 0.186046, -0.328691, -0.00595126, -0.0112338, 0.300746, 0.277842, 0.978414, 0.237588, 0.0432158, -0.187608, 0.400162, 0.4455, 0.0327045, 0.504664, -0.491917, -0.172705, -0.240562, -0.140353, -0.0852585, -0.00269178, 0.30897, -0.0236719, 0.374908, 0.0616351, 0.108736, -0.00965664, -0.891266, -0.00732925, 0.215365, 0.343417, -0.0275322, -0.53819, -0.133276, 0.00377591, 0.31542, -0.640758, 0.286886, 0.528006, -0.0694258, -0.288805, -0.0137608, -0.018034, -0.0935442, 0.384592, 0.242452, -0.0181453, -0.442253, 0.170378, 0.553327, -0.920349, -0.229386, 0.374008, 0.286664, -0.19678, -0.00703072, 0.499175, -0.34698, 0.00260788, -0.662744, 0.34194, -0.0692615, -0.00705071, 0.00452854, -0.00865462, 0.361778, -0.206192, -0.104895, 0.203047, -0.0260985, 0.0666357, -0.292581, 0.0518654, 0.707822, -0.0107024, 0.0428497, -0.207604, 0.321893, 0.390877, -0.518474, 0.359116, 0.0899711, -0.443084, 0.48939, -0.0142733, -0.00713069, -1.43179, 0.18037, -0.73589, -0.423822, -0.0263112, 0.0141754, 0.314869, 0.370299, -0.386985, -0.0249077, -0.349331, 0.00568589, 0.0754338, -0.165216, -0.230481, 0.0442705, 0.0174628, -0.370212, 0.500881, -0.105948, -0.0226583, 0.195041, -0.196049, 0.475109, 0.169558, -0.157308, -0.359634, -0.36912, -0.133156, -0.0153198, -1.11681, -0.0129374, -0.307436, -0.0771208, -0.0193396, -0.014769, -0.000912603, -0.254742, -0.339124, -0.143229, -0.000689689, -0.184418, 0.32042, -0.289158, -0.417579, -0.00660682, 0.00531984, -0.262198, -0.266091, -0.174374, -0.325538, -0.116214, 0.329854, -0.260757, -0.272471, -0.000124274, -0.423813, -0.309957, -0.506965, -0.872599, -0.307109, -0.05775, -0.0145817, -0.282335, 0.00297654, -0.262688, -0.0337941, -0.287538, -0.0131317, -0.127699, -0.00728102, 0.314514, -0.038148, -0.0128559, -0.235425, -1.94701, -0.00879518, -0.155639, -0.306084, -0.257573, -0.224324, -0.0405343, -0.467077, 0.00146518, -0.0164093, -0.10071, -0.241248, -0.310087, -0.0055216, -0.348928, -0.555019, -0.00335892, -0.304068, -0.427186, -0.191886, -0.266125, -0.401474, -0.0359364, -0.171518, -0.239184, -0.000484168, -0.46204, -0.271367, -0.336017, -0.0273976, -0.00259476, 0.00882292, -0.266774, -0.202796, -0.338335, -0.129172, -0.0246141, 0.000564678, -0.211979, -0.529875, 0.0904339, -0.0134065, -0.312935, -0.318324, -0.254443, -0.287237, 0.276575, -0.239322, -0.437884, -0.869323, -0.158105, -0.0184097, 0.00303164, -0.175653, 0.396341, -0.294613, -0.184679, -0.0123592, 0.336167, -0.285189, -0.121268, 0.275437, -0.0185057, 0.242416, -0.0170784, 0.198355, -0.242677, -0.201863, -0.0415991, 0.497256, -0.251263, -0.0708089, -0.273885, -0.0167746, -0.259445, -0.379301, -0.0714852, 0.639095, -0.325302, -0.0777462, -0.351867, -0.266184, -0.0118487, -0.765254, 0.00207432, -0.146766, -0.544181, -0.00431705, -0.0196126, -0.0395306, 0.246606, -0.32569, -0.288668, -0.0222399, -0.069486, 0.0397763, 0.358328, 0.531574, 0.0107761, -0.0060142, 0.242446, 0.198551, 0.527624, -0.175964, -0.554417, -0.0019014, 0.41019, 0.405272, 0.00339814, -0.0270375, 0.331669, -0.0211034, 0.210286, 0.198603, -0.0632743, -0.00450281, 0.298844, -0.0185594, 0.162845, -0.559501, -0.161411, -0.0189771, -0.366057, -0.012282, 0.255332, 0.271104, -0.0245712, -0.484591, 0.493833, 0.0121121, -0.196286, -0.361249, 0.17223, -0.281064, -0.0602005, -0.441444, -0.00416962, 0.0017998, -0.330315, 0.399318, -0.149044, -0.00859707, -0.0610013, 0.307891, -0.121007, -0.863801, -0.357147, -0.170406, 0.174805, -0.362433, -0.00458993, -0.391404, 0.0248912, 0.0142591, 0.371618, -0.105819, -0.189449, -0.0248763, -0.0169711, 0.00300114, 0.276272, -0.378314, 0.242118, -0.378204, -0.0172243, -0.186468, -0.0535917, 0.0885664, 0.400531, -0.0107416, -0.248655, 0.454724, 0.280025, 0.219572, 0.00522957, 0.577605, -0.110153, 0.308549, -0.33553, -0.0258348, 9.03373e-05, -0.602899, 0.294514, 0.415879, -0.27703, -0.027365, 0.201184, 0.353213, -0.370277, 0.156738, -0.00980577, -0.0254667, -0.0317588, 0.439319, 0.0447849, -0.231986, -0.536687, -0.428409, -0.0647388, -0.43839, -0.240767, 0.00403208, 0.423872, -0.326267, -0.325986, 0.14542, 0.271869, -0.326307, -0.0890383, -0.233411, -0.00691827, -0.194861, -0.00106439, -0.216348, -0.664349, -0.0409634, -0.00707177, 0.00423379, -0.787888, -0.195751, -0.407289, -0.00268655, -0.403922, -0.295383, -0.754725, 0.107087, -0.023123, 0.00296773, -0.799067, -0.765218, -1.34705, -0.137458, -0.704654, 0.823794, -0.672867, -0.605591, -0.024734, -0.88827, -0.238082, -0.289997, 0.180342, -0.119244, 0.0105105, 0.0177225, -0.698833, -0.0332097, -0.756874, -0.561387, -0.20362, -0.0411935, 0.261433, -0.00926122, 0.910347, 0.203897, -0.0189634, 0.124016, 0.378715, -0.0352269, 0.260996, -0.459584, -0.768573, -0.563284, -0.0403943, -0.0833247, 0.000990753, -0.0883892, -0.373482, -0.671879, -0.13831, -0.0202999, -0.199271, 1.02318, -0.562678, -0.0204703, 0.102087, -0.452539, -0.718701, -0.181888, 0.000132652, -0.564653, -0.211235, -0.0138638, 0.109843, -0.699919, -0.37889, -0.0231387, -0.014526, -0.0274408, -0.715606, 0.0316174, -0.106519, -0.120042, -0.0216726, 0.354668, -0.207271, 0.694412, -0.384766, -0.0194762, -0.240822, 0.0616453, -0.721124, -0.738325, 0.0305231, -0.619061, 0.582428, 0.414152, -0.619998, -0.01587, -0.0223166, 0.0802237, -0.243001, 0.299392, -0.40095, -0.0475888, 0.681588, -0.746328, -0.61421, 0.816815, -0.0279346, 0.630518, 0.0102154, 0.0407169, -0.00689087, -0.167016, -0.730142, 0.806519, 0.0394215, -0.659187, -0.886454, -0.0153638, -0.640426, -0.207709, -0.697096, 0.486078, -0.046477, -0.402842, -0.271871, -0.16324, -0.018506, 0.439414, -0.0196452, -0.690727, -0.338899, -0.0500498, -0.0127514, -0.0357524, 0.256621, -0.13237, -0.183515, -0.0110479, -0.0864884, -0.395778, 0.207793, 0.222623, -0.000400559, -0.0216813, 0.203876, 0.19428, 1.58979, -0.402261, -0.948603, -0.546041, 0.460096, 0.37551, -0.0694072, 0.272481, -0.605599, -0.436024, -0.477355, -0.0198671, -0.051743, -0.0220083, 0.287746, -0.0525146, 0.208377, -0.217076, -0.744717, -0.0669502, -1.5353, -0.0194153, 0.0980011, -0.0716174, -0.00528822, -0.630293, 0.118576, -0.00834788, -0.0343334, -0.861081, 0.241965, -0.23466, -0.0547851, 0.166151, -0.0237051, 0.0093812, -0.828519, 0.290392, -0.457122, -0.01357, 0.204576, -0.0948988, -0.0145451, 0.401077, 0.0293864, 0.173184, 0.203865, -0.107175, -0.0714259, -0.00542203, 0.127468, -0.00734157, 0.50704, 0.0977443, -0.0775998, -0.0231158, -0.00586125, -0.0489011, 0.249505, -0.950235, -0.0387471, -0.288259, -0.0236037, 0.0589165, 0.165351, -0.574457, 1.12965, -0.0137052, 0.0821073, -0.275218, 0.298538, 0.241336, 0.539189, 0.408881, -0.850303, -0.806788, -0.015879, -0.00354467, -0.0152839, 1.55344, 0.932755, -1.13627, -0.895973, -0.00381664, -0.460906, 0.199917, -0.0623132, -0.427662, -0.0303176, -0.245197, -0.0249776, 0.463627, -0.212367, -0.104368, -0.620756, -0.381437, -0.01127, -0.0415543, -0.718011, -0.0308555, 0.272144, -0.0767581, -0.486928, -0.752434, 0.0657652, -0.81382, -0.379245, -0.181578, -0.0232841, -0.954504, -0.0113626, 0.00851365, 0.0661601, 0.0051685, 0.000466439, -0.00865272, 0.0384698, 0.0619202, 0.0535343, -0.0193413, 0.0650945, -0.098184, 0.071926, 0.0481738, 0.00494872, -0.00370766, 0.0702855, 0.0575629, -0.0247658, -0.0204135, 0.0582652, -0.00659156, 0.0548361, 0.0586808, -0.0245599, 0.0366506, 0.042281, 0.0736256, 0.0649787, 0.0629545, -0.0686044, -0.015167, 0.0574427, -0.0140869, 0.0461157, 0.0576575, -0.00273279, -0.00379845, 0.0470867, 0.0064052, -0.0413624, -0.0185703, 0.00501218, 0.0278499, -0.0870725, -0.00342119, -0.107916, 0.0363855, 0.0608512, 0.0505351, -0.0513475, 0.0595889, 0.00766333, -0.0307177, 0.0633172, 0.0642469, 0.0166783, -0.00591115, 0.0581427, -0.0895135, 0.0532174, -0.146849, 0.0670316, 0.0570373, 0.0480115, 0.0496573, 0.00347143, 0.0413273, 0.0468539, -0.0088232, 0.067694, 0.0489634, 0.0618837, -0.0164216, 0.00120235, -0.00246161, 0.057721, 0.000838026, 0.0676186, 0.0504329, -0.02109, -0.270248, 0.0521347, -0.0421461, 0.0404617, 0.00431429, 0.0776005, 0.0638997, 0.0391366, 0.0819832, -0.0736169, 0.0538561, 0.0670153, 0.0709276, 0.0456219, -0.016085, 0.00437195, -0.237622, 0.0399573, 0.0634451, 0.0425196, -0.0314503, 0.00223623, 0.0366826, 0.0584282, -0.00720298, -0.0011809, -0.0141712, -0.007924, -0.255579, 0.0422537, 0.0623058, 0.0600279, -0.0500739, 0.0681991, 0.0460232, 0.0382535, 0.0236719, 0.0519111, 0.079656, 0.0398548, -0.0692589, 0.0572631, 0.0627132, 0.0642891, 0.0441254, -0.0110896, -0.0778041, -0.00106896, -0.0220061, -0.139367, -0.0377711, -0.0207056, 0.0208283, -0.141628, -0.140924, -0.0835199, 0.000255449, -0.183127, -0.0944727, -0.0818359, 0.448522, -0.0229387, -0.00452679, -0.261652, -0.18755, -1.11201, 0.170225, -0.315434, 0.615905, -0.387195, -0.476675, -0.041859, -0.131111, -0.528656, -0.225147, -0.471014, 0.0544461, 0.00260898, 0.012829, -0.153065, 0.00643539, -0.324444, -0.177639, -0.30822, -0.0108824, 0.617126, -0.0104486, -0.0181688, 0.0774581, -0.0142162, -0.782155, -0.239172, -0.00353492, 0.873855, -0.346107, -0.109615, -0.0419689, -0.0274518, -0.252643, -0.00435257, -0.0334366, -0.493782, -0.149119, 0.120077, -0.0117795, 0.0461158, -0.0980463, -0.177318, -0.403487, -0.217091, -0.174711, -0.165087, -0.0620022, -0.00714881, -0.0861522, -0.00736781, -0.00539506, -0.0858498, -0.374785, -0.0277784, -0.0141901, -0.00461334, -0.00714407, -0.227294, -0.245562, -0.0340651, 0.249253, 0.00853692, -0.235383, 0.0843923, 0.159372, -0.621318, -0.0204403, 0.0981276, 0.0217069, -0.123688, -0.315551, -0.101074, -0.392135, 0.291872, -0.305655, -0.223439, 0.00475993, -0.0239983, -0.817104, -0.907025, 0.32487, -0.561788, -0.0182758, 0.307268, -0.113371, -0.29888, 0.752692, -0.0143959, 0.120698, -0.0096431, 0.222974, -0.178076, 0.0379227, -0.184685, -0.171732, -0.164923, -0.211697, -0.423058, -0.012923, -0.00247967, -0.106291, -0.241732, -0.201788, 0.111231, -0.41873, -0.32868, -0.159625, -0.0150928, -0.784938, -0.00478688, 0.340998, -2.86364, -0.000630205, 0.00277336, 0.0163604, -2.6335, -2.20254, -2.94085, -0.00289631, -2.10223, -0.890534, -2.77468, -2.63126, -0.00818035, -0.0241146, -2.61456, -2.55758, 0.309957, 0.343406, -2.83371, -0.274545, -2.44469, -2.52273, -0.00731075, -2.06389, -3.65522, -2.56469, -2.76901, -2.12141, 0.00245386, -0.0152079, -2.42939, -0.0214135, -2.50139, -2.99469, 0.323957, -0.0340327, -0.785281, -0.00807409, -0.158595, -1.82483, 0.00383105, -0.666999, -0.607791, -0.0118706, 0.777577, -3.62942, -2.55944, -2.13503, -0.0224752, -2.93696, -0.0140608, 0.000969582, -3.03738, -2.62362, 0.330198, -0.00440691, -2.4288, -0.0470919, -2.13807, 0.50475, -2.49181, -1.76775, -2.65217, -2.50731, -0.013214, -1.98507, -2.19742, -0.0103503, -2.78199, -2.42274, -2.51995, -0.021606, 0.0120639, -0.0194798, -2.51306, 0.134079, -2.07446, -2.08332, -0.00799733, 0.773065, -2.22088, -0.230191, 1.51091, -0.0160056, -2.28835, -2.5597, -2.61511, -2.43236, -1.11641, -2.41491, -3.01103, -2.66591, -2.25001, 0.00780212, 0.0105931, -0.0747015, 1.39109, -4.29555, -3.11427, -0.0184891, -0.164547, -2.42597, -2.25568, -0.165305, 0.000593404, 0.285363, -0.00714198, 0.962283, -2.40559, -2.40314, -2.62501, -0.651059, -2.53637, -2.10904, -2.90276, -0.0189647, -2.54194, -2.35616, -2.37122, -0.7987, -2.09295, -3.23274, -2.94156, -2.50333, -0.0140401, -2.11029, -0.00451807, 0.0285809, 0.0258201, 0.00264559, -0.00205281, 0.00275982, 0.0588727, 0.0508475, 0.0433213, -0.0177723, 0.056338, -0.107702, 0.0333002, 0.021588, 0.00755367, -0.0235629, 0.0525931, 0.0341023, -0.177672, 0.0113624, 0.0377299, 6.52904e-05, 0.0598875, 0.0612202, -0.0907162, 0.0483344, 0.0382012, 0.0456637, 0.0494099, 0.0182617, -0.0400588, -0.00443569, 0.0705781, -0.00629486, 0.0480868, 0.0726803, 0.030984, -0.0176249, 0.0338846, -0.0160176, 0.0296524, -0.0539719, -0.0279112, 0.0633771, 0.0199796, 0.00066023, -0.153007, 0.0554601, 0.0604962, 0.0357969, -0.0342659, 0.0179029, 0.0157271, -0.0211157, 0.0378058, 0.0621913, 0.028074, -0.00488468, 0.00988662, -0.0763078, 0.0657858, -0.148077, 0.0166103, 0.0535689, 0.070957, 0.0150154, -0.00911439, 0.0347318, 0.0389316, -0.0142981, 0.0440842, 0.0400972, 0.0288095, -0.0139417, -0.00151422, 0.00229451, 0.0520857, -0.0431473, 0.0310628, 0.0249791, -0.0279332, -0.233648, 0.0242547, 0.0133392, 0.0184158, 0.0101066, 0.0165312, 0.0208131, 0.070746, 0.0571147, -0.120579, 0.0610804, 0.0550366, 0.0567887, 0.0535895, -0.00404654, -0.0220026, -0.0884952, -0.0031002, 0.00291126, 0.0379486, 0.010962, -0.0443902, 0.0273009, 0.054059, -0.0414517, -0.00287548, -0.0217453, -0.00568515, -0.205297, 0.0251148, 0.0163408, 0.0482011, 0.0381142, 0.0270327, 0.0659871, 0.050175, -0.00307909, 0.0386406, 0.0168372, 0.061654, -0.0891642, 0.0297504, 0.0384097, 0.0237396, 0.0211415, -0.0166979, 0.00475747, 0.0162424, -0.177232, -1.72921, -0.0113498, 0.00399137, -0.0270491, -2.21572, -1.14498, -1.37903, -0.0373825, -1.7505, -0.76366, -2.28649, -0.673157, -0.0357884, -0.00951407, -2.32219, -2.15531, 0.58803, -0.142308, -2.00843, 0.0422893, -1.75656, -1.98842, -0.049416, -1.01762, -1.90786, -0.974121, -1.25412, -1.04013, -0.0360262, -0.023993, -1.63741, -0.0344246, -1.84644, -1.7681, -0.274004, -0.0376782, -0.0122475, 0.00385202, -0.0541992, -0.287471, -0.0252487, -1.14401, -0.168273, -0.0311977, 0.213087, -1.83388, -1.98861, -1.78894, -0.0386767, -0.660445, -0.0188397, -0.0566347, -1.58399, -1.85076, -0.118549, -0.0166244, -1.06209, 0.00559875, -1.79449, 0.236496, -1.47345, -1.57558, -1.8981, -1.11613, -0.0328279, -1.52603, -1.02247, -0.0142949, -0.129195, -2.19132, -1.11286, -0.0421071, -0.0258617, -0.0200624, -1.80433, -0.48691, -1.00449, -1.53607, -0.0146266, 0.258804, -1.04999, -0.0527897, 0.327899, -0.021214, -1.44676, -0.73673, -1.98072, -1.83618, -1.10101, -1.96054, -1.12627, -1.09793, -1.76861, -0.0635619, -0.0109235, 0.920831, 0.346603, -0.871937, -1.78052, -0.0135133, -0.0367649, -1.75618, -1.71011, -0.0483251, -0.00726381, 0.499452, -0.0232005, 0.482312, -1.07162, -1.17097, -2.3926, 0.0482904, -0.968907, -1.74928, -2.72957, -0.0250305, -1.98884, -1.47027, -2.14424, -0.265235, -0.86361, -1.82834, -0.712839, -1.41405, 0.000995695, -1.84083, -0.0143275, -0.168407, 0.211089, -0.0191493, -0.00196681, -0.0181969, 0.27389, -0.0804713, 0.256632, -0.00595737, -0.16173, -0.697699, 0.249855, 0.29258, 0.00303114, 0.006505, 0.214756, 0.144855, 0.749237, -0.146243, 0.477602, -0.128983, 0.0455221, 0.0934702, -0.0505619, -0.552898, 0.606151, -0.29358, -0.115534, -0.182355, -0.0425846, -0.0145753, 0.02128, -0.0253143, -0.0125514, 0.119889, -0.198483, -0.018545, -0.296568, 0.00291325, -0.208945, 0.822336, -0.00538336, 0.0219928, 0.778074, -0.00194167, -0.859076, 0.51334, 0.20031, -0.149762, -0.0177369, -0.318336, -0.00927626, -0.0298619, 0.265899, 0.139729, -0.142697, -0.00837678, -0.276912, -0.272236, -0.221784, -0.399545, -0.417883, -0.417874, 0.142644, -0.312119, -0.00774459, -0.330621, -0.29722, -0.00329231, 0.695905, 0.206239, 0.112787, -0.0212263, -0.0159944, 0.00751247, 0.1111, -0.327238, -0.154645, 0.00342641, 0.000452387, -0.650201, -0.313601, -0.438319, -0.137704, 0.0279975, -0.261092, 0.610395, 0.136463, 0.0645181, -0.465158, 0.0959906, -0.107601, 0.0851697, -0.443322, -0.00843101, 0.0107014, -0.371586, -0.777213, 0.719787, 0.302414, 0.0149958, 0.0741938, 0.120792, -0.305716, -0.00738602, -0.018997, 0.415319, -0.0297871, 0.128263, -0.148539, -0.00584297, 0.234912, -0.463188, -0.220302, -0.246709, 0.444663, -0.0237194, 0.20678, -0.29819, -0.00413149, 0.617567, -0.153585, 0.542524, -0.14993, -0.173705, -0.0339714, 0.657073, -0.0106026, -0.0732739, 0.0854894, 0.00632102, 0.00512384, 0.0223178, 0.0934395, 0.0935793, 0.0939835, 0.00667185, 0.0991478, -0.0754688, 0.0709432, 0.093879, 0.00267227, 0.000123907, 0.0876277, 0.0884734, -0.00638059, -0.0169955, 0.0936979, 0.11404, 0.0927808, 0.0788473, -0.0930579, 0.099318, 0.0848537, 0.103089, 0.0869506, 0.0775394, -0.0246253, -0.0146948, 0.0911024, -0.0176808, 0.0880609, 0.0767701, -0.0463361, -0.0122101, 0.0743346, 0.00683783, 0.177849, -0.0138218, 0.00465903, -0.096291, 0.103316, -0.0079795, -0.159203, 0.0917223, 0.109241, 0.0842463, -0.0284084, 0.0938011, 0.00152017, -0.035674, 0.0790833, 0.0989136, -0.00772168, 0.0123687, 0.0897631, -0.044407, 0.0932781, -0.0744141, 0.0767904, 0.0840131, 0.104217, 0.0882229, -0.0268199, 0.096999, 0.0947329, 0.000238579, 0.0848269, 0.103414, 0.0810878, -0.000369369, -0.00443565, -0.00905612, 0.101874, 0.00810743, 0.090278, 0.076369, -0.0101013, -0.415844, 0.0927877, 0.176592, 0.0688096, -0.0064042, 0.128438, 0.0915091, 0.0861759, 0.100255, -0.0433795, 0.0976569, 0.0778234, 0.0888878, 0.0814047, -0.00751241, -0.00352095, -0.183201, 0.100136, 0.0938212, 0.0974336, -0.00647231, 0.125045, 0.0601795, 0.088441, 0.0585021, -0.0184223, 0.0570796, -5.77433e-05, -0.471825, 0.0843215, 0.0875114, 0.0762419, 0.238588, 0.0990066, 0.09311, 0.0813572, -0.000845391, 0.0786434, 0.074998, 0.0917048, -0.0327433, 0.0734462, 0.0829974, 0.0876523, 0.0836231, -0.00476221, -0.0415958, -0.015683, 0.0415709, 0.0552555, 0.0239263, -0.00559187, 0.0380314, 0.0739875, 0.0810248, 0.0795971, 0.00858401, 0.0861275, -0.0391645, 0.0737089, 0.0680664, -0.0182504, -0.00510982, 0.0829701, 0.0784463, 0.0101581, 0.0543512, 0.0678283, 0.089869, 0.0732473, 0.0608221, -0.0615216, 0.0790069, 0.0667765, 0.0784473, 0.0626322, 0.0849427, -0.0461016, 0.00573538, 0.0927037, -0.01351, 0.0635124, 0.0615856, 0.025777, 0.00943273, 0.0896072, 0.0110403, -0.0184888, -0.0365727, 0.0148353, 0.0417301, -0.0678244, 0.00314457, -0.195943, 0.0841939, 0.100188, 0.0653286, -0.0251343, 0.0955971, -0.0089595, -0.0261749, 0.0510021, 0.0766059, 0.0752773, -0.00780648, 0.0771407, -0.11216, 0.0848543, -0.107073, 0.0811097, 0.0806609, 0.0758178, 0.075505, -0.0100479, 0.077322, 0.0727032, -0.00296243, 0.0834039, 0.0874183, 0.0822301, -0.00376463, -0.0223435, -0.00562872, 0.0941536, -0.00915171, 0.104576, 0.0684974, -0.0233024, -0.405972, 0.0702063, -0.0260611, 0.00554432, -0.0111683, 0.0880776, 0.0670187, 0.0721435, 0.0882114, -0.0259411, 0.0742976, 0.0482347, 0.0874273, 0.0831455, -0.0256152, 0.0043006, -0.230908, 0.0302275, 0.0730996, 0.0617867, -0.00396986, 0.0189409, 0.0644034, 0.0812253, 0.0342937, -0.00408608, -0.0281866, -0.0204793, -0.440131, 0.0774087, 0.076768, 0.0852069, -0.0233736, 0.0906997, 0.101146, 0.071895, -0.00768844, 0.0931224, 0.0462217, 0.0857365, -0.0388885, 0.0618492, 0.0653924, 0.0580994, 0.0970503, 0.00430713, -0.0665177, 0.00863819, 0.0511905, -3.05546, -0.0399265, -0.0305194, -0.00483999, -2.41906, -2.24145, -2.62641, 0.00785884, -2.04948, -0.80239, -2.54223, -1.42814, -0.00460683, 0.00187663, -2.46289, -2.51732, 0.523884, -0.0759337, -3.21641, 0.210927, -2.0836, -2.25621, -0.0781721, -2.04769, -2.59015, -2.26051, -2.35674, -1.66583, -0.00549813, -0.0334101, -2.36149, -0.0245061, -2.35451, -2.96638, 0.0261326, -0.0183668, -0.473364, -0.0140516, 0.257587, -0.749045, -0.0168012, -1.78683, 0.223579, -0.0254788, 0.378348, -3.04074, -2.58573, -1.9947, -0.0848182, -2.2369, -0.0207893, -0.0163944, -2.51018, -2.37745, -0.0653961, -0.0293928, -1.91147, 0.144976, -2.07405, 0.590618, -2.12866, -1.63953, -2.5296, -1.98552, -0.0242116, -1.82066, -1.62051, -0.022, -2.81622, -2.114, -2.45056, -0.0173176, -0.00891369, -0.0411854, -2.37835, 0.381893, -1.4365, -2.12553, -0.0119659, 0.511235, -1.5225, 0.0572732, 0.259242, 0.00558, -1.7815, -1.83894, -2.47698, -2.34568, -1.33547, -2.11612, -0.384948, -2.14803, -1.9674, -0.0152818, -0.0248095, 0.0762433, 0.550196, -1.80012, -2.47023, -0.0274558, 0.26811, -2.44509, -1.98931, 0.142167, -0.0336597, 0.27187, -0.02295, 0.439178, -1.59506, -1.75166, -3.10894, -0.220518, -1.732, -1.97232, -2.90339, -0.00396704, -2.35718, -2.15263, -2.58002, -0.523048, -1.36055, -2.5787, -2.05722, -1.98634, -0.0184691, -2.14287, -0.0213248, -0.150098, 0.193138, 0.00691132, -0.00751269, -0.100223, 0.111371, -0.0101527, 0.174809, 0.0141784, 0.246581, 0.0504061, 0.182722, 0.145031, -4.1995e-05, -0.000141344, 0.218117, 0.214211, 0.711623, -0.200684, -0.00972536, 0.15301, 0.657833, 0.617669, -0.0223872, 0.253042, 0.187529, 0.0281126, 0.141508, 0.242733, -0.0737053, 0.00322348, 0.147674, 0.00185957, 0.336743, 0.219854, -0.0657288, -0.0147488, 0.174013, -0.0198179, -0.21121, 0.154641, 0.00036025, -0.225999, -0.121633, -0.00286315, -0.33234, -0.178928, 0.182149, 0.323047, -0.0199096, 0.01806, -0.0100106, 0.000234868, 0.0302094, 0.204685, -0.201334, -0.0122302, 0.0515873, -0.111986, 0.425872, 0.429916, -0.0693739, 0.185642, 0.247547, -0.0119929, 0.00151285, 0.340794, 0.140552, 0.00247369, 0.207868, 0.260624, -0.00477275, -0.0251263, 0.0020914, -0.000975462, 0.190385, 0.405519, 0.00413369, 0.163947, -0.00441754, -0.320842, 0.128896, 0.0654922, 0.528693, -0.00552686, 0.0203851, 0.106516, 0.226094, 0.218913, -0.381463, 0.728024, 0.0156814, 0.143094, 0.369184, -0.0155427, 0.00586298, -0.0366008, 0.704445, -0.375995, 0.033303, 0.00392418, -0.0828387, 0.131675, 0.326173, 0.31719, -0.0114696, -0.215219, -0.0171645, -0.253311, 0.0684242, 0.112065, 0.185184, -0.0342643, 0.0484068, 0.322525, -0.111047, -0.0115336, 0.190453, -0.000346874, 0.260767, -0.406661, 0.113249, -0.179302, 0.0297945, 0.102394, -0.016341, -0.301886, -0.00574994, 0.0112415, 0.206357, -0.00882805, -0.00239652, 0.00668713, 0.175474, -0.0338999, 0.0813137, -0.000176377, 0.253859, -1.26873, 0.147604, -0.485195, -0.00497526, -0.00909014, 0.165645, 0.184676, -3.87636, 0.106739, 0.298399, 0.446276, 0.110174, 0.117704, -0.0637185, 0.265459, -0.243983, -0.513352, -0.668932, -0.325055, -0.0253866, 0.00240356, 0.112879, -0.0124138, 0.123649, 0.430692, -0.0549556, -0.0248926, 1.70808, -0.00357917, 0.171558, 0.213151, 0.00110345, -0.709523, 0.555702, -0.0115487, -0.000689683, 0.0369975, 0.16896, 0.33821, -0.0252992, -0.513805, -0.0119609, 0.00420445, -0.129726, 0.0980405, 0.102458, 0.00711839, -0.453395, 0.199372, 0.332095, -1.27682, -0.133913, 0.108856, 0.145561, -0.37864, 0.0107383, 0.168287, -0.453101, -0.0325283, -1.12905, 0.218791, -0.149211, -0.0269464, -0.0113557, 0.00246108, 0.0981968, -0.237179, -0.30804, 0.299559, 0.00651566, 0.0530663, -0.434055, 0.0148526, -2.3635, 0.0126004, -0.190568, -0.762672, 0.1349, 0.121668, -1.06547, 0.134966, 0.464618, -0.550365, 0.21451, -0.0200363, -0.0303875, -0.742012, -2.73834, -0.183787, -0.18873, -0.0132786, 0.306913, 0.06113, 0.199279, 0.421445, -0.0245445, 0.484191, -0.024839, 0.0190709, -0.309699, -0.54586, 0.37753, -0.226197, -0.443894, 0.197418, 0.323941, -0.0673786, 0.0345272, -0.276548, 0.405206, 0.36187, -0.361516, -0.115117, -0.765686, -0.234307, -0.0209155, -0.451014, 0.00809607, 0.0202412, -0.0447432, -0.00587385, 0.00952409, 0.0179825, 0.0158443, 0.0341391, -0.0270529, -0.0113063, 0.0113997, -0.118698, 0.0190637, 0.0375901, -0.00568672, 0.0109114, 0.0246266, -0.000835935, -0.0255548, 0.0375938, 0.0252829, 0.0240981, 0.0175623, 0.00426005, -0.0656887, 0.0255005, 0.0113773, 0.0127178, 0.0300327, 0.0145237, -0.0290861, -0.0163426, 0.0258985, -0.0196108, 0.0139501, -0.00356628, 0.00959795, -0.00248454, -0.0186138, -0.0198356, -0.0583685, -0.120151, -0.00289088, 0.00765768, -0.136902, 0.0245091, -0.169649, 0.0196659, 0.0279741, 0.00379676, -0.0342553, -0.006127, 0.0131554, -0.00527619, 0.00784904, 0.0298987, 0.0559364, -0.00861318, 0.0137015, -0.122649, 0.0202619, -0.136275, 0.0188009, 0.0197915, 0.0415507, -0.00227057, 0.00452791, 0.0291203, 0.0143666, 0.0131425, 0.00515904, 0.0179104, 0.0290417, -0.00245504, -0.00269655, 0.0118442, 0.0422145, -0.00505442, 0.0275793, 0.00408941, -0.028181, -0.239735, 0.0130038, -0.0535081, 0.0392098, -0.0131214, 0.0290297, 0.0117369, 0.0199364, 0.0332948, -0.126233, 0.0252119, -0.0222435, 0.0329027, 0.000172047, -0.0315914, -0.0221961, -0.00934114, 0.00908163, 0.00156781, -0.00435583, 0.0012945, -0.0195648, 0.0192727, 0.0383021, -2.7058e-05, 0.0125496, -0.0572092, -0.0109824, -0.227694, -0.00507762, 0.015868, 0.0228715, -0.10822, 0.0144762, 0.0192657, -0.00405657, -0.00756947, 0.0272553, 0.0186794, 0.0194473, -0.0902406, 0.0205818, 0.0247121, 0.00574199, 0.0040654, -0.00609919, -0.0671006, 0.0025966, 0.260963, 0.728038, 0.00924413, 0.00182566, -0.0176506, 0.771121, 0.262759, 1.03367, 0.0180137, 0.846374, -0.798841, 0.65728, -0.386307, -0.0300327, 0.0164334, 0.710029, 0.77242, -0.537253, 0.247475, 0.127011, 0.0244754, 0.628988, 0.726514, -0.0163017, 0.862922, 0.715204, 0.385217, 0.0116495, 0.06902, -0.024078, 0.00410617, 0.666451, -0.0125451, 0.669649, 0.60662, 0.263106, -0.0209879, 0.73869, -0.00681326, 0.240999, 1.125, -0.00891987, 0.112136, -0.123224, -0.0101775, -0.0264427, -0.0155458, 0.668377, 0.645039, -0.0319531, 0.379481, -0.0250065, -0.0247796, 0.839246, 0.720603, 0.243754, 0.0118117, 0.195229, 0.267749, 0.770837, -0.2822, 0.0203437, 0.651957, 0.710321, 0.173618, 0.0111815, 0.622773, 0.260424, -0.00347555, 0.169544, 0.72651, 0.9489, -0.0097221, 0.010006, 0.0152866, 0.741875, -0.42221, 0.0881125, 0.489049, -0.0121205, -0.0464467, 0.252434, -0.0707599, -0.0330148, -0.00823352, 0.164305, -0.115753, 0.799372, 0.747279, -0.121341, 0.663963, 1.01701, 0.0721649, 0.756507, -0.00787674, -0.00771778, -0.0921984, -0.420763, 1.37853, 0.87931, -0.0202258, -0.236307, 0.737614, 0.664548, 0.0422301, -0.0117623, 0.0937493, -0.0129755, 0.0362046, -0.0114533, 0.313091, 0.559163, 0.468238, 0.208265, 0.825027, 0.0436227, -0.0129115, 0.664148, 0.151481, 0.402699, 0.498595, 0.100251, 0.827457, 0.44454, 0.154346, 0.00432929, 0.25666, 0.010428, 0.167356, 0.459229, 0.0202016, -0.0179161, -0.0775719, 0.4991, 0.118341, 0.378571, -0.0214, 0.53698, -0.772037, 0.496773, 0.313628, -0.0337858, -0.014872, 0.505073, 0.505177, -2.24871, 0.24741, 0.38515, 0.825734, 0.400141, 0.428784, -0.0410415, 0.599662, 0.175587, -0.142409, 0.0259768, 0.0631201, -0.0149142, -0.0115049, 0.473476, -0.0388038, 0.427114, 0.479422, -0.0634533, -0.0426148, 0.62041, -0.00545873, 0.457509, 0.377479, -0.0115777, -0.314901, 0.489077, -0.0230551, 0.540645, 0.247676, 0.453168, 0.496819, -0.0130464, 0.088997, -0.00983292, -0.0367505, 0.130913, 0.447598, 0.383429, -0.045326, -0.00339286, 0.560985, 0.530812, -0.303799, 0.103558, 0.400806, 0.48373, 0.0113203, -0.0463066, 0.338384, -0.0300647, -0.0302394, -0.00507811, 0.428058, 0.163015, -0.0157896, -0.00368966, -0.000948165, 0.438109, -0.374814, 0.0809403, 0.558754, -0.029157, 0.481188, -0.0371935, 0.368986, 0.503605, -0.0356887, 0.0746929, 0.157886, 0.519763, 0.410753, 0.535958, 0.41996, 0.199625, 0.0696619, 0.330602, -0.0384317, 0.00223665, -0.29821, 0.300826, -0.274761, 0.0212693, -0.00828203, 0.354239, 0.482576, 0.324568, 0.881172, 0.00231139, 0.676378, -0.00300497, 0.54433, 0.10145, 0.0384901, 0.460494, 0.116053, 0.107531, 0.340985, 0.379849, -0.0131202, 0.523542, 0.00456457, 0.539818, -0.193139, 0.115041, 0.179781, -0.114071, -0.0383791, -0.0175689, 0.283926, 0.00721617, -0.0766283, 0.282648, -0.0125379, -0.0235963, -0.0206822, -0.130997, 0.133163, 0.430838, -0.0102292, -0.441041, 0.203765, -0.110861, 0.393233, -0.0143003, 0.00111842, -0.0808191, -0.287478, 0.857685, 0.0840831, 0.382636, -0.501051, -0.234722, -0.106456, 0.0471259, 0.11823, 0.513837, 0.181355, -1.09321, 0.17062, -0.0663136, -0.000962959, -0.291303, 0.0119578, -0.305924, 0.480818, -0.158307, -0.0372998, -0.933455, -0.0101498, -0.298549, -1.11927, -0.0256974, 0.742211, 0.318758, 0.00342613, -0.281044, 0.328256, -0.279681, -0.558171, -0.0640866, 0.147139, -0.0158999, -0.0124089, 0.315231, -0.206843, 0.0798416, -0.0104991, 0.00412897, -0.0746526, -0.323776, -0.350483, -0.138921, 0.0572118, -0.214787, 0.142773, -0.0149049, -0.461111, 0.00893576, -0.0132097, 0.501921, -0.0874476, 0.458924, 0.00353518, -0.00865037, 0.000757078, -0.194326, -0.482935, 0.310478, 0.10877, -0.0060047, 0.0923978, -0.0831318, 0.054148, 1.53491, -0.027397, 0.087448, -0.0576843, -0.104547, -0.116483, 0.550365, -0.0949983, 0.0639244, -1.04718, -0.550593, -0.00500055, 1.39592e-05, -0.206519, 1.13049, -0.876787, 0.371997, -0.0121634, -0.132326, -0.0260951, -0.517821, -0.584422, 0.00859712, 0.21565, -0.0104569, 0.251313, 0.132098, 0.0411391, 0.374744, -0.00153593, -0.410228, -0.214951, -0.586642, -0.0305997, 0.0105695, 0.224417, -0.25906, -0.30678, 0.202248, 0.479598, 0.238395, 0.346562, -0.00256897, -0.516475, -0.0126411, -0.0842179, -0.00142177, 0.0150632, 0.00201792, 0.0274879, -0.00858321, 0.0102222, 0.00991288, -0.00186822, 0.0293435, -0.135185, 0.0183971, 0.00907614, -0.0130305, 0.006393, 0.0131286, 0.0111495, -0.037337, -0.0437265, 0.00923899, -0.0309433, 0.0157217, 0.00439962, -0.0572957, 0.0126534, 0.000228886, 0.0104419, -6.45475e-05, 0.0368064, -0.0530097, -0.0162405, 0.0275296, -0.00991179, 0.00604078, 0.0146465, -0.0686315, -0.00566267, 0.0213629, -0.00115355, -0.140986, -0.0776172, -0.00772363, 0.0317151, -0.155985, -0.0164403, -0.21882, 0.0285429, -0.016139, 0.00464995, -0.02517, 0.0116445, 0.0103766, -0.0100738, 0.0100662, 0.0126882, -0.0359038, -0.00872282, 0.0266163, -0.182419, 0.0196212, -0.160435, 0.0187491, 8.90274e-05, -0.000882549, 0.0106302, 0.000749158, -0.000921383, 0.017939, 0.0207656, 0.0132954, 0.0192262, 0.0104425, -0.0135985, -0.00914494, -0.00719684, 0.0202904, -0.00669951, 0.0168274, 0.0317452, 0.00798785, -0.392506, 0.0212422, -0.0983307, -0.0635404, -0.00310032, 0.0320152, -0.00239551, 0.0104726, 0.0177576, -0.14012, 0.0155526, 0.0199923, 0.00934052, 0.00902458, -0.0251014, -0.00710313, -0.210604, -0.121586, 0.00596415, -0.00340511, -0.00494869, -0.079091, 0.00366885, 0.00104713, -0.0339991, 0.00119383, -0.0348948, -0.0101442, -0.436432, 0.00486614, 0.0187094, 0.0170542, -0.147526, 0.0117832, 0.0162055, 0.0131784, -0.0187231, -0.00405635, 0.0197186, 0.00176987, -0.124342, 0.0233791, 0.00996719, 0.00285986, 0.0127648, -0.0025571, -0.0998304, -0.00531199, 0.432122, 0.108882, -0.0568465, -0.00999963, -0.0494982, 0.236383, -0.0337805, 0.0628214, -0.0388301, 0.185669, 0.153363, 0.256367, 0.139927, -0.0407508, -0.0323636, 0.230702, 0.231266, -1.73346, 0.416654, 0.139376, 0.630473, 0.247332, 0.289662, -0.0889144, 0.295853, 0.253144, -0.0951726, 0.195965, -0.0790605, -0.0855358, -0.0297462, 0.253752, -0.046325, 0.18802, 0.159592, 0.386543, -0.0727497, 0.125653, -0.0253856, 0.925287, 0.00868864, -0.0235743, -0.050461, 0.286056, -0.0513345, 0.744353, 0.320231, 0.191283, 0.121841, -0.0515004, 0.0618979, -0.0158843, -0.0418992, -0.159302, 0.290653, 0.378794, -0.0304795, -0.101274, 0.933988, 0.117962, -0.54927, -0.0103143, 0.038991, 0.25171, -0.125702, -0.0129689, -0.012027, -0.103748, -0.0385327, 0.0645083, 0.255055, -0.136097, -0.0551774, -0.0272397, -0.0367469, 0.264531, -0.125079, -0.109565, 0.312441, -0.0215063, 0.780457, -0.121695, 0.773857, 1.03326, -0.0357848, -0.145869, 0.0246777, 0.252717, 0.213218, 0.449534, 0.314831, 0.374528, 0.340802, -0.0806404, -0.0407165, -0.0403539, -0.485551, 1.14347, 0.102079, -0.187594, -0.047138, 0.700898, 0.233736, -0.0724486, 0.694952, -0.0166441, 0.732312, -0.0370275, 0.65626, -0.0960755, -0.0734233, 0.127648, 0.773611, 0.107424, -0.0153836, 0.28793, -0.0273244, 0.315712, -0.160727, 0.143336, 0.398014, -0.0381465, -0.102504, -0.0703993, -0.113377, -0.0285904, 0.13518, -0.0248722, -0.103541, -0.299246, 0.00350251, -0.0110656, -0.00793483, -0.446911, 0.0775723, -0.150155, 0.0245735, -0.17807, 1.20659, -0.41418, -0.373046, -0.00711255, 0.0240314, -0.401525, -0.442206, 2.47266, -0.128292, -0.486173, 0.914351, -0.15863, -0.264036, -0.0581417, -0.523299, -0.389171, 0.000922692, 0.654224, -0.00071877, -0.116556, -0.015144, -0.306149, 0.00851564, -0.248606, -0.397648, -0.101709, -0.0023795, -1.12496, -0.0108654, 0.928901, -0.127725, -0.0259279, 0.742438, 0.971909, -0.00857677, -0.389365, -0.483713, -0.386684, -0.164987, 0.0242673, 0.299738, 0.015763, 0.0432367, -0.14662, -0.337894, -0.139096, 0.00572746, -0.00734224, 1.0722, -0.26712, -2.30817, 0.440493, -0.283645, -0.354544, 0.00205362, -0.00884918, -0.0763604, -0.00883119, -0.0151454, -0.733733, -0.216056, -0.0671854, 0.0161898, -0.0128307, -0.00360193, -0.243405, -1.08914, 0.0176388, -0.172389, 0.0239535, -0.481444, -0.00589907, 0.583242, -2.11363, 0.00526056, -0.0138606, -0.541873, -0.345225, -0.266894, 0.820784, -0.376077, -0.395827, 0.744223, -0.199806, -0.00386485, -0.0205885, -3.2527, -2.51957, -1.05286, -0.11187, -0.00980646, 0.526519, -0.283504, -0.131796, 0.859048, -0.0148449, 0.9786, -0.00759394, -0.496791, -0.00900483, -0.0161583, -0.48503, 0.872974, 0.379278, -0.137452, -0.686502, -0.00385027, -0.371077, -0.00443822, -0.336086, 0.969407, -0.0220737, -0.102834, 0.0448488, -0.00787734, -0.0139818, -1.64969, -0.0141274, -0.649712};
policyNetwork.Parameters() = {-1.12126, -0.885592, -1.22718, -0.317446, -1.32801, -0.580976, 0.0767691, 0.00555064, -0.0122336, 0.163829, -0.0456355, -0.0255867, -0.0328012, 0.00551459, -2.01826, -1.06721, -0.0173299, 0.83304, 0.202024, -0.500576, -0.00304098, -0.0120421, -0.0506905, -0.0260477, -1.06481, 0.00174265, 0.240928, 0.000724373, -1.45052, 1.30338, 0.0457176, 0.0291735, 0.598555, 0.136851, 0.499379, -0.736964, -0.0694305, -0.00225736, 1.13612, -0.0174587, -0.0130857, 0.21689, 0.674259, -1.31215, 0.245803, -0.94721, -0.607024, -1.59045, 0.760382, 0.744703, 0.771836, 0.587837, 0.0204749, -0.810577, 0.0101868, -0.61228, 0.184054, 0.695286, 0.013766, 0.00535832, -1.20768, 0.148365, -0.303299, 0.808067, 0.823746, -0.0019849, 0.0279802, -0.230417, 0.236567, 0.00253842, -1.6755, 0.300642, 0.0207048, -0.0429176, 2.29518, -0.402168, 0.641183, -0.814602, 0.843145, -0.710994, 1.64317, 0.414384, 2.46557, -0.0169695, -1.67959, 0.450845, 2.06375, 0.150318, 0.00252751, 1.07443, 0.0034509, -1.69262, 0.303838, 0.00211267, 0.287937, -0.227662, -1.65503, 0.00874001, 0.553132, -0.00558256, -1.61189, -0.982398, -0.723132, 0.00472583, -1.06374, -0.0201481, 0.557627, 0.0345595, 1.14717, -0.0874391, 0.146047, -0.091972, -0.676641, -1.73215, 0.215228, 0.247936, -1.68997, -1.00018, -1.18612, -0.377934, -0.612678, 0.00692408, -0.0195093, -0.00865354, 0.629515, -0.0229282, 0.0078825, 0.728852, -1.9078, 3.52101, -3.94227, -0.183029, -6.16484, -0.910147, -0.676075, 0.0584649, 0.0210332, -1.52073, -0.0870047, 0.0232032, -0.0291334, -0.00800313, -5.04651, 0.931583, 0.233554, 0.125216, -2.61156, 1.35944, 0.0928147, 0.0757604, 0.022339, 0.0175167, -2.94234, 0.0802583, 0.216556, 0.0292728, -4.42873, 2.51003, -0.0581737, 0.00855802, 1.83168, 0.0878149, 1.11115, 0.101487, -0.999671, 0.856959, 0.438544, 0.382092, 0.0142683, 0.202587, 0.802078, -1.81123, 1.27302, -1.72211, -0.755041, -5.79169, 2.19619, 1.57931, 1.39546, 1.47683, 0.0429746, -1.59611, 0.0657754, 0.521311, -1.45623, 1.0472, -0.0081553, 0.0241366, -3.94476, -0.270165, 1.11358, 3.11269, 2.15021, -0.00318028, 0.0579373, 0.0290543, -0.660494, 0.0963529, -2.96301, 0.0933746, 0.0506514, 0.0366129, 6.60329, -1.54623, 1.27435, -1.43189, 1.32101, -0.192104, 4.17265, -0.17512, 4.9236, 0.0508381, -4.30754, 3.56138, 5.21339, 0.582762, 0.0100461, 3.15604, -0.0114486, -2.61441, 1.89071, 0.0272309, 1.55022, -1.47822, -2.21313, 0.141138, 0.93232, -0.0159849, -4.0034, -1.51774, -0.552357, -0.0104212, -1.99324, 1.25305, 1.36774, 1.98424, 2.37253, -0.615086, -0.032637, 0.309609, -2.01487, -2.52297, 0.490498, 1.69908, -4.20373, -3.12397, -1.50714, -0.110666, -0.412579, 0.0197596, 0.0615136, 0.065006, -0.916537, -0.260582, 0.0240113, 0.941218, 1.75227, -1.80267, 1.95818, 1.35691, 0.554925, -1.45997, -0.843481, -0.0633564, 0.0335011, -2.44214, -0.0397658, 0.0108823, 0.124079, -0.0242923, 1.61181, -0.593635, -0.942445, -0.69633, 2.10045, 1.28386, -0.0806168, 0.000281586, -0.0232558, -0.0372369, 2.67556, -0.0447205, -0.599961, 0.00868352, 0.2675, -0.367262, -0.124239, 0.0109117, -1.48099, -0.0521501, 0.816396, -0.363493, -1.72336, -1.63787, 0.427585, 0.134058, -0.0282419, -0.815007, -1.23274, 0.959117, -0.496285, 0.681578, 0.609141, 1.53009, -1.47127, -2.23387, -2.50111, -0.570331, 0.00156426, 0.327944, -0.0532424, 0.74968, 0.21045, 0.360345, -0.00579383, 0.0148224, 1.76498, 0.957433, -1.18601, -1.51632, -0.47725, -0.00717129, -0.0239306, -0.10988, 0.967754, 0.00376127, 2.1389, 0.345654, -0.0062817, 0.0292866, -0.849436, 1.57992, -1.44132, 0.699925, -1.35774, 0.868631, 0.0578065, 0.857926, -0.96604, -0.119313, -0.750569, 0.342787, -0.892751, -0.108741, 0.0206601, 0.0223767, -0.00135452, -0.119596, -0.926094, 0.0164363, -0.617288, 0.683777, 1.88391, -0.227052, -0.204095, 0.0367331, 0.0410781, 2.19033, 0.0762307, 0.00301032, -0.157805, 0.31238, -1.19345, -0.952793, -1.18989, -1.79283, -0.00570847, -0.258271, -0.271993, -1.50687, -0.196157, 1.07407, 1.07824, -2.72178, 0.828354, -0.754738, 0.935795, -0.11126, -0.11204, -0.139343, 1.89732, 0.883685, 0.032044, -1.68774, -1.24972, -0.634306, -0.952372, -0.992546, 1.91327, -1.34467, 1.65103, -0.0577543, 0.0686871, 2.51033, -0.0648108, 0.0812891, 0.331208, 0.00823118, -0.0760559, 0.484592, -1.28019, -0.684358, 3.33416, -1.07641, -0.0511956, 0.117447, 0.0216681, -0.0330026, 0.77536, -0.0112907, -0.730101, 0.0360727, 3.19487, -0.665912, -0.310564, 0.0619918, 1.96995, -0.157135, -1.52577, -1.11282, -0.318833, -1.03264, 0.570366, 0.966151, 0.0388369, -1.07513, -0.585426, 0.538506, 0.0794037, 0.557134, -0.53706, 1.67019, -1.80541, 2.25513, -0.201285, -0.962592, -0.00831699, -0.683336, -0.0764732, 1.07186, 2.4629, 1.13792, -0.000397973, 0.0374445, 2.05061, -2.00621, -1.31752, -1.02944, -0.669632, 0.0117516, 0.0663619, 1.09136, -1.07361, 0.0672268, -0.458249, -0.907831, 0.0650543, -0.0748368, 0.768767, 0.940683, -0.749594, -3.14197, 0.663426, -0.851197, -0.155209, 0.139212, 3.22204, -0.00417758, 0.841781, 1.57882, 3.52543, 0.518772, 0.021584, 0.672127, 0.0569116, 1.86944, -0.453383, 0.0427191, -1.1309, -0.798016, -0.136943, -0.174155, -0.217733, 0.03713, 2.5676, -0.665436, -0.735035, 0.0551826, -3.31104, 0.115497, -2.08202, 1.09168, -1.57201, -0.494677, 0.27386, 0.836001, 1.51328, -0.524435, 0.0273031, 0.117626, 2.98608, 0.666658, -1.09716, -0.343305, -1.74502, 0.0471624, -0.0237138, -0.0346644, -2.09233, -1.22698, 0.0297174, -0.144199, 0.143761, 1.48209, -0.458003, 1.25661, 0.192006, -0.5552, -0.783571, 0.0345099, 0.0026327, 0.129928, -0.0555515, 0.0476743, -0.00326606, 0.0404201, 0.197022, 0.0135898, -1.69754, -0.734368, 0.549671, -0.702067, -0.0190694, -0.0102429, -0.0316208, 0.000888857, -1.22932, -0.0113656, -0.601169, 0.0244705, -0.895879, -0.482055, 0.324636, 0.0041303, -0.222084, -0.00847598, -0.156245, -0.174209, -0.439724, -0.442948, -0.319369, -1.31905, 0.0194907, -0.624266, 1.04376, -0.40239, 1.12442, -1.1073, 0.800016, -1.1548, 0.329697, -0.224175, 1.38774, -0.478516, -0.00104264, -0.49719, 0.0199663, 0.34793, -1.62584, 1.12565, 0.0196178, 0.0261157, -0.136485, 0.730628, -0.572168, -0.469807, -0.0870424, 0.00433652, -0.00399152, 0.26588, -0.313273, -0.0298503, -1.52027, -1.41382, 0.00394581, 0.0146399, 0.586072, -0.391437, 0.0909672, 0.320516, -0.543992, 0.500085, 1.43163, 1.18882, 0.41585, 0.0222805, 0.361838, 0.287029, 0.0295436, -0.0983046, -0.0130273, 0.665026, 0.00900989, -0.216268, -0.0363555, 0.0324752, 0.910336, -2.11862, -0.0455954, 0.0577202, 1.52774, -0.00134277, -0.199517, -1.60691, -0.741632, 0.0583555, 0.993579, -1.73755, -0.0387407, 1.26263, -0.864227, 1.69393, -0.0190262, 1.27366, -0.780336, -2.02679, -1.4511, -0.916603, 0.464561, -0.547503, -1.29957, -0.669001, -0.468439, 0.0274252, 0.0204945, 0.0675205, -1.72317, 0.174636, 0.029539, -0.317693, 0.0086134, 0.0906536, -0.293124, 0.0723534, 0.0709353, -0.343023, -0.213415, -0.00100421, 0.0794816, 0.50042, -0.0454514, 0.0167952, 0.0265609, 0.012987, -0.295725, 0.354242, 0.0428985, 1.09387, 0.0921323, 0.0464209, -0.0384584, 0.021724, 0.00702112, -0.0140323, -0.173617, 0.0489765, 0.372278, 0.007618, -0.0012813, 0.0569144, -0.233311, -0.00471294, -0.42536, 0.00704257, -0.371381, -0.0920223, 0.099147, 0.859045, 0.428536, 0.188289, 0.00779551, 0.737316, 0.218291, 0.0125741, -0.084019, 0.00551281, -0.210947, -0.347424, -0.173068, 0.280624, 0.0270293, -0.237895, 0.0211601, -0.0464738, 0.0238433, 0.106195, 0.0166621, 0.187623, 6.67987e-05, 0.00396621, -0.0272348, 0.272819, -0.199997, -0.301771, -0.261397, -0.00665004, 0.00768238, -0.079154, 0.21642, -0.0215292, 0.31445, -0.541194, 0.0302981, 0.0352395, 0.0733522, 0.214594, -0.0299638, 0.101512, -0.110298, 0.161108, -0.13766, -0.267127, 0.371851, 0.0557151, -0.564062, 0.187415, -0.0681782, 0.355409, 0.000590564, 0.289416, 0.0104514, -0.0837608, 0.13849, -0.0037309, 0.181632, 0.285257, -0.701571, -0.166718, -0.443538, -0.0118289, 0.0424213, -0.0841993, -0.185918, 0.00814757, 0.530248, 0.20772, 0.137589, 0.535665, 0.0607006, 0.0516855, -0.186104, -0.342748, 0.916667, 0.08714, -0.0940088, 0.110987, -0.115174, -0.302816, -0.370789, 0.0869184, 0.228294, 0.00761233, 0.0376053, 0.00740385, -0.0205127, 0.728887, 0.0295204, 0.169206, 0.458525, 1.27074, -0.222425, 1.89547, -0.219267, -0.969036, -0.223371, 0.00782218, -0.0130649, 0.359198, 0.192461, 0.0754823, 0.113108, -0.0295792, 0.1205, -0.477745, -1.24602, -0.802172, 0.662034, -0.319408, 0.0793938, -0.0105124, -0.0144643, 0.0249801, -0.835835, -0.00989282, -0.346073, 0.0141739, -0.640867, 0.480995, -0.259975, 0.00781686, 0.687312, -0.0106356, 1.69921, -1.5448, -0.239066, 0.115136, -1.72427, -1.70058, 0.0218415, -0.30805, 0.939167, -0.618041, 0.52263, 1.36, 1.23214, -0.359569, -0.645378, -0.0251139, 0.468763, -0.526054, 0.0273982, -0.772264, 0.0148578, -0.567185, -0.180314, 0.424493, -0.0140702, 0.0330969, 0.389976, 0.315135, -0.330627, 0.627465, -0.486045, -0.0160715, -0.0137218, 0.370897, -0.0273295, -0.0424176, -0.807374, -0.33247, 0.00218634, 0.0428922, 0.274693, 0.276943, -0.660022, -0.270693, -0.492953, 0.346715, 0.434053, -1.15661, 0.681401, 0.0328913, 0.267869, -0.959196, -0.16222, -0.443416, 0.0074892, 0.886394, -0.000859228, 0.129304, -0.84372, -0.00753158, -0.0491179, -1.31676, 0.852651, -0.0304723, -0.0105722, 0.0166479, -0.891493, -0.938627, 0.712575, -0.0022721, 1.26384, -0.719184, 0.563451, -1.4711, -1.33, 1.38225, 0.0257077, 0.793105, 0.38909, -0.588855, -1.50651, 0.840229, 1.07342, 0.289997, -0.71897, -0.0187203, 0.220174, -0.0418909, 0.00932617, 0.0653799, -0.893821, 1.13662, 0.0312645, 0.0154762, 0.0145362, -0.0355627, -0.237712, 0.353506, -0.256662, 0.130276, -0.26737, -0.0247626, 0.0582857, 0.125026, 0.0684069, -0.0330192, -0.0491962, -0.0337733, -0.215591, 0.364514, 0.0196033, -0.182151, 0.178678, -0.0141045, -0.0749296, 0.010567, 0.0349578, 0.0282684, 0.0845595, -0.00258656, -0.510673, -0.0165956, -0.22639, -0.122087, 0.175632, -0.00375742, 0.241822, 0.0833113, 0.131571, -0.3066, 0.582157, -0.0307044, 0.380978, -0.1535, -0.0288147, 0.966147, 0.0204165, -0.0090336, -0.172201, -0.297513, 0.0951879, 0.0669389, -0.141502, 0.425731, 0.102804, -0.92954, -0.00292506, 0.158961, -0.0257614, -0.0770601, -0.191176, 0.0462401, -0.0118297, -0.00861048, 0.0659457, -0.162738, -0.581812, 0.292278, 0.265975, -0.00435415, -0.0060606, -0.106463, 0.130437, 0.00133388, 0.0857369, 0.19591, -0.00644252, -0.0265263, -0.0681064, -0.171399, -0.313771, -0.123873, -0.241475, -0.244246, 0.141957, 0.392639, 0.156961, -0.029182, 0.115276, -0.445497, -0.0879747, -0.148618, -0.00655808, -0.016398, 0.0274653, -0.0877455, -0.055382, -0.0150809, -0.386054, 0.0939673, 0.7751, -0.100647, 0.0147746, -0.0102919, 0.157755, -0.0387558, 0.540235, 0.00257939, 0.0380908, 0.12051, 0.070107, 0.320199, -0.475595, 0.0584478, 0.0366872, 0.0821685, 0.115645, -0.0480285, -0.485717, 0.377443, 0.10544, -0.234864, 0.153013, 0.0854238, -0.0875302, 0.00455898, -0.0294285, -0.056719, 0.193279, 0.685111, -0.0171741, 0.384828, -0.498331, 0.0921895, -0.0697803, -0.975349, -0.0597593, 0.800306, -1.67767, -0.0826342, -0.0636664, -0.390061, -0.283043, -0.130666, 0.160681, -0.070554, -0.224493, -1.08635, -0.623203, -0.282796, 0.165499, -1.40935, -0.098712, 0.0575619, -0.0695048, -0.0823204, 1.53778, -0.0724917, -0.350622, -0.0330038, -0.607499, -0.907548, -0.0669608, 0.0124353, -0.372179, -0.0278945, -0.0992677, -1.08286, 0.196057, -0.479413, 0.292725, -0.0980021, -0.0860493, -0.192693, -0.146549, -0.147448, 0.39535, -1.71535, 1.14627, 0.326308, 0.121014, 0.416624, 0.736862, -0.280944, -0.0158317, 1.79488, -0.0899244, -0.808547, -2.46882, -0.517758, -0.0150088, -0.00489562, 0.0770404, -0.3675, -0.659817, 0.66676, 0.773199, -0.00218697, -0.00134251, -0.2452, -0.431563, -0.023495, -0.279571, 0.131638, -0.00968402, -0.186777, -0.111247, -1.37947, 1.06334, 0.270633, 1.4494, -1.07818, 0.0996574, 0.98656, -0.140973, -0.101805, -0.930196, -1.61525, 0.0493713, 0.414474, 0.00772326, 1.31354, -0.00983081, -0.853039, 0.120724, -0.00919587, -0.627005, 0.663755, 0.673456, -0.0753756, 0.698619, -0.000554318, 0.759168, -0.0591701, -0.769197, -0.0157779, 0.361525, -0.796178, -0.502578, 0.478386, -0.117204, -0.304401, 0.0946607, 0.502003, 0.752552, -0.325977, -0.474008, -0.122251, -0.0349879, -0.959381, -1.06555, -0.218571, -0.198027, -0.185101, -0.099801, -0.0976851, 0.0477474, 0.359917, -0.0319161, -0.38969, -0.371372, 0.83195, 0.875633, -1.30585, 0.644258, -1.71016, 1.20084, -0.0742224, -0.097895, -0.388343, -0.147293, -0.048564, 0.00962435, 0.0593867, -1.64889, 0.232081, 1.08259, -1.09903, -1.01892, -1.62533, -0.0873313, 0.0158606, -0.0270174, -0.0196416, -0.120514, -0.0173981, -0.0801416, -0.0276648, 0.0728259, 0.790462, -0.0779034, 0.0044859, 0.370182, -0.0608931, 0.443886, -0.84857, -0.577278, 1.15252, 0.0476437, 0.166086, 0.0542313, 0.449933, -0.129482, 0.127538, -2.30298, 0.419902, -0.0799937, -0.35703, 0.634232, 0.557922, -0.744341, -0.163172, -0.0149212, -1.15713, -0.0577027, 1.76583, 0.762074, -2.06619, -0.00950901, -0.0328733, -1.91184, -1.23022, 0.298953, 0.266949, 0.0371148, -0.00954489, 0.00725124, 0.100844, -2.3006, 0.00926943, -0.0691565, 0.284287, -0.000692078, -0.0868919, 0.715723, 1.19075, 0.212778, -0.689713, 1.14857, 0.958525, 0.58082, -0.304598, -0.248226, -0.038579, -0.292599, -0.449614, 0.868, -0.514769, -0.00299611, 0.418097, 0.00910317, 0.299596, 0.707182, 0.00811015, 0.646477, -0.418854, -0.532339, -0.045724, -1.30843, 0.0116703, 1.3437, 0.408715, 0.00349374, 0.0103221, 0.328786, 0.728371, 2.18461, -0.0664464, 0.686988, -2.75824, -0.177353, -0.0419929, 0.684302, 1.59966, 1.09182, 0.772648, -1.68642, -0.107213, -0.947784, 0.00491232, 1.71774, -0.0395695, -0.06732, -0.0614457, 0.640491, -1.69409, -0.0207905, -0.265523, -0.897518, 0.24272, -0.121182, 0.366115, -0.296234, -0.168458, 0.408568, 0.027881, -0.043257, -0.650872, -0.163553, 0.0406469, -0.0587859, -0.0340317, -0.0508826, 0.221593, 0.23676, -0.343029, 0.0225603, 0.0618901, -0.00113794, 0.0117468, 0.0441513, -0.0194893, 0.158388, -0.0386202, -0.471465, -0.00360753, 0.17938, -0.0637542, 0.195562, -0.00800381, 0.459559, 0.0564732, 0.774534, 0.23481, 0.275981, 0.132179, -0.184591, -0.572444, -0.0143051, -0.329052, 0.284877, -1.23847, 0.116916, -0.0214719, 0.147897, -0.00880838, 0.278934, -0.183073, 0.373174, 0.101188, 0.0458135, 0.220173, 0.014426, -0.31485, -0.128626, -0.323779, -0.000228682, -0.00579879, -0.4384, -0.588791, -1.6541, 0.0389328, 0.21185, -0.00438973, 0.00502541, 0.122911, -0.294296, 0.0241436, -0.341329, 0.217388, -0.00174571, 0.0399538, -0.0763089, -0.143369, 0.24811, -0.624985, -0.0803452, 0.676917, 0.380814, 0.61116, 0.158975, -0.0349856, 0.806361, -0.355811, 0.328257, -0.327713, 0.00635584, -0.198001, 0.0143727, -0.145056, 0.399621, -0.018621, 0.427553, 0.286795, 0.408353, -0.0583788, 0.247905, 0.00292955, -0.0023817, 0.0954464, 0.201097, 0.00315265, 0.0550162, -0.203082, -0.113322, -0.115479, -0.166241, -0.534963, 0.187405, 0.657916, 0.203337, -0.0132384, 0.408966, 0.682302, -0.251711, 0.207039, 0.239047, -0.0445531, -0.331264, -0.0922965, -0.0119627, 0.00514001, 0.616772, -0.131447, -0.00769235, -1.14805, -0.7143, 0.726259, 0.296475, -0.738724, 0.160988, 0.660372, 1.40009, -0.0504175, -0.00638046, -1.89526, 0.0532036, -0.0382521, -0.130883, 0.0336608, -1.14354, 0.149418, 0.69126, -0.600801, -0.551016, -0.430497, -0.0467635, -0.0108659, -0.125965, -0.0173913, 0.101462, 0.00251225, -0.107471, -0.0242153, -0.842087, 0.532575, -0.445976, -0.0140196, 1.45284, 0.0181116, 0.277942, 0.833706, 2.32739, 0.131025, 0.670236, -0.202272, 0.0617696, 0.755481, 0.333742, -0.180839, -1.6256, -0.915762, 0.860264, -0.234186, 0.848771, 0.173193, 0.514164, -0.593166, -0.0217384, 0.268066, -0.0484585, 0.945116, -1.06281, 0.138067, 0.000151827, -0.0203631, -0.846863, -0.162369, -0.223795, 1.47909, 0.398084, 0.000358213, 0.0120418, 1.0176, -1.31335, 0.0277365, 0.389142, 0.900376, -0.038858, -0.0111315, 0.0747065, -0.163785, 1.44433, -1.69043, 0.586231, -0.240599, 0.0262441, -0.404932, 0.281213, -0.0458816, 0.7823, 0.932493, 0.403923, -0.404938, 0.00257585, -0.276584, -0.0560923, 0.522491, 1.45584, 0.0360816, 0.682175, 0.247683, -0.0177111, -0.00609206, 0.140382, 0.0164612, -0.119426, -0.361587, -0.792374, -0.000993639, -1.20761, -0.194203, -0.224224, -0.780901, 0.243196, -0.900572, 0.129634, -0.753568, -0.38078, 0.442865, 1.02311, 0.297502, -0.147131, -0.249602, 0.5139, -0.794813, -0.316934, 0.0122676, -0.063237, -0.0470637, -0.166589, -0.453879, -0.0076497, 0.0877204, 0.0422127, 0.0667999, -0.0920827, -0.250942, -0.0529803, -0.359019, 0.221523, 0.00516364, 0.0431337, -0.298267, -0.0471671, 0.0774087, 0.0525929, -0.0237382, 0.0790561, -0.271887, 0.227281, 0.347837, 0.0693444, -0.0139343, -0.0654772, 0.00585555, 0.0662505, -0.0439033, -0.0825126, 0.000628758, 0.238558, 0.0128055, 0.0325301, -0.276774, -0.402774, 0.000831248, 0.26094, -0.160626, -0.0142152, 0.589013, 0.0690516, -0.139042, 0.710256, -0.238265, 0.000834843, 0.477565, -0.514931, -0.290177, -0.238435, 0.225212, -0.239043, -0.294841, 0.0106923, -0.131389, 0.0694682, 0.176754, -0.0116857, -0.420893, 0.0201548, -0.0233091, -0.180326, -0.401032, -0.0167268, 0.00980812, -0.271888, 0.129226, -0.14419, 0.190662, 0.30579, -0.00889934, 0.0178257, -0.115304, -0.0447192, 0.0108044, 0.102847, -0.103538, -0.00576917, 0.00617626, -0.126156, 0.226115, -0.164704, 0.113025, 0.281083, 0.0255803, 0.123129, -0.0905098, -0.102142, 0.0301354, 0.530024, 0.0604237, 0.0810872, 0.294919, -0.00244323, -0.0176335, 0.000987391, 0.13942, -0.100581, -0.00304277, 0.47554, 0.269909, -0.366386, 0.0418903, 0.110156, 0.019389, -0.0503967, 0.226942, -0.166063, -0.0358948, 0.225331, -0.0243658, -0.102175, 0.163251, -0.129815, -0.296619, -0.257188, 0.0368964, -0.335619, -0.0904133, -0.429087, 0.499387, -0.0163848, -0.0102237, -0.186404, 0.0936917, -0.356204, -0.00411038, 0.00167308, -0.0391401, 0.326152, 0.80216, -0.00293105, -0.412466, -1.10369, -0.0199351, -0.202331, -0.428847, 0.809822, 0.583812, 0.10183, 0.0447951, 0.0313467, 0.270735, -0.112648, -0.132711, 0.107322, -0.0235178, 0.790614, 0.221574, 0.378082, 0.764901, -0.594455, -0.307307, 0.0729572, -0.0419904, -0.00876148, -0.00242641, -0.3437, 0.0197374, -0.274658, -0.00642083, 0.653558, 0.612523, -0.13775, -0.00242722, -0.81603, -0.113956, -1.20285, 0.503551, -0.380172, 1.71908, 0.162821, 0.479048, -0.0615448, 0.422058, 1.03766, -0.42545, 0.268019, -1.28951, 0.576969, -0.040087, -0.858313, -0.0883743, -0.0703552, -0.754251, -0.00585043, -0.464271, 0.0358444, -0.459091, 0.353904, -0.132528, -0.0172618, -0.0296943, 0.703498, -1.12892, -0.731586, 0.762858, 0.219457, -0.0062143, -0.0351328, 0.917285, 0.534989, -0.0576691, 0.358949, -1.18164, -0.045812, 0.0283546, 0.490281, 1.11141, -0.12245, 0.343593, 0.518602, 0.154442, 0.316742, -0.584598, -0.0554326, 0.0259298, 0.0724307, -0.038838, 0.160261, 0.669937, -0.0105293, 0.272153, -0.0251943, -1.08485, -0.69797, -0.000168221, -0.0760823, -0.165315, -0.072576, -0.0601848, -0.824449, -0.0488436, -0.350459, -0.16747, -0.276491, -0.0077161, 1.08085, -0.17611, 1.14645, 1.03202, 0.202295, 0.0977971, -0.14054, -0.817923, 0.567839, -0.645044, 0.250639, 1.26547, -0.229483, 0.0492921, -0.586169, 0.171088, 1.85756, -0.0584087, 0.0500281, 0.0680133, -0.426854, -0.695111, 0.00422704, 0.64204, -0.597289, -0.312685, -0.102087, -0.137277, 0.22391, -0.147679, 0.174215, -0.0948138, -0.106693, 0.0254113, -0.264884, -0.107243, -0.128805, -0.0551363, 0.0559419, -0.336194, -0.415163, -0.211712, 0.0277942, -0.0582311, -0.0755358, -0.0692918, -0.12358, -0.131453, -0.23106, -0.126553, -0.693646, -0.0414657, 0.18918, -0.187117, -0.164171, -0.0765046, 0.0725483, -0.366756, -0.0830793, -0.177082, 0.307473, -0.178713, -0.324795, -0.059372, -0.0981962, -0.374563, -0.112536, -0.174433, -0.377833, -0.0569683, -0.760553, 0.185471, -0.510465, 0.458522, 0.119259, -0.283451, -0.0413601, -0.246678, -0.105002, 0.00483272, -0.271553, 0.0780771, -0.0348242, -0.0336972, 0.277036, 0.0158995, -0.285617, -0.432594, -0.768498, -0.0182254, -0.0485971, -0.484376, -0.255926, -0.110896, -0.190607, -0.238793, -0.0667695, -0.128152, 0.168023, -0.147565, -0.707708, -0.0178351, -0.212257, -0.809439, 0.0701307, -0.392523, 0.288153, -0.108988, -0.302646, 0.408277, 0.367015, -0.692632, -0.0161102, 0.255165, -0.0555574, 0.238575, -0.24768, -0.049752, -0.658808, -0.332626, -0.12595, -0.239787, 0.0360159, -0.0400407, 0.117831, -0.1417, -0.384389, -0.065011, -0.641681, 0.529386, -0.1015, 0.0734233, -0.0175338, -0.208005, -0.452787, 0.187756, -0.218343, -0.0618624, -0.253061, 0.0192006, 0.201896, 0.286404, -0.0745803, -0.701475, -0.30447, -0.152879, -0.11171, -0.0884996, -0.0394733, -0.381422, -0.0277791, -0.679019, -0.571829, -0.314188, -0.105654, -0.11214, 0.214457, -0.1483, 0.194715, -0.0732143, -0.123652, 0.0129649, -0.27459, -0.107549, -0.131898, -0.0876851, 0.0450815, -0.328334, -0.423213, -0.20944, 0.0335692, -0.0623442, -0.0896799, -0.0612005, -0.147317, -0.111005, -0.227286, -0.128561, -0.678566, -0.0249864, 0.178204, -0.170255, -0.17541, -0.0509915, 0.0924956, -0.332978, -0.087103, -0.159231, 0.280068, -0.186746, -0.324416, -0.0630964, -0.0958143, -0.378891, -0.142521, -0.183968, -0.344872, -0.0742977, -0.781173, 0.201682, -0.507358, 0.463812, 0.139643, -0.296402, -0.0634189, -0.23267, -0.0828376, 0.006487, -0.252999, 0.0827944, -0.0219322, -0.0357359, 0.270161, 0.035458, -0.273869, -0.419269, -0.752697, -0.00174809, -0.0596361, -0.489187, -0.269569, -0.0906247, -0.186511, -0.215088, -0.0805569, -0.121872, 0.161237, -0.135057, -0.691957, -0.0410667, -0.200322, -0.812847, 0.0852681, -0.386019, 0.276561, -0.117101, -0.30435, 0.412947, 0.38658, -0.677308, -0.0171633, 0.258747, -0.0463147, 0.234914, -0.237593, -0.0350936, -0.681228, -0.330428, -0.150563, -0.256979, 0.0375175, -0.0420226, 0.12432, -0.128924, -0.377938, -0.0768514, -0.639111, 0.520887, -0.0947007, 0.0642753, -0.0307257, -0.233659, -0.446011, 0.178889, -0.209623, -0.073327, -0.228352, 0.0405076, 0.192658, 0.281252, -0.0865277, -0.695298, -0.335084, -0.150427, -0.107534, -0.0975224, -0.0485915, -0.373983, -0.0394287, -0.649186, -0.574091, -0.289497, -0.109807, -0.122943, 0.206884, -0.14268, 0.185173, -0.0896727, -0.110597, -0.000312457, -0.291488, -0.0988027, -0.11504, -0.0664987, 0.0157313, -0.331475, -0.411894, -0.198253, 0.0337361, -0.0736527, -0.09895, -0.0583898, -0.143201, -0.112815, -0.221991, -0.112493, -0.701272, -0.0547109, 0.168509, -0.161968, -0.160992, -0.0449545, 0.0980941, -0.348027, -0.0918994, -0.173782, 0.284144, -0.182352, -0.317868, -0.0555843, -0.111004, -0.374751, -0.118485, -0.190567, -0.358778, -0.0448218, -0.770648, 0.161656, -0.518004, 0.433761, 0.114948, -0.287078, -0.0571166, -0.253203, -0.0853748, 0.0118529, -0.253216, 0.0844797, -0.0321363, -0.0377762, 0.261637, 0.0642738, -0.285702, -0.424217, -0.73164, -0.0117084, -0.0703168, -0.505593, -0.275045, -0.0954889, -0.187552, -0.24122, -0.0758648, -0.14092, 0.181554, -0.123943, -0.690125, -0.0333458, -0.212931, -0.811072, 0.086546, -0.3633, 0.296531, -0.102506, -0.28288, 0.388987, 0.387646, -0.665174, -0.0119044, 0.264727, -0.0543612, 0.223785, -0.224873, -0.0305869, -0.683702, -0.335428, -0.152344, -0.263024, 0.0543458, -0.0190912, 0.138324, -0.138521, -0.377767, -0.064562, -0.63441, 0.508036, -0.104168, 0.0601089, -0.0308097, -0.201187, -0.43748, 0.176728, -0.19067, -0.0764688, -0.24036, 0.0431755, 0.188881, 0.283859, -0.110521, -0.702074, -0.32928, -0.136302, -0.0954587, -0.123464, -0.0648269, -0.390452, -0.035881, -0.663555, -0.557996, -0.302451, -0.0938844, -0.113241, 0.19304, -0.160147, 0.183288, -0.0913063, -0.126802, 0.015744, -0.282062, -0.119161, -0.132856, -0.0856619, 0.00955689, -0.339399, -0.413903, -0.223399, 0.04303, -0.0671428, -0.102229, -0.0694648, -0.128976, -0.126879, -0.257452, -0.128326, -0.690298, -0.0344017, 0.149969, -0.180014, -0.174324, -0.0309371, 0.077735, -0.345839, -0.0873839, -0.181792, 0.248991, -0.189197, -0.284007, -0.045093, -0.0955616, -0.381017, -0.126674, -0.175882, -0.339928, -0.0495645, -0.784032, 0.14987, -0.488776, 0.417312, 0.143019, -0.289894, -0.045694, -0.259384, -0.0830402, 0.0258156, -0.250868, 0.125227, -0.0355167, -0.051331, 0.248954, 0.0806938, -0.279645, -0.44485, -0.775745, -0.00325385, -0.0629779, -0.485447, -0.270089, -0.0747197, -0.17966, -0.243989, -0.0624034, -0.143229, 0.20063, -0.10659, -0.683202, -0.0638319, -0.208513, -0.801929, 0.114083, -0.33984, 0.287899, -0.0981626, -0.280121, 0.410076, 0.431816, -0.682788, -0.00114652, 0.285213, -0.0488827, 0.235844, -0.213777, -0.0288586, -0.652692, -0.344933, -0.134556, -0.227343, 0.0673468, -0.0511295, 0.154384, -0.142667, -0.36031, -0.0655474, -0.648006, 0.518565, -0.0884304, 0.0709597, -0.0188304, -0.216254, -0.45547, 0.17627, -0.17114, -0.0593769, -0.181416, 0.030596, 0.222831, 0.296662, -0.120612, -0.719604, -0.331968, -0.151952, -0.120153, -0.116312, -0.0570639, -0.396861, -0.0495778, -0.667676, -0.57072, -0.273329, -0.131456, -0.0989229, 0.179989, -0.154311, 0.18009, -0.0726576, -0.118073, -0.00193845, -0.289984, -0.115896, -0.14102, -0.0853987, -0.00736617, -0.340348, -0.407289, -0.208901, 0.0293733, -0.0767191, -0.0929857, -0.074934, -0.150811, -0.106988, -0.259091, -0.139065, -0.691023, -0.0562343, 0.160033, -0.170809, -0.160231, -0.0562444, 0.0672541, -0.344378, -0.0816943, -0.160239, 0.253809, -0.174795, -0.299485, -0.0237099, -0.0961347, -0.384841, -0.12991, -0.181234, -0.343121, -0.054956, -0.758881, 0.135715, -0.499147, 0.397586, 0.146447, -0.296451, -0.0474885, -0.268912, -0.0835718, 0.0242174, -0.265646, 0.116848, -0.0116799, -0.0511589, 0.218099, 0.0768977, -0.297488, -0.412962, -0.755829, -0.00842231, -0.05413, -0.490205, -0.281666, -0.101981, -0.204749, -0.245533, -0.0856979, -0.118285, 0.220066, -0.119972, -0.677807, -0.0845719, -0.197362, -0.810916, 0.163066, -0.338125, 0.314908, -0.0991271, -0.279583, 0.418586, 0.438715, -0.694768, -0.00691647, 0.28213, -0.0590762, 0.22199, -0.208985, -0.0340897, -0.635428, -0.3483, -0.148536, -0.261586, 0.0656394, -0.0490554, 0.148581, -0.136353, -0.380453, -0.0854203, -0.642913, 0.494849, -0.0881147, 0.0843808, -0.0452339, -0.191439, -0.42973, 0.188716, -0.185458, -0.0628897, -0.188802, 0.0633376, 0.206763, 0.275746, -0.121031, -0.708168, -0.319147, -0.131021, -0.0914457, -0.120151, -0.0760292, -0.355668, -0.0308713, -0.666211, -0.567676, -0.248565, -0.128615, -0.108758, 0.161153, -0.178959, 0.178342, -0.088334, -0.117874, 0.000862466, -0.290351, -0.123562, -0.137237, -0.0895814, -0.000107964, -0.317651, -0.398109, -0.233595, 0.0454309, -0.0830605, -0.0994689, -0.0744954, -0.156551, -0.110881, -0.256777, -0.123163, -0.699256, -0.032481, 0.107454, -0.152033, -0.171086, -0.0481173, 0.0771368, -0.36299, -0.101186, -0.177031, 0.22181, -0.166595, -0.275575, -0.0442177, -0.0977963, -0.379398, -0.113373, -0.168424, -0.344143, -0.0518459, -0.738581, 0.106372, -0.507993, 0.410659, 0.126438, -0.296459, -0.0386355, -0.284257, -0.0975181, 0.00929299, -0.250925, 0.124454, -0.0135214, -0.0413582, 0.211999, 0.0925591, -0.296385, -0.418427, -0.717208, -0.00733153, -0.0553406, -0.491664, -0.298926, -0.0953835, -0.21464, -0.27174, -0.079744, -0.14698, 0.220671, -0.144222, -0.64939, -0.106746, -0.190068, -0.796618, 0.182205, -0.340661, 0.327861, -0.115483, -0.245047, 0.408468, 0.45801, -0.698031, -0.0114533, 0.28825, -0.0691851, 0.215725, -0.172443, -0.0426559, -0.654688, -0.340853, -0.138262, -0.255609, 0.0824712, -0.057632, 0.129209, -0.15145, -0.384992, -0.0600568, -0.650426, 0.471634, -0.0792412, 0.0948177, -0.0341767, -0.210424, -0.431781, 0.202111, -0.221159, -0.062327, -0.181109, 0.0550397, 0.220769, 0.281384, -0.1277, -0.698585, -0.344351, -0.142747, -0.108328, -0.0971938, -0.0956798, -0.385487, -0.0262359, -0.638212, -0.544439, -0.230401, -0.159756, -0.109537, 0.128912, -0.160627, 0.175281, -0.0897026, -0.0913173, -0.0277248, -0.295383, -0.131665, -0.141474, -0.0868445, -0.0510337, -0.33746, -0.368459, -0.179843, 0.0263625, -0.0753133, -0.103264, -0.0516494, -0.127509, -0.106933, -0.257816, -0.111477, -0.683034, -0.0311346, 0.0980917, -0.161339, -0.156488, -0.0587267, 0.0728741, -0.359695, -0.0835554, -0.170761, 0.217318, -0.141735, -0.263626, -0.022765, -0.111014, -0.35812, -0.0946368, -0.164867, -0.31567, -0.0478171, -0.739773, 0.0832966, -0.49779, 0.413159, 0.145708, -0.287056, -0.0670057, -0.287029, -0.0865017, -0.023458, -0.251115, 0.149598, -0.00379222, -0.0399931, 0.17943, 0.111441, -0.317049, -0.409697, -0.733465, -0.00595894, -0.075063, -0.496005, -0.292201, -0.0914041, -0.238583, -0.268259, -0.0379541, -0.142398, 0.241314, -0.129842, -0.645375, -0.147781, -0.164808, -0.812741, 0.189062, -0.313257, 0.33141, -0.1021, -0.260784, 0.401732, 0.493595, -0.700275, -0.00485758, 0.325967, -0.0506754, 0.211454, -0.14426, -0.0430418, -0.651087, -0.344692, -0.153237, -0.237876, 0.104258, -0.054646, 0.129184, -0.131964, -0.386933, -0.0788854, -0.64524, 0.453857, -0.0930119, 0.110019, -0.00114808, -0.151993, -0.43961, 0.213951, -0.191507, -0.0646555, -0.158367, 0.0619822, 0.229892, 0.266601, -0.142932, -0.705476, -0.354654, -0.134609, -0.104546, -0.108176, -0.119084, -0.350175, -0.0533898, -0.640241, -0.573445, -0.227213, -0.164804, -0.0753652, 0.125918, -0.173813, 0.187256, -0.101461, -0.109225, -0.0325936, -0.303601, -0.127583, -0.145072, -0.056349, -0.052981, -0.324147, -0.339808, -0.198063, 0.0271325, -0.0774802, -0.0875992, -0.0804225, -0.163043, -0.105252, -0.242237, -0.121006, -0.675633, -0.0246304, 0.0519321, -0.134521, -0.166055, -0.0771011, 0.0701241, -0.347776, -0.063293, -0.16676, 0.200877, -0.0951433, -0.243153, -0.0425777, -0.104873, -0.366108, -0.0996653, -0.139871, -0.323182, -0.0198214, -0.768541, 0.0566568, -0.462829, 0.384415, 0.124497, -0.310832, -0.041848, -0.296038, -0.0931848, -0.0172769, -0.277036, 0.160425, -0.0309973, -0.0350117, 0.197011, 0.120181, -0.342294, -0.391187, -0.747663, -0.00157737, -0.0503721, -0.504738, -0.259005, -0.0959902, -0.253809, -0.282487, -0.0760274, -0.131749, 0.226837, -0.151199, -0.638284, -0.127646, -0.153694, -0.829746, 0.217924, -0.301832, 0.315725, -0.115714, -0.292433, 0.40884, 0.497259, -0.694571, -0.00469494, 0.350123, -0.0802282, 0.223005, -0.130471, -0.0491337, -0.635611, -0.336273, -0.185626, -0.254554, 0.119604, -0.0500858, 0.0599708, -0.142971, -0.384953, -0.081991, -0.671424, 0.463494, -0.104674, 0.110828, -0.0128105, -0.169355, -0.437005, 0.208267, -0.230314, -0.075237, -0.137593, 0.0642915, 0.219839, 0.272831, -0.132137, -0.712654, -0.350982, -0.167275, -0.102641, -0.107372, -0.116597, -0.358611, -0.0489396, -0.636867, -0.375897, -0.119687, 0.108037, 0.0242204, 0.219294, -0.0194518, 0.0517167, -0.0992114, -0.132179, -0.193676, -0.26134, -0.120336, -0.130779, -0.0782018, 0.18823, -0.361453, -0.214204, -0.106884, 0.0167167, 0.0777975, -0.0997176, -0.0603255, -0.149834, -0.106384, -0.0383136, -0.115249, -0.633759, -0.0568564, -0.0533705, 0.0467438, -0.0967866, -0.0622906, 0.0840346, -0.341458, -0.0798972, -0.284568, -0.104261, -0.146289, -0.226379, -0.144612, -0.101231, -0.394616, -0.0288989, -0.158381, -0.367404, -0.0186759, -0.333443, 0.186476, -0.51994, -0.0454117, 0.000945128, -0.303684, -0.0828137, -0.138194, -0.0733439, -0.110173, -0.329429, -0.097794, -0.0313926, -0.0499262, 0.260341, -0.0455459, -0.348442, -0.330788, -0.630296, -0.0166718, -0.063741, -0.485902, -0.105566, -0.109592, -0.13938, -0.115892, -0.0735238, -0.133624, 0.132632, -0.275477, -0.54735, 0.178901, -0.138544, -0.661932, 0.0497006, -0.323312, 0.191786, -0.0916542, -0.22385, -0.0394705, 0.00911304, -0.637032, -0.0195073, 0.156453, -0.0609445, -0.0488522, -0.0994429, -0.0410615, -0.543297, -0.148669, -0.157506, -0.262415, -0.113261, -0.0549782, -0.0582673, -0.0506902, -0.305689, -0.0587368, -0.358693, 0.248366, -0.109701, -0.0456096, 0.0266811, -0.253838, -0.42155, 0.113988, -0.414877, -0.0603341, -0.231607, 0.128226, 0.273216, 0.00983397, 0.0485794, -0.676191, -0.371085, -0.160139, -0.0985428, -0.10746, 0.240406, -0.342679, -0.0375552, -0.62021, 0.0687229, -0.319723, 0.36298, 0.10269, 0.00785893, 0.252135, -0.286549, -0.0701313, -0.110278, -0.0623659, -0.200323, -0.118123, -0.154542, -0.0778561, 0.335199, -0.700354, 0.129062, -0.126106, 0.106323, 0.074096, -0.0931511, -0.0790211, -0.107974, -0.1062, 0.0789192, -0.114114, -0.222787, -0.055715, -0.00960571, -0.0489699, -0.0621095, -0.0727657, -0.348635, -0.280643, -0.168042, -0.251901, -0.342328, -0.431544, 0.0442263, -0.362091, -0.107295, -0.0766214, 0.294852, -0.471782, 0.160665, -0.197985, 0.172601, 0.144361, -0.255502, -0.550401, -0.172625, -0.137467, -0.0712465, -0.270317, -0.0749803, -0.526644, -0.267382, -0.46087, -0.0311848, -0.0469209, 0.15198, -0.0082549, -0.368269, -0.26182, -0.327555, -0.0208447, -0.0682238, -0.794272, -0.0475041, -0.103672, 0.237334, -0.0374531, -0.0585468, -0.105892, 0.178057, -0.501764, -0.385875, 0.0750447, -0.183976, -0.31375, -0.154045, -0.368983, 0.0920854, -0.073967, 0.0182875, -0.496089, -0.508194, -0.344172, -0.00789446, -0.29633, -0.0637574, -0.386118, 0.0981597, -0.0408363, -0.0974482, 0.00515958, -0.390415, -0.219748, -0.0813681, -0.050386, -0.221139, 0.58527, -0.0897768, -0.0783561, 0.230909, -0.017027, -0.159899, -0.375021, 0.321939, 0.092328, -0.339031, -0.10919, -0.792029, -0.0339136, -0.153868, -0.115559, 0.444599, -0.151691, 0.234274, -0.553579, -0.660597, -0.140072, -0.0738289, -0.0879574, 0.185739, 0.103817, -0.0322158, -0.618416, 0.107464, -0.198137, 0.313967, 0.141652, -0.109087, 0.330509, -0.369833, -0.0787701, -0.0839216, -0.117469, -0.158521, -0.120961, -0.187445, -0.0837461, 0.284573, -0.780761, 0.114802, -0.0293317, 0.142127, 0.0864236, -0.0729433, -0.114441, -0.11993, -0.110209, 0.105106, -0.106552, 0.0786677, -0.0550081, -0.0826311, -0.297535, 0.00305817, -0.0840767, -0.351706, -0.1906, -0.0818283, -0.242236, -0.27139, -0.559629, 0.0426485, -0.392771, -0.0821288, -0.0658321, 0.471635, -0.549499, 0.240041, -0.221402, 0.34899, 0.0703348, -0.0884729, -0.603676, -0.103582, -0.104142, -0.0691321, -0.261561, -0.0802844, -0.619741, -0.390362, -0.538737, -0.037318, -0.0524881, 0.0317348, -0.044867, -0.290339, -0.0534406, -0.369238, -0.015342, -0.0644404, -0.697163, -0.0887127, -0.117455, 0.24178, 0.000448403, -0.0766948, -0.13766, 0.18816, -0.631694, -0.289123, -0.00899627, -0.376493, -0.0595266, -0.132024, -0.426859, 0.152724, -0.0930563, -0.0119781, -0.540522, -0.52715, -0.293483, -0.0233529, -0.631082, -0.0711833, -0.49396, 0.181256, -0.0538529, -0.0111585, -0.000843903, -0.317475, -0.193715, -0.113825, -0.0558119, -0.335706, 0.428548, 0.0954346, -0.0545345, 0.38293, -0.182748, -0.204929, -0.419638, 0.551875, 0.20848, -0.266666, -0.216664, -0.827038, -0.0276879, -0.274287, -0.421188, 0.32934, -0.230625, 0.321864, -0.271793, -0.731117, -0.124739, -0.0922482, -0.0925643, -0.0206608, 0.054343, -0.0536555, -0.309656, 0.555053, 0.00875566, -0.0212627, 0.747614, -0.000236861, -0.0010944, -0.00248452, -0.00715838, 0.326122, -0.840916, 0.00471192, -0.97964, 0.774446, -0.0396574, 0.00111232, 0.259881, -0.557235, 0.369777, -0.849774, -0.0227379, -1.53847, -0.0145015, 0.371344, 0.489822, 0.037746, -0.858726, -0.00319797, -1.97044, -0.37093, -0.0819431, 0.320141, 1.46339, -0.0236076, 0.691923, 0.0886717, -0.702593, -0.594072, -0.0111507, -0.858897, -0.059932, -0.110558, -1.56395, 0.673782, 0.00272866, -0.056259, 1.30419, 0.216914, -0.0199699, 0.101934, -0.00448549, 0.000299989, -0.0160535, 0.0433579, 0.198729, -0.42769, -0.0596622, -0.00762726, 0.335077, 1.38462, -0.0340427, -0.0175474, -0.294225, -0.028811, -1.01615, -1.56902, -0.0066106, 0.322595, 0.00173782, -0.279747, 0.0602474, -0.0225012, -0.00311411, 0.594173, 0.267345, -0.000944081, 0.01021, 0.0517565, 0.209722, -0.303221, -0.0432328, 1.06425, -0.000428324, -0.0303773, 0.00318901, -0.0262565, 0.273304, -0.69583, -0.168111, 0.00139488, 0.00373537, -0.0234621, -0.0147233, -0.0120375, -0.0377821, -0.0193471, -0.00223392, 0.0760381, -0.00974777, -0.790224, -0.621292, -0.0251614, -0.764689, -0.649175, -0.0250326, -0.0345273, -0.00485451, 0.869808, 1.55051, -0.0283958, -1.16542, -0.276999, -0.0609674, -0.00839169, -0.275015, -1.03143, -0.845308, 0.00579752, -0.00305181, -0.660527, 0.505621, -0.29113, 1.12228, -1.45194, 0.0306144, -0.0326245, -0.0191613, -0.00337652, -0.00258853, -1.9058, -0.0125441, -0.0177035, -0.755929, 0.018313, 0.0207883, -0.0184761, -0.00759327, 1.17018, -1.44373, -0.00135573, -1.17682, -0.807965, -0.0114923, -0.03587, -0.434523, -1.29735, 0.422707, -0.523891, 0.00721966, 0.357572, -0.000572835, -0.380739, -1.26934, -0.049966, 0.423306, -0.00833302, -0.646016, -0.193283, -0.0137617, -0.0823809, 0.521517, -0.0598903, -1.12419, -0.712373, -0.144054, 0.376492, 0.0124822, 0.397346, -0.0149777, -0.578227, 2.07443, 0.00800472, 0.0148898, -0.0749551, 0.572419, -1.32908, -0.0186504, -0.36254, -0.0218975, -0.0119772, -0.0105584, 0.0171908, -0.814313, -0.131305, 0.0784189, 0.938137, 0.0960419, -0.602345, -0.0270324, -0.0528674, 0.180815, -0.0252731, -0.801417, 0.129516, -0.0154948, 0.458413, -0.0161245, 0.195626, -0.0280572, -0.0179967, 0.00893054, 0.209531, -0.229979, -0.055329, -0.00143108, 0.153374, -0.149467, 0.0304767, -0.0210961, 0.206419, -0.00697661, -0.0221237, -0.0549152, -0.0216772, 1.10019, -0.540735, 1.03705, -0.0170955, 0.00661257, -0.0446873, 0.00291587, -0.0247612, -0.0878031, 0.0174515, -0.0539007, -0.411002, -0.00440885, -0.595668, -0.138278, -0.0350028, 0.315642, -0.402349, -0.0132375, 0.0461085, -0.000329984, -0.465989, -0.862293, -0.0122842, 0.112632, -0.431662, -0.0271258, 0.389897, -0.873074, -0.884287, -1.00286, -0.0254811, -0.00141433, -0.927443, 0.62216, -0.564879, -0.318317, -0.799382, -0.395609, -0.00848706, -0.773691, -0.0285283, 0.000725758, 0.45403, 0.00861135, -0.0212829, 0.311064, -0.0174499, -0.0179632, 0.00508206, -0.026986, -1.18382, -0.270414, 0.00051344, -1.66924, -0.294792, -0.0368943, -0.0123404, -0.957664, -1.89962, 0.287763, 0.0334444, 0.000138664, -0.54045, -0.0215976, -2.12384, -0.32268, -0.105767, -0.435418, -0.0170688, -0.766848, -0.641759, -0.049258, 0.0253883, -0.267632, -0.0611474, 0.205636, 0.476831, 0.602499, -0.399497, -0.0114799, -0.706097, -0.0236685, 0.474821, -0.376422, 0.428806, -0.00300829, -0.109306, 0.208026, 0.654925, -0.0118774, 0.0292047, -0.0214766, -0.00335969, -0.0334605, -0.0498559, 0.0536924, -1.28634, -0.334398, 0.211111, 0.511099, 0.0270284, -0.0348023, -0.0250852, -0.881029, -0.0341399, 0.0493656, -1.29007, -0.0111847, 0.253056, -0.0231742, -0.896702, 0.511585, -0.0276106, -0.00302819, -0.649662, -0.535329, -0.0394272, -0.00296653, 0.340143, -0.176694, -0.328905, -0.0272696, 0.0668999, -0.00735496, -0.0365208, -0.0331461, -0.0103573, -0.228252, 0.679997, -0.687701, -0.0129499, -0.0372354, -0.0295298, -0.0193633, 0.00020348, -0.0302609, -0.00509743, -0.0171372, 0.0758264, -0.0305118, 0.0114158, 0.157496, -0.00880442, -2.2233, -0.0970709, -0.00239269, -0.0228873, -0.00425253, -0.367052, 0.535609, -0.0247574, -0.040315, 0.0237334, -0.0731933, 0.28713, 0.855364, -0.134968, -1.02439, -0.012605, -0.0135096, -0.768826, -1.18896, -1.21055, -0.114381, -1.64598, 0.0231, 0.000602398, -0.000910987, -0.0418891, -0.0178469, 1.08271, 0.0041984, 0.015897, -0.941734, 0.00242938, 0.00486668, -0.00764312, -0.00267825, -0.0746494, -0.430302, -0.0246637, 0.541003, -0.579389, -0.00019737, -0.0113442, -1.4896, -0.83559, -0.0935545, -0.59347, -0.0138698, -0.0271071, 0.00383969, 0.0514972, -0.889744, -0.108235, -0.0144038, -0.0136962, -0.290965, 0.105181, -0.0482339, -1.17689, -0.355463, -0.0193546, 0.386218, -0.399684, -1.14014, -2.12825, 0.0177144, 0.54805, -0.0227756, -0.852088, -0.783007, 0.444243, -0.0122153, 0.0235026, -0.227592, 0.788552, -0.00468659, -0.0419277, 0.0153601, -0.0214147, 0.00194419, -0.0109178, 0.072784, -0.524843, 0.288225, -1.28494, -1.92763, 0.0305661, -0.012735, 0.0022198, -0.72182, 0.0222242, -0.53431, -0.963314, 0.000459383, -0.783252, 0.0214702, -0.873861, 0.5634, -0.010043, -0.00155811, -0.100293, -0.482473, -0.0165854, 0.0176501, 0.352763, -0.204549, -0.276788, -0.0107344, -0.952085, -0.0127048, -0.0127278, -0.00106201, -0.00212175, 0.0766868, -0.916647, 0.440166, 0.016254, 0.00227958, -0.0195219, -0.000403981, 0.0113994, -0.00472268, 0.000387013, -0.00641411, 0.272096, -0.0236297, -0.577453, 0.226241, 0.00631776, 0.723512, -0.159772, 0.0061567, -0.0338073, -0.00334765, -1.91744, 0.321314, -0.0112916, 0.396247, 0.241532, 0.0134799, -1.55183, -0.62829, 0.191137, -0.651721, -0.017272, -0.00853492, -1.04273, -1.35536, 0.0700681, -0.458541, -0.268627, -0.139113, -0.00314637, -0.850378, -0.0148426, 0.000558782, 0.274295, -0.0166864, -0.0179572, -0.255821, -0.0150698, -0.051474, -0.0104854, -0.0154908, -1.36973, -0.620244, 0.00591838, -2.85363, 0.795225, -0.0318316, -0.00663333, 0.623321, -0.869687, -0.370646, 0.914599, 0.000975972, 0.0339111, -0.0253062, -0.0025629, -0.276739, -0.108192, -0.5857, -0.00855309, -0.989798, -1.3843, -0.028866, -0.49269, -1.11677, -0.0448709, -0.505845, 1.07975, 0.564734, -0.866573, -0.00716622, 0.30902, -0.011736, 1.31398, -0.635552, 0.268703, -0.0419404, -0.0548546, 0.0274521, 0.266133, 0.0158207, -0.510909, -0.0232553, -0.0232925, -0.0270626, -0.0110268, -0.873046, -0.608661, -1.02198, -0.601833, 0.315867, -0.691319, -0.0157629, -0.0284053, -0.0324553, -0.0422093, 0.0643497, 0.284032, -0.0193446, 0.018034, -0.029704, -0.127698, -1.98132, -0.0506857, -0.0213444, -0.189247, 0.589729, -0.0228232, -0.0243017, 0.503847, -0.556508, -0.659276, -0.0313213, -0.946264, 0.000248609, -0.0307786, -0.0132062, -0.00923057, -0.605972, 0.106482, -0.557596, -0.0231942, -0.0176214, -0.018945, -0.0158362, -0.0324422, -0.00733512, -0.0203864, -0.0265837, 0.644154, -0.00395694, -0.0759608, 0.255427, -0.00677237, -0.43798, -0.0200583, -0.0171882, -0.0830742, -0.0191836, 0.285321, 0.457253, -0.0183258, 0.550369, -0.19015, -0.043887, 0.16402, 0.192818, 1.01755, 0.423747, -0.00114601, -0.0300148, -0.92492, -0.387796, -1.15341, 0.193559, -0.115417, 0.0877871, -0.0394241, -0.159709, 0.00917821, 0.00212809, 0.934566, -0.00219437, -0.0145647, 0.20243, -0.0400078, -0.0887994, -0.0277272, -0.00219579, -1.05652, 0.036047, -0.00442132, -0.619843, -0.779037, -0.0282266, -0.017973, -0.710732, 1.05841, 0.683808, 0.228934, -0.00227628, 0.0940307, -0.00498611, -1.17685, 0.690995, 0.00711164, 0.322881, -0.00711339, 0.296228, -0.406932, -0.0264096, 0.671163, 0.143628, 0.0545533, -0.174893, -0.421412, 0.47699, 0.493864, -0.030303, -0.0454545, -0.0278942, 0.508408, 0.261734, 0.962185, -0.0263905, -0.0342632, -0.355457, -1.45926, -0.004056, 0.247111, -0.0185628, -0.000639947, -0.0011106, -0.0151018, 0.169361, 0.599161, -0.172593, 0.0164039, -0.103521, 0.291952, -0.029066, -0.0483541, -1.18598, -0.0292484, -0.372562, 0.189257, -0.033158, 0.496276, 0.00377918, -0.115376, 0.383382, -0.000890707, 0.00291053, -0.32921, 0.162823, -0.0385254, -0.0427145, -1.1796, 1.16638, 0.321531, -0.0227895, 0.39388, 0.000349238, -0.0334922, -0.0436319, -0.0265612, 0.619735, -0.537274, 0.753766, -0.0185028, -0.0373711, -0.0247151, -0.01455, -0.0361049, -0.0433338, -0.0167185, -0.0254612, 0.661722, 0.0139226, 0.16498, -0.134937, -0.0339287, -1.08996, -0.396117, -0.0376063, 0.041719, -0.01994, 0.204782, 0.332456, -0.0473289, 0.65673, 0.585705, -0.0192839, 0.339018, 1.17734, 0.933506, -0.225495, 0.00814042, 0.00314459, -0.643174, 0.16877, 0.634488, -0.57231, -0.340043, 0.101999, -0.0172074, -0.252988, -0.0703591, -0.00749243, -0.0981885, 0.0118395, -0.0122573, 0.103864, 0.0100626, -0.00059973, -0.0190945, -0.0122814, 0.303007, -0.100725, -0.000339148, -0.0810095, 0.818261, -0.014902, -0.011102, -0.699281, 0.0960964, -1.14485, -0.0837099, -0.00530058, 0.346913, -0.0110232, -0.702737, -0.404879, 0.00184984, -0.760322, 0.00566828, 1.36775, -0.0074659, -0.000879787, 0.0130699, -0.174835, 0.000340076, 0.0540661, 1.66254, -0.395554, -0.186328, -0.00555302, -0.252943, 0.0199721, 0.294245, -0.944997, 0.224465, 0.0119403, -0.00438915, -0.282501, 0.334606, 0.0147193, 0.00165403, 0.0093999, 0.00441207, -0.00948789, -0.00347912, 0.00776648, -0.162602, -0.479005, 0.208278, 0.268995, 0.66979, -0.00148435, 0.00117854, 0.111877, 0.00262704, -0.0482222, -0.00810415, 0.00247886, 0.310336, -0.00331084, -0.226354, -0.214589, -0.0334534, -0.0121842, -0.414865, 0.105671, -0.00073934, -0.0197586, 0.276273, 0.19232, -0.792095, 0.0132017, -0.574778, 0.00111142, -0.00960036, 7.98585e-05, 0.00556042, 0.216565, 0.41144, 0.443597, -0.00171739, 0.00615723, -0.0160121, 0.00741808, 0.00985656, -0.0133181, 0.00641138, 0.00431074, -0.0419626, -0.00690367, 1.10191, 0.146348, 0.00432497, 0.163461, -0.309457, -0.00392287, 0.0130047, 0.00222892, -0.349824, -0.372205, 0.00663424, 0.344196, -0.0218939, -0.00565964, -0.545506, 0.101109, 0.525426, -0.297235, 0.0216908, -0.0108607, -0.0508461, -0.162267, 0.576609, -0.0903381, -0.0717882, 0.0114014, 0.00433859, -1.19142, -0.00174576, 0.0167963, 0.062567, -0.00550097, -0.00255092, 0.405716, -0.0193109, -0.0146722, -0.0280023, -0.0127452, 0.113294, 0.0415353, -0.00488685, 0.0636005, 0.0954583, -0.0024948, -0.0171203, 0.0760046, 0.120776, 0.0623768, 0.0358358, -0.00962647, 0.0344299, -0.028365, 0.0254836, 0.0596338, -0.0127479, 0.22242, -0.00467738, 0.0747184, -0.00269252, -0.0120302, 0.191581, 0.134439, -0.00013245, 0.202853, 0.00224447, 0.0802666, 0.172452, -0.0161816, 0.0511022, -0.054773, 0.0437747, 0.0621381, 0.00590757, -0.00770635, 0.00528213, 0.0752171, 0.076771, -0.00783153, 0.0548841, -0.0108497, -0.0110823, -0.00428304, -0.0346512, 0.0770607, 0.12601, 0.0578118, -0.0801457, 0.0520215, 0.0632387, -0.00462968, -0.0198534, 0.0595105, -0.00870661, 0.00465066, 0.143081, -0.0117701, -0.0143256, -0.00168724, 0.0851118, 0.108592, -0.00857312, 0.0113709, -0.126128, 0.0100858, 0.00456004, -0.0156535, 0.00398317, 0.0745489, 0.0597062, -0.0194311, 0.135583, -0.0131228, 0.000494796, -0.0194068, 0.0030717, 0.0747226, -0.00403079, 0.129741, -0.0129568, 0.0207324, -0.020338, -0.0155961, 0.00535379, -0.00857027, -0.00777128, -0.0287308, 0.0646841, 0.0338416, -0.00115487, 0.0138444, -0.029598, -0.0101592, 0.0201742, -0.0281251, 0.00354984, -0.0119045, 0.0778672, 0.052907, -0.0181435, 0.0122982, 0.0603849, -0.0251839, 0.13196, 0.0693126, 0.101127, 0.0366926, -0.0054366, -0.00312952, 0.0663764, 0.0566342, 0.0514547, 0.0343779, 0.0239477, 0.0368928, -0.0287477, 0.0620709, -0.00304053, -0.0124775, 0.000909258, -0.0188804, -0.0146428, -0.026437, 0.00739438, 0.00269689, 0.00218811, 0.0183503, -0.0634524, -0.105255, 0.0314379, 0.0110071, 0.0260712, 0.0133738, -0.00246354, 0.00567666, -0.00388562, 0.00505826, -0.0202823, 0.0135202, -0.0208074, -0.00935347, 0.139033, -0.00598509, 0.0128715, 0.0523705, -0.0260387, 0.0381696, 0.0646618, 0.000352237, -0.00901205, 0.00340078, 0.00364627, -0.0172795, 0.0856743, -0.0504545, 0.0228957, 0.00261794, -0.0255996, -0.0123352, -0.0582138, -0.0159596, 0.0426362, -0.00403684, -0.00107065, -0.0374059, -0.0129718, 0.00971402, -0.0399112, -0.00181752, 0.0130279, -0.00830612, -0.00874424, -0.0165323, 0.0265847, -0.00659743, 0.0470778, -0.0538349, -0.0123939, 0.0205524, 0.00861035, -0.0632295, -0.0105957, -0.0144113, -0.0384756, 0.00199494, 0.0245456, -0.00244264, -0.031214, 0.0224226, -0.00927887, 0.000932065, -0.0268801, -0.00660336, 0.00523032, 0.00714536, 0.0151897, -0.0614099, -0.0550718, 0.00232949, 0.00301656, -0.0025507, -0.0154192, 0.00320644, 0.00722059, -0.04021, 0.00293627, 0.0104029, -0.00868498, 0.00518524, 0.00498796, -0.0051662, -0.00334043, -0.0252618, -6.63551e-05, 0.0260679, 0.0218787, -0.00462991, -0.000906671, 0.0271812, -0.0115982, 0.0135827, 0.0100785, 0.0112402, 0.00953637, -0.0169963, 0.0219038, -0.0496882, 0.00652114, 0.0338101, -0.0163415, -0.00396905, -0.0284613, -0.011701, -0.00725254, 0.0433735, 0.0082686, -0.00179168, -0.00888878, -0.147758, -0.0851158, -0.0235465, -0.0381497, -0.0243208, 0.00302326, -0.134068, -0.0126283, -0.00752489, -0.132479, -0.0107257, -0.00136825, -0.261097, -0.0104557, -0.0752462, -0.0121505, -0.0119209, -1.17092, -0.573178, 0.00159099, 0.0118993, -0.346932, -0.0200223, -0.0195066, 0.587533, 0.560196, 0.493491, -0.527882, 0.000326451, -0.0271167, -0.0236184, -0.581911, 0.507934, -0.0660746, -0.0595573, -0.0183618, 0.51716, 1.09039, -0.0307244, -0.0544133, -1.14714, -0.0345666, -0.559193, -0.117717, -1.08649, 0.855824, -0.0243366, 0.168566, -0.0373899, 0.21854, 0.432759, -0.298358, -0.0192025, 0.0092411, 0.133062, 0.191239, -0.0130509, -0.364382, -0.0047934, -0.0412377, -0.00735891, -0.0511976, -0.258692, 0.162167, -0.913795, 0.444326, 0.37845, 0.592387, -0.0118139, -0.0403955, -0.220329, -0.0283528, 1.02488, 1.18579, -0.0340672, -2.90306, 0.0117552, -0.149825, -0.235859, 0.000401431, -0.0306722, -0.629623, 0.113504, -0.0310293, -0.0348593, -0.518645, 0.32461, 0.0286488, -0.0164551, 0.677242, -0.00906818, -0.0402516, -0.0383809, -0.00861697, -0.669748, -0.54707, -0.0109734, -0.00865645, -0.0306232, -0.0515612, -0.040548, -0.0343644, -0.0657345, -0.040193, -0.00922729, -0.348814, -0.00352921, -0.262001, 0.866155, -0.0282998, 0.951625, 0.460651, -0.0364749, 0.0641558, -0.0202788, -0.299521, -0.271952, -0.0244637, 1.14485, 0.0671268, -0.0336931, -0.336647, -1.39739, -0.0251499, -0.300661, 0.000387496, -0.0109743, 0.100747, 0.37122, 0.108041, 0.262745, -0.28685, 0.120846, -0.0195016, 0.651559, 9.40081e-05, -0.00653803, 0.16114, -0.000628732, -0.0125048, -0.141348, -0.0362394, -0.0751607, -0.0243325, -0.00784688, -0.234081, 0.0713139, -0.0124588, -0.103796, 0.162762, -0.013487, -0.00499522, -0.134749, 0.0159257, 0.0506091, 0.157723, -0.0315422, 0.166332, 0.00122739, -0.305713, 0.200589, 0.00420525, 0.466828, 0.0039851, 0.0852529, -0.0147625, -0.0102058, 0.308815, 0.025139, -0.00325675, 0.265387, 0.75768, -0.0127346, 0.157306, -0.0239065, 0.0398127, -0.0279322, 0.142724, 0.226256, 0.611007, -0.0307067, -0.0480408, 0.0833453, -0.146552, -0.0276243, -0.0538757, -0.0165521, -0.0124895, -0.00320429, -0.0155593, 0.0845784, 0.0679946, 0.0755828, 0.307335, -0.335388, 0.126575, -0.0342781, -0.0695095, -0.233206, -0.0120542, 0.246522, 0.102188, -0.020148, -0.0146345, 0.00911107, -0.104387, 0.046807, -0.00163474, 0.00349714, -0.418281, -0.039158, -0.0412684, -0.0190225, 0.167518, 0.305711, -0.0425497, -0.0242844, -0.154761, 0.0115232, -0.0265883, -0.0356156, -0.0344552, -0.0948085, 0.0416007, 0.0359837, -0.0181837, -0.0394562, -0.0378079, -0.00163651, -0.0262499, -0.0351661, 0.00132348, -0.0247164, 0.656406, 0.00647145, -0.00915956, 0.286989, -0.0300068, 0.00671428, 0.200718, -0.0418907, 0.0378901, -0.0144609, 0.0455045, -0.016067, -0.0181646, -0.0211643, 0.113251, -0.0405579, -0.233679, 0.123205, -0.0151444, 0.0684019, 0.00246192, 0.00256196, 0.156809, 0.0327979, 0.0665497, 0.093742, 0.132299, 0.147997, -0.00581631, 0.334847, 0.00252262, -0.008414, 0.0228046, 0.0028398, -0.0084158, 0.134338, 0.00781086, 0.00556776, 0.00456507, 0.00316631, 0.00447211, -0.00914889, -0.00242336, -0.0161391, 0.0633529, -0.015072, 0.00442578, 0.0462048, 0.0263402, -0.0323794, 0.0010523, -0.00480231, -0.0718952, -0.00581997, 0.0836812, 0.0154005, -0.0237818, 0.0677899, 0.00764652, -0.0742286, -0.0844695, -0.00204285, 0.0933289, 0.0185865, 0.00195749, -0.123437, 0.0407476, 0.0613977, 0.0413266, -0.00891705, -0.0170586, 0.0125272, 0.0479883, -0.00216986, 0.00126801, 0.00648956, -0.00853702, -0.0818561, 0.133282, -0.00192585, 0.018132, 0.000358434, -0.0132012, -0.0276963, 0.00601519, 0.0259478, -0.0339771, -0.0439654, -0.12059, -0.00980267, -0.103426, -0.00418666, 0.000109877, 0.0620716, -0.00626682, -0.0306566, -0.040169, 0.017436, 0.0353471, 0.00439513, -0.00636356, 0.0019528, 0.00821124, 0.000587298, 0.0244395, -0.0641721, 0.0160538, -0.00618123, -0.0417321, -0.21765, 0.0032554, 0.014728, 0.00513286, -0.000797222, 0.00441399, 0.00937362, 0.00545216, 0.0982594, 0.0524247, -0.0420038, -0.0046151, 0.000380427, 0.0117489, -0.00131256, -0.0169622, 0.0140984, 0.00187826, -0.0107001, -0.00558178, 0.00300672, 0.0830153, 0.117365, -0.00123344, 0.1384, -0.142282, -0.00105929, -0.0193153, -0.00186056, -0.0481367, 0.028844, -0.00997905, -0.0445063, 0.032636, 0.0241199, -0.0230584, 0.0637819, 0.0707597, 0.167229, -0.01625, -0.0128413, -0.117501, -0.0230711, -0.103066, 0.0625088, -0.0250368, 0.000330122, -0.00357216, 0.0227716, -0.0174844, -0.0122028, 0.00370938, 0.00939394, -0.0161681, 0.0869436, -0.0264966, 0.0374275, -0.0256934, -0.0139135, 0.0981267, -0.234437, -0.00254933, 0.0260658, -0.238364, 0.005088, -0.00121518, 0.0382282, 0.111839, -0.23934, -0.188572, 0.000976488, -0.175839, 0.00279604, -0.257729, -0.129443, -0.0735409, 0.0207138, 0.00245846, -0.0143359, -0.129614, 0.008104, 0.0797492, -0.184283, -0.0297193, 0.121334, -0.0737505, 0.00300698, 0.0874879, -0.00483524, -0.15374, -0.0104155, -0.199354, -0.17263, -0.0400743, -0.0238246, -0.111529, 0.0202566, -0.149336, 0.00334454, -0.0155095, -0.0151727, -0.027999, 0.00630678, 0.00217472, 0.010555, 0.0565335, -0.0044579, 0.00843622, 0.221501, 0.0514943, -0.0338566, -0.00018028, 0.220163, -0.00573888, -0.0303583, 0.0341749, 0.00784867, -0.0135753, -0.00059879, 0.0175142, 0.0164533, 0.00284398, -0.00218323, 0.114226, -0.0365925, -0.0591731, 0.00870853, -0.0578461, 0.0299473, 0.0615701, 0.00533562, -0.188177, 0.00184983, -0.0563593, 0.0264157, -0.0012635, -0.000979421, -0.0544909, -0.242714, -0.0088607, -0.0261802, -0.0242108, -0.014012, -0.0204441, -0.015903, -0.00681743, -0.017652, -0.155256, 0.00550662, 0.012514, 0.149535, -0.0327179, -0.00454149, -0.0876687, -0.03009, -0.00961759, 0.009631, 0.0247035, -0.187689, -0.0389938, -0.0274378, 0.0111455, -0.00725048, -0.17238, -0.158228, 0.0960465, 0.565197, 0.022677, -0.00952462, -0.155063, 0.103812, -0.19021, -0.174316, -0.215194, -0.175169, 0.00367307, -0.181159, -0.016835, 0.0172787, 0.0289575, 0.00573283, 0.00691899, 0.0182976, -0.00616286, 0.00519844, -0.00821735, -0.00350612, 0.0203996, 0.013229, -0.000749221, -0.00699026, 0.00980342, 0.0103813, -0.0191979, -0.0379965, -0.0137133, 0.00762383, 0.00644408, 0.0208852, -0.00211046, 0.00778136, -0.0402652, -0.00760372, 0.00231865, -0.00654635, 0.00044841, -0.00438974, -0.00455881, -0.0168275, -0.0127718, -0.0204964, -0.0194005, 0.00111407, 0.00325481, -0.0171348, -0.00370667, 0.00918352, -0.0118359, -0.00281251, 0.00710372, 0.0233259, 0.0126874, -0.00172946, 0.0143391, -0.00641625, 0.0292486, -0.00933296, 0.0130786, 0.0109745, 0.00927238, 0.013073, 0.00988371, -0.0201397, 0.00773657, -0.0206392, 0.00467146, -0.00846429, -0.0247233, -0.00513684, -0.00715302, 0.00815213, 0.0150781, -0.0066668, -0.00914262, -0.011364, 0.00502928, -0.00771764, -0.0153857, -0.0384014, -0.00362719, -0.000306705, 0.00152591, 0.0028437, -0.000820181, 0.00405356, 0.00511884, -0.0307222, 0.00319544, -0.00951475, 0.00109258, 0.00975197, 0.00631245, 0.00288202, -0.00316987, -0.0188127, -0.00991675, -0.00181032, -0.0127018, -0.00342347, 0.000619935, -0.0110272, -0.012429, -0.00750409, 0.00616173, -0.0215814, -0.00147976, -0.0106632, -0.010033, -0.00114376, -0.0156739, -0.0217143, -0.0116534, -0.00550454, -0.00452656, -0.00400023, 0.0221648, 0.0396395, 0.00552685, -0.00615096, -0.00807115, -0.00169588, -0.0285807, -0.0209138, 0.0260399, -0.00646594, 0.00703034, 0.0025859, 0.0107126, 0.0119093, -0.0139568, -0.0148447, -0.013456, 0.017355, -0.0237659, -0.00873169, 0.0046192, -0.0164285, -0.0747773, -0.00874663, 0.00586902, -1.55647, -0.0149853, -0.0258065, -0.0102098, -0.00439535, -1.77568, -0.435646, 0.000328037, -0.451309, 0.236843, -0.0020058, -0.0145944, -3.47036, 0.282822, -2.14914, 1.0348, -0.0101106, -0.461164, 0.0101838, -0.349928, -0.425414, -0.0350482, -0.272581, -0.00482563, -0.0271423, 0.004875, 0.000933506, 0.931445, 1.05817, -0.0181994, -0.121856, 0.480451, 0.594788, -0.116808, -0.0167675, -0.32641, -0.0599327, 0.364463, -0.243854, 0.942441, 0.00124306, -0.0240334, 0.546549, -2.5231, -0.0160185, -0.27888, -0.0143969, -0.00363243, 0.00760263, -0.0129986, -0.206064, -0.129404, 0.224845, -1.804, -0.0332781, -0.0926498, -0.0218441, -0.0062175, -1.1403, 0.00894847, 0.1707, 1.33926, -0.011283, 0.260528, -0.0249972, -0.304148, -0.731049, -0.018353, -0.00787566, -0.140156, -0.904647, -0.00113312, 0.022092, 0.498811, -0.374371, -1.36664, -0.00579662, 0.280091, 0.00119996, 0.00379709, -0.0302849, -0.0129964, -1.32081, -0.221578, -2.24045, -0.0230139, -0.0075118, -0.0101424, -0.0200985, -0.0183219, -0.0160109, -0.0671582, -0.0216244, 0.993591, -0.00600206, -1.79156, -1.63271, -0.0218806, 0.586081, -0.218142, -0.000931128, -0.0243612, -0.00814909, -0.666129, 0.331885, -0.0376516, 0.491316, -0.5066, -0.0309527, 1.24842, -0.27404, 0.935549, -0.135189, -0.0338095, -0.00896068, -1.91612, -0.711203, -1.41395, 0.322007, 0.231854, -0.303322, -0.0253378, 0.373406, -0.0405655, -0.0147134, -0.355581, -0.00857208, -0.0369375, -0.191848, -0.0456662, -0.0618884, -0.0252439, 0.00511628, -1.08027, 0.23448, 0.00101915, 0.641353, 0.443468, 0.00248877, -0.0051103, -0.0556111, -0.166435, -0.629315, 0.00164247, -0.00640894, -0.152702, -0.0270893, -0.799143, -0.462549, -0.057528, 0.0554379, -0.0318478, 0.0580541, -0.436268, -0.0217808, -0.38674, 0.873415, -0.00609412, 0.836483, 0.391037, 0.414505, -0.529747, -0.0179904, -1.07197, -0.0245631, -0.131042, 0.323845, 0.132164, -0.0161883, -0.0211576, 1.51641, 0.112388, -0.00380912, 0.0507439, -0.0055853, -0.0197713, -0.000764814, -0.0523717, -0.184906, -1.67058, 0.0765393, 1.29933, 0.188986, -0.884192, -0.0208036, -0.0328672, -0.167146, -0.0364609, -0.422558, -0.994677, -0.0146415, -0.319405, -0.0133583, -0.324776, -0.59483, -0.00186909, -0.00640129, -1.56452, 0.309603, -0.0431715, -0.0239648, 0.127447, -0.663811, -0.103109, -0.0154303, -0.659778, -0.000477471, -0.0279693, -0.0309736, -0.0268255, -0.25403, -0.365535, 0.292073, -0.00216923, -0.0230316, -0.0342992, -0.0295181, -0.0331344, -0.0384842, -0.00141823, -0.0207519, 0.298004, 0.0202514, -1.09775, 0.741876, -0.0303216, -0.29626, 0.695351, -0.0398725, 0.0969844, -0.0121415, 0.0542668, -0.272133, -0.0368751, 0.0283909, 0.439204, -0.0494606, -0.588903, 0.698985, -0.892708, -0.296345, 0.0106915, -0.0145097, 0.723303, -0.110773, -0.93807, 0.826571, -1.19798, 0.0449176, -0.0292434, -0.593949, -0.0145039, -0.00604329, -0.841401, -0.00588187, -0.0120827, -0.391604, 0.00148816, 0.0159127, -0.0140548, -0.00826272, 1.17075, 0.0306131, -0.0200552, -0.688908, -0.886956, -0.00686071, -0.00911322, -1.21538, -0.441872, -0.893224, 0.183047, -0.00770824, 0.064446, 0.0107161, -0.223643, -0.0400054, -0.0843811, -0.28935, 0.014036, 1.39429, -0.317742, -0.0025453, -0.000440732, -1.64199, 0.00508575, -0.236885, 0.45078, -1.06311, -0.553577, 0.000276698, 0.209868, 0.00210377, 0.709531, 0.259823, -0.149059, -0.00256416, -0.00314517, -0.485355, 0.806486, 0.00496451, -0.437706, 0.0156569, -0.00437042, -0.00360743, -0.00548279, -0.642028, -0.431006, 0.0463849, 2.18623, -0.869794, 0.715802, -0.000138613, 0.00666784, -0.783269, -0.00369465, -0.7563, -0.454497, -0.0103658, 0.139959, -0.0084413, -0.551024, 0.0388557, -0.0215759, -0.00357971, 0.450923, 0.0860961, -0.00442924, 0.0041942, 0.426358, -0.625765, -0.0864707, 0.010954, -0.48481, 0.00282228, -0.00763665, 0.00216268, 0.00930754, -0.8786, -0.72644, -0.529657, 0.00138963, -0.00187928, 0.0119618, -0.000226796, 0.0110439, -0.00578511, -0.00269404, -0.0172134, -1.04983, 0.00839991, 0.317362, -0.126572, -0.00491965, 1.41856, 0.266535, 0.0198833, -0.000669047, -0.0187433, 0.0989259, -0.188802, 0.00926242, 0.072288, -0.507081, -0.00560531, -0.218824, -0.153408, 0.140505, -0.713049, 0.0042041, 0.0269529, -1.93168, -0.172855, -0.011948, -1.17764, -0.597892, -0.0111081, 0.0142085, -0.244744, 0.0201959, 0.0101477, -2.35822, 0.0059113, -0.0191705, 1.84283, -0.000708741, -0.00392742, -0.0151319, -0.0386063, 0.513898, 0.0855602, -0.0156759, -0.708231, -1.08536, -0.00423148, -0.0253422, 0.256748, 0.292635, -0.263041, 0.781255, -0.0189185, 0.0266564, 0.000554848, 0.447319, -0.299657, -0.0368018, 0.217053, -0.0191917, 0.723783, -0.223544, -0.000500523, -0.340889, -0.413267, 0.0109844, 0.866377, -0.890809, 0.389, -0.527361, -0.00367342, -0.219882, -0.0401384, 0.667592, 1.32446, 0.00184737, 0.00736493, -0.0581699, 0.167615, -0.175119, -0.00781329, 0.30389, -0.0163167, 0.0210978, 0.0044695, 0.00512785, 0.263664, 0.00748641, -0.238691, 0.0489974, 1.11715, -0.23125, -0.0331058, -0.0124468, -1.12622, -0.00403872, -0.342467, -0.698154, -0.0105212, 0.0187072, -0.0051306, 0.420517, -0.287386, 0.00655341, -0.0054676, -0.847352, 0.268303, -0.0117186, -0.0145932, -0.394821, 0.707297, -0.682757, -0.0173823, 1.26125, -0.002477, 0.00532768, -7.93649e-05, -0.0150346, 0.339271, 0.0747784, 0.694576, -0.0112388, -0.00114722, -0.00733673, 0.00101054, -0.00156775, -0.0480341, -0.00772258, -0.0218244, 0.665427, -0.000600404, -0.517699, -0.208092, -0.017654, 0.583734, -0.165145, -0.0434879, -0.00999087, -0.0250279, -1.86838, 0.750962, -0.0299083, 0.213328, 0.569031, -0.00975394, 0.697865, -0.110322, 0.530673, 0.450863, -0.0197955, -0.014017, 0.549373, -0.0407134, 0.768971, -0.348232, 0.216674, 0.155326, -0.0271862, -0.554544, 0.0230957, -0.00405793, 0.321949, 0.00386682, -0.0323436, -0.0140606, -0.0041189, -0.0119742, -0.0109591, -0.0265722, -1.27859, 0.0386766, 0.00588152, -0.720473, -0.0800044, -0.0114292, 0.00149694, -0.168774, -0.828661, -1.35364, 0.755106, -0.0184094, 0.269471, -0.0195284, 0.700152, 0.0358815, -0.00850045, 0.799726, -0.0128785, -0.81044, -0.280849, 0.0109607, 0.124816, 0.422609, -0.0137711, -0.469472, -0.0863724, 0.995319, -0.359427, -0.0102905, -1.01942, -0.018093, 0.585608, -0.181362, 0.512366, 0.00450977, -0.0848099, 0.305893, -0.226241, -0.0182894, -0.394265, -0.00632943, -0.00137086, 0.0148434, -0.00489919, -0.473432, -1.54796, 1.01482, -0.265368, -1.46135, -0.262429, -0.0388927, -0.0353567, -0.0504418, -0.0368129, -0.483388, -1.0556, -0.0215219, 0.348902, 0.00409125, -1.26381, -1.08792, -0.00702907, 0.0048284, -0.64088, -0.882922, -0.0153811, 0.00570676, -0.176662, -0.106081, -2.45636, -0.020572, -1.13911, -0.00639351, -0.0446715, -0.030187, -0.0223121, -0.852938, -0.730918, -1.61299, -0.0124443, -0.0140779, -0.0372142, -0.0430688, -0.0436243, -0.0510991, 0.0119487, -0.0233504, -0.352598, -0.00237992, -0.870331, 0.193226, -0.0110416, -0.346425, 0.167651, -0.027872, -0.0130238, -0.00876487, 0.574546, -0.307598, -0.0218916, 0.499389, -0.789731, -0.0245702, 0.576093, -0.393606, -0.0404179, 0.0289182, -0.0171747, -0.0290255, -1.61974, -0.541051, -0.441641, -0.203554, -0.354334, 0.0571583, -0.0064328, 0.268269, 0.00374382, -0.00537642, 0.0120326, -0.0109762, -0.0118977, 0.551492, 0.0111334, -0.031258, -0.0132419, -0.00304877, 0.216349, -0.972033, -0.00894197, 0.263656, -0.0563518, -0.0155929, -0.00997343, -0.294935, -0.0704694, -0.476736, -1.63609, -0.00190783, -0.416314, -0.00111633, 0.597817, 0.167225, -0.0136886, -0.265562, -0.0216354, -0.313495, 0.192736, -0.0654575, -0.210355, 0.421451, -0.0477098, 0.213344, -0.205576, 0.653495, -1.3164, -0.023156, 0.363499, -0.0149171, -1.16597, -0.115451, 0.815548, 0.00533653, -0.12021, 0.481601, -0.414704, -0.0111981, 0.183459, -0.04743, -0.00417503, -0.0310468, -0.038778, -0.644969, -0.780908, -0.205809, 0.410606, 0.439817, -0.274908, -0.0241373, -0.033256, 0.21895, -0.0357381, -2.06665, -0.129564, -0.0187718, 0.0399019, -0.0429348, 0.240358, 0.151791, -0.0163405, -0.0117901, -0.779152, 0.0112069, -0.00202123, -0.0331298, 0.0620887, 0.288788, 0.183535, -0.0312107, 0.478339, -0.000862133, -0.016802, -0.0213077, -0.0324864, 0.0661263, -0.930108, 0.185458, -0.0357218, 0.00794386, -0.00436661, -0.0120727, -0.0278222, -0.0628827, -0.0165822, -0.0179232, 0.280448, -0.0106532, -0.239847, -0.392748, -0.0266516, -0.138213, -0.655096, -0.0206471, -0.0306158, -0.0183625, 0.428107, 0.552527, -0.0312896, -2.76041, 1.11366, -0.0421543, -0.0695048, 0.30429, -0.586658, -0.306875, -0.0160448, -0.0296352, 0.929394, -0.778577, 0.050223, 0.874185, -0.304865, -0.0852986, -0.0184018, -0.15149, 0.0401449, -0.000607728, 0.00298319, -0.00395042, -0.0159148, 0.401704, 0.0104339, -0.0128726, -0.00574534, -0.0086995, -0.10471, 0.040579, -0.00598438, 0.069573, 0.0827932, -0.0264136, 0.00115745, 0.0504606, 0.119679, -0.0154824, 0.052754, 0.0016913, 0.0567476, 0.00773922, 0.00878533, 0.0396771, -0.00779707, 0.188377, -0.0153443, 0.0734928, -0.0134012, 0.0211209, 0.208557, 0.117797, -0.00121343, 0.192656, -0.00518578, 0.0616744, 0.134501, -0.0176564, 0.0555151, -0.0286777, 0.0720801, 0.0820183, 0.00789796, -0.0206081, 0.0488751, 0.0706029, 0.0602717, -0.00374136, 0.0745346, 0.004047, -0.00556728, -0.000913564, -0.0104054, 0.0662143, 0.110507, 0.0370597, -0.0151513, -0.126477, 0.0305971, -0.0387021, -0.010676, 0.128054, 0.00191758, 0.00875449, 0.120339, -0.0126053, -0.0659357, -0.0104928, 0.0908867, 0.0429783, -0.00568752, -0.00312996, 0.0139931, 0.00339105, -0.0238198, 0.00442389, -0.00335378, 0.045329, 0.0391807, -0.00454957, 0.136109, -0.00939043, 0.0081829, 0.00651355, -0.00546905, 0.0423883, -0.0955983, 0.138491, -0.0252212, -0.00449775, -0.00537309, -0.0064046, 0.00622705, -0.0148707, -0.00244855, -0.00912108, 0.0134255, -0.000908308, 0.0246191, -0.0364558, -0.0095709, 0.00845948, -0.0187318, -0.0177455, 0.0427637, -0.0148407, 0.0627892, 0.00795869, -0.0208944, 0.0330897, 0.0362115, -0.0245697, 0.134139, 0.0475361, 0.132851, -0.0669938, -0.00144448, 0.000606917, 0.0495522, 0.091864, 0.0602878, 0.0504144, 0.0121302, 0.0285984, -0.0160712, 0.0371033, -0.00220056, -0.0186276, 0.0104178, -0.00183347, -0.0187753, 0.022487, 0.00577024, 0.0319339, -0.00600914, 0.014477, 0.00428713, 0.0338999, -0.0133321, 0.0623156, 0.00221968, -0.00216837, -0.0206739, 0.00457572, 0.0254109, 0.0253991, 0.0194154, 0.0066198, 0.0293256, 0.000631844, 0.016002, 0.0104476, -0.0889415, 0.103669, -0.00400549, 0.0621087, -0.0633631, 0.00270717, 0.0669016, 0.0186599, -0.0825712, -0.0160281, -0.110409, 0.0207081, 0.0625335, 0.0817304, 0.0609483, 0.00233526, 0.0232082, -0.0399641, -0.0171816, -0.000824747, 0.00352164, 0.00456984, 0.0131447, -0.0131251, 0.0361218, -0.0172195, 0.0066033, -0.0439588, -0.0156367, -0.0132939, 0.0903276, -0.00478983, -0.00291211, 0.0412553, -0.0166657, 7.42445e-05, 0.0119632, 0.00701648, -0.00471208, -0.13442, 0.113802, 0.0756872, -0.00982, 0.0153445, 0.0110777, 0.0329378, 0.00598966, 0.00490664, 0.00774518, -0.0765738, 0.0101123, 0.00410834, -0.0441373, -0.010411, 0.12884, 0.00155746, -0.0154889, -0.0187039, -0.0313587, -0.018509, -0.00732768, 0.0256518, -0.0119256, 0.0179718, -0.017868, 0.0158403, -0.0254932, -0.0162973, -0.0250133, 0.0233083, 0.00116689, -0.00285872, -0.00469852, -0.0489047, 0.00151359, -0.0445566, 0.012357, 0.00425169, -0.0178336, 0.00382928, -0.00630021, 0.000683027, 0.00741796, 0.05064, -0.00692268, -0.0552644, -0.0131126, -0.0199798, -0.00237593, -0.0134752, 0.0457926, 0.0375789, -0.00650721, 0.00296658, -0.0102185, -0.014132, 0.0241497, 0.0259813, -0.000682883, 0.0046581, -0.0232782, 0.00435117, 0.00186279, -0.00444781, 0.0102741, 0.00985472, -0.00259081, 0.400935, 0.0291324, 0.0175795, -0.0112956, -0.0137413, 0.299246, 0.0326823, 0.00135213, 0.123071, 0.102351, -0.0157633, -0.00986767, 0.104642, 0.111554, 0.0404214, 0.0132988, -0.0227185, 0.0392217, -0.0183536, 0.00748612, 0.0551315, 0.0172066, 0.224514, -0.00444318, 0.0819189, -0.00251423, -0.0113757, 0.186555, 0.0327594, 0.0149097, 0.18408, -0.0266308, 0.0697036, 0.156742, -0.00678754, 0.0567223, -0.020044, 0.0534702, 0.0430995, -0.10207, -0.00889971, 0.0140292, 0.0488801, 0.0655088, -0.0124138, 0.0765301, -0.0188641, 0.00107069, 0.0107381, -0.0250403, 0.0958629, 0.102862, 0.0567945, -0.0574854, 0.296346, 0.0735975, -0.0259257, -0.0320239, 0.112503, -0.00159251, -0.0565312, 0.113504, -0.0178232, -0.0294767, -0.000820539, 0.0926703, 0.0906491, 0.00102507, 0.00117329, -0.0444452, 0.0126726, -0.0154707, -0.00805759, 0.0603172, 0.0653424, 0.0693045, -0.00991135, 0.129007, -0.0141449, 0.00920313, -0.0219869, -0.0275664, 0.0767384, -0.053068, 0.111144, -0.0150311, 0.000574133, -0.01744, -0.0222612, -0.000938298, -0.0174005, 0.0194466, -0.00834661, 0.0627431, 0.00815115, -0.0227606, 0.144318, -0.0175997, 0.0220752, -0.015505, -0.0439891, -0.00569382, 0.000712286, 0.0936107, 0.0329014, -0.0309922, -0.10649, 0.0745103, -0.00987372, 0.146588, 0.0625451, 0.018582, -0.123489, -0.00469946, 0.0246512, 0.0668065, 0.0872966, 0.0457371, 0.0591379, 0.0204952, 0.0529228, -0.00822133, 0.0854675, -0.00426401, -0.0217263, 0.072899, 0.00941805, -0.0046736, 0.419436, -0.0129365, 0.0425735, -0.0125083, -0.0116971, -0.0153026, 0.0462464, 0.000815527, 0.0937247, 0.0783505, 0.00155903, -0.0209394, 0.0874286, 0.0993274, 0.0332352, 0.0396695, -0.01674, 0.0331837, -0.00348915, 0.0590439, 0.0655036, 0.00357469, 0.179978, -0.0123796, 0.0236377, -0.0173292, -0.0222888, 0.233989, 0.134252, -0.00433908, 0.152003, -0.0299616, 0.0744302, 0.137958, -0.0211344, 0.0643273, -0.0347084, 0.0400572, 0.0377987, -0.00508209, -0.0291036, -0.0133962, 0.0545832, 0.0702965, -0.0140433, 0.0774252, -0.0310029, -0.00499914, -0.00202357, -0.00784211, 0.0797942, 0.123467, 0.0573241, -0.0966018, -0.0187395, 0.069207, -0.0304036, -0.0224331, 0.0572172, -0.0190963, 0.073845, 0.122906, -0.0226439, -0.0131331, 0.0167944, 0.0817297, 0.0793072, -0.00963283, -0.00355785, -0.0805527, 0.0109692, -0.00482633, -0.0218347, -0.051408, 0.0510693, 0.0225135, 3.83255e-05, 0.13195, 0.0301181, -0.00485787, -0.0431002, -0.0181756, 0.0788218, -0.0843758, 0.127212, -0.0350262, -0.0258955, -0.0118069, -0.015254, 0.000486406, -0.0115524, -0.0023552, -0.00620967, 0.0505231, -0.0248405, -0.0129146, 0.15032, -0.0248583, -0.0026154, -0.0387088, -0.0411181, 0.00185486, -0.00170007, 0.104531, 0.0341615, -0.0191612, 0.067461, -5.90807e-05, -0.0126134, 0.137396, 0.0673642, 0.0929914, 0.111472, 0.00925952, 0.0120188, 0.0500234, 0.0472315, 0.0378932, 0.0185894, 0.0206884, 0.061377, -0.0198796, 0.0432029, -0.0226303, 0.00322323, -0.4995, -0.0215625, -0.0148685, 0.703589, -0.0234558, -0.0457016, -0.0070548, -0.0247935, 0.0189705, 0.397434, -0.00256784, -0.958998, -1.09582, -0.0253634, -0.0144244, -0.454687, -1.38389, -1.36056, 0.917763, -0.00720505, 0.319973, -0.0150772, -2.12417, 0.0870479, -0.0471012, -1.00497, -0.00173568, 0.0578437, 0.327908, 0.0229444, 0.472058, -0.00550553, -0.0478625, 0.0196139, -0.63463, 1.00238, -0.289308, -0.0251289, 0.292453, -0.0276239, 0.604761, -0.164042, 0.545479, -0.0183202, -0.0779868, 0.294539, -2.85119, -0.0087599, -0.243924, -0.0261285, -0.0213528, 0.00890152, -0.0471027, -0.328172, -0.59998, 0.139223, -1.04686, -0.865306, -0.488052, -0.0293634, -0.0407446, -0.107057, -0.0244398, -0.224967, -0.451055, -0.0156384, 0.717205, 0.00191623, 0.346589, -0.723723, -0.0409067, -0.0212014, -1.54789, -0.822656, -0.0303823, -0.0131996, 0.893078, -0.65837, -0.736399, -0.00457545, -0.404226, -0.0405169, -0.0386667, -0.0305547, -0.020952, -0.935252, -0.407991, -1.9067, 0.00143514, -0.0324442, -0.0423463, -0.0116724, -0.0319072, -0.0486758, -0.049774, -0.00837946, -0.134498, 0.0145692, -0.718234, -0.114243, -0.0418979, -2.24901, -0.0579602, -0.0548926, -0.0045361, -0.0125093, -0.225998, 0.0332888, 0.00127023, 0.630383, -0.646092, -0.0155864, 0.956779, 0.180177, -0.357688, -1.17971, -0.00976343, 0.000586266, -1.28607, -0.43288, 0.0761458, 0.365389, -0.35783, -0.117297, -0.00235704, 0.584501, -0.0350295, -0.0151978, 0.0492302, -0.0114219, -0.0188712, 0.466652, -4.75636e-05, 0.0321398, 0.0015932, -0.00670918, 0.406142, 0.039774, 0.0123928, 0.0893305, 0.100493, -0.000159587, 0.0100219, 0.0570363, 0.116502, 0.04009, 0.0687938, -0.0356731, 0.0658812, -0.0131638, 0.0430958, 0.0365107, -0.0117998, 0.196133, -0.00884635, 0.105705, -0.0265482, 0.0106088, 0.232788, 0.12199, 0.00908868, 0.0722404, -0.0130434, 0.111114, 0.155353, -0.0190389, 0.06041, -0.041006, 0.0120084, 0.0605665, -0.0369818, -0.0284876, -0.00910469, 0.10959, 0.0396575, -0.00662052, 0.064533, -0.0116969, -0.00882472, -0.00722654, -0.0267886, 0.0762753, 0.140248, 0.0826785, 0.0340193, 0.421941, 0.0254071, -0.0329374, -0.00994419, 0.153696, 0.00207181, -0.0596102, 0.126696, -0.0406141, -0.0514706, -0.00208752, 0.0774919, 0.0848525, -0.0145135, 0.0169077, -0.0214189, 0.0225285, 0.000918562, -0.00827717, 0.0043794, 0.0674552, 0.0778233, -0.029853, 0.172813, 0.00353842, 0.00273135, -0.0348325, -0.019073, 0.071327, -0.0376036, 0.0922583, -0.0396289, -0.00581386, -0.00511348, 0.00179744, -0.00290469, -0.000136529, -0.000401831, -0.00841223, 0.106458, 0.0285742, 0.00292849, 0.158951, -0.0400013, -0.00194728, 0.0722136, -0.0259427, 0.00375741, 0.00301859, 0.0686491, 0.00795069, -0.0376383, 0.0329068, 0.0350604, -0.0194727, 0.0981752, 0.0402781, 0.110304, 0.00601318, -0.010858, -0.00262044, 0.047317, 0.086772, 0.0372149, 0.0320744, 0.0218828, 0.0526883, -0.014736, 0.0105835, -0.00322884, -0.0117669, 0.65373, 0.0020797, -0.00255533, 0.26467, -0.0108261, -0.0664166, -0.0037564, -0.0106792, -0.633024, -0.481818, 0.00769638, -0.0677827, -1.08271, -0.00820791, 0.00829938, 0.015588, -1.87591, 0.216309, 0.00317808, 0.00178442, 0.27717, -0.00866366, 0.368466, 0.0214432, 0.00537102, 0.214463, -0.0162309, -0.248189, 0.125929, -0.0281651, 0.487243, -0.214085, -0.0223489, 1.04627, 0.248995, -0.198115, 0.168175, -0.0133493, -0.0327886, -0.0357366, 0.868714, -0.151937, 0.361365, -0.0295531, -0.0794211, 0.466935, -0.578004, 0.00117202, 0.262887, -0.0247951, -0.0139706, -0.00404802, 0.00885957, -0.0382969, -0.949175, 0.0111549, -0.469739, 0.149108, 0.601566, -0.0286237, -0.0400582, -1.00958, -0.0479759, 0.462185, -1.15167, -0.0267969, -0.381216, -0.01025, -0.341353, -0.0919489, -0.00850012, -0.0146525, -2.0897, 0.614638, -0.0271988, -0.0403368, -0.857552, -0.278239, 0.226756, -0.00813544, 0.822858, 0.0134672, -0.0204699, -0.0280004, -0.0291216, -0.258013, -0.351141, 0.364934, -0.00498676, -0.0268286, -0.0103619, -0.0608524, -0.0077478, -0.0645364, -0.0251475, -0.0238836, -0.954928, -0.00798969, 0.288471, 0.247417, -0.0111175, 0.336905, -0.220668, -0.0206888, 0.0622676, 0.00572188, 0.0838173, 0.594746, -0.0251627, -0.103377, 0.229496, -0.0254758, 0.69988, 0.0465824, -0.881672, -0.708167, -0.00858578, -0.018531, 0.173484, -0.860494, 0.789633, 0.175069, -0.351073, 0.0648769, -0.0408899, 0.226145, -0.0644263, 0.0086999, 0.0249601, -0.00837399, 0.00254975, -0.00482476, -0.00308433, 0.0243783, -0.0034351, 0.00568811, -0.00325292, 0.00575807, 0.00216985, 0.0132896, -0.0267784, 0.0163002, -0.00708283, 0.00862126, -0.0179013, -0.00135777, -0.0137893, -0.00233608, -0.0164641, 0.0107405, -0.000689162, 0.00665305, 0.00863035, -0.0449065, 0.0079926, 0.0217377, -0.00986137, 5.85761e-05, -0.027637, -0.0102366, -0.00514744, -0.00400211, -8.58232e-05, 0.030303, -0.0085824, -0.0121164, -0.00780282, 0.00255197, -0.0198313, -0.0035182, 0.00763306, 0.00252074, -0.00252035, 0.0194659, 0.014224, 0.00113723, 0.0103021, -0.0133345, 0.00677229, -0.00114464, 0.0136033, 0.0241345, -0.0205, 0.0221688, -0.00173613, -0.00807688, 0.0245151, -0.00911876, 0.00961583, -0.0116744, -0.0137021, 0.0100459, -0.00981636, -0.00423885, 0.0044839, 0.0230509, 0.014157, 0.00612431, -0.00496428, -0.00458705, 0.00180964, -0.0096032, 0.00384433, -0.00785491, 0.00518911, 0.0210003, 0.0370939, 0.0158955, 0.00316434, 0.00403817, 0.00719414, 0.00422046, 0.000866004, 0.0135009, -0.00573515, 0.00952387, 0.0166707, 0.0177751, -0.0162798, -0.0124874, -0.0341722, 0.00974129, 0.0161817, 0.00633079, -0.013311, -0.00961227, 0.00582661, 0.00211333, 0.0108798, -0.0011235, -0.00290719, -0.00328179, -0.0138453, 0.00346092, 0.0128045, -0.0293237, -0.000684894, 0.0126217, 0.0228689, 0.00755194, -9.54372e-05, -0.00247038, -0.033796, 0.0118206, 0.000616384, 0.0122939, -0.000776438, -0.0249468, -0.0166863, -0.00278536, -0.00519467, -0.00623825, 0.00234347, 0.00567272, -0.017159, 0.00387686, 0.7783, -0.00752619, 0.0043604, 0.277384, 0.00140713, 0.00178214, -0.00900746, -0.0207571, -1.40724, 0.0428283, -0.0108821, 0.0617538, -0.214208, -0.00136276, 0.000128439, -1.6818, -0.333029, -1.57144, 0.234799, -0.0220404, -0.732957, -0.0124144, -0.0216548, -0.126916, 0.0424019, -0.835015, -0.0134657, -0.0236556, -0.998063, -0.0347681, 0.199353, 0.0184146, -0.0198233, -0.661831, 0.605172, 0.874992, -0.972689, -0.020092, 0.0341237, -0.0688869, 0.367066, 0.317263, 0.0705669, 0.0158566, -0.0585727, -0.205238, -2.19771, -0.015033, 0.0834818, -0.00229214, -0.0208247, 0.0145662, 0.00421663, -0.067305, -1.29378, -1.08247, -0.372497, 0.61165, 1.0368, -0.0301645, -0.00996069, 0.162678, -0.00735202, -0.512086, 0.300236, -0.0162223, 1.13593, -0.0132192, -0.121823, -1.84064, 0.0144512, 0.0111245, -1.36403, -0.0694902, -0.0113186, -0.00681814, -0.0596422, 0.623021, -1.9839, 0.00260399, -1.23589, -0.0113058, -0.0238989, -0.00952525, -0.0346685, -0.225562, -0.614946, -1.37553, -0.0175924, -0.00374405, -0.0315298, -0.00964915, -0.0577975, -0.00825096, -0.0259271, -0.00292201, 0.401708, -0.00813336, -0.0865015, -0.819906, -0.00112529, -0.214407, -0.164958, -0.00877459, 0.00458912, -0.0101185, 1.2178, 0.896275, -0.0514069, 0.163121, -0.191518, -0.02912, 0.219616, -0.554335, 0.820485, -0.484687, -0.0107316, -0.00608584, -1.1361, -1.09845, -0.718939, 0.83578, -0.710155, 0.142584, -0.0189223, -0.0575906, -0.0758867, 0.0119822, -0.546945, 0.00232636, 0.00343072, 0.23319, 0.0231666, 0.0118868, -0.00351568, -0.0247175, 0.644272, 0.886085, 0.00298456, 0.0519693, -0.285884, -0.0109449, -0.0104823, -0.111041, 0.35483, -0.12112, 0.276549, -0.0139271, 0.523369, -0.0141695, 0.813431, 0.424737, -0.0856707, 0.63867, -0.0036881, 0.984732, -0.162481, -0.061487, 0.562791, 0.318342, 0.0164167, -0.305262, -0.441294, -0.503463, 0.125789, -0.00668502, -0.443689, -0.0329532, -0.491221, -0.0779845, -1.69929, -0.00164381, -0.0589761, -0.637593, 0.905379, -0.0205575, -0.250077, -0.00166174, 0.00211669, -0.00568898, 0.0029006, -0.944026, 0.332442, 0.153838, 0.526834, 0.560647, 0.458631, -0.0356769, -0.0725386, 0.0489996, -0.0276162, 0.0806723, -0.427812, -0.0253989, -0.421985, -0.000396266, -0.812245, -0.361325, -0.000280807, -0.0211874, -0.477776, -0.30543, -0.0181032, 0.0108441, -0.677927, 0.282604, -0.783258, -0.0116322, -0.146385, -0.00686655, -0.0352088, -0.015926, -0.00736362, -0.205545, -0.502096, 0.180552, -0.0314366, -0.0110502, -0.029421, -0.0533347, -0.0397467, -0.0320138, -0.0065575, -0.0211606, 0.0862774, 0.00202783, 0.80425, 0.803998, -0.0122377, 1.11661, 0.373914, -0.0226135, 0.00940599, -0.0173865, 0.011118, 0.0490833, -0.0349197, -2.12285, 0.021502, -0.00720397, -0.86748, -0.0146168, 0.440301, 0.843809, -0.00572768, -0.0187707, -0.486396, 0.0885332, 0.0103122, -0.21588, -0.229982, 0.0234528, 0.00481923, 0.136231, -0.0500329, -0.0204643, 0.480172, -0.00694183, 0.0133653, -0.289114, 0.000639132, 0.0141871, 0.00681764, 0.0240645, 0.135006, -0.0291124, 0.00430015, 0.0448727, 0.0587696, 0.00892361, -0.00898798, -0.0715552, -0.00680238, 0.38588, -0.0619388, -0.0204081, -0.0211109, -0.00236993, 0.0637543, -0.0367025, 0.00325308, 0.294937, 0.0099705, -0.175679, -0.0065065, 0.00242475, 0.103924, 0.0808434, 0.0154866, -0.286814, -0.164914, 0.146598, 0.00519124, 0.00484587, 0.00600979, 0.0168753, 0.272548, -0.267395, -0.590579, 0.00249224, -0.00543658, 0.04209, 0.0865606, 0.0183634, -0.00433217, 0.0148647, 0.00642802, -0.0112363, 0.00980126, -0.0358404, -0.0258965, 0.0491056, -0.213079, 0.0539513, -0.189695, 0.0109253, 0.0229436, 0.546778, -0.00787234, 0.061859, 0.158525, 0.00197694, -0.633161, -0.00842408, 0.00697599, 0.0131081, -0.0043799, -0.0136836, 0.0140959, 0.214324, -0.0173512, -0.0176243, -0.176635, -0.0304991, -0.350646, -0.00268884, 0.471655, -0.0135907, -0.00538973, -0.0172495, -0.0105176, -0.0224178, -0.269962, 0.225909, -0.0107837, -0.00503484, -0.00331915, 0.011342, 0.00857607, 0.00359002, -0.0127661, 0.0125992, 0.0607896, 0.00838586, 0.103509, 0.00718292, 0.0138657, 0.435152, -0.0725228, 0.000182538, 0.00444511, 0.00354456, -0.0148656, -0.0250903, -0.0129798, 0.297945, 0.0233443, -0.0123623, 0.00406145, 0.177183, -0.137096, 0.103491, 0.0108532, -0.010366, -0.00575488, 0.0357726, -0.052297, -0.0820788, -0.00310316, 0.0160857, -0.00147027, 0.260387, -0.00146741, -0.0132919, 0.0120583, -0.00949404, 0.0144802, 0.0431938, -0.0090749, 0.0042829, 0.018276, 0.0190413, -0.00338047, -0.004234, -0.00140855, 0.0203243, 0.014359, -0.0197654, 0.00268123, -0.000601878, -0.0281134, -0.0124367, -0.0286599, 0.00122901, 0.00117566, -0.0069826, 0.0107302, 0.0121865, 0.00359371, -0.0501147, -0.00113431, -0.000751968, 0.00255887, 0.00263005, -0.00113106, 0.000307315, -0.00330619, -0.00633649, -0.0101787, 0.0371642, -0.0334002, 0.000877227, -0.00371677, 0.000848709, -0.0182071, 0.00194327, -0.0119531, -0.00215348, -0.00435497, 0.0287747, -0.00577936, 0.00022819, -2.50641e-05, 0.00255518, -0.00254926, 0.00266586, 0.00271448, 0.0149777, -0.0194593, 0.0220561, 0.00659283, -0.0174158, -0.0236643, 0.00976005, 0.00609136, 0.0341712, 0.0118931, -0.00449567, -0.0194707, -0.00657681, 0.00357951, -0.0181221, 0.0213771, 0.00674496, 0.0286497, -0.00339771, -0.0151226, 0.00572908, -0.000167701, -0.00712191, -0.0250231, -0.00543225, 0.0257203, -0.00436549, 0.0163112, -0.00370281, 0.00722395, 0.000306122, -0.00884477, 0.0259166, 0.00651727, 0.0108751, 0.01242, -0.000828981, 0.0118768, 0.0101568, 0.00953453, 0.000541075, -0.0031456, -0.00742976, 0.0163001, -0.00795116, 0.0126955, -0.0266922, 0.00777386, -0.00575351, -0.0110875, 0.0103194, 0.00801047, -0.00803827, -0.00635646, -0.0283476, 0.0024318, 0.0314983, 0.0166049, 0.0137672, -0.0165217, 0.0217847, 0.0368827, -0.0132396, -0.0150323, -0.00198774, 0.00381123, 0.0206572, 0.00808273, -0.0204421, -0.011844, 0.00377041, 0.0135691, 0.00577816, -0.00814565, 0.00732116, -0.881614, 0.0111626, 0.0172871, 0.584827, -0.0011243, -0.00489574, -0.000145984, -0.0140838, 1.0488, -1.08055, -0.00997364, -1.56854, -0.46324, 0.00653183, -0.0398731, -1.08344, -0.0893451, 0.185184, 0.329036, 0.00262586, 0.7656, -0.00462577, -0.509398, 0.871498, -0.0655394, -0.587848, -0.0137558, 0.167518, -0.402314, -0.0534815, -2.20765, 0.114732, -0.0519129, 0.158508, 0.556314, 0.0369336, -0.116241, -0.00158142, 0.123803, 0.0319886, 0.18059, -0.240366, -0.322015, -0.0109768, -0.0290831, 0.148515, -0.162439, -0.0203238, -0.341026, 0.00246184, 0.000335487, 0.00960107, 0.0104623, -0.278602, -0.279426, 0.238939, 0.873182, 0.317585, 0.232431, -0.00938348, 0.00393958, 0.289641, -0.00959834, -0.270735, -1.7745, 0.00578239, 0.754423, 0.00771635, -0.0450074, -0.058283, -0.0132477, -0.00179327, -0.127105, 0.0220952, 0.0144162, -0.00239587, -0.815851, -0.130363, -0.235284, -0.00582961, -0.696239, -0.0326758, -0.00840341, 0.00433485, -0.0137928, 0.29266, 0.247816, -0.139775, 0.00939718, -0.00941154, 0.0105456, -0.0248144, 0.0113959, -0.0164087, -0.023816, -0.0291242, -0.0479111, -0.00866524, 0.397786, 0.0987668, 0.0117868, -0.563379, -0.219558, -0.00729904, -0.0365783, 0.003118, -0.725413, -0.49461, -0.00413826, 0.00526667, 0.353651, -0.0150593, -0.386674, -0.292346, 0.345326, -0.690559, -0.0297782, 0.0143271, 0.282545, 0.0419314, -0.0810567, 0.123819, 0.538792, -0.250647, -0.0143465, -0.589002, -0.0330468, -0.00792613, 0.328383, 0.000542657, -0.00775136, -0.0580675, -0.0413904, -0.084642, -0.0290437, -0.00498197, -0.332054, -0.142237, 0.00523096, 0.119079, -0.0972102, 0.0248052, -0.0059513, 0.301781, -0.0534508, 0.213088, -0.580149, -0.014775, -0.131857, -0.000564851, 0.481563, -0.359238, -0.0181907, 0.0986359, -0.0120213, -0.12417, -0.0334828, 0.0022056, 0.117425, 0.305347, -0.000183095, 0.170852, 0.0662047, 0.151699, 0.0700156, -0.0129173, -0.0590781, -0.0430522, -0.512236, 0.0730886, -0.0479832, -0.0111495, -0.162174, 0.0850195, 0.137679, -0.00799206, 0.126166, -0.0197169, -0.00934293, 0.00686542, -0.0649515, 0.131644, -0.0318358, 0.17967, -0.0161238, -0.273736, 0.0567477, -0.0430003, -0.00224287, 0.276337, 0.00442741, 0.235824, -0.15669, -0.018629, -0.00744135, -0.00694745, 0.038498, 0.307818, -0.0133175, -0.0124028, -0.140658, -0.213323, -0.0375432, -0.0306551, -0.240118, 0.0644084, 0.269583, -0.00864514, 0.340025, -0.00787726, -0.0128366, -0.0338884, -0.0251322, 0.0755155, 0.320222, 0.471971, -0.0053933, -0.0239822, -0.00061523, -0.00298194, -0.013525, 0.0196552, -0.0125076, -0.00525868, 0.0978899, 0.0055748, -0.0141431, -0.0157597, -0.00744034, -0.0739851, 0.144722, -0.0274316, -0.0294149, -0.000577101, 0.383201, 0.396863, -0.0388407, -0.544799, 0.120514, -0.0200297, 0.384457, -0.205918, 0.010845, -0.0327836, 0.00269334, -0.00915758, -0.126015, 0.0170367, -0.0891661, -0.205217, -0.572707, -0.165567, -0.0392588, -0.30193, -0.00462712, -0.0103551, 0.828055, -0.0107883, -0.0155659, 0.322436, -0.026006, -0.0299647, -0.0114008, -0.0164889, 1.02956, -0.172485, -0.0156965, -0.184189, 0.18076, -0.0379785, -0.0129127, -0.386658, 0.152913, 0.0940333, 0.924361, -0.00383304, -0.296842, -0.0250606, 1.17889, -0.941113, -0.129007, 0.886218, -0.0156288, -0.343185, 0.117768, -0.0214556, -0.469912, -0.343301, -0.0465304, 0.608005, -1.05245, -0.145224, 0.269369, -0.022986, 0.214111, -0.00544414, 0.459951, -1.01746, 0.318749, -0.0314968, -0.102089, -0.578822, 0.828488, -0.0194082, 0.106596, -0.0272353, -0.0363004, -0.0388262, -0.015674, 0.00938664, -0.45094, -0.517348, -0.62379, -0.133825, -0.215949, -0.0191611, 0.00649827, 0.173545, -0.0280027, -0.566161, -1.47943, -0.0179616, -0.483223, -0.0116057, -0.512201, -1.07676, -0.0332648, -0.00979277, -0.123481, 0.2197, -0.0233243, -0.0341827, 0.0115653, -0.856939, 0.151163, -0.0411563, -0.587581, -0.0257251, -0.0306187, -0.000474621, 0.00878645, -0.149231, 0.485247, 0.154758, 0.00309294, -0.0341855, -0.0209632, -5.65618e-05, -0.0137866, -0.0358981, -0.0495536, -0.0457591, -0.16566, 0.00550305, 0.35495, 0.582945, -0.0190111, 0.557034, -0.842684, -0.0206218, -0.0794639, 0.00438771, -0.204742, -0.301832, -0.00531659, -0.0405356, -0.414929, -0.0256041, -1.11041, 0.101704, 0.0850985, 0.151475, -0.0102378, -0.0189573, 0.386719, -0.203975, -0.0779055, -0.475656, 0.149203, 0.0564685, -0.00895034, 0.742157, -0.0516198, -0.0168597, 0.254568, -0.0103748, 0.0159158, -0.228451, 0.0215626, 0.0199911, -0.0152275, -0.0135065, 0.0543522, -0.129667, -0.00573904, -0.658055, 0.344715, -0.00752609, -0.00581043, 0.0794116, 0.67381, -1.08315, -0.57538, -0.0269935, -0.427235, 0.00304019, -0.0721129, 0.405096, 0.0175208, -1.08797, -0.0235303, 0.484059, 0.067334, -0.0142868, 0.419932, 0.55207, -0.00101421, 0.32592, 0.29553, -0.189587, -0.826184, -0.0214539, 0.219562, -0.0224566, 0.126112, 0.665997, 0.288536, -0.0155393, -0.038028, -0.176191, -0.855497, 0.00942895, -0.516712, -0.0257815, -0.00976583, 0.0136111, -0.0165222, 0.114218, 0.317903, -0.74223, -0.117368, 0.328113, -0.0641419, -0.0215191, -0.00135147, 1.72334, -0.0100647, -0.772894, 0.93405, -0.0340122, 0.388423, 0.00518438, 0.697368, -0.677196, -0.0264551, -0.0186884, 0.412503, -0.384028, -0.0181436, -0.00739501, -0.72463, -0.399896, 1.06228, -0.0236846, -1.45274, -0.0132446, 0.0176025, -0.0513849, -0.0179829, -0.283928, -0.884008, 0.964131, -0.00186869, -0.0140217, 0.00727215, -0.00646146, 0.00245309, -0.0213231, 0.0110536, -0.0160067, 1.17797, -0.00619881, -0.277267, -0.555581, -0.0314104, -0.650925, 0.247877, -0.0349619, -0.00593327, -0.0325416, 0.248342, -0.231221, -0.0361082, 0.146059, -0.0185956, -0.0193877, -1.02977, 0.870225, 0.846783, 0.791083, 0.00165614, 0.00105188, 0.445718, -0.645854, -0.047916, 0.615444, -0.220976, 0.0360006, -0.00990456, -0.739112, -0.0217499, -0.016969, 0.971059, -0.000196879, -0.0288726, -0.975919, 0.00212193, 0.000813218, -0.00519621, 0.00562141, 0.114434, -1.02532, -0.0124942, -0.933304, 0.668303, -0.0214208, -0.0334377, 0.364528, 0.25952, -1.6918, -0.475732, -0.00345575, -0.57938, -0.0108162, -0.62203, -0.738026, -0.0459696, 0.0488082, 0.000496457, -0.328156, -0.187551, -0.0287814, 0.29953, 0.232198, 0.0107031, 0.167343, 0.537727, 0.225564, -0.336211, 0.000811511, -0.248484, -0.0259858, 0.26263, -0.564923, 0.138193, -0.00843968, -0.10588, -0.047431, 0.0896252, -0.00060184, -0.691938, 0.00830447, 0.00278808, -0.0229968, 0.0225031, -0.762075, 0.449547, -1.10255, -0.327524, -0.314265, -0.162659, -0.0281643, -0.0325118, 0.551649, -0.0106061, -0.145803, 0.91293, -0.00905845, 0.358424, -0.00407155, -0.450515, 0.0585573, -0.0372626, -0.000999575, 0.101363, -0.119107, -0.0468288, -0.016186, -0.460389, -0.506814, 0.757003, -0.0308998, -0.393821, -0.00229492, -0.0317893, -0.0306807, -0.0232054, 0.291149, 0.488412, 0.748992, -0.0149497, 0.00243202, -0.0358764, 0.00621876, -0.0162621, -0.0398034, -0.0214117, -0.0233708, 0.679253, -0.0076282, 0.223176, -0.806016, -0.00943001, -1.32254, -0.532992, -0.0226, 0.0042343, -0.0103706, 0.512068, -0.828753, -0.0329788, 0.101057, 0.000579265, -0.0179044, -0.654048, -0.698555, 0.800296, -0.575478, -0.047779, -0.0198855, 0.0613516, -0.552487, 0.011431, -0.803526, -0.365441, -0.0563257, -0.0242476, -0.257952, -0.0115162, -0.0117477, -0.953054, -0.000809635, -0.0392489, -0.333984, -0.0468931, 0.0276916, -0.0253077, 0.00914161, 0.311867, 0.363811, -0.00497114, -0.115833, 0.018426, -0.00272881, -0.0130309, 0.721122, -0.290629, 0.638164, 0.427731, -0.00256262, 1.22472, -0.00623865, -0.993772, 0.153637, -0.18382, -0.814074, -0.00458801, 0.766393, -0.0818076, -0.0211956, 0.424275, -1.3952, -0.0215787, -0.313989, -0.889204, -0.185451, 0.429129, 0.00359325, -1.2878, -0.0108979, 0.203971, -0.69344, -0.804028, -0.0240006, -0.0296538, -1.05243, 0.656233, -0.0224113, -0.469618, 0.0109842, -0.0265429, 0.0146007, 0.0190857, 0.345373, -0.0401428, 0.0677454, 1.05691, 0.450521, 0.896438, -0.0329355, 0.000233858, 0.941513, 0.0139564, 0.0480346, 0.523449, -0.00903953, -1.40783, -0.0349717, -0.543129, 0.255795, -0.00300162, -0.00197897, -0.441069, -0.234647, -0.0383789, 0.0158916, -0.417487, -0.0135308, 0.876965, -0.00285191, -1.07145, -0.0182672, -0.0304884, 0.0426373, -0.00362789, -0.459097, -0.309213, 0.628444, 0.00555699, -0.0272571, -0.0364751, -0.0555939, -0.0266322, -0.0400441, 0.00341431, -0.000460432, -0.050933, -0.0154051, 1.38116, 0.271824, -0.0266094, 1.13883, 0.242556, -0.0241133, -0.0231367, -0.00230029, -0.46831, -1.15298, -0.0340767, 0.555921, 0.345927, -0.0158104, 0.599976, 0.503361, 0.279494, -0.755945, 0.000177653, -0.00113894, 0.724379, 0.152008, -0.305501, -0.0991167, -0.489934, -0.24881, -0.00241722, 0.716547, 0.0181739, -0.0166446, -0.347596, -0.0106066, -0.00132884, 0.0245176, -0.000426434, -0.0579312, 0.0111231, -0.00351378, 0.430013, -0.286981, 0.00622124, 0.387684, -2.69904, 0.00467902, -0.014618, -0.069702, -0.138165, -0.450235, 0.0332566, -0.00336573, -0.20332, -0.0187275, 0.428537, -0.780292, -0.0925163, 0.0226714, 0.00169265, 0.654393, -0.666069, -0.0382932, 0.0359018, -0.460999, -0.0480182, -0.519369, -0.159382, 0.211718, -0.251111, -0.042962, -0.623629, -0.0529838, -0.13736, 1.02259, 0.206294, -0.00255873, 0.0331952, -1.13835, -0.889713, -0.0153298, -0.239202, -0.00571353, -0.00126101, -0.013136, 0.0246635, -0.0168144, -0.704747, -0.408887, 0.0155396, -0.899874, 0.862085, -0.0155293, -0.0144724, -0.186065, -0.00256896, -1.68677, -1.01482, -0.0272798, -0.943666, -0.0322765, 0.174907, 1.05973, -0.0269526, -0.0165338, 0.794834, -0.788997, -0.0275484, -0.00428576, -0.776445, -0.497523, 0.780561, -0.0476091, 0.327786, 0.00590009, 0.00860513, -0.0289021, -0.0313692, -0.0587665, -0.623399, -0.693867, -0.0026394, -0.00376502, -0.0361995, -0.00252315, -0.0248319, -0.0466921, -0.0378397, -0.0421221, -0.614111, -0.00383479, 0.183339, -0.739995, -0.0233865, -0.0711755, -0.287888, -0.0252622, -0.0363783, -0.0153001, 0.169603, -0.895871, -0.024566, 0.963969, -0.485103, -0.031416, -0.892995, -0.0249888, -0.291687, -0.167471, -0.0034904, -0.0236899, 0.427661, -0.697943, 0.758091, 0.550944, -0.653577, -0.0874533, -0.0103743, 0.113462, -0.0318253, 0.00246735, -0.798802, -0.00639393, 0.00145646, -0.0334426, -0.0237403, -0.0667225, -0.0106978, -0.00941655, -0.540075, 0.551157, -0.0122129, 0.458304, -0.394859, -0.017808, -0.0160382, 0.449798, 0.0198086, -0.491393, -0.768027, -0.0164102, -0.241746, 0.0139646, 0.245482, 0.117131, -0.0496321, 0.0836402, -0.00445525, 0.227346, -0.50202, -0.0566734, 0.605022, 0.433025, -0.00133564, -0.527157, -0.475528, 0.00601708, -1.36087, -0.0312928, -0.935608, -0.0296246, -1.27883, 0.20255, 0.396141, -0.0371947, -0.0505051, 0.428399, -1.33842, -0.0190479, 0.48291, -0.00842472, -0.0147607, 0.00826555, -0.0507179, -0.0340059, 0.00147302, -0.501517, -0.445585, 0.186423, 0.538118, -0.0297873, -0.016831, 0.786133, -0.0115379, -0.0947323, -0.0875677, -0.0107896, 0.14727, -0.0223569, -0.10273, -0.256215, -0.0179422, 0.00229867, 0.274485, 0.633458, -0.0441819, -0.0435577, 0.885745, -0.402304, 0.327496, -0.0201456, -0.87232, -0.00759963, -0.00809948, -0.0338545, -0.0076879, 0.584013, -0.803489, -1.07465, -0.0131886, -0.0541334, -0.0160489, 0.0030193, -0.00998761, -0.0431681, -0.0158041, -0.000488937, 0.545405, 0.0107719, 1.11343, 0.145301, -0.0340315, 0.993539, 1.48851, -0.0311372, -0.0195378, -0.0167699, 1.28232, -0.898046, -0.0300953, -0.0477615, -0.0436979, -0.0467627, -0.987759, 1.04829, -0.0620608, -0.246328, -0.00583211, 0.00391532, -0.0875185, 0.0389487, 0.216484, 1.20483, -0.766312, 0.0281878, -0.0268067, 0.419606, -0.012629, -0.00433147, 0.0268755, 0.000788167, -0.00118868, -0.0318025, 0.00883965, -0.00998455, -0.00150796, -0.00674016, 0.0415523, 0.0147684, 0.0201661, 0.0188665, 0.00838044, 0.000523784, 0.00557178, 0.0646717, -0.00042253, -0.00485985, 0.00890001, -0.00331938, 0.00999272, 0.0026469, 0.0320888, 0.0127209, 0.00723213, -0.0528043, 0.00655521, 0.0114833, -0.0103774, -0.000859187, -0.00347052, 0.0173683, -0.0117051, -0.00394739, 0.0274904, 0.0126868, -0.0115787, 0.00893836, 0.00207381, -0.0160165, 0.0165168, -0.0363019, -0.0175968, -0.0190695, -0.0163997, 0.0118943, 0.00695447, -0.00300718, 0.0148316, -0.0178723, -0.0163049, 0.0311078, -0.0167339, 0.012539, 0.000517214, 0.0240548, -0.00666407, -0.0311167, 0.0120275, 0.0130008, -0.00535257, 0.022552, -0.0172384, 0.0166321, 0.00139771, 0.0173309, -0.0140876, 0.00342093, 0.0136248, 0.058443, 0.00776472, 0.0109825, 0.0258502, 0.00940442, -0.0054785, 0.0124928, 0.0017047, -0.0390467, -0.00406011, -0.0133563, -0.00649547, 0.00591279, -0.0230643, 0.00244565, -0.0081926, -0.00225964, -0.00209917, 0.0204174, 0.00599902, -0.0157759, -0.0129261, -0.00526874, -0.0159187, 0.017706, -0.0159236, -0.0141547, 0.0149242, 0.0243592, -0.0120182, -0.00820805, -0.0204405, 0.0348657, -0.013523, 0.0108121, 0.0204676, 0.0121907, 0.0504247, 0.0394308, 0.0154696, 0.0233896, 0.00463892, 0.000497193, 0.0240572, 0.0127791, 0.00568071, 0.0023447, 0.00885915, -0.0114603, 0.0210817, 0.0260344, 0.0088962, 0.0177395, -0.00433683, -0.000834666, -0.00362807, -0.00663632, -0.0024745, 0.00701002, 0.55047, -0.0116591, -0.0462452, -0.0145033, -0.0399447, -0.0511022, -0.0280273, -0.0650698, 0.553837, 1.41589, -0.0177362, -0.0043874, 0.737729, -0.0245444, -0.0210617, 0.407676, -0.0560417, -0.151838, 0.375596, -0.00411981, 0.499035, -0.0187319, -0.087361, -0.780098, -0.0467472, 0.322819, -0.00500635, 0.0853263, -0.380788, -0.000215272, 0.561092, 0.181066, 0.00863339, -0.554914, -0.4555, -0.501323, 0.275094, -0.0223345, -0.113708, -0.0341986, -0.229268, 0.324826, -0.504873, -0.0345924, -0.0814975, 0.0862541, 0.641785, -0.0257579, -0.124669, -0.0731702, -0.0133824, -0.0229347, -0.0486093, 0.0319654, -0.515254, 0.570079, 0.233123, -0.909964, 0.0753205, -0.0262992, -0.042651, 0.284786, -0.0321916, -0.288285, -0.23657, -0.0161746, 0.165929, -0.0315059, -0.670179, 0.425789, -0.00166714, -0.0156119, 0.358436, -0.872421, -0.0401692, -0.0440595, -0.813767, -0.312324, 0.770189, -0.0516747, 0.353826, -0.0112117, -0.035118, -0.0319844, -0.00649011, -0.407657, -0.0974589, 0.408286, -0.0325821, -0.0269517, -0.0437184, -0.0203999, -0.0390509, -0.0393986, -0.0155651, -0.0113079, 0.624128, 0.0106902, -0.420292, -0.669592, -0.0208481, 1.07067, 0.112706, -0.0308985, -0.0558202, -0.0254635, 0.037946, 0.233889, -0.0381515, 0.0223746, 0.265046, -0.0430491, -0.665607, -0.24052, -0.0607934, -0.419549, 0.00529756, -0.0468539, -0.533754, 0.945287, -0.930162, -0.308981, -0.0535325, 0.0135986, -0.0175712, -0.77305, -0.0570779, -0.00651549, 0.632433, -0.00460473, -0.0131496, 0.204894, 0.00160978, 0.000401008, 0.0252673, -0.00934544, 0.538824, 0.155563, -0.0199712, -0.00699, -0.382302, -0.000730143, -0.0194248, -0.30004, -0.157061, 0.644614, 0.0243608, 0.00556599, -0.297278, 0.00154651, -0.438506, -0.627235, -0.103355, 0.458017, 0.00370912, -0.626037, -0.0369921, -0.056358, -0.253658, -0.78715, -0.0676331, -0.0288197, -0.730662, -0.634876, -0.36624, 0.00882965, 0.23294, -0.00978887, -0.67506, -0.0236286, -0.378826, 0.00757168, -0.0112541, 0.138929, -0.605844, -0.0229836, -0.332994, -0.00308536, -0.000226365, 0.00125202, 0.00178481, -0.342751, -0.31382, 0.581788, -0.0628415, -0.499874, 0.0625629, 0.00962545, -0.0244883, 0.154787, 0.000360836, -1.46447, -0.0556685, -0.00833041, -0.575325, -0.00211807, -0.0144444, -1.50055, -0.020338, 0.00224947, 0.00427164, -0.365289, -0.00322825, 0.00968922, -0.963647, 0.0730638, 1.05563, -0.0154191, 0.843067, -0.0239526, -0.00111162, -0.0111975, -0.00240012, 0.574502, -0.738106, 0.850289, -0.00015942, 0.00143798, -0.0018214, -0.0185262, -0.00464793, 0.0129649, 0.00277609, -0.0483689, -0.711145, -0.0443962, -0.668567, 0.427785, -0.0149315, 0.273795, 0.234351, 0.0170246, -0.0368795, 0.00927315, 0.223298, 1.0757, -0.00472564, -1.19858, 0.479566, 0.000414264, -0.323126, -0.983585, 0.0665817, -0.807662, -0.0245099, 0.015673, -0.798668, -0.36671, -0.745688, -0.828065, -0.62318, -0.509107, -0.0119679, -0.202529, -0.022581, -0.0100788, -0.846438, -0.00175307, -0.0274794, -1.61187, -0.0133379, -0.041781, -0.0322956, -0.0263831, -0.414664, 0.076786, -0.000826461, -1.03171, -0.283052, -0.025118, -0.019046, -0.0655702, -0.140515, -0.126147, 0.176916, 0.00685419, -0.181188, -0.0140879, -0.512751, -0.204511, -0.0725074, -0.389243, -0.0329988, 0.479923, 0.961898, -0.0459332, 0.809971, 1.00212, -0.0144096, 0.935832, 1.13421, -0.540969, -1.2284, -0.0212669, 0.256312, -0.032428, 0.737062, -1.39078, -0.451187, -0.0227489, -0.0921098, 0.76054, -1.16044, 0.00603259, 0.388471, -0.0450409, -0.0133108, -0.011648, -0.0305954, -0.110633, 0.468657, -0.213296, 0.635705, -0.108883, 0.673897, -0.0174881, -0.0247869, -0.984633, -0.0420527, 0.0591442, 0.67244, -0.0311261, -0.964635, -0.0213572, 0.72053, -0.812548, 0.00219993, -0.0111424, -0.301028, 0.344747, -0.0294005, -0.0323312, 0.152087, -0.283041, -0.191835, -0.0187297, -0.23229, -0.0150104, -0.0401557, -0.0401565, -0.00839552, 0.259695, -0.250152, -1.37442, -0.013642, -0.0427454, -0.0357159, -0.0398046, -0.0268164, -0.0294341, -0.0486452, -0.00487837, -0.340261, -0.0186331, -0.44695, -0.622644, -0.0151501, 0.730681, 0.117041, -0.0346335, -0.0612665, -0.00914526, -0.814842, -0.126269, -0.00767063, -0.811141, -0.418806, -2.22122e-05, -1.15063, -0.126336, -0.283589, -0.0769571, -0.0205571, -0.0366137, 0.736792, -0.661862, 0.193341, 0.471743, 0.510499, -0.036859, -0.0273629, -0.143962, -0.0110629, -0.0133442, 0.700579, -0.0140915, -0.0101565, 1.2024, -0.0213273, -0.0349394, -0.0252645, -0.00261451, -0.339635, -0.552957, -0.00869205, 0.719523, -0.456884, -0.0231155, -0.0311387, -0.256196, -0.0932762, 0.21191, -1.77659, -0.0191746, -0.891583, -0.0374496, 0.0572993, -0.271334, -0.0287983, -0.444872, -0.0194437, 0.531398, -0.365937, -0.0144367, -0.481037, 0.422394, -0.0276922, -0.460438, -2.12241, -0.882578, -1.39161, -0.0304853, 0.211753, -0.0321112, -0.596392, 0.436284, -0.0879685, -0.0251717, -0.105762, 0.384763, -0.291804, -0.00548181, 0.159527, -0.0440579, -0.0172198, -0.0262468, -0.0417379, -0.109854, -0.494408, -0.598267, -1.39579, -0.274011, -0.935427, -0.0321165, -0.051156, -0.247247, -0.0356192, -1.42546, 0.265726, -0.0264388, -1.14609, -0.03905, -0.516242, 0.289527, 0.000641395, -0.0129514, -0.378933, -1.07284, -0.0515636, -0.0410106, -0.577251, -0.813343, -0.122835, -0.00880052, 0.990956, 0.000611568, -0.0405309, -0.0423624, -0.0292002, -0.532103, -0.698861, 0.900426, -0.0184971, -0.024445, -0.0225183, 0.00016647, -0.0465269, -0.0426974, -0.0352629, -0.0230341, -0.696266, -0.00150952, -0.697542, 0.195729, -0.0445647, 1.26875, 0.188439, -0.0380512, -0.0730003, -0.0184174, -2.06969, -0.164358, -0.0213251, -0.468093, -0.749067, -0.043026, -1.12427, -0.274173, 0.521529, 0.543851, -0.0402788, -0.0367187, -0.842505, -1.04116, -1.02492, -1.41001, -0.670473, 0.0312121, -0.0235767, -0.160268, -0.0591134, -0.02853, 1.17365, -0.00609289, -0.0139621, -1.08129, 0.0135599, -0.00343336, -0.00828254, -0.0389209, -0.32752, -0.069141, 0.00239951, -0.845543, 0.0548715, 0.00502511, -0.022764, 0.769077, -1.23446, 2.01181, 0.483186, -0.0023409, 0.494626, 0.00125442, 0.084653, 0.5825, 0.0373924, 1.09329, -0.00993381, 0.0569662, 0.551128, -0.0291782, 0.0933798, -0.0628732, -0.00222743, 0.264369, 0.383087, -0.883854, 0.235996, -0.0177485, -0.456875, -0.0184964, 0.946798, -0.707976, -0.293761, -0.0282247, 0.027911, 0.140825, 0.360666, 0.00754621, 0.212065, -0.00940401, -0.0132795, 0.0192806, -0.0174082, -0.225316, -0.606679, 0.180375, 0.178307, -1.13186, 1.22896, -0.0281922, -0.0201672, -1.23077, -0.0108638, -1.19803, 1.50253, -0.0231402, 0.24104, -0.00875183, -0.644181, 0.743525, -0.00535949, -0.00367624, 0.649184, -0.302155, -0.0110559, -0.00127776, 1.09493, 1.04627, 0.0872508, -0.0122213, -1.17195, -0.0212822, -0.0178968, -0.0562369, 0.00323462, -0.0317959, 0.716574, -0.372675, -0.0173075, -0.0184969, 0.00186851, 0.00158641, -0.00911077, -0.0280002, -0.0261582, -0.0138303, 0.679517, 0.00374208, 1.46597, 0.766297, -0.0196458, -0.898927, 0.246217, -0.0332709, 0.00783987, -0.02728, -0.890652, 1.08171, -0.0313848, -0.18483, 0.467043, -0.0134303, 0.495184, 0.786188, 0.525666, -0.714246, 0.0115236, 0.0109854, 0.37143, 0.0196285, -0.447382, 0.431773, -0.310506, 0.0116422, -0.0159035, 0.742457, -0.0188012, -0.00502295, 0.96224, 0.00857133, 0.00800122, 0.0517566, -0.000966332, -0.0150434, 7.50144e-05, -0.00436668, 0.750583, 0.36872, -0.00356824, 0.511021, -1.69311, 0.00486718, -0.0246966, -0.345437, -0.842895, -0.175634, -0.329564, 0.00157924, -0.782474, -0.0207754, 0.0465349, 0.60354, -0.00259444, 0.333824, -0.0343086, -0.332317, -0.965209, -0.000666968, 0.0214256, -0.973693, -0.00725327, -0.292518, -1.18941, -0.00299098, -0.211168, -0.0219003, 0.064556, -0.0512309, 0.258162, 0.989645, -0.292748, -0.0100307, -0.0622285, 0.735198, 1.50808, -0.00197126, -0.552242, -0.0191843, 0.00268068, 0.00386406, -0.0205883, -0.792772, -0.133006, 0.942372, 0.361308, -0.125792, -0.335455, -0.0400646, -0.0144686, 0.0713666, -0.0107626, -0.602515, -0.155706, -0.0236053, 0.162406, -0.00555071, -0.610962, -0.0123842, 0.00757249, 0.00600912, -0.589291, 0.142214, -0.0131637, -0.000426319, 0.600309, -0.371404, 0.862872, -0.00409253, -0.0766891, -0.00680906, 0.00318995, -0.0249258, -0.0383007, -0.42237, -0.684658, 0.866655, -0.0137621, 0.0160131, -0.00703875, 0.000480697, -0.00313391, -0.0359162, -0.00630379, -0.0224983, 0.15975, 0.00706454, -0.700309, -0.603469, -0.00556457, -0.987773, -0.488541, -0.0121329, -0.0072366, -0.0184344, -0.359318, 0.262086, -0.0329178, -0.0512532, -0.39529, -0.00415005, -1.14715, -0.401734, 0.403622, -1.52901, -0.0136463, -0.00742978, 0.399805, 0.270329, -0.751094, 0.201564, -0.0346671, -0.0157172, -0.0246616, -0.423197, -0.0145138, -0.00502554, 0.524695, 0.00848088, -0.000366605, -0.820559, -0.0246112, -0.11041, 0.00223329, -0.0161194, -1.16921, 0.3504, -0.00229788, -0.602954, -1.38146, -0.0211324, -0.0173061, -1.27423, 0.785599, -0.112799, 1.03898, -0.0114566, 0.0202762, -0.0168197, -0.991, -0.495388, -0.00200012, -1.72187, -0.0025762, -0.067689, -0.355656, -0.0282752, -0.483396, 0.103739, -0.0197304, 0.213322, 0.299261, 1.53587, -1.0462, -0.0297849, 0.338794, -0.0263858, 0.78302, -1.00387, 0.694532, -0.0303188, -0.115027, -0.238161, -1.58879, -0.0182067, -0.781434, -0.00286931, -0.0101342, -0.0125528, -0.034867, -0.317706, 0.537206, -1.00747, -0.575003, 0.208565, 0.324815, -0.0396019, -0.0341066, -1.32106, -0.02802, 0.212098, -0.313457, -0.014842, 0.289577, -0.0194002, 0.697947, -1.38628, -0.0533628, -0.0129935, -0.390945, -0.32955, -0.029266, -0.0497451, 0.401793, -0.9878, -1.47823, -0.0480953, -0.531595, -0.000618808, -0.0369267, -0.0255182, -0.0179443, -0.305902, 0.440272, -2.38573, -0.00504709, -0.0295217, -0.0255483, -0.0522138, -0.0197004, -0.0377205, -5.68512e-05, 0.00521589, 0.109614, -0.00241325, -0.521374, -0.528983, -0.046801, -1.5062, -0.23807, -0.029152, -0.00558741, 0.00222126, -0.0120448, 0.579075, -0.0275839, -0.0387314, -0.479577, -0.0215594, 0.892155, 0.349224, 1.06777, -0.359347, -0.0107695, -0.0117232, -0.758051, -1.01576, -0.109678, 0.682345, 0.539049, 0.200285, -0.0201339, 1.08652, -0.0215149, 0.00214946, -0.0560518, -0.0360041, -0.0400761, 0.145884, -0.0287902, -0.0644015, -0.0392551, -0.0579822, 0.863726, 1.44302, -0.0133107, 0.60811, -0.218016, -0.034437, -0.0556403, 0.0724949, 0.480067, -1.07065, 0.116295, -0.0122529, 1.23264, -0.0262077, 0.628063, 0.0506175, -0.153695, 1.63148, -0.0253913, -0.434223, -0.626289, -0.00952439, -0.542863, 0.081158, -0.0297434, -0.809115, -0.130358, 0.638508, 0.149644, -0.0148429, -1.06394, -0.0453279, -0.169691, 0.190002, 1.74748, -0.0094787, -0.0511981, 0.527238, -0.0552174, -0.024782, 0.0716143, -0.0603002, -0.0231701, -0.0303735, -0.0348147, -0.478462, -1.80446, -0.0311933, -0.403861, 0.0320082, -0.122536, -0.0379416, -0.045078, -0.974792, -0.0468915, -0.696715, -0.648607, -0.00478114, 0.161685, -0.0327616, -0.467431, 0.285881, -0.0400565, -0.0186014, -0.230195, -1.00772, -0.0426207, -0.0327542, -0.495681, 0.455259, -0.770494, -0.0285136, 0.702376, -0.0327444, -0.0335561, -0.0245083, -0.0300249, 1.14964, 0.912293, 0.0978865, -0.0182705, -0.0225863, -0.0516723, -0.00531943, -0.0454308, -0.0361894, -0.0133372, 0.00332461, 0.136049, -0.0295112, -0.0856419, 0.241781, -0.0368673, 1.46789, -0.353192, -0.00618509, -0.0311597, -0.0101309, -0.269802, 0.492837, -0.0305913, 0.219711, -0.107498, -0.0523257, -0.126629, -0.212988, -0.646373, -1.00692, -0.0151313, -0.0252263, -0.469582, 0.0494992, 0.23871, -0.589129, -0.966532, -0.00264091, -0.0238962, -0.784507, -0.0724608, -0.0183086, -0.29825, -0.0204761, 0.00147028, -0.233146, -0.0369612, 0.030626, -0.023753, -0.00316985, 0.475183, -0.00738668, -0.00511675, -0.987721, -0.230556, -0.0133146, -0.0051982, 0.334741, 0.279045, 0.398207, -0.00561632, -0.0169108, -0.865733, -0.00386975, -0.0543755, -1.17662, -0.0466281, 0.424422, -0.0206083, -0.902955, 0.145044, 0.00256775, -0.930915, -0.336557, -0.049798, -0.0955534, 0.119341, -0.325041, 0.076858, 0.00371303, -0.701256, -0.0561722, 0.0763634, 0.156624, 0.163972, -0.0122769, 0.0203524, -0.858786, -0.214344, -0.00839719, -0.878185, 0.00560523, -0.00116999, 0.0222746, 0.0170586, -0.977328, 0.253663, -0.88297, 0.0361011, 0.489479, 0.651634, 0.00732697, -0.00795989, 0.101374, -0.000160428, 0.469543, -0.57618, 0.00969159, 0.311557, -0.0272204, -0.635958, 0.280753, -0.026368, -0.0323248, 0.085782, 1.29548, -0.0579091, 0.0309409, -0.870935, 0.089377, 1.09512, -0.0174115, -0.537755, -0.00788384, -0.00319194, 0.030923, -0.0194107, -0.593562, -0.51183, 0.839831, -0.0124268, -0.00720027, -0.00962276, -0.0487523, 0.00683184, -0.0189838, -0.0121958, -0.00278866, -0.85972, -0.00629536, 0.502054, 0.225828, 0.00172923, 0.493172, -0.366853, -0.0348277, -0.0571529, -0.00868386, 0.93372, -0.486822, -0.0199149, -0.985769, -0.647871, -0.00935155, 0.112663, 0.257642, 0.0623253, -0.24345, -0.00456371, -0.00177177, 0.314103, -0.438008, 0.707146, -0.30041, -0.0858537, -0.224473, -0.00785951, 0.830155, -0.0352319, -0.0142914, -0.121692, 0.0101219, -0.018423, -0.032967, 0.0287388, 0.029154, -0.018157, -0.0193481, -0.17201, -0.587774, -0.00219738, 0.498319, -0.142238, -0.0114286, -0.00357301, 0.488449, -0.976934, 0.17923, -1.613, -0.0102045, -0.760414, -0.0272621, 0.0818335, -1.06918, -0.00703909, 1.02369, -0.000767769, -0.895067, 0.214774, -0.0178839, -0.554044, -0.453567, -0.0541907, -0.200008, -0.560017, -0.575798, -0.097358, -0.0223797, 0.461408, -0.0335206, 0.191496, -0.625382, -1.54341, -0.0145935, -0.0825885, -0.664679, 0.107835, -0.0127909, -0.0787566, -0.00446598, -0.000567671, 0.00845058, -0.00994484, -0.124084, -0.486641, -0.198136, -0.38105, -0.408589, -0.910513, -0.0217692, -0.0327537, -0.165541, -0.0225007, 0.0564534, 0.677395, -0.0273037, -0.603299, -0.0142661, -0.16867, -0.144892, -0.0315414, -0.00513145, -0.436927, -0.17409, 0.0144275, -0.0208756, -1.90338, -0.215112, 0.392599, 0.00722268, -0.107177, -0.0109324, -0.00592727, 0.00297346, -0.0223552, -0.27642, -0.988561, 0.508625, -0.0274557, -0.0130833, 0.00442131, -0.0748408, -0.0222485, -0.0179042, -0.0245763, -0.0117427, -0.653318, -0.00745553, -0.254554, -0.33965, -0.0368045, 0.261209, -0.217429, -0.0264182, -0.0113754, -0.0260171, -0.0114503, 0.513611, -0.0393329, -1.31682, -0.15012, -0.0192275, -1.51218, -1.51869, 0.696452, -0.309534, -0.00302112, 0.0028181, 0.375426, -0.786578, 1.24406, -0.581209, 0.193469, -0.0254488, -0.0340266, -0.263365, -0.0228986, -0.015282, -0.632536, 0.000939511, 0.000580194, 0.671663, 0.020705, 0.0213498, -0.0241781, -0.0217659, 0.0721011, -0.0681591, -0.0187029, -0.593284, 0.230681, 0.00092734, -0.0166473, 0.129909, 1.03239, -0.668441, 0.366754, -0.014817, 0.141405, -0.0178914, -0.0945449, -0.217811, 0.00432424, 0.0216897, -0.0190769, -0.228055, -0.0395749, -0.015879, 0.0837554, 0.242281, 0.0268219, -0.397909, -0.116205, 0.539696, 0.2125, -0.0223837, 0.328107, -0.042627, -0.316836, -0.155382, 0.295861, 0.000797018, -0.0180642, -0.127736, 0.0999655, -0.00656446, 0.0119311, -0.0205749, -0.0149935, 0.0304805, -0.0253007, -0.0038607, 0.0244509, -0.129819, -0.476434, 0.733106, -0.294674, -0.0262379, -0.00827859, -0.437689, -0.00764174, 0.532256, 0.124553, -0.0191577, -0.667466, -0.00143704, 0.659749, -0.159778, -0.0332036, -0.00434651, -0.0219966, -0.250393, -0.012458, 0.00017608, -0.231339, -0.161141, 0.041278, -0.0109062, 0.646305, -0.0133694, -0.00174015, -0.0309507, -0.00991684, -0.0295568, 0.013402, 0.144575, 0.00749182, 0.00821851, -0.00210882, -0.0210885, -0.00762513, -0.00263145, 0.00696299, -0.00319806, 0.526195, -0.00705416, -0.79347, 0.365003, -0.00372495, 0.924737, -0.0902305, -0.0489192, 0.0145387, -0.016562, -0.539971, -0.153386, -0.0508746, 0.873316, 0.378267, -0.0284948, 0.304871, 0.277158, 1.2176, -0.0370783, -0.00166067, -0.0116214, 0.38309, 0.245404, -0.374512, 0.00652812, -0.119651, 0.0441785, 0.000933111, -0.029682, 0.0142098, -0.0123911, -0.0214879, -0.00672535, -0.00343119, -0.0478307, 0.00268757, -0.0108195, -0.0107803, -0.00842639, -0.0233724, 0.00136938, 0.00152472, -0.0282212, -0.0100597, 0.00910772, 0.0277089, 0.0120312, 0.0138469, 0.019917, -0.00482219, 9.08825e-08, 0.0177939, -0.000743153, 0.00514032, 0.0374442, 0.00152764, 0.00531874, 0.0131187, -0.0358629, -0.0336092, 0.0121851, -0.00381903, -0.0541239, 0.0209849, -0.00484759, 0.0551115, -0.0214888, -0.00819812, 0.0135209, 0.00153302, -0.0105034, -0.0073413, -0.0268822, 0.0273251, -0.00513446, -0.0142627, -0.0510871, -0.0350421, 0.000760206, -0.0114555, 0.00073876, 0.0150481, -0.00607635, 0.0115855, -0.0277546, 0.0228878, -0.0441543, 0.0543368, -0.0310118, -0.00189144, 0.00622508, -0.0083189, 0.00983528, 0.00852505, 0.041186, 0.00928612, -0.00944973, 0.00960694, -0.00102469, -0.0141084, -0.042306, -0.0080704, 0.0174128, 0.0476273, 0.0175363, 0.00702995, -0.00295473, -0.0397921, 0.000690533, -0.0312549, 0.00548858, -0.0437582, -0.000773761, -0.00841049, -0.0103812, -0.00952853, -0.0186343, 0.0318324, 0.015722, 0.00179746, -0.000465017, 0.00373418, -0.0124733, 0.00494116, -0.0112149, 0.00803248, -0.00246217, 0.0234732, -0.000238161, -0.00426574, 0.0144862, -0.00365448, 0.0192843, -0.0569656, -0.00720873, 0.00377256, -0.00470303, -0.0384098, -0.0098029, 0.00265045, -0.0187886, 0.0285347, -0.0105968, -0.00701736, 0.00114464, 0.0452016, -0.0129096, 0.00157246, 0.0122164, -0.0102968, 0.0396769, 0.00724357, 0.0148428, 0.00338678, -0.00587463, -0.00561104, 0.00728795, 0.00546759, -0.00824064, 0.634058, -0.00964529, -0.0236427, 0.0251685, 0.00629637, -0.0491758, -0.00748561, -0.00859278, -0.765173, -0.0547447, -0.0120796, 0.371885, -0.334018, -0.0090524, 0.00177716, -1.83745, -0.798017, -1.56543, 0.422032, 0.00654338, 0.0240951, -0.00789916, 0.8342, -0.286453, -0.0166313, -0.965765, -0.00947434, 0.749208, -0.562844, 0.0123666, -0.00610486, 0.0534207, 0.00710132, 0.674357, 0.297247, 0.628863, 0.728528, -0.018239, -0.171564, -0.0374551, 1.68082, 0.220814, -0.283766, -0.0392808, -0.10742, 0.958303, 0.290165, 0.00101185, -0.392905, -0.0123127, 0.00705778, -0.0187884, 0.00209634, -0.65384, -0.636783, -0.0220872, -0.554829, -0.887209, -0.0493949, -0.0132317, -0.0440814, 0.173664, -0.0404745, 0.189608, -0.587361, -0.0181486, -0.276674, 0.00779632, -0.0955258, -0.559522, -0.00969858, -0.00903123, 0.253977, -0.139518, -0.0387817, -0.0329143, 0.396808, 0.0176747, -0.501435, 0.00419099, 0.273368, -0.0245785, -0.037404, -0.0686649, -0.0228208, 0.265371, -1.08435, 0.630977, -0.0150173, -0.0334142, -0.0315548, -0.0349581, -0.0388878, 0.039942, -0.0037483, -0.00691354, -0.3967, -0.00819169, -0.576071, -0.269599, -0.0296634, -0.502446, -0.265938, -0.0200883, -0.0289424, -0.0219707, 0.789285, 0.324359, -0.0235556, 1.48537, -0.00243287, -0.00748398, 0.976621, -0.0896985, 0.400986, 1.09062, -0.00732127, -0.0173677, 0.0942042, 1.37506, -0.11988, -0.425178, 0.33437, -0.00465789, -0.0151811, -0.149781, -0.0253763, -0.013133, 0.0973349, -0.00301632, 0.0106153, 0.367973, -0.0148122, -0.00112437, -0.0196206, -0.00551736, 0.106391, 0.0377105, 0.004531, 0.0682129, 0.086975, -0.00587216, 0.00562831, 0.0646245, 0.130554, 0.053173, 0.0224177, 0.000849186, 0.0272401, 0.0130474, 0.00492922, 0.0506101, -0.00506543, 0.211295, -0.0333416, 0.0627578, 0.0103518, 0.00586054, 0.205553, 0.13635, 0.00925106, 0.19218, 0.0023784, 0.0770264, 0.171577, -0.0385308, 0.0595213, -0.0395724, 0.0377228, 0.0580436, 0.0118116, 0.0202316, -0.0360016, 0.0703143, 0.0654916, -0.0117037, 0.0791604, -0.0204076, -0.0034439, 0.00922597, -0.0184283, 0.0847083, 0.12564, 0.0556296, -0.0483744, 0.0856597, 0.0580389, -0.0451507, -0.0258568, 0.0515189, -0.0102844, -0.000203483, 0.120029, -0.0188957, -0.00973153, 0.0013979, 0.0783288, 0.105747, -0.00284117, 0.00910256, -0.110471, 0.00526113, 0.000456079, 0.00162964, 0.0218013, 0.0652839, 0.0364356, -0.0230126, 0.130214, -0.0093165, -0.0107038, -0.00630186, -0.0122752, 0.0622983, -0.0114473, 0.13081, 0.00117707, -0.00196531, -0.0118749, -0.00635812, -0.0027326, -0.0160252, 0.0184821, -0.0119747, 0.0469567, -0.000429838, -0.0103646, 0.00361502, -0.0256422, 0.0136876, 0.01437, -0.0325383, 0.0340408, 0.00241948, 0.087567, 0.0578471, -0.0524197, 0.0105803, 0.0718249, -0.0185293, 0.135383, 0.0629412, 0.0987757, 0.0184121, -0.00130165, -0.00166219, 0.0511293, 0.0286698, 0.0560963, 0.0625467, 0.0295515, 0.0345092, -0.0130619, 0.0615955, -0.00679749, -0.0148376, 0.985159, -0.0222035, 0.0119569, 1.31159, -0.00500108, 0.0387229, -0.0163207, -0.00712979, 1.08139, -0.711941, -0.00460305, -0.565643, 1.40381, 1.60837e-05, -0.00296074, -1.34102, -0.349277, -1.49381, -0.0299995, -0.0131032, -0.0444472, 0.00172885, -0.421731, 0.304072, -0.0511483, -1.16962, -0.000324659, 1.56084, 0.094757, -0.0243293, -0.871824, -0.548629, -0.072229, -0.247323, 0.556647, -0.379501, -0.880529, 0.00463716, 0.243947, -0.0473985, 0.933818, -1.80681, 0.428286, 0.00146211, 0.00285323, -0.0310376, 0.0908832, -0.0338696, -0.278705, -0.0536539, -0.00549134, -0.0334955, 0.00782301, -0.145801, -0.102597, -0.975887, 0.307935, -0.260742, -0.816179, 0.00789046, -0.0174096, 0.673657, -0.0032926, -0.0223069, -0.0995285, -0.00366257, -0.553804, -0.0401598, -0.716552, -0.40912, -0.0148951, -0.0360598, 0.385671, -0.316419, -0.0407917, -0.0131187, 0.408504, 0.706077, 0.356884, -0.0388957, -0.409993, -0.0353641, 0.0212476, 0.00200136, 0.00527588, 0.603142, -0.0515593, 1.05294, 5.94762e-05, -0.00724433, 0.0061668, -0.045671, 0.00359047, 0.00457046, -0.0250951, -0.0309531, -0.473016, -0.0348156, 0.283308, 0.0734279, 0.0108138, -1.2727, 0.881121, 0.0103645, -0.0183629, 0.000900514, -0.250076, -0.914754, -0.0378131, -1.05282, 0.221163, -0.0251971, 0.0181803, 1.27138, 0.00133876, 0.318489, -0.00839627, -0.0107944, 0.491345, 0.483597, 0.952927, 0.507194, -0.797354, -0.272096, 0.0152756, 0.553319, -0.0152418, -0.0244676, 0.0676659, -0.00367144, -0.0138542, 1.60551, -0.0403096, -0.0698285, -0.0340515, -0.0390065, 0.0272707, -1.18592, -0.00831361, -0.287867, 0.65398, -0.00521417, -0.00445362, -1.40357, 0.142945, 0.259893, -0.473954, -0.00495376, 0.18691, -0.013098, -0.409694, -0.830384, -0.102352, -0.966013, 0.020683, 0.735249, -0.126001, -0.0599159, 0.4173, 0.77647, -0.0250894, -0.334716, 0.685897, -0.0935829, 0.243067, -0.00274029, -0.979534, -0.0111717, 0.214615, -0.15091, -0.59435, -0.026339, -0.0593764, -1.24348, 0.775737, -0.0344387, 0.48873, -0.0546817, -0.0241507, -0.0186733, -0.0577488, 0.0806506, 0.514169, -0.134155, 2.1173, 0.85754, -0.58253, -0.0316965, -0.0104041, -0.561037, 0.0062118, 1.16057, 0.977733, 0.00348908, 0.497583, -0.0505318, -0.345347, -0.783833, -0.0366479, -0.0204328, 1.12185, -0.521657, -0.0292817, -0.0226284, 0.00208134, 0.581873, -1.16621, -0.04687, -1.21278, -0.0274401, -0.0398309, -0.034817, -0.00188312, 1.49429, 1.04492, -0.64604, 0.00370542, -0.0371768, -0.0284816, -0.0507484, -0.0380634, -0.0629194, -0.00370194, -0.0347304, -1.29876, -0.0223591, -0.200249, 0.20537, -0.0260098, 1.98362, -0.528798, -0.0389004, -0.0264309, -0.00962234, -0.501304, 0.559536, -0.00840731, -0.932511, -0.240997, -0.0146927, -0.32908, -0.0870635, 1.5317, -0.969983, -0.00638933, -0.0126349, -0.752835, 0.843956, -0.553744, 0.12781, -1.00814, -0.208698, -0.00535329, -0.544413, -0.00868419, 0.00467795, 0.12637, -0.0131944, 0.000407085, -0.719884, -0.00243366, -0.0117906, -0.020583, -0.0162799, -0.729964, -0.490707, 0.0131708, -0.421807, 0.482447, 0.014815, -0.0111913, 0.656666, -0.0350992, 0.155744, -0.553668, 0.0103256, 0.609225, -0.0142627, 0.368879, -1.16734, -0.0246931, -0.0859722, -0.0219249, -0.458531, -0.868589, -0.0245316, 0.224981, 1.32277, -0.00723635, -0.250283, -0.199034, -0.513587, 0.120229, -0.012892, -0.11795, -0.035107, -1.10213, 0.541623, -0.393169, -0.00589389, -0.0463685, -0.174289, -0.333884, -0.00907529, -0.537038, -0.0338597, -0.000345729, 0.0150197, -0.00883442, 0.0476615, -0.787236, -0.498403, -0.263908, -0.61095, -1.63154, -0.0268645, -0.00898235, -0.725416, 0.00762852, 0.0276423, 0.848001, -0.0202264, 0.334558, 0.00996876, -0.657958, -0.0628474, -0.00922077, -0.0119407, -0.386356, -0.358923, -3.33936e-05, 0.00920588, -0.592393, -0.703989, -1.16079, -0.00660944, -0.0999086, 0.0140944, -0.00311282, -0.0381322, -0.0193892, -0.725168, -1.36858, -0.926735, -0.0184244, -0.00760401, -0.00782225, 0.00620881, -0.0124423, -0.0145251, 0.00685636, -0.0181828, 0.0296968, 0.0221488, -0.668718, 0.0856461, -0.0335039, 0.65555, -1.21055, -0.0158505, 0.0185591, -0.0265641, 0.0432643, -1.47602, -0.00971618, -0.472448, -1.26077, -0.0184554, -0.407995, -1.00076, 0.0832152, -0.252767, -0.00683388, -0.00227252, -0.789917, 0.50904, -0.378928, -2.07519, -0.234905, 0.0701422, -0.0285708, -1.36119, -0.00226937, 0.00159576, -0.0170524, -0.001791, -0.0108994, 0.000121816, 0.00967479, 0.00338691, -0.0029711, 0.00440154, -0.00267351, -0.00650763, -0.00193501, -0.0144831, -0.0251555, 0.0128143, -0.0063642, -0.0192884, 0.0160159, -0.0189036, -0.0227333, 0.00273346, -0.0154512, -0.0125606, -0.0166387, -0.00440693, 0.0115688, 0.0233973, 0.014952, 0.00754942, -0.0138691, 0.000526492, 0.0180915, -0.00556445, 0.0018302, -0.0115094, -0.0172371, -0.0154814, 0.0175377, -0.0159543, -0.0173874, -0.0108545, 0.00863925, -0.0178673, 0.0105797, -0.00789396, 0.0144865, -0.00247734, -0.00403835, 0.00102095, -0.0122672, 0.0175361, 0.0168048, 0.00376101, 0.00530674, -0.00658543, 0.00542529, 0.00423131, 0.00403008, -0.0158664, -0.00376642, -0.0122274, -0.00556365, -0.0156736, 0.0107881, -0.00991893, 0.0121772, -0.0274733, 0.0067248, 0.00239898, -0.0201724, 0.0169943, -0.0120513, -0.0096665, -0.000492681, -0.0111451, 0.00561643, 0.0130007, -0.0136661, -0.0115196, -0.00434451, -0.0042461, -0.00168493, -0.00595469, 0.00045263, 0.0135428, 0.0114563, -0.00378632, -0.00154719, -0.0265454, -0.0086768, 0.00719835, -0.00702251, -0.0142017, 0.00854371, 0.00242487, -0.00973253, -0.00371058, -0.00216608, 0.00308949, 0.00791571, -0.0065583, -0.0152158, 0.015681, -0.014123, -0.00857057, -0.0143154, 0.0155007, -0.0276921, -0.0120096, -0.00424203, 0.00383134, -0.00861259, -0.00484183, 5.38413e-05, -0.00235434, 0.00652077, -0.00226804, 0.0137323, 0.000998241, -0.0287323, -0.00548293, 0.00893323, -0.0225321, -0.0207709, 0.00195015, -0.0139953, -0.0171792, 0.0113689, -0.00393016, 0.0330252, 0.00063763, -0.00281127, -0.0125687, 0.000989222, -0.00486836, 0.0232707, 0.0147091, 0.00381909, 0.019871, -0.000814253, 0.0307062, 0.0348607, 0.0108512, -0.00369604, 0.0313007, -0.0360656, 0.0372491, 0.0137321, 0.00586452, 0.0326198, 0.00992048, 0.011471, 0.0311132, 0.0120288, -0.024165, -0.00541628, 0.0242681, 0.039036, -0.00154696, -0.0179014, 0.0547442, 0.00999361, 0.00419863, -0.00128972, 0.040326, -0.0230433, 0.000511057, 0.0168471, 0.00390116, 0.0231268, 0.0196964, 0.000577799, 0.00274839, 0.015948, 0.0134942, 0.0335711, 0.00633223, 0.0443027, 0.0168124, -0.00741577, -0.00969991, 0.0332597, 0.0223465, -0.0174256, 0.032818, -0.0104996, -0.00171086, 0.0129066, -0.00131765, 0.0190502, -0.0137315, -0.00178284, -0.00855431, -0.0511998, 0.00190667, 0.000500163, 0.00802229, 0.0207356, 0.0376939, 0.00352615, 0.0188289, 0.00239332, 0.0080707, -0.008067, -0.00999116, 0.0080541, 0.0253362, 0.0368854, -0.00578149, 0.0553513, -0.00108937, 0.00221926, 0.0138431, 0.000741793, 0.0250931, 0.00411933, 0.0398998, -0.0198697, 0.00360719, -0.00968502, -0.0032242, 0.00272415, -0.00641279, 0.0231505, 0.0268231, 0.0319929, -0.0151937, 0.0103845, 0.00237534, -0.0020484, -0.0103794, -0.0016342, 0.0130164, 0.00323602, -0.0236629, 0.0162004, 0.0259263, 0.00682271, 0.00723607, 0.0307736, -0.0060489, 0.0470037, 0.0328768, -0.0301422, 0.00853895, 0.0207493, -0.00573715, 0.0394555, -0.034112, 0.0218304, 0.02204, 0.0308874, 0.0182571, 0.00901625, 0.0156389, -0.000623857, 0.0130844, -0.117356, -0.00061105, -0.0138164, -0.647443, -0.0215215, -0.0677592, -0.0361797, -0.0158476, -2.56, 0.114963, -0.0141188, -0.0205464, -0.628119, -0.00622587, -0.0254262, -0.470869, 0.786517, -1.25666, 1.19992, 0.00129048, 0.20438, -0.0160496, 0.934708, -0.822716, -0.0207075, -1.76714, -0.00915397, 0.13278, 0.19209, -0.0171197, -0.371374, 0.496825, -0.0231356, 0.49445, 0.496563, 1.18847, -0.829124, -0.0174844, 0.0866156, -0.0118813, 0.490399, -0.84171, 0.846595, -0.00802215, -0.104734, -0.0488528, -1.60787, -0.051165, -0.255736, -0.02076, 0.00155076, -0.0212887, -0.0515916, -0.206857, 0.629173, -0.159796, -0.662322, -0.420077, 0.366424, -0.0356879, -0.0361897, -2.17385, -0.0438416, 0.070159, 0.753693, -0.0165622, 0.47592, -0.0195274, 0.816168, -1.36898, -0.00928992, -0.011208, 0.0705767, -0.512663, -0.0192192, -0.0396007, 0.0547081, -1.8848, -0.46252, -0.0120913, -0.48218, -0.00740549, -0.029143, -0.0488839, -0.0227766, -0.690525, -0.137533, -2.91988, -0.024183, -0.0323977, -0.0383114, -0.0395686, -0.0198536, -0.0312921, 0.00202401, -0.00927748, 0.0219, 0.0031367, -0.826698, -2.62095, -0.0173962, -0.136217, -0.174597, -0.0324086, -0.0318543, -0.00243781, -0.350435, 0.0604954, -0.0448276, 0.486993, -0.867018, 0.00431848, 0.993438, -0.44866, 1.26701, -0.476747, -0.0148545, -0.00983531, -1.13435, 0.0836252, -0.22096, 0.734993, 0.594368, 0.0163242, -0.0108605, 0.585732, -0.00903443, -0.00904507, 0.790564, 0.00884481, -0.0219638, -0.0410097, -0.0278652, -0.0490281, 0.00867785, -0.0358425, -0.166827, -1.55738, -0.00505742, -1.17912, -0.511905, -0.0280864, -0.0146443, 0.0656105, 0.289142, 0.167884, -0.11721, -0.00725511, -0.0845287, -0.00999196, -0.412446, -0.224232, -0.0915045, 0.521112, -0.0110822, -1.09467, -0.54811, -0.00858193, 0.0122459, 0.618929, -0.0531513, -1.44137, 0.250503, 0.154855, 0.664265, -0.00995166, 0.0648471, -0.0197038, -0.117469, 0.7891, -0.0650167, -1.0223e-05, -0.0450398, 0.258562, 0.135725, 0.00682235, -0.0508268, -0.020221, -0.0189128, -0.0117152, -0.037859, 0.117947, 0.0649869, 0.432044, -0.621913, -0.963087, -0.100145, -0.0262162, -0.0446727, -0.132409, -0.0205703, -1.291, 0.363077, -0.0186978, -1.05222, -0.00702344, -0.448878, -0.102323, -0.00251474, -0.0104689, -0.538261, -0.182548, -0.0205606, -0.0267348, -0.764243, -0.165671, -0.275172, -0.0139088, -0.326018, -0.00738419, -0.0379028, -0.0291732, -0.0281527, 0.0709168, -0.912569, -0.0097572, -0.0134316, 0.000267619, -0.0335013, -0.00130769, -0.0300021, -0.0522017, -0.0068328, -0.0244088, 0.134914, -0.00588404, -0.269168, -0.322184, -0.000187471, -0.129942, -0.707192, -0.0209265, -0.0325623, -0.0114781, -0.675369, -0.194906, -0.037007, -1.03865, -0.222601, -0.0294976, -0.82214, -0.689082, 0.306888, -0.438232, -0.000475959, -0.0230178, 0.23923, -0.0824, -1.39709, -0.612201, -0.130179, -0.0544338, -0.0354326, -0.994089, -0.0241366, 0.00220217, -1.11476, -0.0181113, -0.0227014, 0.161078, -0.0373118, -0.0478593, -0.0178996, -0.011092, -0.0294692, -1.21102, -0.00778812, -0.642056, 0.513263, -0.02649, -0.0256109, 0.0282009, 0.0123325, 0.113553, 0.524342, -0.0287218, 0.426784, -0.0117935, -0.127373, -0.827257, -0.12067, -0.0128538, -0.00880854, 0.0417438, 0.441075, -0.0526345, 0.340714, 0.685112, -0.032316, 0.536107, 0.248448, 0.227798, 0.0068502, -0.0322658, -0.751845, -0.0412142, 0.159269, -0.242265, 0.586672, -0.0285965, 0.00756432, 0.81659, 0.0205974, 0.00436536, 0.340222, -0.0631474, -0.0288525, -0.0294092, -0.0462732, 0.354667, 0.476931, 0.228864, 0.373789, 0.930472, 0.396889, -0.0288615, -0.0422125, -1.36768, -0.0480129, 0.106953, -1.01706, -0.00998504, 0.584718, -0.0394248, -0.309569, -0.465123, 0.000964097, -0.0258145, -0.214806, -0.657136, -0.0242341, -0.0143666, -1.02634, 0.62768, 0.54372, 0.003685, -0.248005, -0.00138446, -0.0293519, -0.0430486, 0.00177319, 0.752848, -0.956375, 0.596866, 0.00433188, -0.0401627, -0.0126982, -0.00197462, -0.0444156, -0.0606383, -0.0102484, -0.0150786, -0.964828, 0.00927947, -0.357474, -0.209732, -0.0292003, 0.568515, 0.35246, -0.0369358, -0.0112402, -0.0184257, -0.663125, 0.450492, -0.033995, -0.981809, 0.349794, -0.0117861, 0.353213, 0.697495, -0.358199, 0.141856, -0.0094493, -0.00995578, -0.311604, -0.287993, -0.481884, -0.247396, -0.283296, 0.003272, 0.000246241, -0.39608, -0.00460385, -0.0211582, 1.25298, -0.02615, -0.0204149, 0.536014, -0.0271859, -0.0489987, -0.0339192, -0.0252854, 0.243573, 0.188202, -0.0166696, -0.38247, -0.986069, -0.0400359, -0.0121405, -0.295069, -0.733978, 0.14269, -0.954091, -0.0151107, -0.462498, 0.00326225, 0.436796, -1.01705, -0.0551547, 1.07318, -0.021178, -0.376702, -0.522837, -0.0210391, 0.331889, -0.620863, -0.00358038, -0.182828, -1.89379, -0.843473, -0.108089, -0.00761442, 0.195449, -0.0310179, -0.994132, 0.02011, -0.779401, -0.0397434, 0.0495598, -0.0804277, -0.14921, -0.0141787, 0.194195, -0.0378692, -0.0166502, -0.00342858, -0.0541607, -0.0312146, -0.85945, 1.03434, -1.31949, -2.69806, 0.0272438, -0.0215277, -0.0407794, 0.209745, -0.0226533, -2.30921, -1.29838, -0.0348093, 0.428558, -0.0375869, -0.595583, 0.18507, -0.0344314, -0.0319325, -0.217798, -0.330873, -0.0223636, -0.0373204, -0.911976, 0.308212, -0.175917, -0.0406847, 0.163756, -0.0257733, -0.0183156, -0.0183679, -0.0260723, 0.0792015, -1.66136, 0.168433, -0.0351078, -0.0214517, -0.0326563, -0.00523886, -0.0235624, -0.0195803, -0.0116937, -0.0310379, 0.0309919, -0.00577087, 0.419329, 0.419932, -0.0140038, 1.41776, -0.580369, -0.0348861, -0.0251469, 0.00399354, -0.0803505, 0.586089, -0.0286537, -0.122808, 0.468046, -0.0454063, -0.675411, 0.854492, 0.274887, -0.562518, -0.0112514, -0.0248432, -0.72823, -0.12082, 0.300995, 0.130126, -0.892687, 0.0463104, -0.00975039, -0.639173, -0.0294029, -0.0124001, 0.133083, -0.00116872, -0.000940135, 0.159069, 0.00250474, -0.0402509, -0.00210512, -0.0109986, -0.331108, -0.463372, -0.00836018, 0.178069, -0.00236842, 0.00249233, -0.0268394, 0.121349, -1.09086, -0.766306, 0.661221, -0.0244472, -0.598771, -0.00546222, 1.29248, 0.133114, -0.0154556, -0.104464, -0.0202885, 0.366656, -0.615522, -0.0239025, -0.256931, -0.490883, -0.0167124, -0.49117, -0.0180655, -0.000313905, -0.341367, -0.0257614, -0.346788, -0.0414335, 0.145933, -0.283026, 0.264262, 0.004763, 0.012687, -0.597951, 0.7387, -0.0196039, -0.0931794, -0.0616966, 0.00339836, 0.0126806, -0.0355345, -0.0418903, -1.89652, 0.5174, -1.51884, -0.646522, -0.112785, -0.0163456, -0.00358394, -0.388934, 0.00365815, -0.449567, 0.378319, -0.0176197, 0.0543361, -0.0413827, -0.0467579, 0.250644, -0.00669326, -0.0263057, -0.530605, -1.41134, -0.0435323, -0.00177492, -1.00855, 0.293421, -0.347184, -0.0398057, 0.701584, -0.0251813, 0.00515004, -0.0802381, -0.0174917, 0.220327, 0.0281911, -0.329183, -0.014477, 0.00816118, -0.0143008, -0.0129142, -0.0079816, -0.0321821, -0.0392944, -0.0154132, -0.0449159, -0.00768584, -0.190533, -0.714685, -0.015929, 0.28691, 1.00311, -0.0410612, -0.0390052, -0.0174775, -0.410475, 0.148147, -0.0201303, 1.54596, 0.473571, -0.01717, 0.425479, -0.47039, 0.595007, -0.0406748, -0.0388492, -0.0266128, -0.583667, -0.329865, 0.373527, -1.20595, -0.634628, -0.0368951, -0.023766, -0.733437, -0.0544396, 0.00689404, -0.00568059, 0.00885767, 0.0037104, 0.0113895, -0.00702016, -0.00341589, -0.00538337, -0.00425436, 0.0144167, -0.00812166, 0.00209876, -0.00314281, -0.0254695, 0.0103208, 0.00751509, 0.0153837, -0.000833857, -0.00687921, -0.00546234, -0.0226565, 0.0102907, -0.0235344, 0.0158448, -0.00782218, 0.0091333, -0.00372524, 0.0086593, -0.00256009, -0.0047824, -0.0138012, -0.00398539, -0.0133195, -0.00893934, -0.00845534, -0.00188761, 0.00125018, -0.015371, 0.000656728, 0.00783511, -0.0231672, 0.00211002, -0.0115471, 0.00731984, -0.0143462, 0.00562324, -0.00550765, 0.0048808, -0.0064842, 0.0186949, 0.00783563, 0.0125292, 0.00873844, 0.00763255, -0.0136568, 0.0327683, 0.0062354, 0.00516009, 0.00939272, 0.0119483, 0.00384636, -0.0247362, 0.00877148, -0.00059785, 0.00509918, -0.00575052, 0.0174095, -0.0168197, 0.00147364, 0.00238906, 0.00169361, -0.00716306, 0.00470513, -0.0149695, -0.0121723, -0.00328208, -0.00704945, -0.00404596, 0.00128115, -0.00440929, 0.0109648, 0.00477281, 0.000919932, 0.00574674, 0.01378, -0.00712971, -0.0072301, 0.00733961, -0.00817247, 0.000100049, -0.00168829, -0.0072943, 0.00772628, 0.000965459, 0.0015007, -0.0057264, -0.0255454, 0.00360739, -0.00233436, -0.0129851, -0.00279431, 0.00539342, 0.0107457, -0.00701712, -0.00836962, 0.0140777, 0.00507911, 0.00610701, 0.00115562, 0.026754, 0.00759747, 0.00154275, -0.00427234, -0.00379542, 0.000738792, -0.0176843, 0.00711755, 0.0224363, -0.018002, 0.0246422, -0.00201521, 0.012306, 0.00872716, 0.00691183, -0.00441742, -0.0164356, -0.0149713, -0.00275372, 0.00014812, 0.00477377, 0.017442, 0.0101219, -0.00896574, -0.000458522, -0.00276282, -0.00382433, 0.0032552, -0.0127288, 0.0271107, -0.00484137, 0.0309129, 0.00861231, -0.00443809, 0.000670393, 0.0120283, -0.0228917, 0.0386362, 0.0317751, 0.0040711, 0.0191706, 0.00755962, 0.0115637, 0.0394312, 0.017444, -0.0321909, -0.00470572, 0.0225122, 0.0398629, 0.00916807, 0.00807752, 0.00950657, 0.0105843, -0.0347781, 0.0190818, 0.0282881, -0.00455556, 0.00178826, 0.0269582, -0.00295704, -0.0251324, 0.0231799, 0.0293245, -0.00310206, 0.0142498, 0.0420652, 0.033292, 0.0201734, 0.0360201, 0.00508009, 0.00287004, -0.00708027, 0.00210658, 0.00642615, -0.0264875, 0.0162492, -0.0182373, -0.0123365, -0.0128228, 0.00767298, 0.00388999, -0.0135486, 0.0183274, -0.0048108, -0.0269256, -0.0121872, -0.0293195, 0.00777291, 0.0220782, 0.0446826, -0.0181537, 0.0122946, 0.013076, -0.0108976, -0.00538487, 0.00439375, -0.0475417, 0.0193065, 0.0384252, -0.00503052, 0.0350693, 0.0120465, 0.0172013, -0.0067666, -0.0120403, 0.0223126, -0.00992412, 0.029783, -0.00356101, 0.0136281, -0.017697, 0.00194944, -0.0103464, -0.00157605, 0.00785377, -0.0117904, 0.0338743, -0.012989, -0.00122753, 0.0109537, 0.000600532, 0.0134252, 0.00219294, -0.00101294, -0.0158119, -0.00102034, 0.0213226, 0.0065137, -0.00104953, 0.0280312, 0.0185851, 0.0163791, 0.0110785, 0.0212864, -0.0411671, -0.00253619, 0.00233903, -0.0134681, 0.0391211, -0.0137479, 0.0216559, 0.00188692, 0.0248003, 0.0370497, 0.0148687, 0.0199386, -0.0111173, -0.00821501, 1.81869, -0.00399996, -0.00170128, 0.107742, -0.0240201, -0.0632338, -0.0117661, -0.00160815, 0.557042, -0.632123, -0.00321027, 0.388004, 0.341007, -0.000299828, 0.00879199, 0.237028, -0.169614, -0.386732, -0.183062, -0.00701011, 0.133103, -0.0293519, 0.121061, 0.101827, -0.047647, 0.55138, -0.00367969, 0.608594, -0.144072, -0.00974956, -0.399058, 0.444274, -0.0392662, -0.789742, -1.49379, 0.0318538, 0.17749, -0.0208472, -0.0695354, -0.0345698, -0.951193, 1.33258, 0.522197, -0.0237213, -0.0981879, 0.211753, 0.668602, -0.00167336, -0.0180664, -0.0300995, 0.0140938, -0.0413159, 0.00356346, 0.0021178, 0.0520703, 0.859294, 0.565207, -0.130688, 0.308091, 0.0106078, 0.0053934, -0.0530503, 0.0059651, -0.438183, -0.460275, 0.0149527, 0.872553, 0.00575656, 0.188087, 0.161217, -0.00668528, 0.00779882, 0.158542, 0.387636, -0.0203897, -0.0195326, -0.454651, -0.185966, -1.0555, 0.00655375, 0.0746947, -0.0150284, 0.00244399, -0.0181412, -0.00795745, 0.287875, 0.142468, -0.0568039, -0.0144454, 0.00851277, 0.00927693, -0.0024609, 0.00427453, -0.0421019, -0.0186159, -0.00240373, -0.820197, -0.0428692, -0.739065, -0.0213891, -0.025536, 0.321238, -0.338338, -0.0361446, -0.0255268, 0.0120298, 0.937745, -1.38069, -0.0196526, 0.775347, -0.327633, -0.0166949, -0.172083, -0.187177, 0.877379, 0.336402, -0.0116001, -0.00717472, -0.77715, 0.425653, -0.0302772, -0.483793, -0.370277, -0.0725024, -0.0154611, -0.0878993, -0.00827501, 0.00792455, -0.773459, 0.000798172, -0.00804295, 0.909675, -0.0140275, -0.0465532, -0.0209442, -0.00770995, -1.48994, 0.525726, -0.0160851, 0.635282, -0.435947, -0.0549129, -0.0181766, -1.1452, -0.608077, -1.17471, 0.886581, -0.0183703, -0.3902, -0.0110489, 0.617947, -0.0331045, 0.00797677, -1.65413, -0.0291933, 1.06078, -0.751729, 0.00350131, 0.282197, 1.9163, 0.0111668, 0.549456, -0.742165, 0.173845, -1.27059, -0.0180219, 0.345489, -0.0347525, 0.7308, -0.192493, -0.138226, -0.0376318, -0.0134436, -0.0438747, -0.0578462, -0.0129864, -0.79302, -0.0343907, -0.0271983, -0.0260554, -0.0321342, -0.348815, -1.23001, -0.891753, -1.14224, 1.07516, -0.00190248, -0.0275019, -0.050649, -1.31787, -0.0396547, 0.309957, -1.58127, -0.0391729, -0.612678, -0.0398973, 0.427107, -0.38829, -0.010587, -0.0261681, -0.58996, -0.885871, -0.0345016, -0.0390479, 0.00385062, -0.872883, -0.983136, -0.00080557, 1.6124, -0.0284343, -0.0256374, -0.0361132, -0.0184957, -0.190202, -0.940747, -0.423075, -0.0187238, -0.021477, -0.0379938, 0.000579493, -0.0431748, -0.0399064, -0.0160202, 0.00938112, -0.343757, -0.00275624, -1.43803, -0.77966, -0.0277063, 1.95579, -0.367125, -0.0289825, -0.0448383, -0.0160369, -1.0573, 0.161902, -0.0399016, 0.568464, -0.682531, -0.0713346, -0.0631509, 0.67834, 0.35552, -0.199846, -0.00891679, -0.0201972, -0.787801, -1.17029, 0.11048, -0.244519, 0.0189876, 0.071492, -0.0293434, 0.0419723, 0.00882807, -0.017817, -0.0402151, 0.0109853, -0.0114088, -0.0213785, 0.0105972, -0.00468124, -0.0107129, -0.0178774, 0.0286998, -0.00658054, 0.000902788, 0.0402836, 0.0225316, -0.00250775, 0.00925617, 0.0158357, 0.0294413, 0.0189008, 0.0268523, -0.00808248, 0.0129354, 0.00647898, -0.031585, 0.0143756, 0.00489833, -0.0190614, 0.0127759, -0.0101175, 0.00799521, 0.0111986, 0.00940907, -0.00526135, -0.0134704, 0.061906, 0.113657, 0.0141964, -0.00649034, 0.00593872, -0.00406592, -0.00628168, -0.00619601, -0.071689, 0.0676714, -0.00694829, -0.0058792, 0.0465466, -0.0271516, 0.0139068, 0.00349725, 0.0178415, 0.0222317, -0.00235476, 0.00170367, -0.00991587, 0.0122167, 0.00376317, -0.0343399, -0.0228338, -0.000176673, -0.00343915, -0.0104557, -0.0248742, -0.00299753, -0.031287, 0.0246534, 0.00724585, -0.00569359, -0.0277906, -0.00197251, -0.0504318, -0.00330874, -0.00674944, -0.0785101, -0.0282078, -0.012334, 0.0111695, 0.0275929, 0.0373944, -0.0142844, -0.00663382, -0.0106581, -0.0106963, 0.0073249, -0.00195679, 0.0120678, 0.0119905, 0.00216198, 0.00831523, 0.020866, 0.00392755, -0.0179017, -0.00404968, -0.0108965, 0.0366642, 0.000548486, 0.0037597, 0.00738371, 0.00712311, 0.00846603, 0.0610559, 0.0107053, -0.0340632, -0.0173365, 0.00938514, 0.00458263, 0.00797939, -0.0216125, -0.00282087, -0.00241417, 0.0180831, -0.0127481, 0.0120783, -0.0114979, -0.00633479, -0.0217986, -0.0428779, -0.0124244, -0.0133298, 0.00400121, -0.0447882, 0.0115161, 0.0554561, 0.0313384, 0.0139388, 0.00790203, -0.0184093, 0.00561922, 0.0249143, -0.105731, 0.00724304, -0.0148119, 1.53669, -0.0125804, -0.0111358, -0.0279657, -0.0157001, -1.46491, -1.60029, -0.000835072, -0.303736, -0.6893, -0.0188535, -0.00213753, -1.02498, 0.485479, -0.226003, -3.08303, 0.00368426, -2.00076, 0.000603963, -0.524488, 0.728477, -0.043962, -0.786747, -0.00131173, 0.812208, -0.465117, -0.0234898, 0.32256, -0.969567, 0.00553793, 0.641218, 0.366805, 0.393648, -0.310746, -0.0165516, -0.107791, -0.00793009, -1.27816, -1.35541, 0.479006, -4.31173e-05, -0.105823, 0.511767, 0.46192, 0.00681505, 0.248328, 0.0155262, 0.00231138, -0.00394106, -0.0137782, 0.365638, -1.23174, 0.0371709, 0.0333627, 0.677844, -0.54769, -0.00222442, -0.0121813, -0.783496, -0.00251742, 0.0721881, -0.774046, 0.0124152, 0.61749, 0.0184286, 0.00682947, -0.520216, -0.0284331, -0.000288939, -2.38124, -0.872594, 0.00488288, 0.00426214, 0.960296, -0.672544, -0.546204, 0.00422045, -0.451845, 0.00106744, 0.0108878, -0.0026026, -0.000304822, -1.92487, 0.0472086, -3.20105, 0.00501464, -0.00181344, 0.031447, -0.0195278, 0.0144074, 0.0491028, 0.00888748, 0.00231416, 1.40584, 0.00173262, -1.14472, -0.601862, -0.0089439, -1.89245, -0.0865313, 0.00316215, 2.69197e-05, -0.0164188, -0.366143, 1.42039, 0.00473431, 0.84285, 0.195636, 0.00792016, -0.52415, 0.332664, -1.27796, -1.06926, -0.0196386, 0.00423793, -0.812203, -1.54941, -2.35918, 0.0721255, -1.83675, -0.848449, 0.00919265, -0.827681, -0.0575538, -0.00839575, -0.664802, -0.00650982, -0.0260706, 0.155008, -0.0377282, -0.0152655, -0.0262942, -0.0435311, 1.11126, 0.333921, 0.00944212, -0.431285, -0.894226, -0.00727735, -0.0101425, -1.29497, -0.00104642, -0.667721, 0.0713181, 0.00574962, -0.170126, -0.0130168, 0.0576042, -0.356514, -0.0694071, 0.208208, 0.0125219, 1.35318, -0.155282, 0.00606253, -1.0424, -0.218323, -0.0153453, -0.00236963, -0.105556, -0.346016, -0.881958, -0.0178968, 0.319527, -0.0116305, 0.512807, 0.132429, -0.425689, -0.0203515, -0.131758, -0.079986, 1.092, -0.0184658, 0.153073, -0.0213494, -0.0125242, -0.0361119, -0.0283428, 0.283587, -0.540826, 0.18238, 0.175874, 0.526965, -0.0100446, -0.00704414, -0.0327451, -0.0418899, -0.0480426, -0.439626, -0.969826, -0.0255745, 0.175577, 0.00350233, -0.606505, -0.546923, -0.0394279, -0.0026216, -0.347612, -0.738898, -0.0202217, -0.0460291, -0.271507, -0.645876, 0.0908388, -0.0399465, -0.632106, -0.0328241, -0.0127633, -0.0121899, 0.0108691, -0.190112, -0.548946, 0.24388, 0.00498084, -0.0140141, -0.016273, -0.023043, -0.0197841, -0.040331, -0.0660615, -0.0293104, -0.265517, 0.00174205, -0.335215, -0.00165101, -0.0134917, -0.552626, 0.0499507, -0.0339765, -0.0710432, 0.00851566, -0.902081, 0.603087, 0.00725562, -0.0159293, -0.186031, -0.019061, -0.432882, -0.0430992, -0.958371, -0.485302, -0.0194358, -0.0411568, 0.393407, -0.302146, 0.293157, 0.0554512, -0.406615, 0.0125234, -0.00641782, 0.141603, -0.0313439, -0.0174744, 0.00376242, -0.01209, -0.00181174, 0.0158841, 0.0089485, 0.0118628, -0.00361188, 0.003952, -0.015563, 0.0371966, 0.00434045, 0.017929, 0.0587466, 0.00285189, 0.00989818, 0.0166528, -0.0271945, 0.0349106, 0.0422637, 0.0104178, 0.0170974, -0.00685738, -0.00970243, 0.0195726, -0.00284985, -0.0479062, -0.00688928, -0.0118087, 0.0401692, -0.00342799, 0.0104217, 0.0361559, 0.012159, -0.0120373, 0.025822, 0.0585273, -0.0118447, -0.00441543, 0.0244331, 0.0176911, -0.011455, -0.00361314, 0.0436909, -0.0142705, 0.0075438, 0.0581614, 0.0409944, 0.0219129, 0.0276309, 0.0201126, 0.00454064, 0.0043768, 0.00426728, 0.0244355, -0.0373943, -0.00674134, -0.00183873, -0.00778059, 0.00659912, -0.00522993, -6.57017e-05, -0.0429109, 0.0010229, -0.0159761, -0.0121086, 0.00104759, -0.00713864, 0.0138012, 0.0175294, 0.0505775, 0.00317056, -0.0166291, 0.00261177, 0.00667755, 0.00257788, 0.00199713, 0.0240592, 0.0286248, 0.0318475, 0.00343911, 0.0382759, -0.0216013, -0.00585356, 0.0054063, 0.00594799, 0.0184022, -0.00382683, 0.045215, 0.00567921, -0.0159201, -0.00629495, 0.00477611, -0.010477, 0.00864029, 0.00366273, 0.00520506, 0.0641735, -0.00357809, -0.0182799, -0.0446394, -0.010403, 0.00348338, -0.0194344, 0.00887889, 0.00868508, 0.0108297, -0.00140199, 0.000495831, 0.0144504, -0.0209942, 0.0236232, -0.0130595, 0.0133936, 0.0192962, -0.0242413, 0.00403935, 0.000716762, -0.00908957, 0.00844103, -0.0115603, 0.012802, 0.0706008, 0.0148156, 0.0370713, 0.000573073, 0.0192207, -0.00115191, 0.0101394, 0.0511493, 0.00122652, -0.00879579, 0.361077, 0.000505632, -0.00187534, -0.00384674, -0.0153134, 0.0155767, 0.0573082, -0.00693538, 0.120088, 0.173219, -0.00569391, -0.0121913, 0.0881151, 0.101345, 0.0320983, 0.163418, -0.021652, 0.0878412, -0.00708847, -0.0518987, 0.0327592, 0.0114762, 0.0962929, -0.031681, 0.0889447, -0.0320669, -0.0299902, 0.0932835, 0.147633, -0.00124914, 0.177736, -0.108257, 0.0131609, 0.0331709, -0.00141957, 0.0734514, -0.0205792, 0.00588242, 0.0368373, -0.0277572, -0.00937294, -0.00125281, 0.0811126, 0.0605989, -0.0151557, 0.0121416, -0.00894827, 0.00423157, 0.0123367, -0.000654186, 0.0801896, 0.097183, 0.0199106, -0.0662245, 0.0753088, 0.0801415, -0.0194582, -0.0110043, 0.119818, 0.0125789, -0.205543, 0.0793995, -0.0171719, -0.01355, -0.00674375, 0.0376928, 0.157219, 0.00170616, -0.000615879, -0.0776162, 0.0737617, -0.00111374, -0.00929118, 0.0613627, 0.0654399, -0.0294546, -0.0275703, 0.201333, -0.00929509, 0.00402987, -0.0217905, -0.00309086, 0.0134156, -0.0621394, 0.155501, -0.010892, -0.0103331, 8.0835e-05, -0.00661295, -0.0106616, -0.0126655, 0.00195952, 0.00833225, 0.17626, -0.00389188, -0.0190592, -0.0551749, -0.0101475, 0.0121272, 0.164578, -0.0235577, -0.00744465, -0.00271282, 0.132557, -0.0214061, -0.0355589, -0.129366, 0.0574873, -0.0177129, 0.127105, 0.0648307, 0.100891, -0.053212, -0.00489907, -0.0145741, 0.027618, 0.0462644, 0.0348206, 0.0490113, 0.0481768, 0.0601051, -0.0233164, 0.085695, -0.0351575, -0.0125134, -0.267784, 0.0121996, 0.00916549, 0.237973, -0.00428535, 0.00151494, -0.00519563, -0.0299609, 0.15489, -1.22037, -0.011967, 0.292716, -1.29808, 0.00387152, -0.0492368, 0.871784, -2.04288, 0.452289, -1.91565, -0.00788674, -1.81691, 0.00801681, 0.793778, -1.28381, -0.0412354, 0.206073, 0.000551704, 0.409017, -0.958228, 0.00297536, -0.0205529, -0.124455, -0.0294058, -2.38631, -1.81333, -1.1186, -0.309307, -0.0209862, -1.59383, -0.0271426, -2.69458, 0.222225, -1.4054, -0.0117256, 0.109677, -1.15476, 0.315025, -0.0174136, -0.339775, -0.00519224, 0.00731629, 0.0135184, -0.0180888, -0.206197, -1.72396, 0.366977, -0.00166116, 0.524336, -0.132782, -0.0249679, -0.0277395, 0.646561, 0.00435825, -1.32117, -0.461485, -0.00783762, -0.319816, 0.000275459, -0.587351, -0.141139, -0.016902, 0.000394463, -2.3901, 0.524115, 0.00202623, 0.00922687, -1.29644, 0.0960397, 0.374251, -0.0039824, 0.373049, 0.0235515, 0.00891541, -0.0192034, -0.0125853, -0.0693877, -0.838149, 0.347227, -0.0155755, -0.0233528, 0.00257403, -0.0128803, 0.00022375, -0.0695562, 0.0390974, -0.0523029, -1.34274, -0.0100843, 0.573651, 0.412043, -0.0104395, 0.353801, -2.10752, -0.0285825, -0.0410622, -0.0128728, 0.248585, 0.14186, -0.0435515, -2.2605, 0.0319518, -0.024767, -1.13556, -0.964845, -1.74454, 0.556916, -0.0541674, -0.000703905, -0.678861, -0.00630509, -1.0216, -1.34493, -2.21872, -0.00403229, -0.0444931, -1.76688, -0.0718792, -0.0026681, 0.839145, 0.00795503, -0.00529549, -0.59737, 0.00428136, -0.0154974, -0.0127094, -0.0281836, -0.37972, -1.53646, 0.0134397, -0.0535072, 0.878593, -0.00189417, -0.00985656, 0.566477, -0.360488, -0.95109, -0.164654, -0.0166357, -0.379033, -0.0130302, 0.911358, -1.62708, -0.100874, -1.00874, -0.0130519, 0.112528, -1.07537, -0.0367801, -0.44853, 0.376211, 0.000639941, -0.702893, -0.210209, 0.10944, -1.06651, 0.00160234, 0.0935999, -0.0411162, 0.680675, -0.148679, 0.120075, 0.0126521, 0.0360333, -0.894099, -0.405472, -0.0326319, -0.597569, -0.00607354, -0.0131477, -0.0028281, -0.0326483, -0.378429, -0.244919, -1.15981, -0.0240367, 0.615786, -0.0530441, -0.0290996, -0.0282313, 0.48342, -0.00365191, 0.0390697, 0.636399, -0.0346482, 0.698192, 0.0120578, 0.110486, -0.226885, -0.00588193, -0.00493704, 0.294756, -0.157096, -0.00325064, -0.011287, 0.183348, -0.397429, -1.53361, -0.0181315, -0.965206, -0.0293282, 0.0125449, -0.010764, -0.0143615, -1.04123, 0.701266, 0.120549, -0.0130995, 0.0113677, -0.0109032, 0.0130493, -0.021627, 0.0164049, 0.00245257, -0.011195, 0.426665, 0.0178578, 0.854809, -1.05663, -0.01239, 2.53287, -0.176627, -0.04253, 0.0157944, -0.00789809, -0.226358, 0.402011, -0.0391584, 0.848545, -0.783695, -0.0193684, 0.182601, 0.29531, 0.684617, 0.241095, 0.0019025, 0.00484122, -0.121591, 0.0671556, -0.51995, 0.394589, -0.730434, -0.201608, -0.00529474, -0.131316, -0.033262, -0.0257899, 0.356306, -0.00760536, -0.0226368, 1.1436, -0.00275936, -0.000523105, -0.0119127, -0.00929098, -0.679769, 0.210584, 0.00491827, 0.615539, -0.84532, -0.0207312, -0.0240192, -0.306029, 0.302012, -0.325583, -1.01003, 0.00019503, -0.819505, -0.0103766, 0.74801, -1.28439, -0.0106961, 1.19928, -0.0192485, 0.490977, -0.512602, -0.0385516, 0.463866, -1.66439, -0.00731836, -0.77795, -0.355835, 0.740327, -0.0291618, -0.0288905, 0.165512, -0.0261482, -1.0284, 0.744543, 0.822047, -0.000715511, -0.116863, 0.0982895, 0.195503, -0.0421466, -0.034564, -0.00393714, 0.0197325, -0.00394536, -0.00656044, 0.150011, 0.487501, -0.0542112, -0.366374, -0.446734, -0.178637, -0.0326079, -0.0522807, 0.245861, -0.0370248, -0.115003, 0.450301, -0.0091174, -1.08448, 0.00164903, -0.198107, 0.63618, -0.0032747, 0.00274094, -0.197138, -0.676588, -0.00336683, 0.0118141, -0.822021, 0.181687, -0.840864, -0.0377389, 0.511734, -0.0150878, 0.0009518, -0.0268563, -0.0210424, 0.239336, 0.0726149, -0.238213, -0.0190676, 0.0124086, -0.0218881, -0.0189356, -0.0347724, -0.0257892, -0.017526, -0.0473026, 0.233768, 0.00714468, 0.352308, -0.19871, -0.0497301, 1.92172, -0.0545989, -0.0234673, -0.055919, -0.0176285, -0.55444, 0.888206, -0.0409944, 1.27623, 0.0819425, -0.0177784, -0.756536, -0.881172, -0.0388469, -1.13155, -0.0325584, 0.00409956, -0.990952, -0.0612853, 0.736049, 0.569289, -0.634341, -0.255052, -0.0167619, -0.305972, -0.0180096, -0.0121517, -0.0280937, -0.00558656, -0.00910819, 0.835808, 0.0107204, -0.0118142, -0.000305625, -0.00353645, -0.241612, 0.485654, 6.04133e-05, -0.724756, 0.469263, 0.0153634, -0.00682551, -0.587849, 0.0412523, -0.68825, 1.18255, -0.0307412, -0.0309195, 0.00220309, -1.1246, -0.0774686, 0.00629362, -1.31396, -0.0302499, -0.125137, 0.275005, -0.038739, -0.549188, 0.898766, -0.0702599, -0.293283, 0.539817, 0.34237, -2.40166, -0.0181669, 0.00892524, -0.0489687, 0.166076, -0.180081, 0.231936, -0.0149852, -0.0246382, 0.27077, -0.937175, -0.00328642, -0.434854, 0.0297142, 0.00936418, -0.0241796, 0.00883047, 0.0520275, -0.790098, -1.12436, -1.08694, 0.720241, -0.35616, -0.0220542, -0.0253563, 0.104499, -0.0335902, -0.15907, 0.304994, 0.00182395, -0.85986, 0.00765302, 0.362226, -0.744865, -0.00415288, -0.0230751, 0.103588, -1.79329, -0.0417011, -0.0346308, 0.226303, -0.0821794, -1.23698, -0.020987, -0.505509, -0.0158702, -0.0165238, -0.0196602, -0.0278003, -1.33784, -0.321961, -0.772806, -0.00479022, -0.00300839, -0.0278439, -0.0250248, -0.0311721, -0.0521429, -0.0277065, -0.0427302, 0.40962, -0.0243046, -0.454221, -0.208848, -0.00686666, -0.258394, -0.450138, 0.00989476, -0.0239335, 0.0169854, -0.854012, -0.55756, -0.0245681, -0.265263, -0.894417, -0.0216533, -0.241269, -0.360585, 0.326728, 0.0439871, 0.000619573, -0.019297, -1.28288, -2.49153, 0.0866997, 0.342682, 0.0989741, 0.102604, -0.0415475, -0.296686, -0.0490984, 0.0106646, -1.23003, 0.00432317, -0.039529, -0.115145, -0.0113284, -0.076534, 0.0120967, -0.00416861, 0.782635, 0.223798, 0.000181878, 0.575986, -0.821083, -0.015149, -0.0451006, -0.0488127, 0.475406, -0.534428, -1.12122, -0.00027443, -0.466407, -0.0107089, 0.31583, -0.0342136, -0.102442, 0.67312, 0.0159976, 1.14006, 0.165902, 0.00690917, 0.314142, -1.96586, -0.0145657, -1.10251, -0.338935, 0.641827, -0.999026, -0.00578284, -0.364973, -0.0153952, -0.973216, -0.371205, -0.410419, 0.0189752, -0.0837611, 0.143356, -0.174248, -0.0217556, 0.0899114, -0.0276298, -0.00205518, -0.0616189, -0.0158674, 0.260428, -0.159769, -0.862806, 0.195407, -1.42973, 0.313811, -0.0247721, 0.00405464, 0.415776, -0.00151099, 0.0568919, -0.29468, -0.0054085, 0.411366, -0.0151793, -0.998199, 0.00685612, -0.0168845, -0.00202013, 0.107411, -0.408794, -0.0330878, 0.00828536, 0.35589, -1.22142, 0.269586, 0.00548761, 0.0435276, -0.0126946, -0.0297786, -0.00798539, -0.0236019, -0.918386, 0.323685, -0.743659, 0.00116747, -0.00588081, -0.0224999, -0.0573804, -0.0173077, -0.0698897, -0.0401032, -0.0145223, -0.069699, 0.00366738, 0.316918, -0.464111, -0.014503, 0.0160817, 0.0901252, -0.037274, -0.035432, 0.013587, -0.158418, -0.327271, -0.0359013, 0.431173, -0.896217, -0.0131539, -1.40082, 0.670299, -0.985486, -0.0344288, -0.0404394, -0.00507595, -0.624319, 0.0931819, 0.612007, 0.385723, -0.273154, -0.0359936, -0.0197025, 1.20624, -0.00740346, 0.00113779, 0.448138, -0.00173571, -0.00680088, 0.640365, -0.0384487, -0.0829063, -0.021793, -0.0223468, -0.136391, 0.477813, -0.0107408, 0.378131, -0.532385, -0.0206698, 0.00965198, 0.877869, -0.424865, -0.340153, -0.55017, -0.0156302, 0.731658, -0.000285675, 0.572622, 0.654972, -0.0231058, 0.080309, -0.0125436, -0.547911, 0.222139, 0.000901597, -0.532233, 0.221423, -0.0323335, -0.164783, -1.32811, 0.63198, -0.50068, -0.0312561, -0.65627, -0.0177432, -0.1887, 0.582307, -1.15451, -0.045765, -0.0142199, 1.42747, -0.762468, -0.00435592, 0.481633, -0.00713053, -0.00513405, -0.00343859, -0.0136186, 0.525607, -1.08583, 1.19301, -0.20689, -0.517516, -0.371884, -0.0318226, -0.0417599, -0.698494, -0.0257607, 0.55723, -0.646995, -0.0309585, 0.534834, 0.00761785, 0.0812141, 0.0641419, -0.0102654, -0.0109768, 0.85888, -0.570544, -0.00346548, -0.00415571, -0.653206, 0.0274489, -0.64616, -0.0353965, -0.0836417, 0.00502985, -0.0399974, -0.0467483, -0.00625609, 0.308399, -0.327108, 0.173412, -0.0212469, -0.0298127, -0.0498105, -0.0208389, -0.0336105, -0.0204773, 0.00750305, -0.0221739, 0.00309208, 0.00417429, -0.154203, 0.438325, -0.0187064, 0.131726, 0.305229, -0.0346146, 0.0100536, -0.0161892, 0.380934, -0.301817, -0.0363467, 1.6437, 0.664258, 0.00472783, 0.540805, 0.49238, -0.547345, -0.236286, -0.00839007, -0.0159967, 0.137062, 0.655273, 1.02534, -0.97332, -0.284915, -0.0960391, -0.0140986, 0.263643, 0.00209238, -0.00990774, 0.157649, -0.00603382, -0.0168957, -0.0362357, -0.0278279, -0.0744426, -0.018945, -0.0135484, 0.14368, -2.10783, -0.00426995, 0.0329226, -0.273661, -0.0139212, -0.0245903, 0.764022, -1.90064, 0.496716, -0.488683, -0.00680757, -0.697153, -0.0139805, 0.760971, -0.278924, -0.0329201, 0.551877, -0.0053158, -0.783622, -1.60483, 0.0227198, -0.939436, 0.0663746, -0.0521847, -0.678027, -0.674851, -1.81131, -0.492073, -0.0167468, -0.865854, -0.0360878, -0.12421, 0.992513, -0.72884, -0.0101033, -0.00238432, -0.334812, 0.557788, -0.0303402, -0.437765, -0.0105335, -0.000194562, -0.0147517, -0.0136259, 0.00516651, -1.69231, 0.855714, -0.906801, 0.70658, -0.133323, -0.0349037, -0.0514121, 0.702675, -0.0329124, -1.01349, -1.52504, -0.00411021, 0.582066, 0.0150814, -0.437567, -0.67548, -0.00637416, 0.00577128, -0.408266, 0.964065, -0.0431947, 0.0160271, -1.83168, 0.177242, 0.353305, -0.0134247, 0.0903699, -0.0193194, -0.027647, -0.0131983, -0.0188752, 0.189439, -1.16413, 0.0607885, 0.00223145, -0.0147356, -0.0258013, -0.0304234, -0.0363364, -0.0425236, 0.00498264, -0.0206984, -1.65051, -0.0105892, 0.869416, 0.357362, -0.0127266, 0.430445, -0.987838, -0.0129733, 0.046924, -0.00591755, -0.0733136, 0.195326, -0.0205403, -1.10961, 0.0334444, -0.0381716, -0.959362, -0.101066, -0.576712, 0.813171, -0.00186452, -0.00237336, -0.0693523, -0.182852, -1.07509, -0.996055, 0.0717365, 0.165148, -0.0216304, -0.868424, -0.0105736, -0.0105722, 1.56025, -0.000106761, -0.00765857, 0.478609, 0.0142274, -0.00246788, -0.00401309, 0.0109753, -1.03987, -0.414337, -0.0018958, -0.973852, -1.14466, 0.00422149, 0.00377294, -0.111342, 0.595012, -0.905773, -0.0769884, -0.00617076, 0.263065, 0.0101203, 0.354561, -0.110182, -0.0282514, 0.725959, -0.0137003, 0.528798, -0.251402, 0.00538742, -0.862223, 0.897273, 0.00708433, -0.520796, -0.49456, -0.471487, 0.0707359, 0.00148039, -0.222368, -0.0178858, 0.376722, 0.120533, -0.127252, -0.0128856, 0.013942, -0.0987151, 0.18365, -0.0090685, -0.160053, -0.00263105, 0.00586481, 0.018454, 0.0129048, 0.0645385, 0.342868, 0.599729, -0.305477, 0.332748, -0.710796, 0.00374046, 0.0199901, -1.18221, 0.0155227, -0.85724, 0.194482, -0.0118659, 0.569359, 0.00825608, 0.22639, -0.0737732, 0.00182876, 0.0135881, 0.311918, 0.0694379, -0.00230701, 0.00493813, 0.126845, -0.872961, -0.858117, 0.0062805, 1.08449, 0.0282071, -0.000798936, -0.0100192, 0.00642759, -0.170435, -0.618554, -1.01295, 0.0035173, -0.0100683, -0.0066323, -0.00499924, -0.00804607, 0.0153327, 0.000110823, -0.00384548, 0.82881, -0.0103246, -0.273929, -0.957661, 0.00681917, -0.245203, 0.270228, -0.00382538, 0.00745076, -0.0034785, -0.00873689, -0.000514546, -0.00826868, -0.123062, 0.210184, -0.00814085, -0.112796, -1.28825, -0.0514846, 0.556034, 0.0106673, 0.0137164, 0.110862, -0.354503, -0.0903093, -0.00590531, -0.334554, 0.018947, -0.0174412, 0.44256, 0.0126829, 0.00632824, -0.473001, -0.00431998, -0.0128872, 0.349013, -0.0186969, 0.0155493, -0.00836712, -0.0140098, 0.483377, -0.219796, -0.00535069, -0.287055, -0.0796814, 0.0021221, -0.0137494, 0.0929817, -0.782924, 0.0717582, -2.30397, -0.0249555, -0.846062, 0.00419484, 0.576542, -1.49369, -0.0525349, 0.610665, -0.0294419, -0.116367, -0.476313, -0.0399757, -0.381039, -0.405381, -0.0184556, -0.838225, -1.35935, 0.205871, -0.273306, -0.0118592, -0.258399, -0.0377242, -1.62293, 0.38044, -1.8384, -0.0204143, -0.0505301, -0.393632, 0.440477, -0.0147078, -0.474201, -0.00707916, -0.0250064, 0.0116432, -0.0104382, -1.04512, -0.821868, 0.345839, 0.13766, -0.976866, 0.0712143, -0.017468, -0.00768941, 0.642303, -0.00523328, -0.60568, -0.108714, -0.00472678, 0.7707, -0.0239108, -0.765959, -0.24437, -0.0383118, 0.00503673, -1.15029, 0.181975, -0.00565523, 0.00350468, -2.24367, 0.115927, 0.610452, -0.0104541, -0.339508, -0.0268158, -0.000614527, -0.0403991, -0.0205061, -0.149688, -1.02344, 0.26854, -0.0164149, -0.0121534, -0.0164031, -0.0405328, 0.00466957, -0.000503745, -0.0321012, -0.0259338, -1.10973, -0.00665194, 0.164231, 0.705671, -0.0232149, 0.0383845, -0.855961, -0.0283828, -0.0109322, -0.0187557, 0.752479, -0.237513, -0.0354204, -1.90204, 0.319525, -0.0334024, -3.9217, -1.3949, -0.126479, 0.203003, -0.0153558, -0.00377654, -0.154304, -0.968511, 0.558714, -0.961503, -0.989719, -0.0269719, -0.0162797, -0.337568, -0.0836309, -0.0217762, 0.0792958, 0.000928464, 0.00942052, 0.346251, -0.00317584, 0.00182952, 0.00614832, -0.0078079, 0.0570354, 0.0321876, 0.0104908, 0.0829491, 0.0264927, -0.0143373, -0.00967768, 0.0820076, 0.140091, 0.032512, -0.0377141, 0.0039874, 0.00446794, 0.0154274, 0.0299618, 0.0727163, 0.00628328, 0.261921, -0.0120451, 0.0308591, -0.0212464, 0.00201655, 0.219751, 0.0864019, 0.0111345, 0.193251, 0.00860514, 0.0569652, 0.187245, -0.0209001, 0.0211229, -0.0377107, 0.00220323, 0.0916651, 0.0460423, -0.00147891, -0.0050861, 0.0529899, 0.0160283, -0.0105947, 0.048228, -0.0148852, -0.00447017, -0.010119, -0.0157513, 0.0561736, 0.114959, 0.0374514, -0.0512747, 0.0773194, 0.0865722, -0.0363691, 0.000415395, -0.00344498, 0.00137616, 0.0267803, 0.144254, -0.0257445, -0.0292139, 0.00786446, 0.0705202, 0.0805554, -0.0111209, -0.00596047, -0.10101, 0.004576, -0.0172652, 0.00424091, -0.0303004, 0.106237, 0.029893, -0.0137151, 0.124887, 0.00401723, -0.00612972, -0.00560803, -0.0152323, 0.0546714, -0.023623, 0.0508353, -0.0226021, -0.0055092, -0.0119391, -0.00625861, -0.0139351, -0.0135585, 0.0050759, -0.00303542, 0.0619343, -0.00248285, -0.0223492, 0.0520626, -0.0337045, -0.00837858, -0.038836, -0.04486, 0.0204889, -0.0110179, 0.0560868, 0.0179696, -0.0297504, 0.0330499, 0.0676117, -0.0190176, 0.086228, 0.0980913, 0.0920675, -0.00924859, -0.0030524, -0.0159496, 0.0728368, 0.0336567, 0.0220265, 0.0203701, -0.0180782, 0.0492685, -0.00175873, 0.035418, -0.0301917, -0.0154399, -0.808304, 0.00668275, -0.0162591, 1.57991, -0.0226737, -0.0874459, -0.0237712, -0.0243206, -0.583269, -0.669714, 0.0040613, -0.203168, -0.55156, -0.00120124, -0.0101555, -0.814438, 0.135928, -1.17151, -0.511924, -0.0192614, 0.226462, -0.000774778, -1.11574, -0.337041, -0.02859, -0.0598495, -0.0261335, -0.190532, -0.366427, -0.0214254, 1.15393, -1.48614, 0.0089502, -0.0975716, 1.5953, 0.776569, 0.37724, -0.0080801, -0.461816, -0.0354436, -0.536362, 0.521105, 0.579193, -0.0202789, -0.0656604, 0.888046, -0.535343, -0.00717307, -0.382161, -0.0237359, 0.00461049, -0.0163954, 0.00330606, -0.480927, -0.615317, 0.671257, 0.775436, -0.825074, 0.156304, -0.0290119, -0.0421194, -1.23756, -0.0389944, 0.663553, -0.33522, -0.0283101, -0.678261, -0.0140354, -0.0112908, -0.661114, 0.00449585, -0.00618766, -0.274821, -0.390999, -0.0351471, -0.0376712, 0.664553, -0.716503, -0.02456, -0.00340603, -0.722756, -0.0141373, -0.0301557, -0.0424547, -0.0241216, -0.176871, -1.624, -0.0857617, -0.00141751, -0.036224, -0.0406989, -0.0346469, -0.0675461, -0.0448128, -0.01839, -0.020953, 0.307711, 0.0131238, -0.263353, -0.242465, -0.0205777, 0.349778, -0.22395, -0.0278762, 0.0108297, -0.0261399, 0.178163, -0.472485, -0.0325217, 0.240936, 0.151153, -0.0341341, -1.09062, -0.386694, -0.616217, -0.949469, -0.0101533, 0.00110529, -0.831539, 0.851937, -0.618864, -0.288605, 0.0223896, -0.0128897, -0.0297818, -0.714282, -0.0321771, -0.0182984, -0.0790512, -0.0244709, 0.0112256, -0.449049, -0.0505709, -0.0725657, -0.0149419, -0.0267831, -0.097376, -1.01909, -0.0137101, -0.899853, 0.404629, 0.00492209, -0.04236, 0.358376, 0.684937, -0.647976, -1.04203, -0.0159252, -0.0279207, -0.0125158, 0.976124, -0.280558, -0.0818154, 0.376222, -0.00387418, -0.265758, -0.184779, -0.0382544, -0.684781, 0.475177, -0.061493, -0.74449, -0.374023, 0.338071, 0.021768, -0.00981883, -0.651224, -0.0506818, -1.85985, 0.260542, -0.0521363, -0.0207997, -0.0506557, -0.517304, -0.290392, -0.00916526, -0.07685, -0.0402786, -0.0161785, -0.00385633, -0.024262, -0.0924025, 0.152394, 0.34769, -0.701274, -1.06326, 0.175306, 0.00660071, -0.0136047, 1.30292, 0.0226261, -0.641844, 0.676523, -0.0193685, 0.0932343, -0.0287579, -0.180195, 0.572206, -0.0223638, -0.00709978, 0.273268, -0.602647, -0.0154385, -0.0321716, -0.885157, 0.401868, 2.0198, -0.0319564, -0.421321, -0.0262698, 0.00581381, -0.0175638, -0.0256223, 0.621908, -0.00497555, 0.544705, -0.0052923, -0.0244647, -0.0118126, 0.00118304, -0.00203988, -0.00628477, -0.0242581, -0.0260143, -0.472978, -0.0485722, 0.787448, 0.0344969, -0.014884, 0.499124, 0.0974811, -0.049627, -0.0460646, -0.00774807, 0.909961, -1.25834, -0.0195182, -0.860174, -0.0640822, 0.0123617, -1.51374, 1.43553, 0.044234, 0.150006, 0.0010761, -0.0266298, 0.391968, 0.0601138, -0.195267, -0.135017, -0.569525, -0.41001, -0.0183007, -0.118255, -0.00147725, 0.000356877, -1.57408, -0.00158321, -0.0318762, 0.744147, -0.0243632, -0.0824994, -0.00880836, 0.000134667, -0.0557819, -1.62208, -0.00299758, -0.559397, 0.416543, -0.0298829, -0.00231562, 0.0490681, -1.30556, 0.0799167, -2.76321, -0.00168651, -1.23873, -0.00593152, 0.890421, -0.0564716, -0.0640916, 0.394126, 0.00118379, -0.135418, -0.470398, -0.0432698, -0.366445, -0.739457, -0.047764, -1.02792, -1.72456, -1.17036, -0.550349, -0.02165, -0.0858898, -0.0336189, -1.67462, -0.20301, -0.701656, -0.0141242, -0.0747992, -0.572263, 0.780035, -0.0110063, 0.0784216, -0.00641396, -0.0235877, -0.0205223, -0.000953281, 0.0229907, -1.86027, -0.05742, 0.32269, -0.369574, 0.107214, -0.0468463, -0.0537267, 0.404929, -0.0370529, -1.85581, -1.45225, -0.000827564, 0.768375, -0.0226807, -1.3434, 0.0590199, -0.0480197, -0.0226666, -2.14127, 0.812872, -0.0426486, -0.0245021, 0.0615136, 0.345471, 0.347801, 0.00211081, -0.0799905, 0.000431062, -0.0251016, -0.0411716, -0.0201779, -0.00165902, -1.42624, 0.903999, -0.000525297, -0.0166139, -0.0179095, -0.0490351, -0.0422769, -0.0470271, -0.0235219, -0.0377026, -2.0482, 0.0128225, 0.331677, 1.23038, -0.0473092, 0.206427, -0.648265, -0.0162478, 0.0199419, -0.00544138, 0.842619, -0.0246851, -0.0231414, -2.6926, 0.214345, -0.0184816, -1.56181, -0.253585, -0.100118, 0.225377, -0.0023573, 0.0153178, 0.359956, -0.229739, -1.53369, -0.957097, 0.968383, -0.0401628, -0.014698, -0.893615, -0.0470395, -0.0204109, -0.147995, -0.0170704, -0.00356635, 0.58864, -0.0124042, -0.0944497, -0.00855726, -0.0154083, -0.645309, 0.405807, 0.00210213, -0.460843, -0.0982088, -0.0361736, -0.0461776, 0.493419, -0.886937, 0.668405, 0.295405, -0.016368, 0.0755731, -0.0205314, 0.29529, 0.281715, -0.0342814, -0.0932643, 0.00412505, -0.407352, -0.622841, -0.0296638, -0.0404127, -0.203014, 0.003191, -0.316848, 0.0208023, -0.406546, -2.64414, -0.00832144, -0.445473, -0.0389331, -0.0435543, 0.223992, 0.259657, -0.037053, -0.00767999, 0.00933315, 0.143214, -0.0203059, -0.137227, -0.0206429, -0.00452126, -0.0130095, -0.0628748, 0.291262, -1.17214, -0.556881, -0.808162, -0.59582, -0.129402, -0.0328058, 0.000317306, 0.0643802, -0.00626863, 0.586661, 0.825455, -0.0301091, -0.0391713, -0.0231266, -0.434965, 0.732641, -0.00845695, -0.00457101, -0.106202, -0.732738, -0.0331502, -0.0419898, -0.555379, -0.037971, 0.450621, -0.00405862, -0.0986404, -0.00384746, -0.00903074, -0.0352422, -0.0225734, -0.592166, -0.250142, 0.834151, -0.0193453, -0.0119507, 0.0168556, 0.00244173, 0.00112654, -0.0482451, -0.0117882, -0.020591, 0.0967317, -0.00227485, 0.0409543, -0.395455, -0.0261108, 0.0564522, -0.174268, -0.0464888, -0.0191883, -0.0402655, 0.22391, 0.171738, -0.0341707, 0.604014, 0.137642, -0.0336185, -0.23534, 0.331583, 1.09169, 0.262953, -0.00944948, 0.0117216, -0.48829, -0.947771, 0.423695, -0.560643, -0.539767, -0.0178084, -0.0110298, 0.643765, -0.061246, -0.00737238, 0.0202975, 0.000364569, 0.0184831, 0.0137678, 0.00206565, -0.00136514, 0.0058374, 0.00999589, -0.00824081, -0.00131587, -0.0125737, -0.00538487, 0.00422085, -0.00208258, 0.011304, 0.00164465, 0.00717083, -1.76164e-05, 0.00758645, 0.00907904, -0.00136887, -0.00516541, -0.0184734, 0.0123062, 0.00798076, 0.0095547, 0.00196431, 0.0199421, -0.00708801, 0.0034235, -0.0196675, -0.00694805, 0.000459244, 0.00831879, 0.00553561, 0.0144351, -0.0190096, -0.00310877, 0.00349582, 0.00419478, 0.0177775, -2.88756e-05, -0.0035924, 0.0243362, 0.0137802, 0.0163857, 0.00419371, -0.00825119, 0.0128039, -0.00185974, -0.00720379, 0.0052232, -0.0100177, 0.00648739, 0.0046, 0.0116477, 0.000204605, -0.000286349, 0.00811468, 0.000266896, 0.0087536, -0.0105808, 0.00419333, -0.00548353, -0.00751134, -0.00493898, -0.0116302, -0.00182971, 0.0105077, -0.0125283, 0.00429832, 0.0219491, 0.0233985, -0.0138677, -0.0193599, 0.00772183, -0.0082379, 0.000173546, -0.00788289, 0.0103708, 0.0034511, 0.012767, 0.0161549, 0.00156299, -0.00928343, 0.0107533, 0.0220782, -0.00611215, 0.00455255, 0.0130321, -0.00813614, -0.012425, -0.000440664, 0.000822021, -0.00122689, 0.00672702, 0.0230007, -0.000285065, -0.0206469, 0.0163275, 0.00562989, -0.0307528, -0.0202452, 0.0151086, -0.00274752, -0.00903757, 0.0137858, 0.00509347, 0.00916078, -0.0119308, 0.0149736, -0.00148478, -0.00616322, -0.0120903, -0.00975425, 0.00868078, 7.69734e-05, 0.0035433, 0.00186206, 0.010717, 0.0152506, 0.00152021, 0.00441384, -0.00309208, 0.00257746, -0.00953522, 0.0089127, 0.0127433, -0.426236, -0.00688711, -0.029439, 0.141811, -0.0399668, -0.0733562, -0.0353188, -0.0399924, -0.466052, -0.213321, -0.00818698, -0.863708, 0.591043, 0.000102018, -0.0346905, 0.212626, 0.141355, 0.632254, -0.152064, -0.00659071, -0.320482, -0.0132251, 0.769551, -0.688835, -0.353261, 0.163233, -0.0186311, -0.972643, -0.0782052, -0.0398357, 0.153859, 0.982496, -0.33971, -0.127405, -1.25509, -0.582637, -0.0283034, 0.00233206, -0.118142, -0.0469801, 0.13822, 0.819684, -1.26644, -0.0309216, -0.0702518, -1.40619, -0.0995085, -0.0335144, -0.0491045, -0.0594583, -0.0165323, -0.264478, -0.0615413, -0.0833231, 0.144034, -0.230584, -0.206229, 0.517877, 0.0536431, -0.0229707, -0.0416171, 0.900649, -0.0379044, 0.341261, -1.15699, 0.0120532, -0.333578, -0.0351064, -0.0129382, -0.13051, -0.025234, -0.0291157, -0.974945, 0.740553, -0.0134848, -0.0500166, -1.91873, -0.417531, 0.772351, -0.0247395, -0.502895, -0.0346486, -0.0259308, -0.027326, -0.0220393, -0.633932, -0.239516, -0.0151779, -0.0177511, -0.0310066, -0.0407294, -0.0423107, -0.0182263, -0.0240296, -0.0475952, -0.0327382, -0.408553, -0.248838, 0.546845, 0.484651, -0.0184749, 0.486232, -1.61843, -0.0197943, -0.0396026, -0.0188544, 0.216028, -0.604553, -0.0169497, -0.798692, -0.234263, -0.0185445, -0.508217, 0.180156, -0.726752, 0.521338, -0.035922, -0.0303993, 0.0377501, -0.157393, -0.308226, -1.18639, -0.0966957, -0.247831, -0.0167482, 0.544445, -0.0529472, -0.0335224, 0.0512319, 0.0017397, -0.00966693, 0.0600331, 0.0167981, 0.00198023, -0.0208785, -0.0109412, 0.0112777, -0.0134216, 0.00448303, 0.0244138, 0.00872236, 0.00409303, 0.000758416, -0.0300821, -0.00747082, -0.0252801, 0.00244686, 0.00831887, 0.00567669, 0.000228513, 0.0380366, -0.0308641, 0.00374648, -0.00112595, -0.0167409, -0.0242919, -0.0218811, 0.0114347, -0.00274706, -0.0130505, 0.00929319, -0.0139787, -0.056373, 0.042314, -0.0259481, -0.00179956, -0.0148518, 0.00916737, -0.0337746, 0.0317276, -0.00445029, -0.00438523, 0.00685035, 0.0300882, 0.00940522, 0.00294129, 0.0305704, 0.00473826, 0.00317861, 0.00685769, 0.00596096, 0.0172431, -0.020078, 0.0509794, -0.019071, 0.0231973, -0.00899527, -0.00596257, 0.00437128, 0.0254755, -0.00898133, 0.00547485, -0.0362383, 0.00473547, 0.0209697, 0.00395659, 0.0257569, -0.0187589, 0.00985672, -0.00482951, -0.00135508, -0.0354165, -0.00176185, -0.00688299, -0.0279083, 0.0085126, 0.0258919, -0.011929, 0.0252312, -0.013463, -0.0027999, -0.00824013, -0.0133941, 0.0458363, 0.0135693, 0.0141938, -0.00167378, 0.0176907, -0.00162748, -0.00153341, -0.00472726, -0.00186637, -0.0171315, 0.00925285, -0.0152491, -0.00214708, -0.0079302, -0.00097811, 0.00189979, 0.0110971, 0.0230511, 0.00411992, 0.00080615, -0.0040812, 0.0451295, -0.0571494, 0.00116573, 0.0157298, -0.0555209, 0.0095573, -0.000230035, -0.0280487, 0.0263543, 0.0173199, 0.00694939, -0.0172374, -0.0296216, 0.0189915, 0.0121877, -0.0193826, -0.0217925, 0.0044673, 0.00386401, 0.00935335, -0.0111062, -0.0152055, -0.057298, 0.00408311, -0.0405824, 0.43975, -0.0312947, -0.0465725, -0.013757, -0.0480294, 0.0730166, -0.357717, -0.00932821, -0.379319, 1.60364, -0.0250012, -0.0258318, -0.438529, 0.162586, -0.323154, 0.54746, -0.0247211, -0.415671, -0.00875686, -1.66754, -0.394721, -0.054833, 0.397428, -0.0179108, -0.530517, -0.16245, -0.0025922, 0.314342, 0.0239223, -0.0482075, 0.773552, 0.846254, 0.853695, -0.0231089, -0.00442391, -1.84649, -0.0248661, 1.11862, -0.298511, 0.670102, -0.0117544, -0.0330698, 0.622736, 0.276902, -0.00364929, -0.238764, -0.0233367, -0.0118215, -0.0216197, -0.0387195, -0.309527, -0.797458, 0.0993151, -0.580272, -0.428944, 0.745856, -0.0373003, -0.0226093, -1.85579, -0.0283492, 0.385173, -0.0874699, -0.00162056, -0.519071, -0.0326822, -0.619005, -0.496115, -0.0346581, -0.0053322, -0.532291, 0.247909, -0.0230767, -0.0247235, 0.687993, -0.275557, -0.96281, -0.027238, 0.250746, -0.00644898, -0.0287066, -0.0225732, -0.0188397, -0.321426, 0.219399, -0.350319, -0.017905, -0.0185974, -0.0294556, -0.0173606, -0.0281604, -0.0596403, -0.0074942, -0.0028702, 0.534216, -0.0200519, -0.526598, 1.02756, -0.00263289, -1.04441, 0.207495, -0.0142355, -0.0249163, -0.00650514, -0.672795, 0.0650972, -0.00995683, -0.198545, 0.199118, -0.0162603, 0.606977, 0.291284, 0.339892, -1.08347, 0.0138192, -0.0320412, 0.0364828, 0.388052, -0.629816, 0.525282, -0.225241, -0.0189401, -0.00743633, -0.106764, -0.0406925, -0.00512986, 0.944287, -0.0117393, -0.0323165, -0.461898, -0.0454219, -0.045453, -0.0154359, -0.0556012, 0.693936, -0.395036, 0.00306204, 0.680846, -0.312894, -0.0395825, -0.0393214, -0.0199325, -0.225553, -1.39423, -0.66446, -0.0234942, -0.367765, -0.00664595, 0.151006, -0.642348, -0.13604, 0.0400431, 0.00213436, 0.954813, -0.383131, 0.00531889, 0.241304, -0.177081, -0.0174648, 0.0105467, -0.0638503, -0.546661, -0.679514, -0.0219854, -0.6353, -0.012298, -0.84345, -0.456905, 1.12686, -0.0151184, -0.00263879, 0.32021, 0.0815541, -0.0185973, 0.314885, -0.0466498, -0.0271943, -0.0209694, -0.0538352, 0.00421121, -0.672828, -0.225706, 1.24019, -0.724616, 0.176812, -0.0436901, -0.0459855, -0.304388, -0.0213823, -1.23785, -0.140887, -0.0258844, 1.50366, -0.0250696, -0.0808267, 0.413137, -0.0334591, -0.0151056, 0.984846, -0.269767, -0.0305548, -0.0276878, -0.288517, 0.0564698, -1.04849, -0.0488597, 0.0748952, -0.0427808, -0.0219803, -0.028661, -0.0295434, 0.587553, 0.377184, -0.0641166, -0.00732905, -0.0286684, -0.0452234, -0.0183501, -0.0333028, -0.0224193, -0.00811026, -0.0171951, 0.803694, -0.00167708, 0.106562, -0.156598, -0.034946, -0.329749, 0.570256, -0.0449134, -0.00407897, -0.00814214, 0.248313, -0.786956, -0.0401588, -0.567307, -0.636893, -0.0491635, -0.769575, -0.0656698, -0.342882, -0.850357, -0.00136105, -0.024418, -0.100385, -0.034377, 0.376654, -0.473502, -0.853134, -0.0949073, -0.0390096, -0.895473, 0.0183434, -0.0186302, 0.00758734, 0.00365102, 0.0104696, -0.00326422, -0.00481607, 0.0034688, -0.00275073, -0.0158918, -0.0061511, 0.0132654, 0.00410114, 0.000252426, -0.000304552, -0.000592734, 0.00829303, 0.00947346, 0.00610908, -0.00762369, 0.0114126, 0.00270473, -0.00325871, -0.00794891, -0.0108151, -0.0071538, 0.0315827, -0.0106198, -0.0110002, -0.0101667, 0.0105306, 0.00835546, 0.00463715, -0.0136832, 0.00191136, -0.00666126, 0.000511546, -0.00496062, 0.00159149, 0.0183703, 0.00519446, 0.0103384, 0.007687, -0.0112667, 0.0070575, 0.00440267, -0.00218248, -0.0100158, 1.57497e-05, 0.0188152, 0.000103174, -0.00511348, 0.00496653, -0.00679927, -0.00119303, -0.00176404, 0.000412054, -0.0119288, 0.00121298, 0.0102469, -0.00339052, -0.000191366, 7.1759e-05, -0.00405121, -0.0141013, 0.0270326, 0.0116218, -0.00979213, 0.00386387, -0.0125276, 0.00970615, 0.0113213, -0.00472521, 0.0049748, 0.0177526, 0.0140129, 0.011822, -0.000544669, 0.0200429, 0.0059069, 1.52834e-05, 0.00267655, -0.00703307, -0.000238228, 0.00487022, 0.00670731, 0.00305112, -0.00814666, 0.0121902, -0.00854941, -0.0124891, -0.0208092, -0.000548823, -0.013662, -0.000661495, -0.00174427, -0.0201903, -0.00975355, -0.0217033, 0.00596148, -0.0034623, 0.00104501, -0.0204891, -0.00273835, 0.0202177, -0.0108711, -0.0110236, -0.000162792, 0.0278612, -0.0069022, 0.00985101, 0.00703167, 0.0156144, -0.0109414, -0.0136987, -0.0271109, 0.0112568, -0.000260287, 0.00824824, -0.00135013, -0.00875581, 0.0203187, 0.0165485, 0.0117277, -0.00468455, -0.00792601, 6.65197e-05, -0.00127145, -0.00239389, 0.00173813, -0.167187, -0.0188948, -0.0219826, -0.943579, -0.0425582, -0.0652028, -0.0195995, -0.0100381, -0.285168, -0.577232, -0.0391474, 0.348838, 0.104978, -0.0191379, -0.0320477, 0.568527, -2.00084, -0.347432, -0.146181, -0.0132763, -0.48197, -0.0282141, 0.177285, 0.396124, -0.0713278, 1.1191, -0.0108649, 0.0151774, 0.543837, -0.0307893, 0.184793, 0.672559, -0.0154, 0.585206, -1.05427, -0.407096, 0.461164, -0.0294673, -0.580337, -0.0182445, 0.227081, -0.467531, -0.509323, -0.0150127, -0.0247689, -0.343093, -0.207403, -0.0176117, 0.0147379, -0.066413, -0.000402901, -0.0348715, -0.0350812, -0.0357239, -1.49386, 0.337423, -0.282529, 0.784875, 0.643187, -0.0254051, -0.0348515, -1.30646, -0.024038, -0.585113, 0.0429987, -0.0190235, 1.02132, -0.0408824, -0.260664, -0.201746, -0.00763295, -0.0156907, -1.71554, 0.163426, -0.0315575, -0.0100015, -1.56635, 0.835137, -0.454652, -0.00791832, 0.625145, 0.00206636, -0.0115849, -0.050056, -0.0276507, 1.27401, -1.28894, -0.330875, 0.00147252, -0.0399892, -0.0319155, -0.0236266, -0.0622073, -0.0662445, -0.00751219, -0.045937, 0.750479, -0.0167399, -0.197514, 0.112137, -0.0215298, 0.788529, -0.876829, -0.0151377, -0.0393591, -0.0187549, 0.0934216, 0.289991, -0.0398582, 0.594648, 0.830151, -0.0176125, 0.561298, 0.0609146, -0.484137, -0.631791, 0.0021064, -0.007405, -0.938223, -0.343545, -0.462691, -0.520436, -0.0629152, 0.0460688, -0.0373862, -1.18462, -0.0238635, -0.0162883, -0.41571, -0.0197172, 8.74584e-05, -1.14579, -0.00971442, -0.0104225, -0.0298831, -0.0351163, -0.0301186, 0.574324, -0.00532907, -0.303526, -0.155768, -0.0258342, -0.00556012, -1.27249, -0.51741, -1.35219, 0.872367, 0.00564203, 0.4486, -0.01353, -0.595551, 0.345727, -0.0418418, -1.612, -0.00826471, -0.175384, -0.755815, -0.0416781, -0.310037, 0.298351, -0.065942, 0.915648, -0.372474, 0.209848, -0.00939133, -0.0207275, -0.431116, -0.0335227, 1.21409, -0.719923, 0.189901, -0.029278, -0.0343111, -0.138906, -1.70346, -0.0054567, 0.000376131, -0.0300987, -0.0395422, -0.0247169, -0.0225823, -0.132748, -1.79212, -0.695194, 2.2082, -0.854503, 0.682262, 0.014868, -0.0179032, -0.299012, -0.0210927, 0.216821, -0.608422, -0.0317207, 0.174426, -0.0326985, -1.5894, -0.654516, -0.00791911, -0.0160302, -0.0137578, -0.50091, -0.0140299, -0.0167314, -0.501351, -0.276084, 0.179787, 0.00220524, -0.929866, -0.0404154, -0.0249953, -0.0307281, -0.0195347, -0.128325, -0.0425251, -1.29753, -0.0139118, -0.0362909, -0.014779, -0.0314652, -0.0428884, -0.00793257, -0.0396492, -0.000345083, -0.873808, -0.0132448, -0.24967, -1.08162, -0.0199242, -1.37115, -0.136837, -0.00156907, -0.0700249, -0.00244875, -0.489883, 0.170873, -0.000385972, -0.0386907, 0.342214, -0.0159036, 1.11401, 0.11015, -0.377152, 0.0998466, -0.0259818, -0.0435175, -0.616866, -0.58707, 0.331156, 0.0453303, -0.776558, 0.129701, 0.0093219, 0.336004, -0.059959, -0.011234, -0.607719, 0.010784, -0.0171202, -0.0128506, -0.0284027, -0.0204332, -0.0364759, -0.0418753, -0.847666, 0.358856, 0.0020666, 0.0257184, -0.66298, -0.0256937, 0.00906586, 0.95576, -0.845674, -0.0580476, -0.104108, 0.00212481, 0.598168, -0.00830315, 0.423044, 0.897699, -0.012295, 0.0870489, 0.0022031, -0.128301, -0.385877, -0.0610801, -0.340252, -0.174054, -0.0519103, 0.405088, 0.0953825, 1.42255, -0.606553, -0.0324956, 0.60009, -0.0329847, -0.502433, -0.572534, 0.516459, -0.0308249, -0.0471675, 0.655336, 1.01752, -0.0187142, 0.132008, -0.0310226, -0.0256644, -0.0408491, 0.0308394, -0.058944, -2.0849, 0.88534, -1.33563, 0.226646, -0.487458, 0.0012971, -0.0277685, 0.350133, -0.014595, 0.158963, -0.93322, -0.0163607, 0.961989, -0.0244196, -1.18188, -0.137676, -0.00220893, -0.0152838, 0.348845, -0.22641, -0.0505878, -0.0098673, 1.48615, 0.426262, -0.288418, 0.00182722, -0.767101, -0.0171108, -0.0188868, -0.0103528, 0.00822125, -0.0992755, -0.879621, -0.774082, -0.00073015, -0.0353371, -0.00283228, 0.00209115, -0.0141755, -0.0357341, -0.0597073, -0.00817162, 0.427438, -0.000191087, -0.818988, 1.0874, -0.0395261, -1.14383, -0.496693, -0.00242852, -0.0559143, -0.000192681, -0.134935, -0.210605, -0.0181151, 0.182353, -0.471674, -0.0377214, -1.03448, -0.571656, -1.62284, 0.251102, -0.032562, -0.00806248, -0.345949, -0.477465, -0.409342, 0.797609, -0.136443, 0.108947, -0.0111904, 0.508718, -0.0707581, -0.00910991, 0.289708, 0.00440399, 0.00838071, 0.303732, 0.00319469, -0.0416496, -0.00512567, -0.0121457, -0.0290604, -0.0354716, -0.00186956, 0.212563, 0.144436, -0.00576014, -0.0204276, 0.21268, 0.024188, 0.0268067, -0.0678926, -0.00763834, -0.0273659, -0.00682251, 0.347908, 0.193162, 0.00998182, 0.172775, -0.00254718, 0.0680992, -0.00349272, -0.000596219, 0.251242, 0.134234, 0.0151604, 0.281854, -0.0836789, 0.178165, 0.182665, -0.031045, 0.0693767, -0.0364618, -0.0425355, 0.19857, -0.134748, 0.00689897, -0.0497969, 0.192822, 0.0279823, -0.0186485, 0.163462, -0.0194435, 0.00970801, 0.00353597, 0.022999, 0.202798, 0.123181, 0.322316, -0.0282117, 0.184818, 0.207922, -0.0434654, -0.00803128, 0.326316, -5.97078e-05, -0.21504, -0.05937, -0.0301969, -0.0618009, -0.0230527, 0.177086, 0.0354445, 0.000172555, -0.0212543, 0.107459, -0.0291698, 0.00109856, -0.000243722, 0.0329835, 0.168189, -0.00254465, -0.03593, 0.110485, -0.00421512, 0.00717746, -0.0353319, -0.0240134, 0.228555, 0.0317763, 0.0692208, -0.0207923, -0.00354834, -0.00607998, -0.00199249, -0.023981, -0.0701268, -0.00590022, -0.0285583, -0.0415068, -0.0165891, 0.0921217, -0.0757787, -0.0082404, -0.0404841, 0.0114181, -0.0137258, -0.010118, -0.00434945, 0.211257, 0.147966, -0.0418289, -0.0754505, 0.22616, -0.0240377, 0.214093, 0.182324, 0.128448, 0.341644, 0.00398031, -0.0211481, 0.204144, 0.235045, -0.0364916, -0.0537924, -0.126271, -0.0235026, -0.0124672, -0.0361188, 0.00283325, -0.00404248, 0.246947, -0.012123, -0.0432397, 0.195816, 0.0153552, -0.0321212, -0.0224118, -0.011072, -0.47419, -0.51631, -0.00106615, -0.972306, -0.559779, -0.00835836, -0.0243179, 0.293818, 0.0858429, -0.188042, -0.519494, -0.00891109, -0.413221, -0.0137738, 0.423114, -0.724912, 0.000884666, 1.03272, -0.0111808, -0.865096, -0.602676, -0.0406907, 0.0274001, 0.879587, -0.012451, -1.30059, -0.129085, -1.22716, 0.184955, -0.0204375, 0.0137728, -0.0174089, -0.425158, 1.0429, 0.798964, -0.0204261, -0.0115492, -0.167403, 0.561909, -0.0131174, -0.212138, -0.0205735, -0.00627283, -0.0161739, -0.00654214, 0.0993783, -0.158461, 0.021119, -0.340413, -0.834366, -0.757449, -0.0186331, -0.0425175, 0.249751, -0.0319541, -0.720979, 0.630415, -0.0173575, 0.0516186, -0.007743, -0.372018, -0.201057, -0.0161085, -0.00383025, -0.139845, -0.375729, 0.00149422, 0.00787268, -0.631234, -0.926351, -1.02135, -0.0135278, 0.222136, -0.0263458, -0.022746, -0.00869128, -0.0309597, -1.11017, -2.14421, -0.125078, -0.0146007, 0.0102288, -0.0261842, -0.0293189, -0.0488341, -0.0364675, -0.0216024, -0.0183283, -0.528618, -0.00840188, -0.603292, -0.13025, -0.0122835, 0.399619, -1.34857, -0.00635189, -0.0377543, 0.0115139, -0.43203, -0.0872381, -0.0406597, 0.630992, -0.860808, -0.0269506, 0.00533927, -1.51906, 0.958503, 0.521782, -0.0112395, 0.00446048, -0.204861, -0.0983523, 0.0419428, -1.13269, -0.798805, -0.288964, -0.0273045, -0.877823, -0.017033, -0.0171822, 0.030027, -0.00169091, -0.0141783, 0.0325636, 0.000585933, -0.00759489, 0.00500076, 0.00754052, -0.0202966, -0.00364534, 0.00274261, -0.00678344, -0.0225817, 0.00674007, 0.0282312, -0.0295574, -0.0091388, 0.0209372, -0.0157729, 0.00172128, -0.00207048, 0.0138996, -0.0216962, 0.00555552, -0.000715134, 0.0318753, 0.0167867, 0.013193, -0.002989, 0.00426547, -0.00457795, -0.00662729, -0.00832241, -0.00455337, -0.00512484, -0.00500948, -0.00309877, 0.0115097, -0.000732531, 0.00216189, 0.00818472, 0.0188898, 0.00446662, 0.00318734, 0.00913368, -0.0114451, -0.0327616, -0.00975856, -0.0172384, -0.0059953, 0.00118036, 0.00733121, 0.00990689, -0.0110828, 0.0263172, 0.00480351, 0.00103923, 0.00216034, 0.00617643, 0.00395491, -0.00282724, -0.00584259, 0.00767243, 0.00423147, 0.0193821, -0.00674116, -0.00277555, 0.0172976, -0.015857, -0.0161067, 0.00829113, 0.00588832, 0.00897441, -0.00336135, 0.00152568, -0.0206524, 0.0179837, 0.0266056, 0.0128696, 0.00837136, 0.0135194, -0.0140053, -0.0092617, 0.00169249, 0.0130481, -0.0209378, 0.0140057, 0.0354772, 0.0141563, -0.0170086, 0.0115112, 0.00214924, -0.0194173, -0.00543211, 0.0117842, -0.0106776, 0.00302505, -0.0187929, 0.0029225, -0.016344, 0.00878532, -0.0508937, 0.00673801, -0.0111285, -0.00190237, 0.0178324, 0.0308179, 0.0216618, 0.0112395, -0.00233059, -0.00281859, 0.00717888, -0.00537258, -0.00590811, 0.0151712, 0.00817696, -0.0114999, -0.00191707, -0.00696261, 0.00534894, -0.0134747, -0.0163396, 0.000825691, -0.0193094, 0.00219841, 0.00950493, -0.00791282, -0.000453596, 1.06193, -0.00230223, -0.0101159, 0.727892, -0.0107, -0.0894502, -0.00668678, -0.0180465, -2.46634, -1.06311, 0.00554638, -0.908719, 0.588992, -0.0113881, -0.00822744, -0.852961, -0.409785, -1.23257, -0.102936, -0.00119944, -1.49167, 0.00521315, -1.16712, -0.642603, -0.0582846, -1.06542, -0.0289089, 1.11291, -0.689855, -0.00807331, 0.219559, 0.0658016, -0.0297344, 0.0216565, 0.227017, -0.0264539, -0.352885, -0.0227929, -1.89231, -0.0438271, 0.743047, 0.149716, -0.113724, -0.0200989, 0.00322264, -0.160139, -0.31968, -0.0185708, -0.295857, -0.0259999, -0.0151705, -0.0113924, -0.00735612, -0.229175, 0.117431, 0.526121, -1.02565, -0.312352, -0.337025, -0.0332234, -0.0165907, -0.320509, -0.0331632, 1.10808, -0.321154, -0.0146065, -0.324358, -0.0148307, -0.599571, -0.509468, -0.0371438, -0.00415976, -1.16614, 0.225712, -0.00366155, 0.00476273, 1.22001, -0.0055343, -0.475324, -0.0205497, -0.468863, -0.0114646, -0.0229906, -0.0359005, -0.0176365, -0.548762, 0.121366, -1.3042, -0.0112006, -0.0255327, -0.0117694, -0.0293912, 0.0107816, -0.038011, -0.00757007, -0.0086841, -0.400286, -0.0153995, -0.124493, 0.616537, -0.0237489, -1.03464, 0.599909, -0.0247574, 0.00616558, -0.00910192, 0.467826, -0.0292359, -0.0352629, 0.551896, 0.323775, -0.0264561, 0.344082, 0.513055, 0.540437, -0.921795, 0.0124787, 0.00234414, -1.0533, -1.10449, -1.22304, 0.0145618, -0.25157, -0.343215, -0.025486, 0.336551, -0.0321233, -0.00464443, -0.357872, 0.0113403, -0.00619146, 1.29883, 0.00758846, 0.0288191, -0.0184066, -0.0122991, -0.874765, -2.15293, -0.00741124, -0.83182, -0.607144, -0.0115861, -0.024443, -1.40642, 0.625064, -0.0696187, -1.17423, -0.00811458, -1.91066, -0.00848687, -1.05216, -0.0637471, 0.046155, -0.0581591, -0.0122827, -1.59634, -0.895925, -0.0520745, 1.51554, 0.70017, 0.00251535, 0.800817, -0.0518981, -0.251945, -0.0891733, -0.00374877, -0.911771, -0.0404172, -0.0163951, -0.895115, -0.474498, 0.0133117, -0.0718237, 0.447245, -1.77407, 0.0037765, -0.0414519, -0.0329599, -0.00691421, -0.0109246, -0.0160933, -0.318687, -0.32317, 0.179797, 0.0622843, -0.317777, -0.611487, -0.0283678, -0.0470701, -0.481091, -0.0280299, 1.22228, -0.0278961, -0.00426802, 0.399284, -0.00845996, -0.462121, -0.0294362, -0.00191485, 0.00123766, -0.000716654, 0.11582, -0.00862776, -0.00772673, -0.265744, 0.193454, 0.600859, 0.00411678, -0.0815277, -0.00247915, -0.0274951, -0.0238555, -0.0206425, -0.526548, -0.98794, -0.875795, -0.0113906, 0.0150036, -0.0242899, 0.0202156, -0.0365372, -0.00747796, -0.00993015, -0.00942556, -1.17266, 0.0104867, -0.244279, -0.88433, -0.0353366, -0.256399, -1.68481, -0.0396881, 0.0106857, 0.00711244, -1.01278, 0.35389, -0.0286456, 0.215005, -0.674075, -0.0374248, -0.139673, 0.00765413, -0.778546, -0.967747, -0.0102706, -0.00901011, -1.4655, 0.512861, -1.73835, 0.234087, -1.42779, -0.0857035, -0.0200375, -0.456603, -0.00267803, 0.0150989, 0.529816, -0.000924836, -0.0153366, 0.299714, -0.026489, -0.0903114, -0.0208125, 0.00416109, -0.273344, 0.442871, 0.00698301, -0.686966, -0.359994, 0.000147326, -0.0168571, 0.308445, -0.191101, 0.146078, 0.25068, -0.00233139, 0.307676, -0.0299637, -0.410182, 0.253174, 0.0361524, 0.063381, -0.0210146, 0.1346, -0.42747, -0.00678675, -0.637476, -0.187567, 0.016537, -0.0759215, -0.389909, 1.26145, -0.241985, -0.0352138, -0.185643, -0.0241325, 0.21528, 0.554709, -1.1351, -0.0218504, -0.0427829, 0.0653606, -0.498827, -0.00762637, 0.295143, -0.00777569, 0.00478086, 0.0053607, -0.0269268, 0.409791, -0.298659, 1.18146, -0.412297, -0.495031, 0.561019, -0.040056, -0.0481433, -0.0882158, -0.0318084, 0.269787, -0.888397, -0.0279375, 0.226354, -0.0137527, -0.333258, -0.0850601, -0.0015733, 0.00220355, -0.197131, 0.303532, -0.0543632, -0.0245347, 0.216711, 0.997029, -0.076437, -0.00290567, -0.0649443, -0.0315026, -0.0209843, -0.0323492, -0.00667308, 0.481191, -0.866829, 0.111568, -0.0289816, -0.0264909, -0.0370183, -0.0324539, -0.05029, -0.0191806, -0.00613325, -0.018067, 0.102839, -0.0104056, -0.146225, 0.884963, -0.00999844, -0.0842259, 0.784778, -0.0370229, 0.0506181, -0.0159633, 0.228976, 0.446006, -0.0215105, -0.218262, 1.2694, -0.0239984, -0.12252, 0.0426814, -0.647191, -0.029855, 0.00149659, -0.000470728, 0.835155, 0.212105, 0.467485, 0.863335, 0.246101, 0.114967, -0.0223893, 1.8913, -0.00843419, -0.0288067, 0.000542515, -0.000446442, -0.0119236, 0.0326593, 0.0169955, -0.0159801, -0.0106056, -0.0179784, -0.0130336, 0.0156349, 0.000881335, 0.0480359, 0.0064128, -0.00849759, 0.0160156, 0.0153956, -0.0391838, -0.0231173, -0.00630533, 0.00114284, -0.0274839, -0.0144113, -0.0134847, -0.00149498, -0.016765, 0.0202125, -0.0104284, -0.0658865, 0.00657089, 0.00349693, -0.0502216, -0.00503106, -0.00281874, -0.00267768, -0.00289386, -0.00653425, -0.0228043, 0.0186587, -0.00729041, -0.0058272, -0.0144257, -0.0460256, 0.00750098, 0.0139949, 0.00184334, 0.00995826, -0.0576508, -0.00683195, -0.00696811, -0.0131303, -0.023897, 0.0076664, 0.0122594, -0.0095872, -0.0197625, -0.00216746, -0.025955, -0.0110926, 0.0035361, -0.000281747, 0.00525499, 0.0180152, 0.00891547, 0.0164699, -0.028713, -0.0111829, 0.0255564, -0.020647, -0.0160319, 0.02178, -0.00269781, -0.0148375, 0.00802329, 0.00160647, -0.000648409, 0.00979739, -0.0586951, 0.00581983, 0.0252096, -0.00277402, 0.0161038, 0.0174893, 0.00908744, -0.00416965, -0.00602481, -0.00181132, -0.000754032, 0.0108067, -0.00278893, -0.000861443, 9.62565e-05, 0.00501895, -0.000649347, 0.00107463, -0.00417488, -0.000830315, -0.00740081, 0.0145043, 0.00857054, 0.0256052, 0.00589809, 0.00130097, -0.0519979, 0.0107285, -0.0132408, 0.00433543, 0.0484653, -0.0149731, -0.00684946, 0.0149143, -0.012205, -0.00405365, 0.0192941, 0.0045515, 0.0178342, -0.00990939, -0.000706233, -0.00436682, 0.00723587, 0.0265288, -0.0277297, -0.0103557, -0.0209148, 0.0115211, -0.00724222, 0.0170884, 0.00290236, -0.00616536, 0.632194, -0.0231456, -0.0102105, -0.577052, -0.0269177, -0.101035, -0.000721497, -0.00966459, -0.220666, -1.72087, -0.00694715, 0.438249, 0.200452, -0.00301923, -0.0151103, -0.404949, -0.912774, -0.83215, -0.0671207, -0.0102103, -0.486006, -0.00668776, -1.30577, 1.04065, -0.0506264, -0.0243021, -0.00307999, 0.237887, -0.283386, -0.0424807, 0.505502, -0.682929, -0.019508, -0.663859, -1.50935, -0.14808, 0.145385, -0.0255967, -0.543077, -0.0485677, -0.424607, -0.188869, -0.46939, -0.00496412, -0.0457663, 0.743943, -0.058639, 0.00474216, 0.403987, -0.0118165, -0.00693862, -0.00522906, -0.0641893, 0.243879, -0.014973, -0.926213, -1.22982, 1.90453, 0.136227, -0.0431891, -0.0147216, 0.879228, -0.0154908, 0.63821, -0.24288, -0.0162692, -0.991676, -0.0108751, 0.485441, 0.278985, -0.0202419, -0.0213525, 0.0723383, -0.11164, -0.0498799, -0.0338819, 0.181568, 0.119416, -0.771582, 0.00264669, -0.664583, -0.00183765, -0.00781076, -0.0222774, -0.0163465, 0.207284, -0.371203, 0.275361, -0.0248162, -0.0351769, -0.00258168, -0.00280409, -0.0186066, -0.00539289, -0.000623278, -0.012071, -0.0750834, 0.0023524, -0.113789, -0.626259, -0.0178123, 1.26981, -0.232213, -0.0457587, -0.0117522, -0.00656709, -0.540426, -0.636002, -0.0386385, 0.499497, -0.479044, -0.0222786, 0.140389, 0.21237, -0.93654, 0.0527861, -0.0170394, -0.00388151, -0.625447, 0.200309, -0.356704, -0.274411, -0.281431, -0.0924094, -0.0335779, 0.862729, -0.0290382, 0.00103723, -0.482887, -0.0150607, -0.0238475, -0.389004, -0.0302874, -0.0780603, -0.0218427, -0.0149441, 0.690217, -0.721122, -0.0103653, -0.598687, 0.158149, 0.0107731, -0.00650177, -0.025961, 0.509025, 0.111319, -0.224649, -0.015819, -0.171535, -0.00124385, -1.39258, -1.04593, -0.22888, -0.89463, -0.00283406, -0.337692, -0.508988, -0.0378314, -0.10831, 0.633187, -0.0849798, 0.604709, -0.718802, 0.799769, -0.249177, 0.00699419, -0.683416, -0.00449437, -0.548811, -1.25839, -0.00727887, -0.0326589, -0.0677197, -0.846766, -0.248228, 0.00140577, -0.0150247, -0.0686506, -0.0310831, -0.0911398, -0.0721414, -0.471761, -0.00604237, -0.737464, 0.354576, 0.414663, 1.07436, -0.0390815, 0.00333585, 0.703907, -0.00778896, 0.228012, -0.638324, 0.00304638, -0.652731, -0.0390423, -0.0130262, 0.0640192, -0.0358418, -0.0152086, 0.546476, -0.0420474, -0.0262151, -0.0433692, -1.17489, -0.122811, 0.963784, -0.0330582, -1.66466, -0.0174326, -0.0256839, -0.0306352, -0.00534122, -0.167345, 0.0792731, 0.608419, -0.0111196, -0.0252725, -0.0335155, -0.0708504, -0.0482758, -0.0272232, -0.0301954, -0.0102391, -0.378792, -0.0361752, 0.533324, 0.598593, -0.0282725, -0.173758, -0.335213, -0.0242668, -0.0521608, 0.00256015, 0.422099, -0.721976, -0.0296725, -1.42626, -0.195682, -0.0179305, -0.583038, 0.710205, -0.447576, -0.536942, -0.0096854, -0.0340956, 0.766805, 0.112255, -0.0940709, 0.0680657, -0.52169, -0.30005, -0.0031754, -0.288981, -0.0645684, -0.0162755, -1.42106, -0.00648067, -0.0149346, 0.450917, -0.0183563, -0.0950835, -0.0187242, -0.0192925, 0.62631, -0.553089, -0.0221476, -0.949924, -0.169671, -0.0240575, -0.0396014, 0.300503, -0.769218, 0.860235, -1.05993, -0.0150456, 0.374333, -0.0121063, -0.520945, 0.135915, -0.0339241, 0.244271, -0.0134538, 0.271003, -0.43914, -0.0395512, 0.114357, -1.22435, -0.0136277, -1.10515, -0.195247, -0.358223, -0.00731018, -0.013997, -0.167455, -0.0177489, -1.11778, 0.459102, -1.04798, -0.0139753, -0.0896047, -0.670318, -0.252139, -0.0184915, -0.312555, -0.0442646, 0.00409322, -0.00633075, -0.0579764, -0.600777, -0.169839, 1.18059, 0.713959, 0.4836, 0.279262, -0.0234061, -0.0328316, 0.784326, -0.0217285, 0.314502, 1.14366, -0.00700403, -1.69212, -0.0400315, -0.594071, 0.132929, -0.0472978, -0.0251281, -0.850962, 0.901307, -0.0188489, -0.0299908, -0.992639, 0.162786, 0.8113, -0.0281747, -0.0231615, 0.000868754, -0.0375667, -0.0393539, -0.0353375, -0.444883, 0.0761611, 1.48103, -0.0280486, -0.0108184, -0.0323493, -0.000858336, -0.044991, -0.0629706, -0.00727229, -0.04793, 0.89323, -0.00250666, 0.640149, 0.554342, -0.0294455, 0.0649863, -0.458164, -0.0260649, -0.0732284, -0.00470157, -0.167821, -0.744326, -0.0134768, 0.0663418, 0.313888, -0.0267639, -0.638971, 0.373685, -2.08137, -0.158204, 0.00341013, -0.0207875, -0.596091, 0.423301, -0.803595, -1.10126, -0.586697, -0.106779, -0.0129515, 0.0394026, -0.00642717, -0.000509973, -0.123453, -0.00421329, 0.0193145, 0.20273, -0.0358572, 0.00446539, 0.000333788, 0.00174009, 0.416846, -1.10339, -0.000660475, -1.27702, -1.16783, -0.0196506, -0.00327312, 1.27726, 0.864752, -0.107198, 0.183015, 0.00737334, -0.317909, -0.0011275, -0.0130334, -0.728232, -0.109187, -0.197299, -0.0164025, 1.50507, 0.54451, -0.0196329, 0.225153, -0.432851, -0.0789403, -0.638271, -0.56771, -1.57524, 0.595599, -0.015022, -0.749561, -0.0713406, 0.270904, 1.62036, -0.649666, -0.00946305, -7.54832e-05, -0.671993, -0.845907, 0.00249938, -0.678895, 0.0138987, -0.00790746, -0.0123189, 0.0222869, -0.963648, -0.0059783, -0.240159, 1.21113, 0.801177, -0.00488003, -0.00485459, -0.0153567, 0.944272, 0.00171624, -0.933051, 0.671187, 0.00190747, 0.333115, -0.0151028, -0.0919529, 0.403487, 0.000142964, -0.0211053, 0.664165, -0.369743, -0.0361047, -0.000204639, -0.988516, 0.160073, 0.830676, -0.0439717, 1.14564, -0.0415545, 0.0119582, 0.041902, -0.00476327, -0.0861745, -1.00375, 0.776113, -0.0111884, -0.00237375, 0.00693397, -0.0215655, -0.00877352, 0.0121762, 0.0117985, -0.0499296, -0.247995, -0.0249041, -0.26236, -0.566452, 0.00466148, 0.382673, 0.205361, -0.0272527, -0.0119036, -0.00907943, 0.0818617, -1.1378, -0.0221887, -0.12607, -0.352429, -0.000456744, 1.15472, -1.00515, -0.165893, 0.642503, -0.0336218, -0.0122188, 0.383469, 0.393275, -0.739173, -0.477964, -0.361517, -0.446801, -0.0105497, -0.135182, -0.020963, 0.00871687, -0.198803, -0.00942021, 0.0126935, 0.13796, 0.00201584, 0.00167404, -0.0107815, -0.00185175, 0.39022, -0.929881, 0.00765354, -0.326509, -2.54482, 0.00127309, -0.00864106, -0.279934, -0.865984, 0.0844988, -0.831224, 0.00466839, -0.0979627, 0.00256944, -0.289784, -1.0859, -0.0334881, -0.346974, 0.000490833, 1.0133, -0.212012, -0.0472398, 0.66939, -0.844485, -0.0234253, -1.8439, -1.13509, 0.169218, -0.757265, -0.00910748, -0.238123, 0.00417837, -0.739908, -1.11531, -0.1605, 0.0132994, 0.0121475, -0.124934, -0.713618, -0.00693553, -0.019486, 0.0162892, 0.0105052, -0.0114898, -0.0222594, -0.168695, -0.531712, -0.495215, 0.165433, -0.622578, -0.449035, 0.0104576, 0.01659, -0.227083, 0.00191066, 0.512328, -1.60631, 0.00566083, -1.01932, 0.00411366, -0.658458, -0.197095, -0.034212, 0.00471721, -0.768351, 0.102679, -0.00257563, -0.0183124, -0.167821, -0.341838, 0.429463, -0.0140975, 0.547035, -0.0134068, -0.0011598, 0.00426232, 0.00766535, -0.0156079, -0.102321, 0.297225, 0.0218341, -0.00168615, 0.00746006, -0.0119978, 0.0140335, 0.00744486, 0.0220009, 0.00151278, -0.897698, 0.00117183, 0.681816, -1.4915, -0.00680967, -0.0434741, 0.267183, 0.00449231, 0.0322768, 0.0130299, -0.587207, -0.13904, -0.00551721, 0.171243, -0.0970588, -0.0100826, -0.625256, -0.633077, -0.716729, -0.230075, -0.00284668, -0.0267698, -0.438357, -0.668379, 0.786519, -0.294585, -0.526873, -0.119357, -0.00312504, -1.05182, -0.0316606, -0.0136801, 0.427533, -0.00936177, -0.00203558, -0.993197, -0.00762725, -0.0201981, -0.00146633, -0.0197082, -0.805374, -0.234236, -0.0171936, 2.30845, 0.476013, -0.0147264, 0.00109921, -1.62208, 0.00837034, -0.0482649, -0.956313, -0.00701861, 0.655945, 0.00399808, -1.52666, -0.83193, -0.0577227, -0.328811, -0.0297546, -0.254922, -0.792775, -0.045193, 0.154723, 0.304904, -0.0467656, -0.132732, 0.34455, -0.448669, 0.528271, 0.000598861, 0.38237, -0.0468469, -0.135613, 0.86798, -0.205829, -0.0121849, -0.0738226, 0.867736, -0.553955, 0.00445034, -0.142854, -0.0226094, -0.0119241, 0.0109987, 0.0168855, -0.70108, -1.3792, 0.20255, 0.786078, -2.61613, -0.771259, -0.0268078, -0.0111719, -0.218013, 0.0177756, -0.472904, 0.382787, -0.0252989, -0.681251, -0.0204764, -0.686006, -1.10892, 0.00025656, -0.0274917, 0.509573, -0.632844, 0.00459109, 0.00602587, -0.264343, -0.960238, -0.799045, -0.0107996, -0.16019, -0.0132582, -0.00525674, -0.0414779, -0.0261633, -0.881497, 0.0997574, -0.317426, -0.0270674, 0.000616181, -0.00271542, -0.00611118, -0.0196713, -0.00302775, -0.0112111, -0.0042797, -0.485771, 0.000985726, -1.55912, -0.425603, -0.0120693, 0.0214558, -0.468137, -0.0115121, -0.0480005, -0.0239612, -1.15364, -1.10078, -0.0270695, 0.451695, -1.37353, -0.0309835, -0.25786, -0.408772, 0.0649279, -2.51802, 0.0116033, 0.0128349, -0.551068, 0.961861, -0.56476, -0.385016, 0.159211, -0.00338312, -0.0188592, -0.0492283, -0.012949, 0.0181003, -0.0661523, 0.0181291, -0.0130952, 0.104141, -0.0233209, -0.121487, -0.023977, -0.0252681, -0.083006, -0.239285, -0.00193172, 0.188432, -0.474281, -0.0222292, -0.0512212, 0.139505, 0.299413, -0.252419, -0.570547, -0.0116352, -0.372448, 0.0203635, 0.812999, -0.469782, -0.0115866, 0.105837, -0.0188388, -0.27442, 0.15811, -0.00709863, 0.271723, 0.692811, 0.00189416, -0.285764, 0.480344, 0.0482972, 0.283306, -0.0267189, 0.225255, -0.0338075, -0.733206, -0.0320706, -0.0265843, -0.0287089, -0.0440914, -0.146247, -0.0710825, -0.0238561, -0.03616, -0.00460368, -0.0112795, -0.013287, -0.0236809, 0.064441, 0.333335, 0.0732929, -0.340318, -0.344565, -0.270056, -0.0351558, -0.0499385, 0.379275, -0.0318888, -0.189564, 0.350579, -0.00662055, -0.0150509, -0.010779, 0.0379131, 0.435526, -0.00218652, 0.00131958, -0.085556, -0.114294, -0.0135948, 0.00510978, -0.437167, 0.0101367, 0.424719, -0.00731025, 0.755158, -0.0225888, -0.0412773, -0.028123, 0.00689452, -0.022529, -0.0606719, 0.816728, -0.030777, -0.030678, -0.016146, -0.0112379, -0.039048, -0.0578785, 0.0103249, -0.0201956, -0.133142, -0.00857995, -0.0521247, -0.140124, -0.035363, -0.101596, 0.000226399, -0.0155458, -0.020866, -0.0222915, 0.499205, 0.0390479, -0.0169877, -0.453133, -0.31671, -0.0188066, 0.91165, -0.16328, 1.09687, 0.0991816, -0.0419106, -0.0058246, -0.305867, 0.131537, -0.259553, -0.624782, -0.786514, -0.495902, -0.0133945, -0.0629549, 0.00252726, 0.00662799, 0.0764013, -0.020376, -0.0345202, 0.379171, -0.0327716, -0.0818587, -0.0274447, -0.0376672, -0.459283, -0.26433, -0.0208888, -0.687748, -0.529881, 0.0120057, -0.0525712, 0.0768661, 0.260739, 0.242743, 0.702635, 0.00887232, 0.221625, -0.00621004, 0.405639, -2.28447, -0.199269, 0.653423, 0.00958501, -0.846388, 0.534646, -0.0117035, -0.942347, 0.297504, -0.0301567, 0.174744, -1.42433, -1.32059, -0.0586238, -0.0013195, -0.491025, 0.00223481, 0.0675505, 0.296585, -0.582353, -0.0405324, -0.0450183, -0.678713, 0.654173, -0.0276838, -0.621024, -0.0579598, -0.0464035, 0.00093494, -0.0496874, -0.471959, 0.152236, 0.550608, -0.433382, 0.13354, 0.355506, -0.0345222, -0.0294241, -0.182356, -0.0209594, -0.663596, 0.134464, 0.00107012, 0.321785, -0.0400615, -1.15247, -0.682627, 0.0093953, -0.0213573, 0.274313, 0.365893, -0.029594, -0.0406205, -0.279443, -0.242252, -0.0444699, -0.0256339, -0.215671, -0.0331156, -0.0235535, -0.0344151, 0.00964358, 0.244697, -0.558897, -0.14615, 0.0168204, -0.0380854, -0.0423634, -0.0756736, -0.0359342, -0.0484125, -0.0567715, -0.0393641, -1.7671, -0.00791334, -0.0414591, 0.131262, -0.030194, 0.86965, -0.232138, -0.0141278, -0.0533841, -0.0182161, 0.36085, -0.440313, -0.0361489, -0.619976, -0.380944, 0.0017809, 0.249997, -0.936284, 0.360368, 0.679751, -0.0113889, -0.0325484, -0.899856, 0.0554028, -0.424598, -0.031075, 0.14429, -0.252387, -0.0153007, 0.175802, -0.0606481, 0.00468679, 0.128662, -0.00171398, 0.00316481, -0.858336, 0.00942902, -0.0685185, -0.029762, -0.0136306, -0.564461, 0.100358, -0.01799, 0.16944, 0.112072, -0.00399886, -0.0139636, -0.0175467, 0.15185, -0.229968, 0.33089, -0.0114764, -0.217799, 0.0141877, -0.608925, -0.497475, -0.0572745, -0.61636, -0.0117396, -0.534615, -0.474674, 0.00343926, -0.257835, -0.414049, -0.00139808, -0.206306, -0.605206, -0.597998, 0.391522, -0.01897, 0.162089, -0.0366659, 1.09862, 1.10545, 0.524839, -0.0217706, -0.0922357, -0.320848, -0.0228722, -0.0241933, -0.48864, -0.0212817, 0.00134276, 0.0108929, -0.00649334, -1.01359, 0.669432, 0.258875, -1.67726, -0.138903, -1.37646, -0.0252963, -0.0248261, 0.114334, 0.00537206, -0.13486, 0.433475, -0.0106282, 0.67427, 0.0232303, -0.0794765, -0.784794, -0.0068448, 0.00614727, -0.617932, 0.360448, 0.00555792, -0.0180078, -0.772917, -0.135518, 0.594375, -0.00286814, -0.522696, 0.00453731, -0.00586691, -0.0161717, -0.0101852, 0.0836321, 0.55238, 0.557662, -0.0230678, -0.0204126, -0.00840458, -0.0129498, -0.010134, -0.035853, -0.0116636, -0.0212492, -0.165501, 0.0103714, -0.0969064, -0.427502, -0.0101348, -1.11067, 0.979289, -0.0314537, 0.00263518, -0.000371095, 1.20216, 0.480164, -0.0143281, -0.396062, 0.485173, -0.015569, 1.30203, 0.381191, 0.767904, -0.197573, -0.0116751, -0.00231274, 0.403946, -0.249105, 0.746786, 0.396222, -0.0603444, 0.0268933, -0.0270321, 1.18255, -0.00616809, -0.00499956, -0.367368, 0.004802, -0.00373519, 0.425709, 0.00800097, 0.0235695, -0.0139189, -0.0222293, -0.996918, 0.137311, -0.00823888, -0.898593, 1.69249, -0.00939764, 0.0102688, -2.29383, -2.15409, 0.111872, -0.159023, -0.0082881, 0.68691, -0.0160183, -2.44244, 0.365288, -0.0215729, -3.02519, -0.0168988, 1.08409, -0.302723, -0.0473944, -0.234606, 0.800476, 0.00330841, 0.34562, 0.805716, -1.18814, -0.572046, 0.0018985, -0.696552, -0.0147949, 0.963895, -0.724437, 0.382137, -0.00437617, -0.0357173, 0.403704, 1.4037, -0.0201754, -0.0839409, -0.00855373, -0.0120426, 0.00472377, -0.0218917, 0.112822, -1.70002, -0.71791, 2.10648, -0.312834, -0.356516, -0.0264894, -0.0451217, 0.821219, -0.0205371, 0.744903, -0.781104, -0.031422, 0.240721, 0.0126594, -0.7713, -0.29252, -0.0209352, -0.00521146, -1.29973, -1.06457, -0.00890593, -0.00342555, 0.805497, -1.11647, -2.30495, -0.0106961, -0.754628, -0.0111694, -0.031113, -0.021189, -0.0106048, -1.88312, 0.0142158, -2.59224, -0.0168676, 0.00381406, -0.018486, 0.00706609, -0.0298229, -0.049879, -0.0277349, -0.0471782, -0.788834, -0.00194184, -1.17955, 0.181659, -0.0306559, -0.799486, 0.0760529, -0.0292544, -0.0303637, -0.0111187, -0.489036, -0.685751, -0.0389363, 0.442589, -0.648029, -0.0310495, 0.0215848, 0.463454, -1.46372, -2.31842, -0.00957579, -0.00312326, -1.18982, -0.315619, -0.831287, -0.43179, -0.404447, -0.266709, -0.0275731, -0.231658, -0.0461378, 0.00672026, -0.315587, 0.00213099, -0.00840018, -0.777509, -0.00448084, 0.0172515, -0.0156805, -0.00801188, -0.820383, -0.0602756, -0.0152644, -0.160012, -0.851129, 0.0025001, -0.00408371, -0.698846, -0.555763, -1.129, 0.216779, -0.0171964, 0.215079, 0.00168561, -0.726439, -0.711453, -0.103004, 0.37238, -0.00919666, 0.988942, 0.23954, -0.00248173, 0.420401, -0.972643, -0.0297501, 0.453116, -0.161348, -0.474906, 0.483314, -0.0169881, -0.338019, -0.0343012, 0.780261, 0.177434, 0.887778, 0.0057915, -0.0433375, -0.323701, -1.00156, -0.0223479, -0.144669, -0.0176609, 0.00327256, 0.0154154, 0.00248474, -0.194419, -0.730883, -0.341369, 1.25059, -0.169739, -0.688781, -0.0216314, -0.0317034, -1.11749, -0.0276823, -0.0323392, 0.177019, -0.00650531, 0.256952, -0.0283786, -0.432783, -0.486291, -0.021012, -0.0211658, -0.516896, 0.0400119, -0.0640869, -0.00542876, 0.1927, -0.934717, 0.152588, -0.00486234, 0.216103, -0.0055544, -0.0246215, -0.00087217, -0.00879057, -0.164662, 0.6048, -0.398476, -0.0231533, -0.0152805, -0.0311873, -0.032163, -0.0230153, -0.05605, -0.0365747, -0.00687633, -0.966554, -0.0100164, 1.43464, 0.30299, -0.0182572, 0.844828, -0.0661059, -0.0373426, -0.0579363, -0.00480765, -0.724355, -0.321568, -0.0392585, -0.370444, -0.152083, -0.0250803, -0.564655, -0.86705, 0.0107706, -0.947654, -0.0120207, 0.00787968, 0.633076, -0.402335, -0.131921, -1.17093, -0.103247, -0.0571262, -0.0336994, -0.431577, -0.021577, -0.0242899, 0.107216, -0.0272887, -0.0382965, -0.00703884, -0.0286978, -0.0585689, -0.0303019, -0.0350924, -0.212769, -0.543506, -0.017995, -1.38335, 0.325319, -0.0551081, -0.0430326, 0.141119, 0.0521301, -0.240903, 0.295398, -0.00579124, -0.40126, -0.0235973, -0.122389, -1.10919, -0.397993, -0.318707, -0.028409, -0.140181, 0.120338, -0.0299822, -0.568573, 0.257481, -0.336114, 0.343454, -0.561729, -0.958243, -0.317484, 0.000702291, -0.185817, -0.0300108, -0.158102, 0.0843161, -0.283013, -0.0342372, -0.0448263, -3.14189, -0.486591, -0.0414759, -0.363238, -0.044534, -0.0341383, -0.267857, -0.0460657, -0.100134, 0.019452, -0.0789219, 0.236487, 0.617806, 0.304971, -0.0234389, -0.0406713, 1.23052, -0.02979, 0.144122, -0.228527, -0.0139986, 0.636278, -0.0216773, 0.0167892, -0.254849, -0.0185769, -0.0292768, -0.22876, 0.182055, -0.0376152, -0.0434251, -2.05951, -0.839118, 0.344027, -0.0335809, -0.519483, -0.0568375, -0.0335005, -0.0337007, -0.0195011, -0.950544, -0.332189, -0.132249, -0.0193255, -0.0338377, -0.0159151, -0.0514361, -0.0372949, -0.0271367, -0.0593348, -0.0488274, -0.168874, -0.274565, 0.228895, -0.0181091, -0.0436025, 0.500965, -1.39103, -0.0426232, -0.0422626, -0.0063844, 0.409124, -1.37557, -0.037142, 0.463237, -0.969551, -0.024549, -0.0895688, -0.0827568, 0.375837, -0.318492, -0.0412852, -0.03503, 0.0602767, -0.147925, -0.518551, -0.61379, -0.0492235, -0.261232, -0.0211021, -0.0215559, -0.0622935, 0.0065036, 0.779981, -0.00592618, -0.0228168, -1.07074, -0.0280406, -0.0585035, -0.00748456, -0.0192889, -2.15835, -0.624028, -0.0195763, 0.605419, -1.66856, -0.00895477, -0.00853156, -0.847996, -0.974609, -0.928893, -0.87839, -0.008943, -2.51246, -0.00905449, -1.24391, -0.345213, -0.0242083, -0.292057, -0.00367121, -0.748909, -0.860442, -0.0209162, -0.213595, 0.119747, 0.00112739, 0.271982, 0.779862, 0.23954, -1.26868, -0.0167762, -0.531657, -0.0592587, -0.0926154, 0.201274, 0.702021, -0.0343108, -0.114384, 0.708944, -3.06911, -0.00964635, -0.0806565, -0.0163881, -0.0115346, -0.0221424, -0.0154474, -0.108033, -1.65027, -0.163795, 0.0756514, -1.41916, -0.331488, -0.0550378, -0.0440292, -2.83501, -0.0249066, -0.350446, -1.38422, -0.0217016, 0.376523, 0.018681, -1.2205, -0.292829, -0.00328762, -0.00937941, -1.34246, -1.14219, -0.0444832, -0.0318713, 0.614981, -0.247672, -1.0076, -0.00233808, 0.442921, -0.00797742, -0.0288217, -0.0473499, -0.0170709, -1.18271, -1.66423, -1.75266, -0.0145778, -0.0277337, -0.0374311, -0.0228697, -0.0505763, -0.014875, 0.00607014, -0.0248922, 0.0703663, -0.0111909, -0.849826, -1.71705, -0.0386987, -0.5653, 0.122295, -0.0355251, -0.0020949, -0.00709671, 0.844363, 1.13869, -0.0213117, 0.846264, -0.205786, -0.00836516, 0.281107, -0.189892, 0.108702, -0.109533, 0.000446927, -0.00661656, -2.0316, -0.474346, -1.49686, 0.375686, -1.66506, -0.193344, 0.0014221, -0.167385, -0.0300778, -0.00932121, -0.180903, -0.0154745, -0.0178366, -0.744838, -0.0013231, -0.00245369, 0.015393, -0.00321875, 0.149474, -0.692367, 0.00246233, 0.0356606, 0.0670484, -0.00763162, 0.00634821, -1.90101, 0.813639, 0.515864, -0.23124, -0.00112878, -0.570818, 0.00601073, -1.44048, -0.252752, 0.0214101, -1.01788, -0.011752, -0.426533, 0.0417251, -0.0414394, 1.05427, 0.760665, -5.10918e-05, -0.557247, 1.63263, 0.542934, 0.121542, -0.00711125, -0.0303604, -0.0564529, -0.048804, 0.245236, 1.19213, -0.00816219, -0.0470343, 0.927321, 0.0902576, -0.00375304, -0.0796473, 0.00280105, -0.00176249, 0.00243954, -0.00712092, -0.0453931, 0.541181, 0.314237, -0.360496, -0.784588, 1.0065, -0.0170098, -0.00426971, -0.23895, -0.0137775, 0.817335, 0.704275, -0.00912886, -0.00961203, -0.0034257, 0.323899, 0.630098, -0.0201081, -0.000201159, -0.51755, 0.364226, -0.00205105, 0.00330448, 0.717242, 0.0899818, -0.379661, 0.0108177, -1.12213, -0.0100008, -0.0319855, -0.00645, -0.0200733, 0.339941, 0.40615, 0.162566, -0.019643, -0.00420098, -0.0280404, -0.0157922, -0.0280756, -0.0119959, -0.00890245, -0.00510148, 0.796776, 0.000876768, 0.326491, -0.287732, -0.00185637, -0.597906, -0.295148, 0.0103324, -0.00366727, -0.0147501, 0.18327, -0.994228, -0.0354786, 0.115377, -0.37057, -0.0354333, -0.834983, 0.685233, 0.496606, -0.136632, -0.00317543, -0.000892837, -0.458977, -0.439065, -0.657283, 0.255375, -0.299364, 0.0405014, -0.00711354, -0.826645, -0.0628359, -0.00931367, -1.15551, -0.000462322, -0.0236452, -0.75236, -0.0265857, -0.0856864, -0.00439793, -0.017445, -0.359233, 0.296344, 0.000907217, -1.41325, 0.98956, -0.0273604, -0.020685, -0.979679, 0.267653, -0.817147, -0.0642219, -0.0150263, 0.107039, -0.0029338, -1.52576, 0.577531, 0.0503613, -2.98593, -0.0195816, 0.134054, -0.414848, -0.00667792, 0.899676, 1.08123, 0.0132177, 0.617163, 0.43029, -0.0477344, -0.483426, -0.0260169, -0.0777601, -0.033415, 0.748222, -1.56952, 0.511263, -0.0209186, -0.0310631, 1.64937, -0.430621, -0.0238032, 0.134306, -0.0104896, -0.00155012, -0.00551077, -0.0203497, 0.27063, -0.232498, 0.183025, -1.39037, -0.385914, 0.941957, -0.0429563, -0.045953, 0.274725, -0.0346418, -0.892335, -0.00560582, -0.0388276, 0.363539, 0.0023256, 0.0674909, -0.521396, -0.00475371, 0.00828247, -0.180978, 0.369268, -0.041685, -0.0435505, 0.433397, 0.497256, 0.178298, -0.00315473, 0.220475, -0.0110952, -0.0386119, -0.0725587, -0.035258, 0.0834357, -0.866882, -0.755097, -0.00944942, -0.0322859, -0.0332967, -0.0178651, -0.0323212, -0.0580943, -0.008099, -0.0304717, -0.0774765, -0.0024888, -0.0979174, -1.43136, -0.0415243, -1.22084, -0.775002, -0.0194753, 0.0613383, -0.0141425, -0.966144, 0.493647, -0.0234454, -1.07654, 0.0454607, -0.0314854, 0.135047, 0.0825514, 0.381076, 0.136631, 0.00813983, 0.0124135, -1.29153, -0.509512, 1.1887, 1.11349, -0.109883, 0.0578805, -0.0282785, 0.0502677, 0.00363003, 0.00276598, 0.757242, 0.0106771, -0.0190317, 0.0940266, -0.0203081, -0.0717894, -0.0129689, -0.013696, -0.761506, 0.602972, -0.00892263, -0.746902, -0.00862837, -0.00286936, 0.00499513, 0.363164, 0.145419, 0.935637, 0.418124, 0.00249686, 0.500235, -0.017733, -0.683572, 0.248059, 0.0465305, -0.11269, -0.0155078, -0.223646, -0.116277, -0.0159494, -0.185242, -0.169336, 0.01459, -0.042761, -0.405126, 0.938301, -0.32671, -0.0309063, 0.00793434, -0.024451, 0.264228, 0.575759, -0.676406, -0.0263729, -0.0535367, 0.149864, -0.448914, -0.00954681, 0.347388, -0.00939983, -0.0157656, -0.00107072, -0.0617555, 0.494438, 0.0654202, 1.16938, 0.110426, -0.0829425, 0.838662, -0.0251459, -0.0478827, -0.419861, -0.0438775, 0.152868, -0.664744, -0.015854, -0.367741, -0.00049133, -0.186728, -0.0370634, -0.00378036, -0.00619033, -0.342642, -0.00141511, -0.0516396, -0.038958, -0.371612, 1.20852, -0.45043, -0.0195676, -0.385823, -0.00725652, -0.0213587, -0.0680711, -0.0188031, 0.809413, -0.352902, -0.0494076, -0.0122875, -0.0262376, -0.0376708, -0.0117293, -0.0352087, -0.029122, 0.00311498, 0.00644135, 0.427542, 0.0124798, 0.0788245, 0.832149, -0.0374205, -0.0400984, 0.642989, -0.0242316, 0.055987, -0.0145355, 0.030674, 0.944152, -0.0208953, -0.508729, 1.29615, -0.0211645, -0.267678, 0.607716, -0.287407, -0.215613, -0.000371964, 0.0193774, 1.02044, -0.235189, 0.550983, 0.748434, 0.450747, 0.0994202, -0.0234739, 1.42217, -0.00395706, -0.0164737, -0.0277106, -0.011186, -0.00725768, -1.22169, -0.0359791, -0.0941176, -0.0167718, -0.0354322, -0.842982, -0.596734, 0.00328535, 0.2178, 1.0824, -0.0127101, -0.00900924, 0.763859, 0.439644, 0.0137031, -0.218674, 0.00846924, 0.384047, 0.00175513, -1.25425, -0.301355, -0.08315, -0.367086, -0.0125653, 0.609668, -0.130029, -0.0119155, 0.724719, -0.136489, -0.00566061, -0.617313, 0.149588, -0.0891117, -0.0255571, -0.0227349, -0.451761, -0.0114885, 0.500126, -0.147205, -0.119062, -0.0336953, -0.0357942, -0.405281, -0.0643144, -0.0166027, -0.681468, 0.00130586, -0.0097402, -0.00550557, -0.0316559, -1.14489, 0.115966, -1.89104, 0.660137, 1.9959, 0.347281, -0.0511031, -0.0585389, 1.12249, -0.0219886, 0.705621, 0.306058, -0.0336629, -1.33518, -0.00551705, 0.350236, 0.493194, 0.00113038, 0.0096509, -0.24189, 0.71291, -0.0157349, -0.00701603, 0.319585, -0.373474, 0.357318, 0.00530135, -1.07764, -0.0121076, -0.0169814, -0.0237592, -0.012218, -0.858947, 0.0812092, 0.760832, -0.0472896, -0.0263627, -0.030159, -0.0107888, -0.0447277, -0.0480576, 0.0510248, -0.0175131, 0.618797, -0.0074126, 1.06172, -0.874717, -0.0387843, 2.31706, 0.620852, -0.025309, -0.0418766, -0.0268272, -1.00618, -0.813081, -0.0334568, 0.725037, -0.617051, -0.00131815, -0.4618, -0.614012, -0.234126, 0.556413, -0.0215122, 0.0204085, -0.018715, -0.325022, -0.741946, -0.568595, 0.406473, -0.117132, -0.0256325, 1.20652, -0.0244588, -0.000341087, 0.0715424, 0.00776376, -0.00163754, 0.0171557, 0.00141955, -0.00189504, -0.00492389, 0.0086142, -0.0362915, 0.119687, -0.0160047, 0.0545076, -0.119418, -0.0170935, 0.0212924, 0.0150061, -0.0258283, 0.0063829, -0.0242316, 0.00659475, 0.0265526, 0.000843588, -0.0226843, -0.00102718, 0.00030783, -0.058867, -0.00251628, 0.0381227, -0.0477146, 0.00368432, -0.130398, 0.0252362, -0.0198185, -0.0202836, 0.020528, 0.00745268, -0.107323, 0.015988, 0.00903617, 0.00268541, 0.0652723, -0.0459709, -0.078135, 0.0391121, 0.0100271, -0.00151568, -0.096012, 0.00118024, 0.0231271, 0.00542469, 0.00357766, 0.000157411, 0.00205506, 0.102371, -0.016052, 0.0376275, 0.0916155, 0.0421428, 0.0539348, -0.0016848, 0.0110248, -0.0854624, 0.00901982, 0.00875388, -0.00905089, -0.00803459, 0.0251107, -0.00324455, -0.0488898, -0.109685, 0.00983477, -0.0156492, -0.0319217, 0.0044956, -0.00747804, -0.00102153, -0.0412292, 0.0460726, 0.018804, -0.00665537, -0.0488545, 0.00574309, -0.0175869, -0.0159376, 0.00641017, 0.0278856, 0.0358907, 0.040019, 0.000240215, -0.00582518, -0.00197401, -0.00597187, -0.000554141, 0.00825419, 0.000957756, -0.00736343, 0.015469, 0.018489, -0.0106574, -0.0288983, 0.00141158, -0.00341618, -0.0695561, 0.00025, 0.00757467, 0.00893253, 0.084408, 0.0150594, -0.0172921, 0.0615522, 0.102097, -0.00306237, 0.00532927, 0.00549748, -0.1283, 0.0306354, 0.00839995, 0.00321759, 0.0196665, -0.0335999, 0.152734, 0.0377659, -0.0190957, 0.0917716, -0.00356497, 0.143965, 0.0145066, 0.00285524, 0.0536638, 0.00330268, 0.0123401, 0.264123, -0.0126918, -0.0144669, -0.0144872, -0.0305275, -0.0281694, 0.0388483, -0.0098408, 0.102953, 0.0623627, -0.0167479, 0.00469072, 0.0799391, 0.10854, 0.04194, 0.0269389, -0.0104165, 0.0223009, -0.0124122, 0.0290655, 0.0630847, 0.00133309, 0.256771, -0.0297178, 0.113927, -0.0114014, -0.00612947, 0.218956, 0.10196, 0.0121494, 0.245891, 0.00892266, 0.0646475, 0.206226, -0.00268114, 0.0411235, -0.0458675, 0.00227822, 0.069066, 0.0275201, 0.00316437, 0.00652309, 0.0229912, 0.0626469, -0.0100493, 0.0508275, -0.00663537, -0.00489181, -0.000994262, -0.0266017, 0.0772561, 0.156565, 0.0384743, -0.0699927, 0.0186279, 0.0723469, -0.0265775, 0.00663747, 0.0173622, -0.0112115, -0.00123263, 0.179297, -0.0307432, 0.00230102, -0.0046508, 0.0490873, 0.0760245, 0.000197492, -0.004145, -0.121176, 0.00905548, -0.0105397, -0.00227368, 0.0170831, 0.0724393, 0.0538471, -0.00419691, 0.100747, -0.0257167, 0.00200236, 0.00481491, -0.0131763, 0.035506, 0.0253727, 0.0781335, -0.00469049, -0.0103684, -0.000426985, -0.015629, -0.0196446, -0.0371669, 0.00723254, -0.0156174, 0.0584775, 0.00886055, -0.0134592, -0.0222276, -0.0136165, 0.00142092, -0.0207578, -0.0408271, 0.042408, -0.0116594, 0.0849452, 0.0602168, -0.0330341, 0.0137944, 0.0700449, -0.0352588, 0.0778696, 0.0676346, 0.0724886, 0.119349, -0.000554691, -0.00260871, 0.0652925, 0.0416431, 0.0563398, 0.0497783, 0.00266239, 0.0663343, -0.0307661, 0.0726042, 0.000235006, 0.00668775, 0.0638783, -0.00913278, 0.00208043, 0.354716, -0.0151364, -0.0155771, -0.0215169, -0.0179684, -0.0118997, 0.0523732, -0.0321275, 0.0881767, 0.0823216, -0.0149828, 0.0113946, 0.0525364, 0.128632, 0.0242222, 0.0425152, 0.00655759, 0.0436287, -0.0119595, -0.000907514, 0.0446825, -0.00596071, 0.21253, 0.00502325, 0.0585919, -0.00863129, -0.00999134, 0.22218, 0.155031, -0.00553209, 0.199961, -0.0373455, 0.0804031, 0.169937, -0.00856062, 0.0540262, -0.0414136, 0.0293005, 0.0543605, -0.000595718, -0.00833441, 0.0600234, 0.0511331, 0.0539708, -0.0084043, 0.070525, -0.0296499, -0.00172878, 0.00178822, -0.0213429, 0.0938814, 0.0993599, 0.0690617, -0.0836281, -0.0282409, 0.0656441, -0.0299024, -0.0256567, 0.0409779, -0.00556083, -0.0242714, 0.127723, -0.00638855, -0.0231766, -0.000790022, 0.0833024, 0.070703, 0.00265493, -0.0218534, -0.110397, -0.0017648, -0.0103861, -0.0113247, 0.0109757, 0.0803588, 0.0528576, -0.00585415, 0.152098, 0.00822581, -0.000579791, -0.0143633, -0.0276031, 0.070431, -0.0375672, 0.14678, -0.0189692, 5.65486e-05, 0.0140255, 0.00341153, -0.0314521, -0.0202803, -0.00732376, -0.0212805, 0.0503804, 0.00612324, -0.0214087, 0.00538604, -0.0281248, -0.00791531, 0.00962634, -0.0273134, 0.0501524, -0.00437508, 0.0738656, 0.0502322, -0.0320778, 0.0489026, 0.0615053, -0.0300671, 0.153543, 0.076487, 0.0839662, 0.0368029, 0.0014384, 0.00182571, 0.0675984, 0.0520121, 0.0473263, 0.0541597, 0.0285011, 0.0475445, -0.0253976, 0.061711, -0.00921977, -0.00661579, -0.609274, -0.00134113, -0.0242721, -0.113714, -0.028058, -0.0685448, 0.00131462, -0.0298939, 0.952645, -0.235306, -0.00433855, -0.629177, 0.092344, -0.0360128, -0.0384974, -1.44119, 0.467168, -1.86753, 0.787351, -0.00951702, 0.219975, -0.00114817, -0.727537, -0.936263, -0.102249, -0.0342135, 0.000178841, -0.0484515, -0.0713422, -0.0222877, 0.000493095, -0.00961764, -0.0548966, 0.481246, -0.761288, 0.298054, -0.747903, -0.00918521, 0.0189072, -0.0369315, 0.217758, -1.21389, 0.0112815, -0.00696936, -0.065049, -1.89938, -0.585208, -0.0213632, -0.53087, -0.0205937, -0.0441069, -0.0251994, -0.0122935, -0.326868, 0.106495, -0.551173, 0.400419, 0.878373, -0.0467414, 0.000427809, -0.0212667, 0.314903, -0.0084044, 0.282788, -1.17726, -0.0126141, -0.771049, -0.0331591, 0.173557, -1.12251, -0.0399688, -0.0281546, 0.706807, 0.151318, -0.0241819, -0.0209511, -0.426462, -1.1001, -0.434268, -0.0214954, -2.0176, -0.0616799, -0.0375432, -0.0244137, -0.0184272, -0.972918, 0.294058, -0.992946, -0.0157673, -0.0299372, -0.0498196, -0.0178661, -0.0289868, -0.0300593, -0.020973, -0.0237581, -0.26657, -0.00547534, 0.0350317, 0.0303274, -0.0270347, 0.0643601, -0.663865, 0.000537991, -0.062389, -0.0173385, -0.999072, -1.14672, -0.0175203, -0.410465, -0.954165, -0.0291971, 0.145109, 0.438712, 0.625544, 0.332362, -0.0279141, -0.0305426, 0.230202, -0.299407, 0.306966, 0.35842, -0.0764826, -0.0671835, -0.0319914, -0.1029, -0.0123459, -0.009515, 0.584375, -0.0106315, -0.00932033, -0.756033, 0.0247361, 0.0182618, -0.0199136, -0.00521422, -0.469532, -0.723303, -0.0275962, 0.74406, 1.06854, -0.010522, -0.0147209, 0.432967, 0.108522, 0.381916, -0.447104, 0.00212314, -0.730686, -0.00103235, 0.211085, -0.484663, 0.0543205, 0.465978, -0.0117902, 0.238832, 0.278829, 0.00162375, 0.775983, 1.15236, -0.0256598, 0.634994, -1.01234, 0.345331, -0.274396, -0.0283686, -0.368503, -0.0355676, 0.668544, 0.747537, -0.853258, -0.0194219, -0.0432314, 0.0568242, 0.23953, -0.0274632, 0.312714, -0.00198202, -0.0011027, -0.00442517, -0.0109241, 0.280792, 0.0930126, 0.333731, -1.42866, -0.49773, 0.122132, -0.048837, -0.0181701, -0.33375, -0.011176, 0.407942, 0.724672, 0.0016616, 0.0956443, -0.00989537, -0.202419, 0.193639, -0.0036467, -0.00213373, 0.384986, -0.414419, 0.0049962, -0.00253675, -0.0137964, -0.502398, -0.0799484, -0.015689, -0.299659, 0.0058081, -0.014984, -0.0324462, -0.0359998, -0.131496, -0.35343, 0.0586619, -0.00334597, -0.00852912, -0.0021174, 0.00668068, -0.0211214, -0.00578565, 0.00378194, -0.00641068, -0.284684, 0.0175101, 0.809177, -0.100662, -0.0339713, -0.272237, -0.313866, -0.0394648, 0.019034, 0.00533333, -0.718362, 0.693478, -0.0264652, -0.449889, -0.0566866, -0.0296072, 0.141139, 1.21942, 0.297351, -0.0120697, -0.00773572, 0.0103639, -0.40623, -0.357664, -0.495841, -0.424575, -0.674319, 0.0573879, -0.00237595, 1.55332, 0.0176616, -0.0331144, 0.0319202, 0.00500044, -0.00757907, -0.00955763, 0.0292856, -0.00148036, -0.0018906, -0.00392768, 0.0143827, 0.00191751, 0.00127661, 0.0127944, -0.00627425, 0.00632182, 0.000935563, 0.0363538, -0.0202918, 0.0119291, -0.018123, -0.00930196, -0.00804185, 0.0322864, -0.00188013, 0.00259478, 7.98154e-05, -0.053757, 0.00465183, 0.0272533, 0.00660221, -0.00389148, -0.032377, 0.00625098, 0.000926554, 0.000155401, -0.00904099, 0.0350859, -0.0153646, -0.00266522, 0.00322651, -0.00677469, -0.010164, -0.0284183, -0.000683408, 0.00140168, -0.00177139, 0.0411658, 7.67253e-06, -0.0109198, 0.0206822, -0.00334186, 0.00161385, -0.00416886, 0.0029943, 0.034693, -0.0107198, 0.0213182, -0.00507814, 0.0144767, 0.0221277, 0.00763251, -0.000492117, 0.0120304, -0.00115696, 0.00463252, -0.027963, 7.11631e-05, 0.00866991, 0.0148516, 0.0284621, 0.0053282, -0.000640455, 0.0149985, -0.00284559, 0.000714402, 0.0105465, 0.0102567, 0.0119978, 0.00900457, 0.0275504, 0.0204536, -0.0116685, -0.00950837, -0.00236417, 0.0200893, -0.0187741, 0.0135421, 0.0186028, -0.00248561, -0.00698286, -0.000205836, -0.0256945, 0.0101422, -0.0210567, -0.00738841, 0.00716621, 0.01188, -0.0104236, -0.00997874, -4.23194e-05, 0.0186494, -0.0132332, -0.015207, 0.00955841, 0.000486552, -0.00287872, -0.00901023, 0.0484001, -0.0116665, 0.00117928, -0.00251837, 0.0362081, -0.00411201, 0.00653207, 0.00532414, -0.0270482, 0.00744212, -0.00362136, -0.00156213, -0.00300099, -0.00536847, -0.00404642, 0.00750068, -0.000732609, 0.00173209, 0.00566885, 0.00578105, 0.0106453, -0.016031, 0.434233, 0.00145128, -0.0369219, 0.545094, 0.0240027, -0.003688, -0.00794794, -0.00765186, 1.12228, 0.06933, -0.00607257, -0.640661, -0.270098, -0.014415, 0.00274099, -0.661296, 0.286167, 1.20218, 1.28508, -0.00487854, -0.198612, 0.00983616, -0.737658, -1.07143, -0.0178801, -0.0520918, 0.000398322, 0.0654233, -0.313243, -0.0152387, -0.610088, -0.586782, 0.00711746, -0.865602, -0.484085, -0.784142, 0.122952, 0.0115918, -0.149561, -0.02323, 0.940484, 0.786056, 0.572718, -0.00753532, -0.0862115, -0.0942437, 0.360449, -0.00671031, 0.0939871, -0.00980142, -0.00633638, -0.0175406, 0.0266255, -0.0729147, 0.772454, 0.0561383, -1.41118, 0.00844334, -0.369313, -0.0268692, -0.03354, 0.455778, -0.0353611, -0.837507, 0.378958, -0.0174224, -0.0938624, -0.000520439, -0.542298, 0.233963, -0.022303, 0.0219063, -0.145786, -0.465006, 0.0107028, 0.0253623, -0.910364, 0.238403, 1.15071, -0.0126128, 0.838813, -0.00676204, -0.0247814, -0.00274116, -0.0137581, -0.431677, -0.99648, 1.76896, -0.0028948, 0.012001, -0.0330684, 0.0111173, -0.033062, -0.0235071, -0.0269816, -0.00775918, 0.842881, 0.00272315, -0.975363, -0.192346, -0.0191418, -0.489477, 0.972676, -0.0192011, -0.0290906, -0.0171398, -0.138563, 1.04975, -0.0240001, 0.173431, 0.552045, -0.0181384, 0.785086, -0.401178, 0.190037, 0.51583, 0.0184789, 0.0128426, -0.937551, -0.337159, 0.103319, 0.140865, -0.161165, -0.186766, -0.0102738, -0.0117941, 0.00991767, -0.00995102, -0.100216, -0.0190438, -0.0349479, 0.20823, -0.0438052, -0.063066, -0.0128774, -0.0227364, 0.376508, -0.265835, -0.0232511, -0.400409, 0.30717, -0.0496591, -0.0358044, -0.251449, 0.0764115, -0.0402729, 0.647877, -0.0174557, -0.0575171, -0.00837826, -0.165352, -0.567614, -0.095365, -0.458955, -0.023338, -0.262904, -0.407887, -0.0637304, 0.061859, 0.239907, -0.0195154, -0.502236, -0.622179, 0.201772, -0.248891, -0.0122098, -0.017701, -0.0654785, 0.204143, 0.0613159, -0.0895807, -0.0370498, -0.0771502, 0.0212624, -0.0868461, -0.0358669, 0.00232799, -0.0385311, -0.0305778, -0.0323572, -0.018902, -0.0588494, -0.286866, -0.245917, 0.163597, 0.613023, 0.11094, -0.0419888, -0.0263905, 0.650321, -0.0390151, -0.0562423, 0.157576, -0.0123973, 0.38744, -0.04699, 0.346682, -0.625934, -0.0524866, -0.0327505, 0.252667, -0.25443, -0.0408819, -0.0232735, -1.17088, -0.667377, 0.305252, -0.0334616, -0.594828, -0.0579591, -0.0387837, -0.0196757, -0.027142, -0.27742, 0.00980224, 0.236926, -0.0369695, -0.0446938, -0.0514967, -0.0553751, -0.0271686, -0.0163809, -0.0581549, -0.0229253, -0.0602886, -0.0320972, -0.232198, -0.155775, -0.0478282, 0.469829, -1.03514, -0.0257085, -0.0911774, -0.0232716, -0.152676, -0.712927, -0.0278334, 0.215026, -0.378719, -0.0536621, 0.442505, 0.312589, 0.845734, 0.0564427, -0.0536165, -0.0298579, -0.311275, -0.125477, 0.0204706, -0.133841, -0.284803, -0.0531723, -0.0286503, -0.0192171, -0.0519957, -0.0222892, -0.99869, 0.402632, -0.0428737, 0.0296756, 0.00217453, 0.00733305, -0.00127586, 0.00183676, -0.0331697, -0.0152743, 0.0320692, 0.0279054, 0.202822, -0.283274, -0.86283, 0.469136, -0.0240881, -0.0414872, 0.0259962, 0.0256895, -0.0457779, -0.029481, -0.0503003, -0.0137387, -0.0383941, 0.0284151, 0.0260852, -0.0278678, -0.00540367, 0.0158255, 0.00177844, -0.0243877, -1.12666, -0.749372, -0.13292, 1.88447, -0.39747, -1.56632, 1.55152, -1.65718, 0.016865, -0.00443731, 0.0139671, 0.017856, 1.82319, 2.49786, -0.506703, 0.246302, 0.395272, 1.27751, -0.59797, 0.576522, 0.00810389, 0.0173019, 0.00764902, -0.0297058, 0.0224986, 0.0431662, -0.0277468, 0.00930465, 0.106479, 2.08152, 1.27786, -0.963796, 0.594052, -0.770149, -1.23338, 0.067818, -0.876712, -0.181216, 1.62631, -0.881155, -0.548823, 0.934174, 1.41078, -0.363374, 0.0081414, 0.0114525, -0.0050452, 0.0178592, -0.0471941, -1.77944, 1.14183, -0.731489, -0.0318868, 0.0192688, 0.0233827, -0.0100369, 0.848109, -0.210075, -2.03224, -0.0951151, 0.175942, -0.435111, 0.394879, -1.31751, 0.00209807, -0.0424404, 0.134472, -0.0194946, 0.636342, -0.61121, -0.52964, 0.488145, 0.010304, 0.00538835, -0.0230582, 0.00756117, 1.34572, -0.64282, 1.21999, 0.093481, -1.18452, -0.23633, -0.506684, 1.37786, 0.0185521, 0.0341508, -0.0349425, -0.0333017, 0.445715, 0.223647, -0.56874, 0.909618, 0.108853, 0.126587, 1.22709, 0.226709, -0.0273708, -0.0379771, -0.00693685, -0.0215704, -0.735086, 0.675747, 0.631518, -0.102156, 0.469807, 0.582705, -0.363943, 0.551656, 0.990976, -1.16172, -1.25797, 0.433455, -1.15038, -0.729113, -1.35844, -0.886675, 0.0058154, -0.00793163, -0.073169, 0.0231217, 0.430335, -1.08823, -0.769648, -0.395795, -0.0311458, -0.019509, -0.0269282, 0.00429838, -0.991831, 0.466223, 0.440719, -0.0911619, 0.248185, -1.22842, 0.649478, 0.269723, 0.636144, 0.511144, -0.505726, 0.0111718, -0.00585456, -0.0213079, 0.000556861, -0.024799, -0.0313155, -0.00945823, 0.0351784, 0.00474701, 1.24526, 0.729033, -0.0553377, -0.293628, 0.416572, -0.484173, -0.516493, -1.2261, -0.0306422, 0.042758, -0.0309864, 0.0240959, -0.421491, 0.0631968, 0.121439, 0.352329, -0.0126243, -0.0195803, -0.025594, -0.00947101, 0.00439397, 0.00146754, -0.0283193, -0.0325368, 0.00846696, -0.0236361, 0.0251656, 0.0593432, -0.0061274, 0.0572584, -0.0105132, 0.00772954, -0.468999, 0.300983, 0.108767, 0.321735, 0.291842, 0.382771, -3.2398, 0.975488, -0.37577, -1.00352, -0.0519988, 0.276903, 0.913353, -0.267903, 0.984402, 1.08954, 0.816551, -0.526109, -1.91729, 0.96238, 0.0569077, 0.319667, 1.56245, 0.0210896, -0.00495781, -0.00220242, -0.00702836, 0.0261698, -0.0183637, -0.00176451, -0.000468823, -0.0184451, -0.974249, 0.302476, 0.243877, -1.4026, -0.00325863, 0.00420061, -0.0211062, 0.01928, 0.0193601, -0.463912, -0.588785, 0.584191, -1.38978, 0.331242, -0.216233, -0.32599, 0.0112132, 0.000661958, -0.0668782, 0.0109431, 0.414496, -0.280015, 0.43033, -0.628514, 0.0270398, -0.00594765, 0.0190561, 0.0202249, 0.203597, -0.90946, -0.528845, -0.176583, 0.301152, 0.299195, -0.00885629, -0.844604, -0.0115825, -0.00119328, -0.00447823, -0.029132, -0.0194347, 0.00435275, -0.00739663, 0.0101191, 0.33532, -0.86152, -0.255329, -0.445602, 0.165965, -0.158021, 0.409941, -1.66161, -0.0362073, 0.039203, 0.0215689, 0.0228233, -0.0151589, 0.00495678, -0.00118816, 0.0141101, 1.55008, -1.5983, 0.462712, -0.548743, 0.0308998, -0.620737, 0.571611, 0.210689, -1.32194, 0.116026, 1.21038, 0.291639, 0.0207792, -0.00853958, -0.0154588, -0.0470428, 0.441357, 0.79769, 0.818051, 0.0609824, -0.0285431, 0.0403597, 0.0195018, 0.0115905, 0.0191435, 0.00436463, 0.00350048, 0.0112656, -0.00397627, -0.0340592, -0.0307006, -0.00147182, 0.00696674, -0.0175458, 0.0209597, -0.0072607, 0.215076, -0.316304, 0.606517, 1.25122, -0.646384, 1.27064, -0.303992, -0.103441, -0.977422, -1.6652, 0.833683, -0.240619, 0.00840895, -0.0225047, 0.00596411, 0.000141409, -0.00387752, -0.0155348, 0.00721189, 0.00273874, -0.0173143, -0.0235528, 0.0305844, 0.0351411, 0.0370745, 0.0941242, -0.0345249, 0.00129792, -0.035967, -0.0071381, 0.0284786, 0.0231604, 0.00769255, 0.0373081, -0.0514193, -0.0241389, 0.0189506, -0.032813, -0.0751007, 0.0393213, 0.012646, 0.00802185, -0.0203327, -0.00746857, 0.264005, 0.269943, -0.446028, 0.09934, -0.00904706, -0.0399331, 0.0192741, 0.00143531, 0.316169, 1.7146, 0.0230188, -0.988572, 0.612348, -0.811516, -1.11345, -0.80788, -0.00466928, -0.0195898, -0.0142132, -0.016106, 0.132452, 0.246991, -0.407457, 1.12193, -0.646938, -1.1781, 0.830534, 0.193046, -0.000247373, -0.00455029, -0.0140857, -0.0173154, -0.0337698, -0.0300911, 0.029857, 0.0340117, 0.00520616, 0.00201924, 0.00477251, 0.00831611, -0.296992, -0.542615, 1.28218, -0.18194, -0.457305, 0.389758, 1.7728, -0.317524, -0.0334285, -0.033526, 0.0262911, -0.0178027, -0.870344, -0.357153, -1.4744, 0.117863, 0.136885, -0.345461, 0.300606, 0.423219, 0.0461238, 0.0208887, -0.0105053, -0.00014499, -1.05348, 0.245448, 0.419063, -0.841363, -0.60077, 0.955416, -0.669571, 0.333676, -0.131568, 0.50396, -0.421234, -0.655758, -0.318324, 0.288936, -0.833544, 0.544027, -0.00867354, -0.0120969, -0.0358021, 0.0222959, -0.014653, 0.018517, 0.0143138, 0.0225158, -0.432406, -0.698727, 1.75014, -0.362655, -0.0885146, -1.22756, -0.559936, -1.12581, -1.65361, -0.590952, 0.907509, -1.45111, 0.0617283, -0.546052, 1.10092, -0.227711, -0.794673, 0.511023, 0.624368, 0.545131, 0.0271308, -0.194324, 0.0931385, -0.0650241, 0.0242644, 0.0187255, 0.0118772, 0.0276857, -0.540064, -0.46509, 0.204313, 0.0513155, -0.018695, 0.00246456, 0.0147285, -0.0220116, 0.016556, 0.0200215, -0.00700903, 0.0169519, 0.852284, -0.185744, 0.116349, 0.39408};

You can train the model from scratch by running the following: 
```c++
// Set up Soft actor-critic agent.
SAC<ContinuousActionEnv, decltype(qNetwork), decltype(policyNetwork), AdamUpdate>
    agent(config, qNetwork, policyNetwork, replayMethod);

const std::string environment = "BipedalWalker-v3";
const std::string host = "127.0.0.1";
const std::string port = "4040";

Environment env(host, port, environment);

std::vector<double> returnList;
size_t episodes = 0;
bool converged = true;
size_t consecutiveEpisodesTest = 50;
while (true)
{
    double episodeReturn = 0;
    env.reset();
    size_t steps = 0;
    do
    {
        agent.State().Data() = env.observation;
        agent.SelectAction();
        arma::mat action = {agent.Action().action};

        env.step(action);
        ContinuousActionEnv::State nextState;
        nextState.Data() = env.observation;

        replayMethod.Store(agent.State(), agent.Action(), env.reward, nextState, env.done, 0.99);
        episodeReturn += env.reward;
        agent.TotalSteps()++;
        steps++;
        if (agent.Deterministic() || agent.TotalSteps() < config.ExplorationSteps())
            continue;
        for (size_t i = 0; i < config.UpdateInterval(); i++)
            agent.Update();
    } while (!env.done);
    returnList.push_back(episodeReturn);
    episodes += 1;

    if (returnList.size() > consecutiveEpisodesTest)
        returnList.erase(returnList.begin());

    double averageReturn = std::accumulate(returnList.begin(),
                                           returnList.end(), 0.0) /
                           returnList.size();

    std::cout << "Average return in last " << returnList.size()
              << " consecutive episodes: " << averageReturn
              << " steps: " << steps
              << " Episode return: " << episodeReturn << std::endl;

    if (episodes % 10 == 0)
    {
        data::Save("./" + std::to_string(episodes) + "qNetwork.xml", "episode_" + std::to_string(episodes), qNetwork);
        data::Save("./" + std::to_string(episodes) + "policyNetwork.xml", "episode_" + std::to_string(episodes), policyNetwork);
    }
    if (averageReturn > -50)
        break;
}

```

## Testing the trained agent

It is so amazing to see how just a matrix of numbers, operated in a certain fashion, is able to develop a walking gait. 

Thats the beauty of Artificial Neural Networks! 

In [14]:
// Set up Soft actor-critic agent.
SAC<ContinuousActionEnv, decltype(qNetwork), decltype(policyNetwork), AdamUpdate>
    agent(config, qNetwork, policyNetwork, replayMethod);

agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "BipedalWalker-v3");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {agent.Action().action};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();
std::cout << url;
auto video = xw::video_from_url(url).finalize();
video

 Total steps: 1069	 Total reward: 257.629
https://gym.kurg.org/dabc6e8bd1604/output.webm

A Jupyter widget